In [18]:
learning_rate_x = 0.0001
learning_rate_y = 0.0001
training_epochs = 10000
batch_size = 100
test_batch_size = 136

In [19]:
train_dataset_x = tf.data.Dataset.from_tensor_slices((x_train, y_train_x)).shuffle(buffer_size=100000).batch(batch_size)
train_dataset_y = tf.data.Dataset.from_tensor_slices((x_train, y_train_y)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset_x = tf.data.Dataset.from_tensor_slices((x_test, y_test_x)).batch(test_batch_size)
test_dataset_y = tf.data.Dataset.from_tensor_slices((x_test, y_test_y)).batch(test_batch_size)

In [20]:
def create_model_x():
    model=keras.Sequential() 
    model.add(keras.layers.Conv2D(filters=1024, kernel_size=(1,4),strides=[1,4],kernel_regularizer=tf.keras.regularizers.l1(0.01), input_shape=(4,4,1)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    return model

def create_model_y():
    model=keras.Sequential() 
    model.add(keras.layers.Conv2D(filters=1024, kernel_size=(1,4),strides=[1,4],kernel_regularizer=tf.keras.regularizers.l1(0.01), input_shape=(4,4,1)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.l1(0.01)))
    return model

In [21]:
model_x=create_model_x()
model_y=create_model_y()
model_y.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 4, 1, 1024)        5120      
_________________________________________________________________
flatten_3 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              4195328   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 1025      
Total params: 4,201,473
Trainable params: 4,201,473
Non-trainable params: 0
_________________________________________________________________


In [22]:
def loss_fn_x(model_x, images_x, labels_x):
#    w1x=sum(sum(sum(sum(tf.square(model_x.get_weights()[0])))))
#    w2x=sum(tf.square(model_x.get_weights()[1]))
#    w3x=sum(sum(tf.square(model_x.get_weights()[2])))
#    w4x=sum(tf.square(model_x.get_weights()[3]))
#    w5x=sum(sum(tf.square(model_x.get_weights()[4])))
#    w6x=sum(tf.square(model_x.get_weights()[5]))
#     w1x=sum(sum(sum(sum(tf.square(model_x.get_weights()[0])))))
#     w2x=sum(tf.square(model_x.get_weights()[1]))
#     w3x=sum(sum(tf.square(model_x.get_weights()[2])))
#     w4x=sum(tf.square(model_x.get_weights()[3]))
#     w5x=sum(sum(tf.square(model_x.get_weights()[4])))
#     w6x=sum(tf.square(model_x.get_weights()[5]))
#+0.001*(w1x+w2x+w3x+w4x+w5x+w6x)
    logits_x = model_x(images_x, training=True)
    loss_x = tf.reduce_mean((tf.square(logits_x-labels_x)))
    return loss_x
def loss_fn_y(model_y, images_y, labels_y):
#    w1y=sum(sum(sum(sum(tf.square(model_y.get_weights()[0])))))
#    w2y=sum(tf.square(model_y.get_weights()[1]))
#    w3y=sum(sum(tf.square(model_y.get_weights()[2])))
#    w4y=sum(tf.square(model_y.get_weights()[3]))
#    w5y=sum(sum(tf.square(model_y.get_weights()[4])))
#    w6y=sum(tf.square(model_y.get_weights()[5]))
#     w1y=sum(sum(sum(sum(tf.square(model_y.get_weights()[0])))))
#     w2y=sum(tf.square(model_y.get_weights()[1]))
#     w3y=sum(sum(tf.square(model_y.get_weights()[2])))
#     w4y=sum(tf.square(model_y.get_weights()[3]))
#     w5y=sum(sum(tf.square(model_y.get_weights()[4])))
#     w6y=sum(tf.square(model_y.get_weights()[5]))
#+0.001*(w1y+w2y+w3y+w4y+w5y+w6y)
    logits_y = model_y(images_y, training=True)
    loss_y = tf.reduce_mean((tf.square(logits_y-labels_y)))
    return loss_y

def grad_x(model_x, images_x, labels_x):
    with tf.GradientTape() as tape:
        loss_x = loss_fn_x(model_x, images_x, labels_x)
    return tape.gradient(loss_x, model_x.variables)
def grad_y(model_y, images_y, labels_y):
    with tf.GradientTape() as tape:
        loss_y = loss_fn_y(model_y, images_y, labels_y)
    return tape.gradient(loss_y, model_y.variables)

In [23]:
optimizer_x = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate_x)
optimizer_y = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate_y)
def evaluate_x(model_x, images_x, labels_x):
    logits_x=model_x(images_x, training=False)
    correct_prediction_x=tf.reduce_sum(abs(labels_x-logits_x))*28.9/len(labels_x)
    accuracy_x=abs(correct_prediction_x)
    return accuracy_x
def evaluate_y(model_y, images_y, labels_y):
    logits_y=model_y(images_y, training=False)
    correct_prediction_y=tf.reduce_sum(abs(labels_y-logits_y))*16.2/len(labels_y)
    accuracy_y=abs(correct_prediction_y)
    return accuracy_y

In [24]:
avg_test_acc_x_ref=1.
avg_test_acc_y_ref=1.
avg_train_acc_x_ref=1.
avg_train_acc_y_ref=1.
stop_rate_x=0.1
stop_rate_y=0.1
for epoch in range(training_epochs):
    avg_loss_x=0.
    avg_loss_y=0.
    avg_train_acc_x=0.
    avg_train_acc_y=0.
    avg_test_acc_x=0.
    avg_test_acc_y=0.
    train_step=0
    test_step=0
    if ((avg_test_acc_x_ref>stop_rate_x)or(avg_train_acc_x_ref>stop_rate_x)):
        for images_x, labels_x in train_dataset_x:
            grads_x = grad_x(model_x, images_x, labels_x)
            optimizer_x.apply_gradients(zip(grads_x, model_x.variables))
    #        loss_x = loss_fn_x(model_x, images_x, labels_x)
    #        avg_loss_x = avg_loss_x+loss_x
            train_step+=1
#    avg_loss_x = avg_loss_x / train_step
    train_step=0
    if ((avg_test_acc_y_ref>stop_rate_y)or(avg_train_acc_y_ref>stop_rate_y)):
        for images_y, labels_y in train_dataset_y:
            grads_y = grad_y(model_y, images_y, labels_y)
            optimizer_y.apply_gradients(zip(grads_y, model_y.variables))
    #        loss_y = loss_fn_y(model_y, images_y, labels_y)
    #        avg_loss_y = avg_loss_y+loss_y
            train_step+=1
#    avg_loss_y = avg_loss_y / train_step
    train_step=0
    
    
    for images_x, labels_x in test_dataset_x:
        acc_x=evaluate_x(model_x, images_x, labels_x)
        avg_test_acc_x=avg_test_acc_x+acc_x
        test_step+=1
    avg_test_acc_x=avg_test_acc_x/test_step
    test_step=0
    for images_x, labels_x in train_dataset_x:
        acc_x=evaluate_x(model_x, images_x, labels_x)
        avg_train_acc_x=avg_train_acc_x+acc_x
        test_step+=1
    avg_train_acc_x=avg_train_acc_x/test_step
    test_step=0
    
    for images_y, labels_y in test_dataset_y:
        acc_y=evaluate_y(model_y, images_y, labels_y)
        avg_test_acc_y=avg_test_acc_y+acc_y
        test_step+=1    
    avg_test_acc_y=avg_test_acc_y/test_step
    test_step=0
    for images_y, labels_y in train_dataset_y:
        acc_y=evaluate_y(model_y, images_y, labels_y)
        avg_train_acc_y=avg_train_acc_y+acc_y
        test_step+=1    
    avg_train_acc_y=avg_train_acc_y/test_step
    

    avg_test_acc_x_ref=avg_test_acc_x
    avg_train_acc_x_ref=avg_train_acc_x
    avg_test_acc_y_ref=avg_test_acc_y
    avg_train_acc_y_ref=avg_train_acc_y
    print('Epoch:', '{}'.format(epoch+1), '|| (avg delta x, avg delta y) ','|| avg train delta = ','(', '{:.4f}'.format(avg_train_acc_x), 'cm, ', '{:.4f}'.format(avg_train_acc_y), 'cm)', '|| avg test delta = ','(', '{:.4f}'.format(avg_test_acc_x), 'cm, ', '{:.4f}'.format(avg_test_acc_y), 'cm)')
    if (avg_test_acc_x_ref<stop_rate_x) and (avg_test_acc_y_ref<stop_rate_y) and (avg_train_acc_x_ref<stop_rate_x) and (avg_train_acc_y_ref<stop_rate_y):
        break
#'|| x_loss =', '{:.12f}'.format(avg_loss_x), '|| y_loss =', '{:.12f}'.format(avg_loss_y),

Epoch: 1 || (avg delta x, avg delta y)  || avg train delta =  ( 6.5352 cm,  1.8912 cm) || avg test delta =  ( 6.2563 cm,  1.7399 cm)
Epoch: 2 || (avg delta x, avg delta y)  || avg train delta =  ( 5.5139 cm,  1.7988 cm) || avg test delta =  ( 5.2849 cm,  1.6595 cm)
Epoch: 3 || (avg delta x, avg delta y)  || avg train delta =  ( 4.4682 cm,  1.8879 cm) || avg test delta =  ( 4.2943 cm,  1.6496 cm)
Epoch: 4 || (avg delta x, avg delta y)  || avg train delta =  ( 3.3512 cm,  1.5189 cm) || avg test delta =  ( 3.2185 cm,  1.3902 cm)
Epoch: 5 || (avg delta x, avg delta y)  || avg train delta =  ( 2.5921 cm,  1.4072 cm) || avg test delta =  ( 2.4841 cm,  1.1608 cm)
Epoch: 6 || (avg delta x, avg delta y)  || avg train delta =  ( 1.8909 cm,  1.3217 cm) || avg test delta =  ( 1.8169 cm,  1.1067 cm)
Epoch: 7 || (avg delta x, avg delta y)  || avg train delta =  ( 1.2486 cm,  1.2173 cm) || avg test delta =  ( 1.2012 cm,  1.0134 cm)
Epoch: 8 || (avg delta x, avg delta y)  || avg train delta =  ( 0.685

In [ ]:
# 17행 31열 (학습이 이루어지지 않은 영역)

In [46]:
test_one=np.array([[[	0.091086428	,	0.360874226	,	0.49478791	,	0.331322881	,],
[	0.268757442	,	0.096542407	,	0.398028818	,	0.371364791	,],
[	0.275943246	,	0.261929296	,	0.101199483	,	0.261007253	,],
[	0.276761246	,	0.407192993	,	0.432672556	,	0.101857965	]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([30/33],dtype=np.float32)
test_label_y=np.array([16/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 1.0200 cm || delta y = 2.1477 cm


In [47]:
# 5행 7열

In [48]:
test_one=np.array([[[	0.097713769	,	0.256358871	,	0.28632803	,	0.276126352	,],
[	0.370912956	,	0.089508408	,	0.263574981	,	0.391965488	,],
[	0.450002353	,	0.319168028	,	0.08937607	,	0.365187627	,],
[	0.348198827	,	0.351123577	,	0.269076932	,	0.09903081	]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([6/33],dtype=np.float32)
test_label_y=np.array([4/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.3281 cm || delta y = 0.0832 cm


In [49]:
# 6행 8열

In [50]:
test_one=np.array([[[	0.100408922	,	0.26100062	,	0.297618806	,	0.284301018	,],
[	0.361398809	,	0.092653412	,	0.268753774	,	0.393201309	,],
[	0.434453358	,	0.310601542	,	0.083866596	,	0.360374302	,],
[	0.333619969	,	0.343804922	,	0.268984386	,	0.098257107	]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([7/33],dtype=np.float32)
test_label_y=np.array([5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.3573 cm || delta y = 0.0322 cm


In [51]:
# 6.5행 3.5열

In [52]:
test_one=np.array([[[	0.09466807	,	0.231550653	,	0.271861704	,	0.280718676	,],
[	0.366122086	,	0.081478087	,	0.258236617	,	0.39898103	,],
[	0.433695657	,	0.292366753	,	0.075579302	,	0.363684319	,],
[0.329897025	,	0.317065743	,	0.245254944	,	0.100792946	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([2.5/33],dtype=np.float32)
test_label_y=np.array([5.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.1408 cm || delta y = 0.0823 cm


In [53]:
# 3.5행 11.5열

In [54]:
test_one=np.array([[[	0.097637581	,	0.276745107	,	0.290970979	,	0.256396771	,],
[	0.333666823	,	0.084733348	,	0.242173971	,	0.34480141	,],
[	0.413081217	,	0.312479193	,	0.073646506	,	0.321776891	,],
[	0.33293432	,	0.363077597	,	0.271692953	,	0.091212038	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([10.5/33],dtype=np.float32)
test_label_y=np.array([2.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.0849 cm || delta y = 0.3371 cm


In [55]:
# 4.5행 21.5열

In [56]:
test_one=np.array([[[	0.088080767	,	0.316322341	,	0.329592912	,	0.252405077	,],
[	0.275418248	,	0.084438994	,	0.246008016	,	0.303934564	,],
[	0.360480699	,	0.318339428	,	0.074965047	,	0.283362866	,],
[	0.30484062	,	0.392951395	,	0.303958395	,	0.087506458	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([20.5/33],dtype=np.float32)
test_label_y=np.array([3.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.1335 cm || delta y = 0.2667 cm


In [57]:
# 7.5행 32.5열

In [58]:
test_one=np.array([[[	0.086169553	,	0.375018307	,	0.412511154	,	0.274506509	,],
[	0.231997682	,	0.093242947	,	0.291770717	,	0.28705421	,],
[	0.31283136	,	0.330191604	,	0.088837348	,	0.25159492	,],
[	0.29226257	,	0.439605341	,	0.371849016	,	0.089095354	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([31.5/33],dtype=np.float32)
test_label_y=np.array([6.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.9310 cm || delta y = 0.1858 cm


In [59]:
# 6.5행 3.5열 (면적이 1/3로 줄어든 경우)

In [60]:
test_one=np.array([[[	0.059325045	,	0.143705082	,	0.166999287	,	0.177885254	,],
[	0.237884857	,	0.051584033	,	0.164642627	,	0.262596442	,],
[	0.285398722	,	0.191246464	,	0.044898621	,	0.240981837	,],
[	0.207429683	,	0.19946683	,	0.151020675	,	0.066049975	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([2.5/33],dtype=np.float32)
test_label_y=np.array([5.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.0731 cm || delta y = 0.1035 cm


In [61]:
# 3.5행 11.5열 (면적이 1/3로 줄어든 경우)

In [62]:
test_one=np.array([[[	0.062022645	,	0.179257345	,	0.187888374	,	0.168764282	,],
[	0.22805242	,	0.063640955	,	0.164666705	,	0.238109121	,],
[	0.286948039	,	0.216060578	,	0.047005556	,	0.22474898	,],
[	0.229754974	,	0.249657508	,	0.185197833	,	0.070145707	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([10.5/33],dtype=np.float32)
test_label_y=np.array([2.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.1417 cm || delta y = 0.1148 cm


In [63]:
# 4.5행 21.5열 (면적이 1/3로 줄어든 경우)

In [64]:
test_one=np.array([[[	0.057285406	,	0.208765163	,	0.216296308	,	0.169361881	,],
[	0.179434061	,	0.052913	,	0.155934171	,	0.200318242	,],
[	0.238937045	,	0.20907716	,	0.044803552	,	0.189227691	,],
[	0.202735671	,	0.260065864	,	0.198237074	,	0.059149701	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([20.5/33],dtype=np.float32)
test_label_y=np.array([3.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.2548 cm || delta y = 0.1688 cm


In [65]:
# 7.5행 32.5열 (면적이 1/3로 줄어든 경우)

In [66]:
test_one=np.array([[[	0.058624565	,	0.254104939	,	0.27661388	,	0.188231565	,],
[	0.148995755	,	0.058024551	,	0.182144665	,	0.186310561	,],
[	0.203605123	,	0.21268779	,	0.052404284	,	0.164372392	,],
[	0.197241651	,	0.296248784	,	0.246232405	,	0.059918039	,]]],dtype=np.float32)
test_one[0,:,0]+=-0.00238
test_one[0,:,1]+=0.001496742
test_one[0,:,2]+=0.009334
test_one[0,:,3]+=-0.00588
test_one[0,0,0]=0
test_one[0,1,1]=0
test_one[0,2,2]=0
test_one[0,3,3]=0
test_label_x=np.array([31.5/33],dtype=np.float32)
test_label_y=np.array([6.5/18],dtype=np.float32)

test_one=np.expand_dims(test_one, axis=-1)

for i in range(1):
    for j in range(4):
        test_one[i,j,0:4,:]=test_one[i,j,0:4,:]/max(test_one[i,j,0:4,:])

x=model_x(test_one, training=False)
y=model_y(test_one, training=False)
print('delta x =','{:.4f}'.format(abs((x[0,0]-test_label_x[0]))*28.9),'cm','|| delta y =','{:.4f}'.format(abs((y[0,0]-test_label_y[0]).numpy())*16.2),'cm')

delta x = 0.9646 cm || delta y = 0.2405 cm


In [67]:
x_train.shape

(560, 4, 4, 1)

In [68]:
x_test.shape

(1330, 4, 4, 1)

In [69]:
model_x.variables

[<tf.Variable 'conv2d_2/kernel:0' shape=(1, 4, 1, 1024) dtype=float32, numpy=
 array([[[[-0.03493034, -0.00839552, -0.01631716, ..., -0.0103792 ,
           -0.02846906, -0.02262888]],
 
         [[-0.00395262, -0.00189935,  0.01184789, ..., -0.00151808,
            0.01752426,  0.00443666]],
 
         [[-0.00351641,  0.03634366,  0.01818467, ..., -0.01192902,
            0.01948604,  0.00402039]],
 
         [[ 0.01326311, -0.00722994, -0.03487876, ...,  0.00558616,
            0.00345415, -0.02396   ]]]], dtype=float32)>,
 <tf.Variable 'conv2d_2/bias:0' shape=(1024,) dtype=float32, numpy=
 array([ 7.7382691e-04, -5.1277573e-04, -6.6239823e-05, ...,
         1.5647316e-04, -1.2521846e-04,  1.3277805e-03], dtype=float32)>,
 <tf.Variable 'dense_4/kernel:0' shape=(4096, 1024) dtype=float32, numpy=
 array([[-0.00795986,  0.02028064,  0.02247395, ...,  0.02632295,
         -0.02162702, -0.00300399],
        [ 0.02672403, -0.00155047,  0.03128961, ...,  0.00815688,
         -0.0088302 ,  0

In [ ]:
remove.model_x.variables[1]

In [ ]:
model_x.get_weights()[6].shape

In [ ]:
w1x=sum(sum(sum(sum(tf.square(model_x.get_weights()[0])))))
w2x=sum(tf.square(model_x.get_weights()[1]))
w3x=sum(sum(tf.square(model_x.get_weights()[2])))
w4x=sum(tf.square(model_x.get_weights()[3]))
w5x=sum(sum(tf.square(model_x.get_weights()[4])))
w6x=sum(tf.square(model_x.get_weights()[5]))

In [ ]:
w6x

In [ ]:
avg_test_acc_x=0
acc_x=0
test_step=0
for images_x, labels_x in test_dataset_x:
        acc_x=evaluate_x(model_x, images_x, labels_x)
        avg_test_acc_x=avg_test_acc_x+acc_x
        test_step+=1
avg_test_acc_x=avg_test_acc_x/test_step
test_step=0

In [ ]:
avg_test_acc_x

In [ ]:
avg_test_acc_ref_x

In [ ]:
not 0

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from random import shuffle
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import os

In [2]:
x_test=np.array([[[	0.109434342	,	0.238589958	,	0.25673569	,	0.258792364	,],
[	0.407905154	,	0.093874632	,	0.254000982	,	0.395377646	,],
[	0.485885885	,	0.323996463	,	0.081595254	,	0.370534408	,],
[	0.382072501	,	0.349828124	,	0.25471866	,	0.099717967	,]],
[[	0.104949193	,	0.232680587	,	0.251049251	,	0.25328152	,],
[	0.407339243	,	0.090059773	,	0.253270834	,	0.394904272	,],
[	0.484895058	,	0.322832457	,	0.082046195	,	0.369377255	,],
[	0.38404278	,	0.351398866	,	0.256389893	,	0.103802613	,]],
[[	0.10782474	,	0.232846618	,	0.251265073	,	0.253957981	,],
[	0.406125877	,	0.090450689	,	0.252042036	,	0.39357787	,],
[	0.488934827	,	0.327310058	,	0.084730739	,	0.373856654	,],
[	0.38378137	,	0.351821789	,	0.256802022	,	0.100781039	,]],
[[	0.106633896	,	0.235484878	,	0.25374128	,	0.255828723	,],
[	0.410157157	,	0.097535099	,	0.256565564	,	0.39773928	,],
[	0.48482842	,	0.322867218	,	0.081031814	,	0.369332907	,],
[	0.382293612	,	0.34996916	,	0.254910605	,	0.100663145	,]],
[[	0.105067121	,	0.232136264	,	0.25057601	,	0.25277285	,],
[	0.406027182	,	0.089102752	,	0.251766802	,	0.393434627	,],
[	0.486316499	,	0.324307948	,	0.08448279	,	0.370797033	,],
[	0.386475575	,	0.353673314	,	0.258782689	,	0.107374858	,]],
[[	0.110968122	,	0.234832693	,	0.253352571	,	0.256015464	,],
[	0.407433438	,	0.092748054	,	0.253255989	,	0.394784844	,],
[	0.486728339	,	0.324901958	,	0.082411458	,	0.371413508	,],
[	0.382743573	,	0.35060686	,	0.255472142	,	0.100032643	,]],
[[	0.105144607	,	0.233302763	,	0.251462638	,	0.253735684	,],
[	0.408169778	,	0.090594896	,	0.253970224	,	0.395457615	,],
[	0.48486011	,	0.322752521	,	0.081488878	,	0.369231541	,],
[	0.383684706	,	0.350942665	,	0.255928281	,	0.102738673	,]],
[[	0.106892874	,	0.232537826	,	0.250907944	,	0.25344066	,],
[	0.405950438	,	0.089707989	,	0.251639058	,	0.393274676	,],
[	0.4881456	,	0.326308258	,	0.087695986	,	0.372702409	,],
[	0.384751744	,	0.352879795	,	0.257886579	,	0.101669266	,]],
[[	0.107635468	,	0.23669354	,	0.254901031	,	0.256876961	,],
[	0.409006874	,	0.095858652	,	0.255259924	,	0.396572813	,],
[	0.485324068	,	0.323281926	,	0.081022164	,	0.369788481	,],
[	0.382287027	,	0.34993716	,	0.254849441	,	0.100120858	,]],
[[	0.105187023	,	0.232438499	,	0.25081358	,	0.253020346	,],
[	0.406578832	,	0.089412024	,	0.252282058	,	0.393857786	,],
[	0.485843041	,	0.323768034	,	0.083568285	,	0.370238774	,],
[	0.385341099	,	0.352563626	,	0.257589058	,	0.105658411	,]],
[[	0.107666511	,	0.24699705	,	0.26266124	,	0.257228048	,],
[	0.387930015	,	0.088555694	,	0.247018824	,	0.379481236	,],
[	0.471076321	,	0.323661157	,	0.085320772	,	0.360577451	,],
[	0.372474497	,	0.354656901	,	0.259792097	,	0.099731247	,]],
[[	0.102063772	,	0.24034251	,	0.256104266	,	0.250857492	,],
[	0.390767276	,	0.094156784	,	0.250244976	,	0.382369171	,],
[	0.46898532	,	0.321336973	,	0.079740732	,	0.358392888	,],
[	0.369947459	,	0.351813415	,	0.256925283	,	0.098461424	,]],
[[	0.104366713	,	0.240154092	,	0.256075378	,	0.25117602	,],
[	0.389067002	,	0.088642978	,	0.248090737	,	0.380524461	,],
[	0.468794822	,	0.321036058	,	0.081262012	,	0.358106239	,],
[	0.373590586	,	0.355016619	,	0.260161441	,	0.104423112	,]],
[[	0.104588162	,	0.24383275	,	0.259523627	,	0.25408938	,],
[	0.388349883	,	0.089942843	,	0.247461695	,	0.379952079	,],
[	0.471570457	,	0.324120697	,	0.082206104	,	0.361249986	,],
[	0.370706452	,	0.352828243	,	0.257975081	,	0.098325342	,]],
[[	0.101760065	,	0.239245267	,	0.255165956	,	0.249879625	,],
[	0.39194248	,	0.091175762	,	0.251001422	,	0.383408655	,],
[	0.468154707	,	0.320416684	,	0.079375934	,	0.357488558	,],
[	0.371258925	,	0.35300391	,	0.258137516	,	0.100705612	,]],
[[	0.10728727	,	0.241847036	,	0.257756397	,	0.252960529	,],
[	0.38825098	,	0.088456814	,	0.247219867	,	0.379706412	,],
[	0.470303499	,	0.322681942	,	0.083944291	,	0.359599199	,],
[	0.373522743	,	0.355665333	,	0.260901337	,	0.100669665	,]],
[[	0.102530686	,	0.241200307	,	0.256850439	,	0.251560357	,],
[	0.3897455	,	0.092619521	,	0.249001826	,	0.381308139	,],
[	0.469528476	,	0.321958726	,	0.080267573	,	0.358916635	,],
[	0.370074686	,	0.351941655	,	0.256975237	,	0.098230618	,]],
[[	0.103358542	,	0.239748678	,	0.255562393	,	0.250492635	,],
[	0.389743305	,	0.089201484	,	0.248799046	,	0.3812243	,],
[	0.468629401	,	0.320813284	,	0.080675673	,	0.357838653	,],
[	0.372863186	,	0.354293342	,	0.259450545	,	0.103208717	,]],
[[	0.105873091	,	0.245055114	,	0.260815815	,	0.255264354	,],
[	0.388392168	,	0.089448237	,	0.247338954	,	0.379784805	,],
[	0.472342069	,	0.324967295	,	0.087437789	,	0.361742915	,],
[	0.371541684	,	0.353595281	,	0.258760001	,	0.098718173	,]],
[[	0.102036153	,	0.239905821	,	0.255770782	,	0.250370499	,],
[	0.392021605	,	0.096316357	,	0.251523843	,	0.383490523	,],
[	0.468438219	,	0.320659148	,	0.079503028	,	0.357689189	,],
[	0.370695767	,	0.352416761	,	0.257511351	,	0.099548785	,]],
[[	0.106305661	,	0.250575368	,	0.262880635	,	0.250040805	,],
[	0.373570016	,	0.087657539	,	0.243215639	,	0.367193998	,],
[	0.454520194	,	0.319633843	,	0.080322125	,	0.347141989	,],
[	0.362904756	,	0.357710228	,	0.262023936	,	0.098405135	,]],
[[	0.100411671	,	0.249179449	,	0.261293058	,	0.247918154	,],
[	0.373375116	,	0.089821471	,	0.243258	,	0.36704425	,],
[	0.456261944	,	0.321402862	,	0.079476793	,	0.348984146	,],
[	0.359922561	,	0.354476473	,	0.258574626	,	0.096389789	,]],
[[	0.102372323	,	0.248627853	,	0.26078454	,	0.247872236	,],
[	0.376314865	,	0.089982586	,	0.24611846	,	0.369976097	,],
[	0.453594691	,	0.318556746	,	0.077918549	,	0.346038435	,],
[	0.362526937	,	0.35666897	,	0.260852937	,	0.101323673	,]],
[[	0.103100772	,	0.25256666	,	0.264581267	,	0.25107873	,],
[	0.372800348	,	0.087733343	,	0.242600775	,	0.366511315	,],
[	0.456776042	,	0.321951943	,	0.083692058	,	0.34929391	,],
[	0.361082727	,	0.355874311	,	0.260075388	,	0.096817515	,]],
[[	0.100088045	,	0.248022897	,	0.260254946	,	0.246901102	,],
[	0.375117317	,	0.092842193	,	0.24528386	,	0.368773973	,],
[	0.45447731	,	0.31947932	,	0.077869631	,	0.347020623	,],
[	0.36031632	,	0.354804025	,	0.258951416	,	0.097570002	,]],
[[	0.10485825	,	0.249823829	,	0.262073225	,	0.249129538	,],
[	0.374114771	,	0.088040824	,	0.243661812	,	0.367661728	,],
[	0.454220164	,	0.319240809	,	0.079408803	,	0.346711213	,],
[	0.364576665	,	0.359365162	,	0.263558583	,	0.099899029	,]],
[[	0.101217742	,	0.250125748	,	0.262186023	,	0.248763206	,],
[	0.373043488	,	0.088914356	,	0.242806227	,	0.366656096	,],
[	0.457370505	,	0.322551711	,	0.080565291	,	0.350144055	,],
[	0.36006331	,	0.354680869	,	0.258763707	,	0.096110025	,]],
[[	0.101295144	,	0.248116966	,	0.260295081	,	0.2471607	,],
[	0.377220174	,	0.090896856	,	0.247066287	,	0.37077144	,],
[	0.453614149	,	0.318661019	,	0.077657395	,	0.346137736	,],
[	0.361697745	,	0.355959289	,	0.260145667	,	0.100000287	,]],
[[	0.104693233	,	0.254018354	,	0.26607914	,	0.252579432	,],
[	0.372920055	,	0.087509281	,	0.24267324	,	0.366631723	,],
[	0.455995181	,	0.321130144	,	0.082399442	,	0.348454596	,],
[	0.361663951	,	0.356477749	,	0.260722056	,	0.097154944	,]],
[[	0.100070938	,	0.248375224	,	0.260525015	,	0.247150466	,],
[	0.3744741	,	0.091595048	,	0.244501478	,	0.368074486	,],
[	0.455225041	,	0.320258455	,	0.078571436	,	0.347788997	,],
[	0.359983922	,	0.35440852	,	0.258498958	,	0.096726467	,]],
[[	0.103161013	,	0.259499379	,	0.270046088	,	0.248298419	,],
[	0.362254035	,	0.095189662	,	0.244775166	,	0.359019922	,],
[	0.438705408	,	0.317308876	,	0.077558422	,	0.335134652	,],
[	0.352640824	,	0.358971181	,	0.264416854	,	0.100053519	,]],
[[	0.099281105	,	0.259349162	,	0.269628278	,	0.247467313	,],
[	0.358181193	,	0.086651507	,	0.240095615	,	0.354907253	,],
[	0.44032867	,	0.319215856	,	0.081717825	,	0.336834928	,],
[	0.34992272	,	0.356709249	,	0.261954401	,	0.093820826	,]],
[[	0.099657234	,	0.257674682	,	0.267994898	,	0.246215407	,],
[	0.359315553	,	0.090238915	,	0.241462752	,	0.356038412	,],
[	0.440212139	,	0.318940936	,	0.078519409	,	0.336679849	,],
[	0.349952451	,	0.356272116	,	0.261705365	,	0.095585252	,]],
[[	0.10209376	,	0.262842091	,	0.27310059	,	0.250787813	,],
[	0.360342919	,	0.088138707	,	0.242256686	,	0.357012392	,],
[	0.438592942	,	0.317316101	,	0.078334967	,	0.334993965	,],
[	0.352828276	,	0.359800115	,	0.265216205	,	0.096342489	,]],
[[	0.098134519	,	0.257550394	,	0.267966098	,	0.245725388	,],
[	0.358139891	,	0.087559418	,	0.240169031	,	0.354865659	,],
[	0.442103156	,	0.321231312	,	0.084919345	,	0.338633883	,],
[	0.349220666	,	0.355880351	,	0.261255388	,	0.093570808	,]],
[[	0.102203759	,	0.258880581	,	0.269388101	,	0.247587377	,],
[	0.361172174	,	0.09350341	,	0.243672677	,	0.357895786	,],
[	0.438955602	,	0.317561905	,	0.077469265	,	0.335340909	,],
[	0.35150566	,	0.357812816	,	0.263203299	,	0.098203099	,]],
[[	0.099613773	,	0.259801213	,	0.269991572	,	0.247986979	,],
[	0.358762082	,	0.086847222	,	0.240523655	,	0.355276865	,],
[	0.439737341	,	0.318521156	,	0.080419471	,	0.335995577	,],
[	0.350557624	,	0.357311825	,	0.262570809	,	0.094290372	,]],
[[	0.099123422	,	0.257618049	,	0.267890886	,	0.245957247	,],
[	0.35874359	,	0.089247563	,	0.240833848	,	0.35544485	,],
[	0.440768807	,	0.319479794	,	0.078882457	,	0.337363122	,],
[	0.34956184	,	0.355963405	,	0.261180087	,	0.094636456	,]],
[[	0.10352429	,	0.264180755	,	0.274450403	,	0.252174058	,],
[	0.361470775	,	0.089138865	,	0.24335736	,	0.358097823	,],
[	0.438491539	,	0.317000386	,	0.077598993	,	0.334642378	,],
[	0.353815936	,	0.360804557	,	0.266270373	,	0.097316412	,]],
[[	0.098746627	,	0.258415674	,	0.268718795	,	0.246646219	,],
[	0.357987079	,	0.086793033	,	0.239838586	,	0.354577854	,],
[	0.441400653	,	0.32036323	,	0.083541959	,	0.337749414	,],
[	0.349454443	,	0.356170927	,	0.261498603	,	0.093554116	,]],
[[	0.10045447	,	0.266662218	,	0.275145891	,	0.244059481	,],
[	0.345812211	,	0.090046856	,	0.238451339	,	0.343769291	,],
[	0.42914956	,	0.319757852	,	0.077807155	,	0.327343424	,],
[	0.345177848	,	0.364329851	,	0.267903041	,	0.094937002	,]],
[[	0.097081649	,	0.266915774	,	0.275223179	,	0.24387703	,],
[	0.347146209	,	0.088111247	,	0.23928281	,	0.345003622	,],
[	0.427938749	,	0.318585731	,	0.078154037	,	0.326039057	,],
[	0.346899199	,	0.366426812	,	0.270032307	,	0.094347319	,]],
[[	0.097197358	,	0.265039856	,	0.273483827	,	0.242295191	,],
[	0.344539376	,	0.087207638	,	0.236882029	,	0.34242702	,],
[	0.432042579	,	0.323225047	,	0.085492366	,	0.330258138	,],
[	0.344079745	,	0.363389936	,	0.266846187	,	0.0924525	,]],
[[	0.100794975	,	0.27118269	,	0.279492764	,	0.247911685	,],
[	0.347610323	,	0.093272546	,	0.240360984	,	0.345463554	,],
[	0.428151244	,	0.318711968	,	0.077135469	,	0.326099106	,],
[	0.347144207	,	0.366014592	,	0.269568584	,	0.098057372	,]],
[[	0.095993079	,	0.265288636	,	0.273732122	,	0.242232549	,],
[	0.345306665	,	0.086788105	,	0.237490964	,	0.343159355	,],
[	0.429158939	,	0.319850187	,	0.080319411	,	0.327052321	,],
[	0.345139727	,	0.364667924	,	0.268096249	,	0.092678127	,]],
[[	0.09951594	,	0.266200275	,	0.274736301	,	0.243574571	,],
[	0.345335737	,	0.089091356	,	0.237765225	,	0.343206567	,],
[	0.430166061	,	0.320682213	,	0.078671683	,	0.328224647	,],
[	0.344496197	,	0.363693081	,	0.267229412	,	0.093820123	,]],
[[	0.098039163	,	0.267965191	,	0.276171628	,	0.244801708	,],
[	0.34835493	,	0.089195179	,	0.240382659	,	0.346016118	,],
[	0.427911779	,	0.318463298	,	0.077653308	,	0.32579967	,],
[	0.347989396	,	0.367471308	,	0.270970278	,	0.095239657	,]],
[[	0.096666526	,	0.264920823	,	0.273299933	,	0.242087925	,],
[	0.344705482	,	0.086931359	,	0.236775812	,	0.34239059	,],
[	0.43103593	,	0.321987195	,	0.083674471	,	0.329116002	,],
[	0.34417836	,	0.363430929	,	0.266843994	,	0.092206198	,]],
[[	0.101949534	,	0.272235351	,	0.280495164	,	0.248927056	,],
[	0.346963255	,	0.092054418	,	0.23958789	,	0.344741526	,],
[	0.428609479	,	0.319097596	,	0.077390872	,	0.326465423	,],
[	0.346280022	,	0.36524177	,	0.268731685	,	0.096557227	,]],
[[	0.096590295	,	0.266060292	,	0.27442522	,	0.243004198	,],
[	0.345795581	,	0.087006962	,	0.237952026	,	0.343570579	,],
[	0.428527173	,	0.319213965	,	0.079290156	,	0.326393847	,],
[	0.345779974	,	0.365283808	,	0.268746397	,	0.093158162	,]],
[[	0.09962501	,	0.281546881	,	0.286960949	,	0.247269192	,],
[	0.337584199	,	0.088852428	,	0.239591785	,	0.338853314	,],
[	0.425908376	,	0.328073222	,	0.082345391	,	0.324789816	,],
[	0.344121039	,	0.376966687	,	0.277533804	,	0.093454775	,]],
[[	0.098044698	,	0.283150627	,	0.288285908	,	0.24841775	,],
[	0.339025458	,	0.092805576	,	0.241262652	,	0.340302929	,],
[	0.426256206	,	0.328331968	,	0.079962196	,	0.32535179	,],
[	0.346500281	,	0.379023147	,	0.279642192	,	0.097864928	,]],
[[	0.096875269	,	0.280290273	,	0.285561887	,	0.245792683	,],
[	0.339552591	,	0.089995862	,	0.241383858	,	0.340648497	,],
[	0.424467409	,	0.326461575	,	0.079350724	,	0.323296352	,],
[	0.345929232	,	0.378610556	,	0.279249461	,	0.094630947	,]],
[[	0.101942302	,	0.287412239	,	0.29263411	,	0.252628142	,],
[	0.337725386	,	0.089735033	,	0.239555542	,	0.338816054	,],
[	0.427931186	,	0.330210554	,	0.085644911	,	0.32686155	,],
[	0.344604331	,	0.377141166	,	0.277693939	,	0.094594284	,]],
[[	0.096507251	,	0.281159666	,	0.286499696	,	0.246484196	,],
[	0.341277033	,	0.096167093	,	0.24354158	,	0.342347318	,],
[	0.424670044	,	0.326568111	,	0.078644003	,	0.323552468	,],
[	0.349211077	,	0.381410369	,	0.282108615	,	0.101719408	,]],
[[	0.098523021	,	0.280994313	,	0.286370879	,	0.246574854	,],
[	0.337942757	,	0.088871691	,	0.239738485	,	0.339025474	,],
[	0.425382549	,	0.327404995	,	0.081201049	,	0.324046212	,],
[	0.344583462	,	0.377313192	,	0.277916946	,	0.093554545	,]],
[[	0.099038897	,	0.284238751	,	0.289380388	,	0.249455859	,],
[	0.33861594	,	0.091760158	,	0.240692117	,	0.339766022	,],
[	0.427270377	,	0.329333531	,	0.080797962	,	0.326296548	,],
[	0.34592952	,	0.378417544	,	0.27895339	,	0.096923837	,]],
[[	0.096413816	,	0.280293883	,	0.285527939	,	0.245663033	,],
[	0.340429801	,	0.09073073	,	0.242194972	,	0.341375429	,],
[	0.424370802	,	0.326258054	,	0.078952373	,	0.32314396	,],
[	0.346786116	,	0.379427916	,	0.280272808	,	0.095362627	,]],
[[	0.101728954	,	0.282776707	,	0.288051089	,	0.24849817	,],
[	0.337626582	,	0.089233736	,	0.239449305	,	0.338687861	,],
[	0.427016601	,	0.329160392	,	0.084127562	,	0.325815397	,],
[	0.344406195	,	0.376847859	,	0.277415004	,	0.094042982	,]],
[[	0.09700784	,	0.281918238	,	0.287105428	,	0.247116488	,],
[	0.340187082	,	0.094672769	,	0.242423841	,	0.341181648	,],
[	0.425250261	,	0.327115669	,	0.079011417	,	0.324112545	,],
[	0.348151125	,	0.380333705	,	0.280955378	,	0.100112492	,]],
[[	0.098735852	,	0.29003677	,	0.293301052	,	0.246069173	,],
[	0.322684182	,	0.088159376	,	0.23593483	,	0.326550645	,],
[	0.415142895	,	0.330204259	,	0.085095848	,	0.316953259	,],
[	0.337546712	,	0.382030003	,	0.281813327	,	0.09284539	,]],
[[	0.09540315	,	0.290076634	,	0.29321276	,	0.245752617	,],
[	0.325122088	,	0.093462142	,	0.238791188	,	0.328952173	,],
[	0.411811831	,	0.326332883	,	0.077558764	,	0.313463012	,],
[	0.341699193	,	0.385805047	,	0.285876665	,	0.09965529	,]],
[[	0.095326441	,	0.28828824	,	0.291352908	,	0.244050416	,],
[	0.324002712	,	0.088147929	,	0.236821457	,	0.327494598	,],
[	0.412706653	,	0.327400601	,	0.080454601	,	0.314312984	,],
[	0.337542312	,	0.382048108	,	0.281794486	,	0.091904656	,]],
[[	0.098864746	,	0.294094842	,	0.297192056	,	0.249667642	,],
[	0.323147553	,	0.089520882	,	0.236319736	,	0.326802991	,],
[	0.414493292	,	0.328910103	,	0.079546204	,	0.316192968	,],
[	0.33868445	,	0.382940704	,	0.28272154	,	0.094966857	,]],
[[	0.09431946	,	0.288643911	,	0.291721114	,	0.244169829	,],
[	0.327337884	,	0.091038969	,	0.24036056	,	0.330882123	,],
[	0.411504491	,	0.325960031	,	0.077789034	,	0.313029125	,],
[	0.340074692	,	0.384632962	,	0.284578325	,	0.093832033	,]],
[[	0.097635323	,	0.289416884	,	0.292681114	,	0.245314454	,],
[	0.3230027	,	0.087829508	,	0.235985284	,	0.326598808	,],
[	0.41456982	,	0.329373871	,	0.083674071	,	0.316081487	,],
[	0.337231667	,	0.381739247	,	0.28153197	,	0.092220836	,]],
[[	0.096190258	,	0.291110816	,	0.294142909	,	0.246667219	,],
[	0.324613353	,	0.09230965	,	0.238235787	,	0.32837067	,],
[	0.412654924	,	0.327022618	,	0.077958244	,	0.314255257	,],
[	0.340607658	,	0.384845995	,	0.284761392	,	0.097946783	,]],
[[	0.094840752	,	0.288221718	,	0.291249998	,	0.24397806	,],
[	0.324848796	,	0.0888275	,	0.237799302	,	0.328417771	,],
[	0.412359203	,	0.326878387	,	0.07968582	,	0.31390043	,],
[	0.338335772	,	0.382720597	,	0.282552363	,	0.092248982	,]],
[[	0.10028477	,	0.295581758	,	0.298589556	,	0.251077465	,],
[	0.322889477	,	0.08884405	,	0.236050115	,	0.326545869	,],
[	0.41536127	,	0.329910106	,	0.08053248	,	0.317144004	,],
[	0.338295863	,	0.382505178	,	0.282279049	,	0.093983269	,]],
[[	0.094694902	,	0.28925368	,	0.292332783	,	0.244745513	,],
[	0.326885095	,	0.096094291	,	0.240561412	,	0.330482997	,],
[	0.411936138	,	0.326311025	,	0.077979097	,	0.313396379	,],
[	0.341106748	,	0.385651055	,	0.285719412	,	0.094781411	,]],
[[	0.101408666	,	0.31290315	,	0.313018145	,	0.254792704	,],
[	0.324492471	,	0.091941129	,	0.24482111	,	0.329978808	,],
[	0.417347905	,	0.341930149	,	0.083484167	,	0.317332269	,],
[	0.343996001	,	0.403307487	,	0.29697506	,	0.094735511	,]],
[[	0.098869463	,	0.313495782	,	0.313412963	,	0.255130327	,],
[	0.322864723	,	0.092152643	,	0.2433656	,	0.328335273	,],
[	0.419467155	,	0.343936646	,	0.083315526	,	0.31985991	,],
[	0.345336427	,	0.404378582	,	0.298073384	,	0.098038278	,]],
[[	0.098106415	,	0.311024575	,	0.311117148	,	0.252757422	,],
[	0.327067128	,	0.099593168	,	0.248192653	,	0.332647273	,],
[	0.416169042	,	0.340603978	,	0.081085346	,	0.316262334	,],
[	0.346095885	,	0.405163846	,	0.299027904	,	0.096312076	,]],
[[	0.102694881	,	0.31763746	,	0.317653344	,	0.259197779	,],
[	0.323118791	,	0.09111403	,	0.243359432	,	0.328439063	,],
[	0.418755355	,	0.343604527	,	0.086238937	,	0.318921011	,],
[	0.344031591	,	0.403113617	,	0.296760852	,	0.095370479	,]],
[[	0.097709481	,	0.312024693	,	0.312021667	,	0.253557661	,],
[	0.324308394	,	0.094814515	,	0.244925605	,	0.329729389	,],
[	0.417259887	,	0.341662157	,	0.081353448	,	0.317427546	,],
[	0.347776763	,	0.406558385	,	0.300250844	,	0.101442787	,]],
[[	0.10025909	,	0.312293842	,	0.31239459	,	0.25402027	,],
[	0.325272263	,	0.09258629	,	0.245582715	,	0.330509658	,],
[	0.416819011	,	0.34136777	,	0.082447475	,	0.316909017	,],
[	0.344241326	,	0.403530153	,	0.297316469	,	0.09475473	,]],
[[	0.100005807	,	0.314647474	,	0.314545411	,	0.25619746	,],
[	0.322876296	,	0.091607922	,	0.243166761	,	0.328220532	,],
[	0.420526593	,	0.345059478	,	0.084432642	,	0.320993565	,],
[	0.344883141	,	0.403932691	,	0.297675221	,	0.097101194	,]],
[[	0.097876315	,	0.311318647	,	0.311212065	,	0.252874243	,],
[	0.326251173	,	0.098115165	,	0.247017675	,	0.331587002	,],
[	0.41643084	,	0.340857933	,	0.080954336	,	0.316519457	,],
[	0.347031414	,	0.406089112	,	0.299970575	,	0.097132488	,]],
[[	0.103549664	,	0.314085639	,	0.314172274	,	0.255973238	,],
[	0.32357851	,	0.09135318	,	0.243729652	,	0.32886206	,],
[	0.418182818	,	0.342863149	,	0.084905189	,	0.318103701	,],
[	0.343849876	,	0.40291427	,	0.296571556	,	0.095018648	,]],
[[	0.098140125	,	0.312484038	,	0.312440989	,	0.254019821	,],
[	0.323750783	,	0.093714692	,	0.244199174	,	0.32899256	,],
[	0.417994719	,	0.342331437	,	0.081961862	,	0.318169421	,],
[	0.346859838	,	0.405630621	,	0.299341762	,	0.099996819	,]],
[[	0.099743528	,	0.323142699	,	0.320650464	,	0.254875013	,],
[	0.316278912	,	0.095573405	,	0.246806735	,	0.324195538	,],
[	0.408711504	,	0.346054656	,	0.082871039	,	0.310267817	,],
[	0.340968986	,	0.412832751	,	0.304706187	,	0.09451774	,]],
[[	0.097320825	,	0.323718416	,	0.320951145	,	0.255133384	,],
[	0.312259638	,	0.092794558	,	0.242964505	,	0.32025253	,],
[	0.41223282	,	0.350299211	,	0.089672906	,	0.31409023	,],
[	0.342122107	,	0.413851311	,	0.305602901	,	0.097501242	,]],
[[	0.096607428	,	0.321484638	,	0.318766394	,	0.25298278	,],
[	0.314316696	,	0.097593233	,	0.245426209	,	0.322378735	,],
[	0.408633354	,	0.345883489	,	0.08169118	,	0.310216482	,],
[	0.343265833	,	0.414920001	,	0.30686574	,	0.096289183	,]],
[[	0.10129585	,	0.328010888	,	0.325308762	,	0.259422676	,],
[	0.314001016	,	0.09355487	,	0.244508777	,	0.32188261	,],
[	0.409568257	,	0.347205561	,	0.08502429	,	0.311219412	,],
[	0.340893223	,	0.41259408	,	0.304389673	,	0.095015969	,]],
[[	0.095995357	,	0.322077035	,	0.319506699	,	0.253450554	,],
[	0.312157615	,	0.093678795	,	0.242915511	,	0.320187581	,],
[	0.410883219	,	0.348169643	,	0.08349253	,	0.312736653	,],
[	0.344472247	,	0.415859519	,	0.307753773	,	0.100895212	,]],
[[	0.098390123	,	0.322225947	,	0.319671435	,	0.253766429	,],
[	0.316403565	,	0.10114605	,	0.247690142	,	0.324401078	,],
[	0.408317657	,	0.345700788	,	0.082183294	,	0.309885516	,],
[	0.341614043	,	0.413425888	,	0.305272433	,	0.094803968	,]],
[[	0.098499161	,	0.325034908	,	0.322202418	,	0.256375765	,],
[	0.312552306	,	0.092639909	,	0.243064024	,	0.320388025	,],
[	0.411731821	,	0.34951249	,	0.08834241	,	0.313357804	,],
[	0.341693751	,	0.41351193	,	0.305267304	,	0.096561326	,]],
[[	0.096386046	,	0.321697489	,	0.318960807	,	0.252988818	,],
[	0.313519004	,	0.096109145	,	0.244450254	,	0.321425161	,],
[	0.409211903	,	0.346466941	,	0.082171266	,	0.310838215	,],
[	0.344235612	,	0.415853499	,	0.307951196	,	0.097186402	,]],
[[	0.101941132	,	0.324410023	,	0.321693872	,	0.256022842	,],
[	0.31489517	,	0.094227426	,	0.245307719	,	0.322699199	,],
[	0.409118361	,	0.346588087	,	0.083899744	,	0.310656028	,],
[	0.341004968	,	0.412697305	,	0.304434269	,	0.094638607	,]],
[[	0.096661726	,	0.322840256	,	0.320044383	,	0.254160251	,],
[	0.312422947	,	0.093409209	,	0.243044922	,	0.320272937	,],
[	0.411776809	,	0.349037515	,	0.084418573	,	0.313616121	,],
[	0.343631247	,	0.415169018	,	0.3070046	,	0.099561133	,]],
[[	0.098282547	,	0.333511794	,	0.329566821	,	0.254944358	,],
[	0.302358764	,	0.098004404	,	0.245969622	,	0.313587879	,],
[	0.40037038	,	0.350380746	,	0.082822606	,	0.304368208	,],
[	0.33753411	,	0.421846598	,	0.313026987	,	0.094298332	,]],
[[	0.098035901	,	0.335767816	,	0.331731298	,	0.257127728	,],
[	0.302398124	,	0.094320425	,	0.24548731	,	0.313700848	,],
[	0.400445496	,	0.350732766	,	0.084659932	,	0.304302728	,],
[	0.337101237	,	0.421464553	,	0.312358707	,	0.095201048	,]],
[[	0.095878721	,	0.33241424	,	0.32839724	,	0.253778124	,],
[	0.300729104	,	0.094618992	,	0.244081264	,	0.312094319	,],
[	0.403121438	,	0.353046298	,	0.085121919	,	0.307175713	,],
[	0.34087651	,	0.425015711	,	0.316275954	,	0.09740354	,]],
[[	0.101835092	,	0.335530617	,	0.331537705	,	0.257101628	,],
[	0.304628891	,	0.101674058	,	0.248535948	,	0.31596488	,],
[	0.399765309	,	0.349807809	,	0.08264375	,	0.303561587	,],
[	0.336475442	,	0.420745091	,	0.311680394	,	0.093587606	,]],
[[	0.095955784	,	0.333495708	,	0.32945476	,	0.254751854	,],
[	0.300949931	,	0.093382118	,	0.243969134	,	0.312165011	,],
[	0.40210546	,	0.352609561	,	0.08763072	,	0.305952983	,],
[	0.338962165	,	0.423196363	,	0.314017315	,	0.098047623	,]],
[[	0.09744404	,	0.332973995	,	0.329121668	,	0.254351975	,],
[	0.301924263	,	0.096980402	,	0.24542036	,	0.313056705	,],
[	0.401094728	,	0.351109308	,	0.08330574	,	0.304999848	,],
[	0.338313694	,	0.422635006	,	0.313677407	,	0.094786879	,]],
[[	0.09884062	,	0.336670075	,	0.332437453	,	0.257861344	,],
[	0.303528842	,	0.095032599	,	0.246403609	,	0.314527535	,],
[	0.400146241	,	0.350279842	,	0.083839349	,	0.303832765	,],
[	0.33672983	,	0.421179415	,	0.31196492	,	0.094497583	,]],
[[	0.095733824	,	0.332624926	,	0.328448249	,	0.253800978	,],
[	0.300683128	,	0.094132723	,	0.244007462	,	0.311966269	,],
[	0.404248073	,	0.355018855	,	0.091146349	,	0.308132735	,],
[	0.340914852	,	0.425046064	,	0.316078633	,	0.101280166	,]],
[[	0.100490149	,	0.334818432	,	0.330809201	,	0.256277655	,],
[	0.303737111	,	0.100215248	,	0.247588173	,	0.315037787	,],
[	0.400091865	,	0.350099607	,	0.08265723	,	0.303829351	,],
[	0.336979137	,	0.421310941	,	0.3122658	,	0.093754433	,]],
[[	0.096671371	,	0.334330103	,	0.330356625	,	0.255588804	,],
[	0.301550375	,	0.093728251	,	0.244533868	,	0.31268136	,],
[	0.401443075	,	0.351782524	,	0.086287658	,	0.305135404	,],
[	0.338168605	,	0.422444066	,	0.313285722	,	0.096837223	,]],
[[	0.097779321	,	0.345827574	,	0.339303771	,	0.25585047	,],
[	0.289929661	,	0.096192178	,	0.243839653	,	0.303655749	,],
[	0.394051228	,	0.356377885	,	0.08398764	,	0.299095547	,],
[	0.33633235	,	0.432740917	,	0.321758745	,	0.095428829	,]],
[[	0.097503135	,	0.347904193	,	0.341295979	,	0.257938323	,],
[	0.293582881	,	0.096833673	,	0.246960915	,	0.307078902	,],
[	0.392702222	,	0.355294173	,	0.084120713	,	0.297603313	,],
[	0.334345183	,	0.430899865	,	0.319565088	,	0.094432554	,]],
[[	0.095266009	,	0.344626514	,	0.337971251	,	0.254543826	,],
[	0.289627644	,	0.094317349	,	0.243280435	,	0.303333614	,],
[	0.396393233	,	0.359552841	,	0.090998294	,	0.30135866	,],
[	0.33813768	,	0.434531643	,	0.323380071	,	0.100821925	,]],
[[	0.101093059	,	0.347690051	,	0.341084459	,	0.257852415	,],
[	0.291610567	,	0.099060962	,	0.245773169	,	0.305244524	,],
[	0.392583914	,	0.355000739	,	0.082933486	,	0.297547834	,],
[	0.334554367	,	0.431021466	,	0.3197527	,	0.093971801	,]],
[[	0.095604625	,	0.345936158	,	0.339232324	,	0.255782557	,],
[	0.291446144	,	0.095036331	,	0.244731456	,	0.304915732	,],
[	0.393730376	,	0.356551236	,	0.086364204	,	0.298641141	,],
[	0.335384191	,	0.431791107	,	0.320446756	,	0.096381053	,]],
[[	0.096626292	,	0.345129244	,	0.33874068	,	0.255219268	,],
[	0.289710667	,	0.095439014	,	0.243391019	,	0.303335855	,],
[	0.394874829	,	0.357131326	,	0.084607264	,	0.2998698	,],
[	0.337455611	,	0.433740126	,	0.322725437	,	0.096279442	,]],
[[	0.09856787	,	0.348949775	,	0.342210673	,	0.258941895	,],
[	0.293866688	,	0.102847248	,	0.248232568	,	0.307484406	,],
[	0.392429553	,	0.354948043	,	0.083415739	,	0.297170979	,],
[	0.334183484	,	0.430681472	,	0.319387787	,	0.094017935	,]],
[[	0.095226994	,	0.344823799	,	0.338191904	,	0.254762034	,],
[	0.289825081	,	0.094061221	,	0.243352293	,	0.303418521	,],
[	0.395538298	,	0.358582801	,	0.089438977	,	0.300493357	,],
[	0.337269401	,	0.433556727	,	0.322313647	,	0.099291903	,]],
[[	0.099664794	,	0.346833536	,	0.340253077	,	0.25698849	,],
[	0.29095659	,	0.097849565	,	0.244945741	,	0.304631927	,],
[	0.393081242	,	0.355377294	,	0.083139362	,	0.298022934	,],
[	0.33540586	,	0.431710015	,	0.320480271	,	0.094435866	,]],
[[	0.096524618	,	0.346879227	,	0.340231342	,	0.256712915	,],
[	0.292436195	,	0.095728613	,	0.245672424	,	0.305836477	,],
[	0.393150209	,	0.355880958	,	0.085254838	,	0.298012533	,],
[	0.335022263	,	0.431411027	,	0.320106407	,	0.095596529	,]],
[[	0.097798292	,	0.358908849	,	0.349569148	,	0.258286877	,],
[	0.278056032	,	0.096157899	,	0.243209265	,	0.294796508	,],
[	0.385879335	,	0.362150576	,	0.083599751	,	0.293164148	,],
[	0.333596082	,	0.443365663	,	0.32958808	,	0.095531654	,]],
[[	0.095931957	,	0.359796095	,	0.350285393	,	0.259113693	,],
[	0.281524757	,	0.102857547	,	0.247288948	,	0.298262685	,],
[	0.384081931	,	0.360713894	,	0.083423711	,	0.291206319	,],
[	0.331476921	,	0.441444491	,	0.327332923	,	0.09439599	,]],
[[	0.094872116	,	0.35736825	,	0.347844816	,	0.2565302	,],
[	0.278660778	,	0.095218543	,	0.243532555	,	0.29526101	,],
[	0.38789143	,	0.364911725	,	0.090190554	,	0.295008066	,],
[	0.335027207	,	0.444808015	,	0.330885783	,	0.10054982	,]],
[[	0.099954097	,	0.363920197	,	0.354408705	,	0.263298703	,],
[	0.279240142	,	0.098415364	,	0.24455743	,	0.295951115	,],
[	0.384474105	,	0.36080012	,	0.082597345	,	0.291580814	,],
[	0.331762235	,	0.441614147	,	0.327671666	,	0.093931965	,]],
[[	0.094275175	,	0.357978592	,	0.348515858	,	0.257088307	,],
[	0.281407342	,	0.097151377	,	0.245953964	,	0.297796196	,],
[	0.385045835	,	0.361775744	,	0.085353263	,	0.29198219	,],
[	0.332326079	,	0.442160251	,	0.328122578	,	0.096229301	,]],
[[	0.096350547	,	0.358027898	,	0.348629799	,	0.257250725	,],
[	0.278172239	,	0.095760936	,	0.243213798	,	0.294822747	,],
[	0.387066436	,	0.363235898	,	0.084696713	,	0.294224609	,],
[	0.334595257	,	0.444337176	,	0.330590122	,	0.096218348	,]],
[[	0.097052322	,	0.360960148	,	0.351436107	,	0.260221009	,],
[	0.281152115	,	0.101782813	,	0.246665626	,	0.297681052	,],
[	0.384240848	,	0.360713671	,	0.082984249	,	0.291215095	,],
[	0.33142002	,	0.441381449	,	0.327363162	,	0.094022765	,]],
[[	0.094603912	,	0.357629147	,	0.348160914	,	0.256810104	,],
[	0.279432042	,	0.095730452	,	0.24414397	,	0.295837574	,],
[	0.38695962	,	0.363938606	,	0.088532482	,	0.294046355	,],
[	0.333988669	,	0.443814013	,	0.329810119	,	0.098797361	,]],
[[	0.099664917	,	0.359893493	,	0.350516498	,	0.259226433	,],
[	0.278690654	,	0.097288366	,	0.243870566	,	0.295301605	,],
[	0.385029006	,	0.36127214	,	0.082873047	,	0.292129648	,],
[	0.33263149	,	0.442346206	,	0.328464829	,	0.094471268	,]],
[[	0.095050837	,	0.358947761	,	0.349461605	,	0.258046459	,],
[	0.282646684	,	0.098308764	,	0.247129633	,	0.298964083	,],
[	0.384481571	,	0.361158441	,	0.084413884	,	0.291478514	,],
[	0.331888937	,	0.441816659	,	0.327721869	,	0.095337941	,]],
[[	0.096059015	,	0.368059689	,	0.356345131	,	0.257871938	,],
[	0.266595291	,	0.095710057	,	0.243413843	,	0.286195379	,],
[	0.378668972	,	0.367737686	,	0.090980674	,	0.287716706	,],
[	0.330095827	,	0.451184188	,	0.336367892	,	0.095540459	,]],
[[	0.096160768	,	0.370458618	,	0.358557838	,	0.260252455	,],
[	0.267364214	,	0.09918185	,	0.244703873	,	0.287049678	,],
[	0.375456756	,	0.363689946	,	0.083739708	,	0.284475959	,],
[	0.326543475	,	0.448053465	,	0.332869035	,	0.09295884	,]],
[[	0.09395448	,	0.36717123	,	0.355412987	,	0.256904164	,],
[	0.268920177	,	0.09701043	,	0.245490645	,	0.288361156	,],
[	0.376155731	,	0.364735827	,	0.086466145	,	0.285050778	,],
[	0.329373777	,	0.450770206	,	0.335630439	,	0.097965601	,]],
[[	0.09951146	,	0.37002233	,	0.358379397	,	0.259997325	,],
[	0.266117911	,	0.096164794	,	0.243128571	,	0.285734028	,],
[	0.37802177	,	0.366017577	,	0.085611704	,	0.287067397	,],
[	0.327974683	,	0.44919788	,	0.334076056	,	0.093679593	,]],
[[	0.094109824	,	0.368297211	,	0.356527629	,	0.257987349	,],
[	0.269096999	,	0.102501444	,	0.246758643	,	0.288769896	,],
[	0.375049124	,	0.363374485	,	0.083896722	,	0.283984215	,],
[	0.327208875	,	0.44853022	,	0.333247347	,	0.094237446	,]],
[[	0.09537011	,	0.367746163	,	0.355954817	,	0.257396217	,],
[	0.267082514	,	0.095863164	,	0.243733047	,	0.286534514	,],
[	0.377724809	,	0.366577843	,	0.089374959	,	0.286628316	,],
[	0.331385405	,	0.452378087	,	0.337642292	,	0.096807637	,]],
[[	0.097386296	,	0.371655119	,	0.359808489	,	0.261506196	,],
[	0.266880349	,	0.098080115	,	0.244056169	,	0.286442329	,],
[	0.376029544	,	0.364129559	,	0.083909664	,	0.284894026	,],
[	0.326739928	,	0.448102844	,	0.33295615	,	0.092815957	,]],
[[	0.093663712	,	0.367251437	,	0.355410493	,	0.256905495	,],
[	0.269946339	,	0.097969506	,	0.246498023	,	0.289344663	,],
[	0.375674084	,	0.364198325	,	0.085551153	,	0.284402024	,],
[	0.328415026	,	0.449850515	,	0.334566097	,	0.096454375	,]],
[[	0.098034224	,	0.369172545	,	0.357373445	,	0.258935583	,],
[	0.266260467	,	0.09580057	,	0.243101785	,	0.285777403	,],
[	0.379157751	,	0.367173187	,	0.086708076	,	0.288163135	,],
[	0.328764059	,	0.449804229	,	0.334893728	,	0.094271841	,]],
[[	0.094962468	,	0.369087976	,	0.357330011	,	0.258781092	,],
[	0.268442465	,	0.10107857	,	0.245905044	,	0.287988278	,],
[	0.375014274	,	0.363290445	,	0.083474171	,	0.2838437	,],
[	0.327059285	,	0.44847979	,	0.333092493	,	0.09376603	,]],
[[	0.099313245	,	0.222909773	,	0.247327616	,	0.25731283	,],
[	0.402810016	,	0.089122864	,	0.251336244	,	0.398787563	,],
[	0.475419865	,	0.308937194	,	0.073310854	,	0.368264857	,],
[	0.369188636	,	0.332426092	,	0.242874453	,	0.09320686	,]],
[[	0.096609505	,	0.218175961	,	0.242738531	,	0.25305985	,],
[	0.399126834	,	0.0809181	,	0.247446673	,	0.395337487	,],
[	0.476498334	,	0.309993391	,	0.076557596	,	0.369241421	,],
[	0.371968841	,	0.334654453	,	0.245205874	,	0.098488666	,]],
[[	0.101667085	,	0.220167131	,	0.244716274	,	0.255408315	,],
[	0.400131144	,	0.084423032	,	0.248555461	,	0.396352802	,],
[	0.477555623	,	0.31126282	,	0.075046908	,	0.370675925	,],
[	0.370282033	,	0.33387652	,	0.244429085	,	0.093511647	,]],
[[	0.097062354	,	0.220161519	,	0.244851101	,	0.25471104	,],
[	0.401381499	,	0.08244305	,	0.249643902	,	0.397521366	,],
[	0.475274854	,	0.308762823	,	0.073760666	,	0.368072297	,],
[	0.369675512	,	0.332932765	,	0.243349225	,	0.094819094	,]],
[[	0.097629596	,	0.218234342	,	0.242851996	,	0.253075753	,],
[	0.398836865	,	0.081340026	,	0.24698242	,	0.394922318	,],
[	0.478314402	,	0.312104871	,	0.07974351	,	0.371062164	,],
[	0.373050952	,	0.336803262	,	0.24740772	,	0.096018723	,]],
[[	0.100476575	,	0.224114387	,	0.24853657	,	0.25847636	,],
[	0.402023883	,	0.087719725	,	0.250462155	,	0.3979874	,],
[	0.4760591	,	0.309575376	,	0.073714095	,	0.36895245	,],
[	0.36931989	,	0.332657675	,	0.243084075	,	0.093105172	,]],
[[	0.0966524	,	0.218794374	,	0.243302105	,	0.253477712	,],
[	0.399615612	,	0.081176222	,	0.247911651	,	0.395734608	,],
[	0.47620267	,	0.309799065	,	0.075689016	,	0.368998328	,],
[	0.371289616	,	0.334076329	,	0.244477378	,	0.097296202	,]],
[[	0.100381007	,	0.219385292	,	0.24389126	,	0.254551716	,],
[	0.39971568	,	0.083401943	,	0.247976987	,	0.395759545	,],
[	0.478496883	,	0.312284326	,	0.075949204	,	0.371678592	,],
[	0.370854754	,	0.334459165	,	0.245006315	,	0.093869123	,]],
[[	0.097912979	,	0.221247101	,	0.245733217	,	0.255571545	,],
[	0.402329893	,	0.083399525	,	0.250678451	,	0.398447113	,],
[	0.47544062	,	0.308918735	,	0.073601872	,	0.36821302	,],
[	0.369433015	,	0.332535041	,	0.242999401	,	0.093966901	,]],
[[	0.097488553	,	0.218397456	,	0.243006376	,	0.253239575	,],
[	0.399005319	,	0.081300065	,	0.247181098	,	0.395102855	,],
[	0.477393345	,	0.311057168	,	0.078190164	,	0.370159593	,],
[	0.373359941	,	0.336023042	,	0.246538557	,	0.100582656	,]],
[[	0.097438524	,	0.225612321	,	0.249412223	,	0.254464124	,],
[	0.388577699	,	0.083362604	,	0.24496056	,	0.386446519	,],
[	0.465497561	,	0.307711715	,	0.073901626	,	0.36178164	,],
[	0.360503654	,	0.332143198	,	0.243589405	,	0.091263159	,]],
[[	0.093952933	,	0.220390026	,	0.244275678	,	0.249633559	,],
[	0.389326149	,	0.080817895	,	0.245673825	,	0.387265646	,],
[	0.463463815	,	0.305535748	,	0.073144445	,	0.359503288	,],
[	0.360745022	,	0.331796575	,	0.243258422	,	0.09335218	,]],
[[	0.098792681	,	0.221999714	,	0.245862269	,	0.25162614	,],
[	0.387096403	,	0.080186455	,	0.243368784	,	0.384990486	,],
[	0.4666025	,	0.309095275	,	0.079251642	,	0.362688172	,],
[	0.363306215	,	0.335137911	,	0.246721119	,	0.093589332	,]],
[[	0.09508092	,	0.22285364	,	0.246638782	,	0.251753711	,],
[	0.390724914	,	0.087106728	,	0.24729667	,	0.388637595	,],
[	0.464101302	,	0.306194325	,	0.072668437	,	0.360237011	,],
[	0.359712934	,	0.331220651	,	0.242636043	,	0.091073588	,]],
[[	0.094892959	,	0.220276187	,	0.244273906	,	0.249671788	,],
[	0.387753808	,	0.07970332	,	0.244038327	,	0.385711689	,],
[	0.464395048	,	0.306519815	,	0.075110079	,	0.360390425	,],
[	0.362583966	,	0.333485347	,	0.245024725	,	0.096321282	,]],
[[	0.098571017	,	0.226823831	,	0.250493612	,	0.255535574	,],
[	0.387983485	,	0.082265728	,	0.244228517	,	0.385879086	,],
[	0.466338437	,	0.308666722	,	0.074655651	,	0.362609291	,],
[	0.361286592	,	0.332769496	,	0.244400747	,	0.091873187	,]],
[[	0.093817708	,	0.220730503	,	0.244607557	,	0.249895333	,],
[	0.390284616	,	0.081507416	,	0.246655888	,	0.388243624	,],
[	0.463620503	,	0.305621839	,	0.072772081	,	0.359630994	,],
[	0.360369324	,	0.331559136	,	0.243042941	,	0.092617774	,]],
[[	0.097368492	,	0.221094518	,	0.245054015	,	0.250707038	,],
[	0.387077712	,	0.07972214	,	0.243238997	,	0.384987827	,],
[	0.466047598	,	0.308321327	,	0.07805316	,	0.361994715	,],
[	0.364232676	,	0.336122907	,	0.247813759	,	0.094571724	,]],
[[	0.095925994	,	0.22381554	,	0.247684008	,	0.252645324	,],
[	0.389787325	,	0.085445114	,	0.246263022	,	0.387536025	,],
[	0.464401209	,	0.306495121	,	0.072961389	,	0.360484469	,],
[	0.360248861	,	0.331697871	,	0.243192873	,	0.091191684	,]],
[[	0.094542934	,	0.220394336	,	0.244327661	,	0.249650257	,],
[	0.388282995	,	0.08002929	,	0.244462551	,	0.386047624	,],
[	0.464067861	,	0.306125792	,	0.07429701	,	0.360040263	,],
[	0.361610264	,	0.332588579	,	0.244054351	,	0.094812681	,]],
[[	0.097169032	,	0.231126165	,	0.253933308	,	0.254159074	,],
[	0.379113677	,	0.080051857	,	0.241815973	,	0.379093478	,],
[	0.458032514	,	0.306945875	,	0.077742101	,	0.356303114	,],
[	0.356970985	,	0.335278483	,	0.247505315	,	0.091643401	,]],
[[	0.093177347	,	0.225619115	,	0.248550775	,	0.248976575	,],
[	0.382113127	,	0.086147738	,	0.245074525	,	0.38187226	,],
[	0.456111409	,	0.304788963	,	0.072080597	,	0.354441328	,],
[	0.354858035	,	0.332749089	,	0.244630928	,	0.090761735	,]],
[[	0.096830271	,	0.226304506	,	0.249195791	,	0.250147925	,],
[	0.379823478	,	0.079695984	,	0.242592964	,	0.379718731	,],
[	0.455814771	,	0.304473009	,	0.073648012	,	0.353988469	,],
[	0.358201642	,	0.335538201	,	0.247627067	,	0.096584249	,]],
[[	0.094548648	,	0.228244819	,	0.251096882	,	0.251358741	,],
[	0.379834129	,	0.081913013	,	0.242635627	,	0.379681569	,],
[	0.45867811	,	0.30756564	,	0.074343239	,	0.357251931	,],
[	0.355490331	,	0.333721444	,	0.245758631	,	0.090455775	,]],
[[	0.093714686	,	0.225256353	,	0.248153342	,	0.24876793	,],
[	0.382677384	,	0.082016716	,	0.245459702	,	0.382506046	,],
[	0.455367179	,	0.304053306	,	0.071858709	,	0.353588727	,],
[	0.355514344	,	0.333231326	,	0.245335668	,	0.0923414	,]],
[[	0.099091042	,	0.232859998	,	0.255598643	,	0.255911851	,],
[	0.378923395	,	0.07952765	,	0.241630759	,	0.378721715	,],
[	0.457182859	,	0.306020665	,	0.076320528	,	0.355456269	,],
[	0.358054784	,	0.336278526	,	0.248470211	,	0.09247392	,]],
[[	0.093345554	,	0.226061596	,	0.2488146	,	0.249381343	,],
[	0.381320719	,	0.084729746	,	0.244199111	,	0.380983904	,],
[	0.456758578	,	0.305433776	,	0.072519168	,	0.355101904	,],
[	0.354769044	,	0.332738174	,	0.244690986	,	0.090250032	,]],
[[	0.095846467	,	0.225822524	,	0.248729724	,	0.249478321	,],
[	0.380779657	,	0.080366079	,	0.243481304	,	0.380593817	,],
[	0.455631338	,	0.304292677	,	0.072956543	,	0.353752303	,],
[	0.357319261	,	0.334610843	,	0.246641438	,	0.095176567	,]],
[[	0.095645115	,	0.229281979	,	0.252135722	,	0.252297583	,],
[	0.379278658	,	0.080799299	,	0.242092212	,	0.379160192	,],
[	0.459338299	,	0.308454869	,	0.079941176	,	0.357638891	,],
[	0.356252347	,	0.334458381	,	0.246521865	,	0.090843605	,]],
[[	0.093379909	,	0.225211449	,	0.248186717	,	0.248675151	,],
[	0.383725601	,	0.088631003	,	0.24675899	,	0.38335975	,],
[	0.455467617	,	0.304138995	,	0.071706892	,	0.353720595	,],
[	0.355060497	,	0.332746754	,	0.24470354	,	0.091225471	,]],
[[	0.097856655	,	0.236729613	,	0.258141361	,	0.254138252	,],
[	0.373262177	,	0.079570582	,	0.241277356	,	0.373774658	,],
[	0.451093807	,	0.305581744	,	0.074933206	,	0.350201224	,],
[	0.354256017	,	0.338827455	,	0.250812656	,	0.092067716	,]],
[[	0.092553624	,	0.23027364	,	0.251811237	,	0.248139455	,],
[	0.374495679	,	0.08352144	,	0.242708185	,	0.374974606	,],
[	0.451985144	,	0.306490059	,	0.073182094	,	0.35139838	,],
[	0.350939922	,	0.335146962	,	0.246922828	,	0.089707554	,]],
[[	0.095286166	,	0.230161937	,	0.251784589	,	0.248444116	,],
[	0.375090353	,	0.080589805	,	0.243280887	,	0.375739867	,],
[	0.449661341	,	0.303954602	,	0.071906277	,	0.348718206	,],
[	0.353333991	,	0.337127588	,	0.24895213	,	0.094680914	,]],
[[	0.09461806	,	0.2334802	,	0.255014716	,	0.250972218	,],
[	0.372625048	,	0.079994148	,	0.240765569	,	0.373152149	,],
[	0.452820708	,	0.307468229	,	0.078060177	,	0.351942299	,],
[	0.352034787	,	0.336624366	,	0.248627774	,	0.09014077	,]],
[[	0.092731265	,	0.229538862	,	0.251227525	,	0.247380458	,],
[	0.376592299	,	0.087006844	,	0.244963053	,	0.376988229	,],
[	0.450650271	,	0.304963429	,	0.071874008	,	0.349834146	,],
[	0.351364584	,	0.335486057	,	0.247284433	,	0.090977108	,]],
[[	0.09829557	,	0.23174648	,	0.253521922	,	0.250168466	,],
[	0.373596234	,	0.079666795	,	0.241566905	,	0.374149824	,],
[	0.450474395	,	0.304950167	,	0.073877086	,	0.349511867	,],
[	0.355467752	,	0.339204618	,	0.251001586	,	0.097996826	,]],
[[	0.092972693	,	0.230861917	,	0.252476263	,	0.248613734	,],
[	0.373770233	,	0.082248484	,	0.24187051	,	0.374214617	,],
[	0.452916173	,	0.307584228	,	0.074045084	,	0.352298296	,],
[	0.351267315	,	0.335528994	,	0.247264479	,	0.089630137	,]],
[[	0.094352657	,	0.229747866	,	0.251383822	,	0.247952882	,],
[	0.376169281	,	0.081465403	,	0.244260655	,	0.376588259	,],
[	0.449959524	,	0.304229537	,	0.071852074	,	0.349003896	,],
[	0.352591814	,	0.336452665	,	0.24817594	,	0.093250538	,]],
[[	0.095911139	,	0.234668108	,	0.256224397	,	0.252102967	,],
[	0.37296186	,	0.079747482	,	0.241008659	,	0.373461216	,],
[	0.451845066	,	0.306419892	,	0.076606821	,	0.350830283	,],
[	0.353145015	,	0.337744738	,	0.249619221	,	0.090890198	,]],
[[	0.092541922	,	0.229796321	,	0.2515682	,	0.247611301	,],
[	0.375545728	,	0.085354293	,	0.243841898	,	0.375960872	,],
[	0.451100426	,	0.305498641	,	0.072221595	,	0.350242826	,],
[	0.350899001	,	0.33507192	,	0.246847841	,	0.090185869	,]],
[[	0.096638822	,	0.242027706	,	0.262721683	,	0.254072198	,],
[	0.367578389	,	0.080452823	,	0.242312851	,	0.370001155	,],
[	0.442748065	,	0.303851107	,	0.072253569	,	0.343635847	,],
[	0.349868569	,	0.34019537	,	0.252574134	,	0.094951223	,]],
[[	0.091230436	,	0.235467997	,	0.256236158	,	0.247820604	,],
[	0.365899328	,	0.080568968	,	0.240520469	,	0.368361384	,],
[	0.446145795	,	0.307625948	,	0.078714859	,	0.347095704	,],
[	0.347714263	,	0.338736716	,	0.251119907	,	0.089408519	,]],
[[	0.0935611	,	0.235240306	,	0.25600416	,	0.247846858	,],
[	0.369104568	,	0.087020019	,	0.24416724	,	0.371412186	,],
[	0.443357831	,	0.304541977	,	0.071984114	,	0.344401573	,],
[	0.347347349	,	0.337994667	,	0.250263576	,	0.090883368	,]],
[[	0.093427483	,	0.238751363	,	0.259486927	,	0.250788462	,],
[	0.365989852	,	0.079428788	,	0.240671625	,	0.368444418	,],
[	0.443552204	,	0.304763369	,	0.074162225	,	0.344342407	,],
[	0.350444756	,	0.341723536	,	0.254271466	,	0.091746018	,]],
[[	0.091087178	,	0.234646058	,	0.255528701	,	0.24702939	,],
[	0.366620383	,	0.082517002	,	0.241504593	,	0.369023794	,],
[	0.446187634	,	0.307476784	,	0.074414494	,	0.347311012	,],
[	0.346787647	,	0.337702908	,	0.250087942	,	0.089061761	,]],
[[	0.096534813	,	0.236883116	,	0.257584561	,	0.249605915	,],
[	0.368573123	,	0.081342706	,	0.243293292	,	0.370909334	,],
[	0.442815985	,	0.30396144	,	0.07194736	,	0.343545351	,],
[	0.349120335	,	0.339534125	,	0.251994049	,	0.093662232	,]],
[[	0.091835901	,	0.236481771	,	0.257146888	,	0.248660145	,],
[	0.365737074	,	0.079933208	,	0.24035229	,	0.368047487	,],
[	0.445260179	,	0.306662795	,	0.077102861	,	0.346033629	,],
[	0.348259152	,	0.339188321	,	0.25170743	,	0.089793813	,]],
[[	0.092796257	,	0.234970641	,	0.255756696	,	0.247521929	,],
[	0.368482059	,	0.085781192	,	0.243380472	,	0.370746842	,],
[	0.44388078	,	0.305072105	,	0.072251172	,	0.344807647	,],
[	0.347039717	,	0.337686575	,	0.249991011	,	0.090063117	,]],
[[	0.094656072	,	0.240015675	,	0.260795651	,	0.251931673	,],
[	0.366575107	,	0.079753008	,	0.241238562	,	0.368861239	,],
[	0.443396875	,	0.304435467	,	0.073431895	,	0.344071015	,],
[	0.35138941	,	0.341418426	,	0.253899848	,	0.097285837	,]],
[[	0.091144881	,	0.235041569	,	0.255880506	,	0.24717669	,],
[	0.366205163	,	0.081515423	,	0.241065722	,	0.368539628	,],
[	0.446964237	,	0.308375177	,	0.07516696	,	0.348166446	,],
[	0.346969435	,	0.33786878	,	0.250326355	,	0.088863832	,]],
[[	0.096163603	,	0.246602001	,	0.266162624	,	0.252281143	,],
[	0.360891209	,	0.080545462	,	0.241027264	,	0.364096972	,],
[	0.435696269	,	0.30338142	,	0.071425288	,	0.338134773	,],
[	0.344651454	,	0.341954918	,	0.253655355	,	0.09309903	,]],
[[	0.09034047	,	0.239962655	,	0.259556398	,	0.245790901	,],
[	0.358112112	,	0.079180191	,	0.23814865	,	0.361260502	,],
[	0.438307239	,	0.306334446	,	0.076856661	,	0.340791143	,],
[	0.34343624	,	0.341344876	,	0.253017623	,	0.088537523	,]],
[[	0.092185918	,	0.239167426	,	0.258863932	,	0.245457873	,],
[	0.36076409	,	0.085054455	,	0.241094028	,	0.363781014	,],
[	0.436846642	,	0.304490815	,	0.071642882	,	0.339349004	,],
[	0.342450921	,	0.339952618	,	0.251553349	,	0.089280953	,]],
[[	0.092679255	,	0.243215164	,	0.262825303	,	0.2488755	,],
[	0.358833493	,	0.078986309	,	0.238822929	,	0.361998237	,],
[	0.436121789	,	0.303889738	,	0.072731723	,	0.338525854	,],
[	0.346696847	,	0.344823744	,	0.256601106	,	0.091497384	,]],
[[	0.089947659	,	0.238749012	,	0.258480824	,	0.244634908	,],
[	0.358576108	,	0.080831414	,	0.238724524	,	0.361678412	,],
[	0.439989382	,	0.307841244	,	0.074479726	,	0.342586793	,],
[	0.342430491	,	0.340242852	,	0.251910577	,	0.087986359	,]],
[[	0.095182839	,	0.240779231	,	0.260355805	,	0.247034172	,],
[	0.362110722	,	0.081574431	,	0.242060295	,	0.365061367	,],
[	0.435928703	,	0.303638219	,	0.071228829	,	0.338318926	,],
[	0.344015074	,	0.341319509	,	0.252956799	,	0.091859418	,]],
[[	0.091025187	,	0.240732935	,	0.260434903	,	0.246676677	,],
[	0.35813181	,	0.078928014	,	0.238181744	,	0.361186715	,],
[	0.437541687	,	0.305390261	,	0.075535533	,	0.339875153	,],
[	0.344345698	,	0.342152087	,	0.253860202	,	0.089178387	,]],
[[	0.091405966	,	0.238956122	,	0.258683392	,	0.245147971	,],
[	0.359871784	,	0.08349418	,	0.240085992	,	0.362812071	,],
[	0.43748364	,	0.305232928	,	0.072191104	,	0.340004628	,],
[	0.342574171	,	0.34002556	,	0.251638079	,	0.088872259	,]],
[[	0.094041051	,	0.244562629	,	0.26422489	,	0.250129213	,],
[	0.359274504	,	0.079298879	,	0.239436544	,	0.362473874	,],
[	0.435910627	,	0.303568124	,	0.072055113	,	0.338197253	,],
[	0.34610815	,	0.343127151	,	0.254792105	,	0.095158983	,]],
[[	0.089933302	,	0.239107572	,	0.258777841	,	0.244968141	,],
[	0.358400038	,	0.080105653	,	0.23841941	,	0.361400431	,],
[	0.439904874	,	0.307967375	,	0.07925592	,	0.342255378	,],
[	0.342774683	,	0.340474086	,	0.252233012	,	0.088006259	,]],
[[	0.095417463	,	0.246212564	,	0.264081137	,	0.247397297	,],
[	0.355349444	,	0.087289354	,	0.241303466	,	0.359546559	,],
[	0.430361518	,	0.30470177	,	0.070994999	,	0.334041209	,],
[	0.339787919	,	0.343232187	,	0.254720546	,	0.090214723	,]],
[[	0.089523505	,	0.24475444	,	0.262570493	,	0.245413736	,],
[	0.351949476	,	0.079185666	,	0.237507758	,	0.356393296	,],
[	0.430016807	,	0.304438936	,	0.072692709	,	0.333660457	,],
[	0.342030352	,	0.345776478	,	0.257397637	,	0.089862395	,]],
[[	0.091128954	,	0.243897716	,	0.261738411	,	0.244852334	,],
[	0.352753611	,	0.082563792	,	0.238466293	,	0.357117355	,],
[	0.433003513	,	0.307543788	,	0.073408445	,	0.336957452	,],
[	0.338797139	,	0.342262429	,	0.253808344	,	0.087861864	,]],
[[	0.092323897	,	0.248505681	,	0.26632774	,	0.249000989	,],
[	0.35410883	,	0.080635389	,	0.239747197	,	0.358483274	,],
[	0.429729659	,	0.303935536	,	0.070894875	,	0.333301564	,],
[	0.341619376	,	0.344789024	,	0.256245545	,	0.093183346	,]],
[[	0.089270928	,	0.243629079	,	0.26162626	,	0.244440328	,],
[	0.351579193	,	0.079755166	,	0.237158575	,	0.355947225	,],
[	0.431416776	,	0.306113481	,	0.075510649	,	0.335049188	,],
[	0.340037371	,	0.343909522	,	0.255453	,	0.088174401	,]],
[[	0.094326893	,	0.245701005	,	0.263570401	,	0.246785228	,],
[	0.354462341	,	0.085737761	,	0.240294464	,	0.358683483	,],
[	0.430882817	,	0.305252177	,	0.071493977	,	0.334662004	,],
[	0.339242266	,	0.342798078	,	0.254273363	,	0.089253986	,]],
[[	0.09014301	,	0.245712009	,	0.263444529	,	0.246238822	,],
[	0.35242321	,	0.079334642	,	0.237767702	,	0.356627001	,],
[	0.430129174	,	0.304459961	,	0.072177324	,	0.333575908	,],
[	0.343042145	,	0.346926212	,	0.258650154	,	0.090958558	,]],
[[	0.090810161	,	0.244093722	,	0.261942028	,	0.245032073	,],
[	0.352302387	,	0.08150551	,	0.237805875	,	0.356481852	,],
[	0.433977385	,	0.308881112	,	0.079429962	,	0.337565999	,],
[	0.338967736	,	0.34253528	,	0.253985979	,	0.087679951	,]],
[[	0.093504846	,	0.249686947	,	0.267458888	,	0.250122981	,],
[	0.35531044	,	0.081731204	,	0.240896731	,	0.359545347	,],
[	0.429769571	,	0.304063669	,	0.070710616	,	0.33328067	,],
[	0.340960597	,	0.344134437	,	0.255573086	,	0.091993765	,]],
[[	0.089283097	,	0.24415131	,	0.262024563	,	0.244899051	,],
[	0.351738636	,	0.079314485	,	0.237229291	,	0.355957653	,],
[	0.431213905	,	0.305695483	,	0.074547507	,	0.334583387	,],
[	0.340496061	,	0.344298506	,	0.255883246	,	0.088440643	,]],
[[	0.09430997	,	0.25135064	,	0.268711432	,	0.247536937	,],
[	0.348402906	,	0.083399652	,	0.23882304	,	0.354342141	,],
[	0.429535806	,	0.309233848	,	0.074109834	,	0.334538294	,],
[	0.337320643	,	0.346782131	,	0.25722799	,	0.088048038	,]],
[[	0.089666456	,	0.250986474	,	0.268179807	,	0.246623564	,],
[	0.349324702	,	0.080764898	,	0.239490368	,	0.355259253	,],
[	0.426416459	,	0.305818408	,	0.071860057	,	0.331050843	,],
[	0.341189927	,	0.350372119	,	0.260827159	,	0.094801985	,]],
[[	0.090507891	,	0.24954207	,	0.266835605	,	0.245530994	,],
[	0.347004259	,	0.080233581	,	0.237169145	,	0.352941373	,],
[	0.428458079	,	0.308232864	,	0.076593144	,	0.333054406	,],
[	0.33731539	,	0.346911982	,	0.257402417	,	0.087107311	,]],
[[	0.092924441	,	0.254928858	,	0.272189328	,	0.250465573	,],
[	0.350241685	,	0.086651756	,	0.240777142	,	0.356019263	,],
[	0.42749295	,	0.30694834	,	0.072198467	,	0.332241469	,],
[	0.338536593	,	0.347818017	,	0.258142794	,	0.090288835	,]],
[[	0.088847687	,	0.249545333	,	0.266866543	,	0.245251145	,],
[	0.347799304	,	0.079656221	,	0.23795025	,	0.353731158	,],
[	0.426352541	,	0.305861501	,	0.072799367	,	0.330887205	,],
[	0.339784367	,	0.349678326	,	0.260190135	,	0.088998184	,]],
[[	0.093038659	,	0.250819164	,	0.268063507	,	0.24680459	,],
[	0.347872904	,	0.082193947	,	0.238115944	,	0.353686413	,],
[	0.430295558	,	0.310315487	,	0.079987178	,	0.334931128	,],
[	0.336833473	,	0.346478577	,	0.256872612	,	0.087338056	,]],
[[	0.090445614	,	0.251993253	,	0.269098423	,	0.247549375	,],
[	0.350445535	,	0.081686714	,	0.240576544	,	0.356307679	,],
[	0.426605393	,	0.305950633	,	0.071738123	,	0.331128387	,],
[	0.340378557	,	0.349573612	,	0.260045129	,	0.093438643	,]],
[[	0.089737307	,	0.249292744	,	0.266497597	,	0.245138476	,],
[	0.347307925	,	0.080034596	,	0.237352091	,	0.35305474	,],
[	0.42772879	,	0.30742147	,	0.075293993	,	0.332230428	,],
[	0.337803362	,	0.34733122	,	0.257850325	,	0.087223345	,]],
[[	0.094346448	,	0.256340834	,	0.27348713	,	0.251767874	,],
[	0.349505641	,	0.08525673	,	0.239923455	,	0.355260377	,],
[	0.428173022	,	0.30772656	,	0.072719304	,	0.332884254	,],
[	0.337783354	,	0.347121335	,	0.257467553	,	0.089026764	,]],
[[	0.089300091	,	0.250262489	,	0.267485113	,	0.245888829	,],
[	0.348175284	,	0.079869647	,	0.238277739	,	0.35400049	,],
[	0.426282216	,	0.305726991	,	0.072100944	,	0.330699319	,],
[	0.340427286	,	0.350280771	,	0.260886515	,	0.0894345	,]],
[[	0.092908178	,	0.255900789	,	0.271245445	,	0.246040942	,],
[	0.340559349	,	0.082546294	,	0.236773881	,	0.347766327	,],
[	0.424106895	,	0.311223527	,	0.079403311	,	0.330387597	,],
[	0.33302138	,	0.349009509	,	0.259159062	,	0.086992701	,]],
[[	0.088966603	,	0.255870423	,	0.271291361	,	0.245647021	,],
[	0.343409848	,	0.082542062	,	0.239459004	,	0.35062522	,],
[	0.420362008	,	0.306852782	,	0.071187719	,	0.326593058	,],
[	0.335857145	,	0.351503619	,	0.261617646	,	0.092289363	,]],
[[	0.089394806	,	0.254180886	,	0.269487216	,	0.244192382	,],
[	0.339941646	,	0.080262836	,	0.235901977	,	0.347152046	,],
[	0.421198189	,	0.307848855	,	0.074407032	,	0.327361958	,],
[	0.334163059	,	0.350237643	,	0.26039383	,	0.08719109	,]],
[[	0.092386162	,	0.259899942	,	0.275452858	,	0.249557045	,],
[	0.341834546	,	0.085334844	,	0.238128744	,	0.348873961	,],
[	0.422106218	,	0.3085913	,	0.072346203	,	0.328563457	,],
[	0.333744324	,	0.349502855	,	0.259477892	,	0.088552482	,]],
[[	0.08781464	,	0.254181873	,	0.269647191	,	0.2440072	,],
[	0.3414193	,	0.080806247	,	0.2373936	,	0.348577777	,],
[	0.419858297	,	0.306333635	,	0.071470267	,	0.325961687	,],
[	0.337469394	,	0.353767984	,	0.264059719	,	0.090268535	,]],
[[	0.091873438	,	0.255327714	,	0.270840309	,	0.245413288	,],
[	0.340246123	,	0.081662929	,	0.236300792	,	0.347367332	,],
[	0.423176218	,	0.310054299	,	0.077836445	,	0.329290218	,],
[	0.333207946	,	0.349275272	,	0.259406941	,	0.086826122	,]],
[[	0.089768803	,	0.256940084	,	0.272316467	,	0.24661373	,],
[	0.344284243	,	0.08917032	,	0.240721455	,	0.351244638	,],
[	0.420541141	,	0.307049518	,	0.071148239	,	0.326790194	,],
[	0.335382577	,	0.351048886	,	0.261065939	,	0.091162062	,]],
[[	0.088598488	,	0.253889153	,	0.269283823	,	0.243751571	,],
[	0.34000912	,	0.079969433	,	0.235840749	,	0.347109296	,],
[	0.420835577	,	0.307469613	,	0.073466335	,	0.326846156	,],
[	0.33515073	,	0.351251277	,	0.261407988	,	0.087882609	,]],
[[	0.093687222	,	0.261226702	,	0.276662763	,	0.250778993	,],
[	0.341393849	,	0.084165124	,	0.237540017	,	0.34838988	,],
[	0.423254423	,	0.309889623	,	0.073502113	,	0.32964385	,],
[	0.333307342	,	0.349144056	,	0.259137354	,	0.087658515	,]],
[[	0.088573873	,	0.255159371	,	0.27058639	,	0.244934986	,],
[	0.342152479	,	0.081309489	,	0.238029004	,	0.349237797	,],
[	0.419932499	,	0.306330958	,	0.071157082	,	0.325918059	,],
[	0.337587889	,	0.353004244	,	0.263092835	,	0.094724833	,]],
[[	0.092412305	,	0.260482111	,	0.275471897	,	0.246058112	,],
[	0.334129572	,	0.081153174	,	0.236016559	,	0.34269187	,],
[	0.416386201	,	0.30899385	,	0.077482614	,	0.324198931	,],
[	0.329412031	,	0.35137735	,	0.26157129	,	0.086990897	,]],
[[	0.088159503	,	0.260222009	,	0.275367225	,	0.245586529	,],
[	0.338129946	,	0.088648732	,	0.240401706	,	0.34651541	,],
[	0.414058621	,	0.306260068	,	0.071190902	,	0.321887813	,],
[	0.332248732	,	0.35379345	,	0.263974232	,	0.092309745	,]],
[[	0.088753618	,	0.258675895	,	0.273653956	,	0.244260601	,],
[	0.334120968	,	0.079717582	,	0.235736226	,	0.342611694	,],
[	0.414065379	,	0.306444469	,	0.073167239	,	0.32165763	,],
[	0.330576886	,	0.352538512	,	0.262793593	,	0.08736555	,]],
[[	0.091198662	,	0.263976951	,	0.279087738	,	0.249264198	,],
[	0.335391806	,	0.083722719	,	0.237314523	,	0.34385943	,],
[	0.416816692	,	0.309113093	,	0.07356187	,	0.324881695	,],
[	0.329851957	,	0.351597441	,	0.261697003	,	0.088377814	,]],
[[	0.087160141	,	0.258694403	,	0.273864601	,	0.244022721	,],
[	0.336299877	,	0.080838589	,	0.237952034	,	0.344827584	,],
[	0.413139158	,	0.305362241	,	0.070928654	,	0.320897658	,],
[	0.333883523	,	0.356091327	,	0.266453906	,	0.090314339	,]],
[[	0.091246551	,	0.259804456	,	0.27484724	,	0.245434614	,],
[	0.333882489	,	0.080309557	,	0.23558727	,	0.342350641	,],
[	0.415591984	,	0.308166615	,	0.07614817	,	0.323251137	,],
[	0.329597338	,	0.35157825	,	0.261724201	,	0.086874648	,]],
[[	0.088825256	,	0.261281594	,	0.276253195	,	0.24653268	,],
[	0.337278858	,	0.087084537	,	0.239376081	,	0.345624072	,],
[	0.414767446	,	0.307044597	,	0.071694087	,	0.322571464	,],
[	0.331055662	,	0.352620749	,	0.262797635	,	0.090535596	,]],
[[	0.08801234	,	0.258361334	,	0.27343111	,	0.243781785	,],
[	0.334538067	,	0.07974013	,	0.236101677	,	0.343006821	,],
[	0.413791319	,	0.306030221	,	0.07244416	,	0.32134549	,],
[	0.331405078	,	0.353261793	,	0.263487347	,	0.087882317	,]],
[[	0.092615151	,	0.265383217	,	0.280423778	,	0.250524702	,],
[	0.334901562	,	0.082649728	,	0.236663842	,	0.343355709	,],
[	0.417592524	,	0.310381001	,	0.079684572	,	0.325336442	,],
[	0.329593393	,	0.351424649	,	0.261477321	,	0.087556399	,]],
[[	0.087461036	,	0.259188927	,	0.274270585	,	0.244550088	,],
[	0.337027091	,	0.081444629	,	0.238642999	,	0.345557793	,],
[	0.41349729	,	0.305795362	,	0.07098478	,	0.321282291	,],
[	0.333623142	,	0.354890797	,	0.265098362	,	0.094365013	,]],
[[	0.091559108	,	0.265289994	,	0.279475478	,	0.245633921	,],
[	0.3284971	,	0.079754011	,	0.235305215	,	0.338197932	,],
[	0.411257381	,	0.309034871	,	0.074353642	,	0.319450444	,],
[	0.32878579	,	0.356718213	,	0.266340531	,	0.086830571	,]],
[[	0.088401187	,	0.265671499	,	0.279871561	,	0.24575036	,],
[	0.330370415	,	0.084576997	,	0.23743293	,	0.339936636	,],
[	0.412340077	,	0.310044419	,	0.072605044	,	0.320737287	,],
[	0.329870959	,	0.357474156	,	0.2670361	,	0.089897491	,]],
[[	0.088200391	,	0.263522831	,	0.277740919	,	0.243793897	,],
[	0.33012621	,	0.080380247	,	0.236866323	,	0.339795649	,],
[	0.410331247	,	0.307944148	,	0.071763893	,	0.318438263	,],
[	0.330937379	,	0.358850945	,	0.268554706	,	0.08845543	,]],
[[	0.091977567	,	0.269921447	,	0.28407751	,	0.249814547	,],
[	0.328692648	,	0.080923433	,	0.235571257	,	0.338297268	,],
[	0.412948284	,	0.311062714	,	0.077505231	,	0.321174047	,],
[	0.328528116	,	0.356270346	,	0.265827629	,	0.087132979	,]],
[[	0.087359664	,	0.264160938	,	0.278419495	,	0.244282144	,],
[	0.332467888	,	0.088263885	,	0.239661989	,	0.341861953	,],
[	0.410646745	,	0.308276071	,	0.071169356	,	0.318866208	,],
[	0.331962904	,	0.35913349	,	0.268760395	,	0.093078993	,]],
[[	0.090388082	,	0.264564582	,	0.278773458	,	0.244830055	,],
[	0.328945037	,	0.079824463	,	0.235684179	,	0.338566552	,],
[	0.410550586	,	0.308362331	,	0.073122515	,	0.31858611	,],
[	0.329368551	,	0.357346734	,	0.266973326	,	0.087169986	,]],
[[	0.089336598	,	0.26686833	,	0.280965367	,	0.246817207	,],
[	0.329806545	,	0.083403267	,	0.236840534	,	0.339383396	,],
[	0.413385903	,	0.311069889	,	0.073525981	,	0.321793404	,],
[	0.32904712	,	0.356757649	,	0.266311939	,	0.088589332	,]],
[[	0.08776684	,	0.263717073	,	0.277805408	,	0.243741262	,],
[	0.331252939	,	0.081308153	,	0.237917168	,	0.340786856	,],
[	0.410117203	,	0.307712992	,	0.071203497	,	0.318239043	,],
[	0.332225789	,	0.360180878	,	0.269909534	,	0.089604055	,]],
[[	0.093430413	,	0.271275922	,	0.285369935	,	0.251130723	,],
[	0.328462314	,	0.080172368	,	0.235151011	,	0.337975395	,],
[	0.412207112	,	0.310119172	,	0.076024757	,	0.320222496	,],
[	0.328572765	,	0.356287289	,	0.265807956	,	0.086864846	,]],
[[	0.087838195	,	0.264884769	,	0.279047914	,	0.244891952	,],
[	0.331775374	,	0.086936504	,	0.23880677	,	0.341123451	,],
[	0.411383785	,	0.308973748	,	0.07156535	,	0.319549182	,],
[	0.330822249	,	0.358256665	,	0.267753277	,	0.091582893	,]],
[[	0.090598936	,	0.270105629	,	0.282549873	,	0.245235394	,],
[	0.324827489	,	0.080178605	,	0.235483632	,	0.33500119	,],
[	0.406496956	,	0.308828321	,	0.071501407	,	0.314653065	,],
[	0.328065688	,	0.361154881	,	0.269895372	,	0.086721777	,]],
[[	0.088440425	,	0.271483653	,	0.28395083	,	0.246290451	,],
[	0.323857623	,	0.081472443	,	0.234563929	,	0.333913894	,],
[	0.409495276	,	0.312216614	,	0.077489702	,	0.317622112	,],
[	0.326893858	,	0.359854752	,	0.26833093	,	0.086988098	,]],
[[	0.087629731	,	0.268892666	,	0.281298128	,	0.243764123	,],
[	0.328105255	,	0.082960296	,	0.238629102	,	0.338125781	,],
[	0.407174333	,	0.309466147	,	0.071197536	,	0.315376499	,],
[	0.331304721	,	0.363804698	,	0.272345648	,	0.094072712	,]],
[[	0.092257921	,	0.275709494	,	0.288202141	,	0.250397126	,],
[	0.323696495	,	0.079772559	,	0.234259597	,	0.333810101	,],
[	0.407234121	,	0.309687766	,	0.07327656	,	0.315264255	,],
[	0.327179402	,	0.360051959	,	0.268668005	,	0.086067453	,]],
[[	0.086921426	,	0.26946007	,	0.281988211	,	0.244286505	,],
[	0.325339413	,	0.084163772	,	0.236193194	,	0.335330891	,],
[	0.409255367	,	0.311532151	,	0.072852269	,	0.317691599	,],
[	0.328635673	,	0.361215301	,	0.269737545	,	0.089633942	,]],
[[	0.089605305	,	0.269676639	,	0.282079296	,	0.244613268	,],
[	0.325690503	,	0.080759452	,	0.236156849	,	0.335760089	,],
[	0.406491314	,	0.308727078	,	0.071072911	,	0.314643839	,],
[	0.329206829	,	0.362258481	,	0.271054097	,	0.087625071	,]],
[[	0.089326065	,	0.272476711	,	0.28492907	,	0.247265704	,],
[	0.323730996	,	0.080761285	,	0.234361643	,	0.333812737	,],
[	0.409021723	,	0.311573307	,	0.076365595	,	0.317089342	,],
[	0.327066203	,	0.359927699	,	0.268502498	,	0.086633562	,]],
[[	0.08692008	,	0.268546546	,	0.281002952	,	0.243333768	,],
[	0.327498913	,	0.087893242	,	0.23850816	,	0.337311849	,],
[	0.407662174	,	0.310025475	,	0.071453048	,	0.315965266	,],
[	0.33048038	,	0.362995712	,	0.271529843	,	0.092620655	,]],
[[	0.092892903	,	0.271214015	,	0.283588391	,	0.246328611	,],
[	0.323974085	,	0.079573328	,	0.234418415	,	0.333996286	,],
[	0.407250789	,	0.309606323	,	0.072706067	,	0.315198723	,],
[	0.327463744	,	0.360393861	,	0.268965653	,	0.086154847	,]],
[[	0.087620591	,	0.270289283	,	0.282909113	,	0.244975808	,],
[	0.324778295	,	0.083046956	,	0.235442875	,	0.334693588	,],
[	0.41085774	,	0.313180614	,	0.074434578	,	0.319260858	,],
[	0.327731406	,	0.360448368	,	0.26890242	,	0.088234785	,]],
[[	0.089765531	,	0.274026811	,	0.286198169	,	0.243915464	,],
[	0.318820617	,	0.080599433	,	0.235388494	,	0.330086573	,],
[	0.399303295	,	0.307946562	,	0.070558392	,	0.309226029	,],
[	0.324903134	,	0.36411757	,	0.27317471	,	0.08654014	,]],
[[	0.088248481	,	0.275749618	,	0.287974164	,	0.245584541	,],
[	0.316744626	,	0.080571516	,	0.233423715	,	0.327979057	,],
[	0.402865372	,	0.311874955	,	0.077257611	,	0.312740227	,],
[	0.323423679	,	0.362361656	,	0.271334921	,	0.086525871	,]],
[[	0.087408173	,	0.273045202	,	0.285251355	,	0.242913819	,],
[	0.320326092	,	0.087543658	,	0.237442137	,	0.331473708	,],
[	0.400307484	,	0.308799651	,	0.070410017	,	0.310193505	,],
[	0.327355415	,	0.366010048	,	0.274879763	,	0.093215979	,]],
[[	0.092471943	,	0.280463759	,	0.29267314	,	0.250033391	,],
[	0.317418992	,	0.079728194	,	0.233856866	,	0.328550173	,],
[	0.40045176	,	0.309191536	,	0.072707493	,	0.310260014	,],
[	0.323554778	,	0.362547587	,	0.271484634	,	0.085525314	,]],
[[	0.086654751	,	0.273829885	,	0.286128778	,	0.243578349	,],
[	0.317879595	,	0.082918427	,	0.23455447	,	0.328960891	,],
[	0.402699159	,	0.311269301	,	0.072584241	,	0.312911379	,],
[	0.324432335	,	0.363244473	,	0.272002227	,	0.088514228	,]],
[[	0.088920369	,	0.273576666	,	0.285899353	,	0.243494373	,],
[	0.319720769	,	0.081303928	,	0.236275793	,	0.330845044	,],
[	0.399783502	,	0.308300001	,	0.070627392	,	0.309568332	,],
[	0.325446479	,	0.36454615	,	0.273716117	,	0.086905449	,]],
[[	0.089437738	,	0.277072766	,	0.289221315	,	0.246748509	,],
[	0.316830961	,	0.080167454	,	0.233360317	,	0.328035444	,],
[	0.402097318	,	0.310999163	,	0.075803977	,	0.311866956	,],
[	0.323232058	,	0.362267378	,	0.27115675	,	0.085855346	,]],
[[	0.087061623	,	0.273220952	,	0.285425026	,	0.242987365	,],
[	0.319750583	,	0.086263407	,	0.23663948	,	0.330828134	,],
[	0.40128657	,	0.309805709	,	0.071249131	,	0.311170662	,],
[	0.32659232	,	0.365238879	,	0.274008013	,	0.091750162	,]],
[[	0.091936998	,	0.275292777	,	0.28724413	,	0.245166947	,],
[	0.317750308	,	0.079770719	,	0.234108434	,	0.32880227	,],
[	0.400359415	,	0.30893902	,	0.072231665	,	0.310066768	,],
[	0.323815504	,	0.362756211	,	0.271671539	,	0.085617873	,]],
[[	0.087422351	,	0.274671555	,	0.287000288	,	0.2443274	,],
[	0.317383158	,	0.081817497	,	0.233929916	,	0.328377376	,],
[	0.404126104	,	0.312646744	,	0.073892527	,	0.314193158	,],
[	0.323916105	,	0.362645602	,	0.271441383	,	0.08753118	,]],
[[	0.088963383	,	0.278038536	,	0.289399775	,	0.24354492	,],
[	0.313538437	,	0.081170337	,	0.235624237	,	0.326212052	,],
[	0.395336071	,	0.309933632	,	0.071042845	,	0.305866893	,],
[	0.322757219	,	0.367884358	,	0.275914076	,	0.08595444	,]],
[[	0.087691667	,	0.279958158	,	0.291380949	,	0.245362228	,],
[	0.310914031	,	0.080451288	,	0.23319831	,	0.323595893	,],
[	0.39838434	,	0.313456766	,	0.077256499	,	0.309006105	,],
[	0.321238103	,	0.366185799	,	0.274029469	,	0.086032357	,]],
[[	0.086299747	,	0.277018072	,	0.288285976	,	0.242453476	,],
[	0.314000316	,	0.086881316	,	0.236681667	,	0.326576582	,],
[	0.39606005	,	0.310641675	,	0.070855137	,	0.306824372	,],
[	0.325566119	,	0.370176837	,	0.278047448	,	0.093016271	,]],
[[	0.091597094	,	0.284352993	,	0.295804938	,	0.249596526	,],
[	0.31153357	,	0.079710027	,	0.233545756	,	0.324056015	,],
[	0.39604339	,	0.310721342	,	0.072894442	,	0.306520618	,],
[	0.321484284	,	0.366466931	,	0.274306716	,	0.085185007	,]],
[[	0.086066907	,	0.278026536	,	0.289491698	,	0.243289154	,],
[	0.311763004	,	0.082397641	,	0.233931193	,	0.324234694	,],
[	0.398937955	,	0.313639505	,	0.073279179	,	0.309828391	,],
[	0.322603968	,	0.367360112	,	0.2751247	,	0.0884528	,]],
[[	0.088094861	,	0.277666471	,	0.288922251	,	0.243011557	,],
[	0.314356515	,	0.081795782	,	0.236497639	,	0.326958809	,],
[	0.395770988	,	0.310290519	,	0.07106464	,	0.306263722	,],
[	0.323661916	,	0.368800426	,	0.27687485	,	0.086643381	,]],
[[	0.088409366	,	0.28099316	,	0.292350882	,	0.246218005	,],
[	0.310990387	,	0.080014846	,	0.233101408	,	0.323517416	,],
[	0.397669779	,	0.31256324	,	0.075947348	,	0.308165916	,],
[	0.321127983	,	0.366081541	,	0.273960849	,	0.08538805	,]],
[[	0.085995825	,	0.277089053	,	0.288363198	,	0.242469452	,],
[	0.313252737	,	0.085377565	,	0.235648112	,	0.325631083	,],
[	0.396982742	,	0.311548704	,	0.07144348	,	0.307642611	,],
[	0.324355226	,	0.368997014	,	0.276805475	,	0.091201421	,]],
[[	0.091186584	,	0.279309131	,	0.290456668	,	0.24487704	,],
[	0.312272899	,	0.08016927	,	0.234236921	,	0.324747372	,],
[	0.395963651	,	0.310601138	,	0.072277165	,	0.306416664	,],
[	0.321780557	,	0.366720038	,	0.274613416	,	0.085129427	,]],
[[	0.086432154	,	0.278632275	,	0.28999293	,	0.243849066	,],
[	0.31129771	,	0.081595674	,	0.233477212	,	0.323764788	,],
[	0.399549677	,	0.314774923	,	0.079350056	,	0.309998821	,],
[	0.32211152	,	0.366859093	,	0.274579922	,	0.087267152	,]],
[[	0.088721701	,	0.283257141	,	0.293379721	,	0.243132225	,],
[	0.308669577	,	0.081647417	,	0.235246464	,	0.321793732	,],
[	0.391656228	,	0.312008567	,	0.071223755	,	0.302875368	,],
[	0.321380204	,	0.372333684	,	0.27919345	,	0.085911307	,]],
[[	0.087118561	,	0.284896712	,	0.295100921	,	0.244696599	,],
[	0.306135606	,	0.080947701	,	0.232806376	,	0.319242077	,],
[	0.394087933	,	0.314834002	,	0.076695592	,	0.305226084	,],
[	0.319912836	,	0.370635613	,	0.277319217	,	0.085778593	,]],
[[	0.085799282	,	0.281910575	,	0.291995018	,	0.241757144	,],
[	0.30901061	,	0.087138359	,	0.236016975	,	0.322005524	,],
[	0.392606769	,	0.312845974	,	0.071077574	,	0.303863017	,],
[	0.324066133	,	0.374414692	,	0.281187539	,	0.092586687	,]],
[[	0.091104996	,	0.28927757	,	0.299595653	,	0.24896936	,],
[	0.306939164	,	0.080415545	,	0.233412315	,	0.319988657	,],
[	0.392054429	,	0.312533769	,	0.072760003	,	0.30316465	,],
[	0.320185377	,	0.370906235	,	0.27772404	,	0.084902185	,]],
[[	0.085449533	,	0.282815036	,	0.293092587	,	0.242476304	,],
[	0.306786241	,	0.08273776	,	0.233410609	,	0.31964529	,],
[	0.395681435	,	0.315934148	,	0.073906073	,	0.307129481	,],
[	0.321241885	,	0.371618185	,	0.278295553	,	0.087972768	,]],
[[	0.087674141	,	0.282571384	,	0.292660072	,	0.24244959	,],
[	0.309866123	,	0.082515798	,	0.236336493	,	0.322918892	,],
[	0.391713572	,	0.312011281	,	0.071049821	,	0.302935028	,],
[	0.32247094	,	0.373257043	,	0.280184035	,	0.086636005	,]],
[[	0.087858104	,	0.285827983	,	0.29607307	,	0.245593424	,],
[	0.305840521	,	0.080155849	,	0.232334436	,	0.31889483	,],
[	0.393661437	,	0.31428795	,	0.075567111	,	0.304649972	,],
[	0.319700951	,	0.370480204	,	0.277161549	,	0.085207928	,]],
[[	0.085670483	,	0.282194396	,	0.292260262	,	0.241924122	,],
[	0.30832357	,	0.085642251	,	0.235121641	,	0.321139223	,],
[	0.393349686	,	0.313551867	,	0.071729548	,	0.304583414	,],
[	0.323027654	,	0.373432236	,	0.280113922	,	0.091072808	,]],
[[	0.090855577	,	0.284295613	,	0.294217242	,	0.244202181	,],
[	0.307418651	,	0.08061823	,	0.233802313	,	0.320391853	,],
[	0.391848893	,	0.312266978	,	0.071995344	,	0.302820383	,],
[	0.320666752	,	0.371433766	,	0.278169719	,	0.085152565	,]],
[[	0.086000608	,	0.283650204	,	0.293844806	,	0.243332767	,],
[	0.30651426	,	0.081801699	,	0.233076265	,	0.319421811	,],
[	0.395429355	,	0.316357926	,	0.078965772	,	0.30653576	,],
[	0.320475422	,	0.371002656	,	0.277496642	,	0.086803677	,]],
[[	0.088451555	,	0.289184647	,	0.297267253	,	0.24300621	,],
[	0.303942854	,	0.089505625	,	0.236153688	,	0.317696041	,],
[	0.386594087	,	0.313202533	,	0.070830465	,	0.298697274	,],
[	0.319285541	,	0.376861876	,	0.283270192	,	0.086354961	,]],
[[	0.087517512	,	0.291493431	,	0.299758831	,	0.245253506	,],
[	0.300227789	,	0.080788382	,	0.232045375	,	0.314171089	,],
[	0.387335777	,	0.314324773	,	0.074025691	,	0.299435872	,],
[	0.316544786	,	0.374135046	,	0.280272657	,	0.084921799	,]],
[[	0.085850912	,	0.288143165	,	0.296283991	,	0.241999329	,],
[	0.301129912	,	0.084530356	,	0.233180173	,	0.315075079	,],
[	0.388802748	,	0.315504578	,	0.072373109	,	0.301109529	,],
[	0.319809142	,	0.377073012	,	0.283217866	,	0.090690241	,]],
[[	0.09186148	,	0.290899637	,	0.29891364	,	0.244948818	,],
[	0.302338036	,	0.08193922	,	0.234039964	,	0.316268075	,],
[	0.386156189	,	0.31299184	,	0.071099636	,	0.298164046	,],
[	0.317543586	,	0.37503166	,	0.281286599	,	0.08494658	,]],
[[	0.085772855	,	0.289313291	,	0.297512564	,	0.242997363	,],
[	0.300044937	,	0.081499445	,	0.231815185	,	0.313777132	,],
[	0.389219064	,	0.316478707	,	0.077262854	,	0.301191614	,],
[	0.31732581	,	0.374613069	,	0.280633916	,	0.086386434	,]],
[[	0.087622989	,	0.28899065	,	0.297075044	,	0.242649227	,],
[	0.303232728	,	0.088058563	,	0.235305579	,	0.316819139	,],
[	0.387047582	,	0.313692417	,	0.070958936	,	0.299201363	,],
[	0.320229928	,	0.377794386	,	0.284245444	,	0.087164538	,]],
[[	0.088744684	,	0.292715406	,	0.300822668	,	0.246371434	,],
[	0.300887289	,	0.080942628	,	0.232501855	,	0.314733997	,],
[	0.386953248	,	0.313824878	,	0.073033618	,	0.298821734	,],
[	0.316656118	,	0.374106717	,	0.280362916	,	0.084549678	,]],
[[	0.085656868	,	0.288435407	,	0.296424206	,	0.242179689	,],
[	0.300443891	,	0.083235064	,	0.232327843	,	0.314327024	,],
[	0.389951773	,	0.316514987	,	0.07342639	,	0.302218727	,],
[	0.318785429	,	0.37604224	,	0.28214426	,	0.089130961	,]],
[[	0.090598923	,	0.2903862	,	0.298377244	,	0.244245328	,],
[	0.303390047	,	0.082556472	,	0.234862553	,	0.317108328	,],
[	0.386444403	,	0.313110642	,	0.071074812	,	0.298444306	,],
[	0.317995691	,	0.375425724	,	0.281688768	,	0.085093108	,]],
[[	0.086196647	,	0.290122292	,	0.298255438	,	0.243725057	,],
[	0.299897278	,	0.080870391	,	0.231665436	,	0.313645136	,],
[	0.388173627	,	0.315287945	,	0.075709359	,	0.300072222	,],
[	0.316811001	,	0.374160586	,	0.280232258	,	0.085612943	,]],
[[	0.086275238	,	0.293792676	,	0.301348546	,	0.242819203	,],
[	0.295421926	,	0.085852292	,	0.234087845	,	0.31147736	,],
[	0.382421293	,	0.31578864	,	0.071522832	,	0.296294042	,],
[	0.318547638	,	0.382467823	,	0.288475637	,	0.087874873	,]],
[[	0.087686866	,	0.297754785	,	0.3054658	,	0.246676965	,],
[	0.295252302	,	0.081526003	,	0.233484332	,	0.311362787	,],
[	0.381342303	,	0.314951552	,	0.072490484	,	0.295038384	,],
[	0.313981055	,	0.377880165	,	0.283583104	,	0.084105226	,]],
[[	0.084475494	,	0.293287111	,	0.300896744	,	0.242387808	,],
[	0.293701243	,	0.082112167	,	0.232116512	,	0.309870364	,],
[	0.385272757	,	0.319456887	,	0.079700762	,	0.298929164	,],
[	0.315716628	,	0.379303034	,	0.284787719	,	0.087705636	,]],
[[	0.089236703	,	0.295085863	,	0.302556489	,	0.244283531	,],
[	0.297535418	,	0.08941271	,	0.236271482	,	0.313400283	,],
[	0.38150185	,	0.314947494	,	0.071202022	,	0.295199497	,],
[	0.316163964	,	0.379905353	,	0.285716212	,	0.085556551	,]],
[[	0.085071614	,	0.295085529	,	0.302853026	,	0.243996397	,],
[	0.293943294	,	0.080756707	,	0.232101308	,	0.310051589	,],
[	0.382655199	,	0.316394684	,	0.075015352	,	0.296327306	,],
[	0.314161749	,	0.377863832	,	0.283327177	,	0.084767746	,]],
[[	0.085472801	,	0.293390167	,	0.301000046	,	0.242339455	,],
[	0.294839869	,	0.084620432	,	0.233291155	,	0.310834448	,],
[	0.383372426	,	0.316671173	,	0.072438418	,	0.297202351	,],
[	0.317929663	,	0.381180035	,	0.286705036	,	0.091134335	,]],
[[	0.088446241	,	0.298595168	,	0.306276191	,	0.247691822	,],
[	0.295815411	,	0.081826958	,	0.23409696	,	0.311947441	,],
[	0.381390299	,	0.31489062	,	0.071997653	,	0.294993554	,],
[	0.314580962	,	0.378420894	,	0.284183567	,	0.084258831	,]],
[[	0.084355948	,	0.293594245	,	0.301102132	,	0.242570837	,],
[	0.293489979	,	0.081392291	,	0.231758248	,	0.309536406	,],
[	0.384503868	,	0.318461397	,	0.078247662	,	0.29805942	,],
[	0.314964262	,	0.378662214	,	0.284106173	,	0.086450006	,]],
[[	0.088146586	,	0.294597993	,	0.302047352	,	0.243668186	,],
[	0.296732151	,	0.087996685	,	0.235393787	,	0.312493997	,],
[	0.38208799	,	0.315480625	,	0.071511614	,	0.295758868	,],
[	0.316697984	,	0.380491457	,	0.2862786	,	0.085909757	,]],
[[	0.085673285	,	0.295818755	,	0.30346024	,	0.244670792	,],
[	0.294262146	,	0.080898487	,	0.232335121	,	0.310342139	,],
[	0.381993988	,	0.315734501	,	0.073776096	,	0.29559308	,],
[	0.313815279	,	0.377554543	,	0.283077305	,	0.084093447	,]],
[[	0.087305375	,	0.300057367	,	0.306711496	,	0.243681445	,],
[	0.290639953	,	0.087476329	,	0.235044983	,	0.307969654	,],
[	0.377248889	,	0.316392583	,	0.071074413	,	0.291524648	,],
[	0.315709735	,	0.385096544	,	0.290820735	,	0.086026384	,]],
[[	0.085899993	,	0.301864616	,	0.308510491	,	0.245384617	,],
[	0.289210222	,	0.081184614	,	0.233005078	,	0.306693055	,],
[	0.377086559	,	0.31657374	,	0.073089406	,	0.291179685	,],
[	0.312678125	,	0.381937519	,	0.287557442	,	0.08419993	,]],
[[	0.084508499	,	0.298792632	,	0.305346744	,	0.242373467	,],
[	0.288535038	,	0.083174087	,	0.232580031	,	0.306023705	,],
[	0.380567558	,	0.319728726	,	0.074085185	,	0.295036502	,],
[	0.31595792	,	0.384975491	,	0.290582417	,	0.089950135	,]],
[[	0.089939481	,	0.306204183	,	0.312899435	,	0.249553414	,],
[	0.291959082	,	0.083321445	,	0.235583608	,	0.309363999	,],
[	0.37659176	,	0.315863907	,	0.071105171	,	0.290717571	,],
[	0.31380239	,	0.382996136	,	0.288659216	,	0.084331255	,]],
[[	0.084204088	,	0.29986668	,	0.306575974	,	0.243256579	,],
[	0.288302976	,	0.081246115	,	0.23207784	,	0.305746656	,],
[	0.378653734	,	0.318291919	,	0.075816091	,	0.292615046	,],
[	0.313564276	,	0.382621238	,	0.288062987	,	0.085779586	,]],
[[	0.086331666	,	0.299531874	,	0.30617818	,	0.243003577	,],
[	0.289868709	,	0.085967697	,	0.234145264	,	0.307272379	,],
[	0.377966563	,	0.317133422	,	0.071745503	,	0.292273161	,],
[	0.316689124	,	0.38598553	,	0.291767793	,	0.086934085	,]],
[[	0.087027021	,	0.303089708	,	0.309758319	,	0.246578002	,],
[	0.289826708	,	0.081649014	,	0.233589943	,	0.307249211	,],
[	0.376961146	,	0.316289518	,	0.072332374	,	0.290960422	,],
[	0.312989955	,	0.382219872	,	0.28779536	,	0.083976455	,]],
[[	0.084484177	,	0.299164939	,	0.305805047	,	0.242641333	,],
[	0.28818719	,	0.082206924	,	0.232242194	,	0.305690827	,],
[	0.380729935	,	0.320681552	,	0.079653622	,	0.294688366	,],
[	0.315215723	,	0.384194234	,	0.289725031	,	0.088565301	,]],
[[	0.089448383	,	0.30108976	,	0.307597179	,	0.24467189	,],
[	0.292340722	,	0.089942646	,	0.236748809	,	0.309468885	,],
[	0.376875955	,	0.316087518	,	0.071096248	,	0.291048651	,],
[	0.314715504	,	0.383894424	,	0.28946797	,	0.084996485	,]],
[[	0.084887851	,	0.300709933	,	0.30738317	,	0.244084965	,],
[	0.288736711	,	0.08131366	,	0.232491445	,	0.306051787	,],
[	0.378010837	,	0.317553332	,	0.074591251	,	0.291921358	,],
[	0.313343503	,	0.382430611	,	0.287820181	,	0.08509771	,]],
[[	0.086735183	,	0.304742432	,	0.311671393	,	0.244042461	,],
[	0.283223439	,	0.083532271	,	0.232387058	,	0.301812468	,],
[	0.376628375	,	0.321255876	,	0.073804743	,	0.291952643	,],
[	0.315710657	,	0.390526219	,	0.295639229	,	0.087565292	,]],
[[	0.086337748	,	0.307355932	,	0.314239673	,	0.246634556	,],
[	0.285923476	,	0.082782351	,	0.234856662	,	0.304724381	,],
[	0.373313427	,	0.318103031	,	0.071853823	,	0.288439273	,],
[	0.311350536	,	0.386392848	,	0.290986994	,	0.083851147	,]],
[[	0.084397344	,	0.303842976	,	0.310726366	,	0.243112533	,],
[	0.282479266	,	0.08104265	,	0.231551677	,	0.301192991	,],
[	0.375324557	,	0.320597623	,	0.076945939	,	0.290392403	,],
[	0.312246849	,	0.386933771	,	0.291510916	,	0.086824808	,]],
[[	0.090243222	,	0.306661705	,	0.313389855	,	0.24602646	,],
[	0.284950072	,	0.086689567	,	0.234394062	,	0.303427547	,],
[	0.374499087	,	0.31917344	,	0.072077345	,	0.28972152	,],
[	0.313318385	,	0.388146468	,	0.293013595	,	0.085209471	,]],
[[	0.084544109	,	0.305427458	,	0.31237221	,	0.244550304	,],
[	0.283752402	,	0.081042031	,	0.232591478	,	0.302330302	,],
[	0.373683599	,	0.318700633	,	0.073315839	,	0.288740769	,],
[	0.311085909	,	0.385938268	,	0.290445068	,	0.084242521	,]],
[[	0.08581919	,	0.304280447	,	0.311222192	,	0.243528523	,],
[	0.283050941	,	0.082788138	,	0.232105565	,	0.301523535	,],
[	0.377183154	,	0.322904417	,	0.080374119	,	0.292376718	,],
[	0.315075854	,	0.389446732	,	0.294050043	,	0.090643109	,]],
[[	0.087734377	,	0.308817679	,	0.315605301	,	0.248027322	,],
[	0.287075144	,	0.083797837	,	0.235954242	,	0.305663012	,],
[	0.373553941	,	0.31832925	,	0.071918291	,	0.288712112	,],
[	0.311438411	,	0.386313295	,	0.291094705	,	0.083647941	,]],
[[	0.084227948	,	0.30418526	,	0.311012119	,	0.243315457	,],
[	0.282826293	,	0.080958033	,	0.231739175	,	0.301439126	,],
[	0.374734363	,	0.319982851	,	0.075720039	,	0.289850154	,],
[	0.312132775	,	0.386918249	,	0.29144998	,	0.086105519	,]],
[[	0.088961266	,	0.305911517	,	0.312715888	,	0.245287221	,],
[	0.284231196	,	0.08534995	,	0.233531075	,	0.302681511	,],
[	0.375503342	,	0.320147229	,	0.072978057	,	0.290700051	,],
[	0.314231757	,	0.389051294	,	0.293868661	,	0.085873822	,]],
[[	0.085247226	,	0.306154861	,	0.3131005	,	0.245252107	,],
[	0.284639845	,	0.081640151	,	0.233423928	,	0.303164524	,],
[	0.37360325	,	0.318403741	,	0.072742763	,	0.28857251	,],
[	0.310926696	,	0.385680301	,	0.290291391	,	0.083632974	,]],
[[	0.089066218	,	0.319404932	,	0.323267573	,	0.249344253	,],
[	0.286776038	,	0.085719315	,	0.238661894	,	0.304703081	,],
[	0.382742998	,	0.333583868	,	0.081312959	,	0.29477278	,],
[	0.321449762	,	0.404920878	,	0.30498295	,	0.09170614	,]],
[[	0.088493136	,	0.321676823	,	0.325560433	,	0.251695906	,],
[	0.290810374	,	0.093409341	,	0.243213279	,	0.308513413	,],
[	0.37997098	,	0.329877401	,	0.074145523	,	0.292090431	,],
[	0.318602137	,	0.402267191	,	0.302510378	,	0.0852452	,]],
[[	0.086657643	,	0.318377938	,	0.322327697	,	0.248465243	,],
[	0.286967898	,	0.084295296	,	0.238630536	,	0.30490015	,],
[	0.380441143	,	0.330824087	,	0.076898428	,	0.292400044	,],
[	0.318912579	,	0.402710554	,	0.302691787	,	0.087477472	,]],
[[	0.092315037	,	0.321047953	,	0.324801536	,	0.251108746	,],
[	0.288169873	,	0.088466824	,	0.240232328	,	0.306068695	,],
[	0.382405555	,	0.332258487	,	0.076092851	,	0.294685381	,],
[	0.321699308	,	0.404974494	,	0.305503098	,	0.087805925	,]],
[[	0.086467659	,	0.319436759	,	0.323368435	,	0.249297364	,],
[	0.289537858	,	0.085664783	,	0.240919416	,	0.307260395	,],
[	0.37961943	,	0.329636684	,	0.074479667	,	0.291505997	,],
[	0.318152661	,	0.401808992	,	0.301773137	,	0.085288183	,]],
[[	0.088119869	,	0.318899509	,	0.32281133	,	0.248781218	,],
[	0.286778416	,	0.085248073	,	0.238535745	,	0.304572883	,],
[	0.382161664	,	0.33290459	,	0.080079488	,	0.294032838	,],
[	0.320653089	,	0.404230748	,	0.304152006	,	0.090317679	,]],
[[	0.089552062	,	0.322789179	,	0.32665536	,	0.252841864	,],
[	0.289852674	,	0.091937673	,	0.242161158	,	0.307590682	,],
[	0.380789068	,	0.330610073	,	0.074676557	,	0.292771616	,],
[	0.319412336	,	0.403010376	,	0.303290733	,	0.085796919	,]],
[[	0.086268182	,	0.318343488	,	0.322246061	,	0.248271543	,],
[	0.28736009	,	0.084426106	,	0.238951141	,	0.305206476	,],
[	0.380175898	,	0.330374307	,	0.075973804	,	0.292026909	,],
[	0.318452375	,	0.402217849	,	0.302105557	,	0.086487799	,]],
[[	0.091072152	,	0.32053991	,	0.324344681	,	0.250412019	,],
[	0.287561979	,	0.087290576	,	0.2395158	,	0.305349967	,],
[	0.383850185	,	0.333679518	,	0.077439168	,	0.296042935	,],
[	0.322933653	,	0.406271968	,	0.306712927	,	0.089087263	,]],
[[	0.087546742	,	0.320498785	,	0.324493091	,	0.250385672	,],
[	0.290595205	,	0.086722324	,	0.242005514	,	0.308266751	,],
[	0.379683126	,	0.329672889	,	0.074226059	,	0.291564163	,],
[	0.318455718	,	0.40205807	,	0.302079584	,	0.085371902	,]],
[[	0.087666703	,	0.325872792	,	0.328737854	,	0.249987174	,],
[	0.281336006	,	0.085080925	,	0.239238492	,	0.30156115	,],
[	0.377984974	,	0.334417982	,	0.078064173	,	0.290324858	,],
[	0.318902266	,	0.408923423	,	0.307989792	,	0.088902836	,]],
[[	0.089191994	,	0.329659293	,	0.332573375	,	0.253978498	,],
[	0.283385166	,	0.090367754	,	0.241580014	,	0.303438328	,],
[	0.378791048	,	0.33453118	,	0.075614995	,	0.29137246	,],
[	0.319605241	,	0.409551385	,	0.308995483	,	0.087023579	,]],
[[	0.085616865	,	0.325201263	,	0.32810516	,	0.249279348	,],
[	0.283076969	,	0.085549177	,	0.24071009	,	0.303200766	,],
[	0.376623561	,	0.332737537	,	0.074886843	,	0.288933884	,],
[	0.317758956	,	0.40786693	,	0.306933227	,	0.086247075	,]],
[[	0.090862435	,	0.327623816	,	0.330434498	,	0.251745241	,],
[	0.281358811	,	0.086239262	,	0.239438797	,	0.301558181	,],
[	0.379894254	,	0.33659324	,	0.081377948	,	0.292177361	,],
[	0.321122063	,	0.41107194	,	0.310101022	,	0.092300532	,]],
[[	0.086771398	,	0.327189705	,	0.330123584	,	0.25130066	,],
[	0.285144323	,	0.093879532	,	0.243761934	,	0.305203298	,],
[	0.377426789	,	0.333253559	,	0.074501347	,	0.289789145	,],
[	0.318131647	,	0.408003139	,	0.307220543	,	0.085747003	,]],
[[	0.086993807	,	0.32565228	,	0.328563406	,	0.249699376	,],
[	0.281610094	,	0.084925704	,	0.239376685	,	0.301635898	,],
[	0.37731919	,	0.33372126	,	0.076792943	,	0.289595086	,],
[	0.318538669	,	0.408479281	,	0.307388342	,	0.087985376	,]],
[[	0.090373911	,	0.330914258	,	0.333693468	,	0.255098486	,],
[	0.282764338	,	0.089012549	,	0.240895416	,	0.302748262	,],
[	0.379980102	,	0.335634615	,	0.076654725	,	0.29246388	,],
[	0.320717887	,	0.410537841	,	0.309993996	,	0.087923766	,]],
[[	0.086007177	,	0.325750472	,	0.32857598	,	0.249819148	,],
[	0.284161369	,	0.08645563	,	0.241694083	,	0.304144586	,],
[	0.376604354	,	0.332573424	,	0.074558597	,	0.288865756	,],
[	0.317675099	,	0.407714034	,	0.306774006	,	0.085802217	,]],
[[	0.089525057	,	0.326923589	,	0.329612122	,	0.250855352	,],
[	0.281552613	,	0.085707158	,	0.239346667	,	0.301518822	,],
[	0.379241089	,	0.335787227	,	0.080066842	,	0.291413122	,],
[	0.320334881	,	0.410226059	,	0.309222803	,	0.090840783	,]],
[[	0.087891638	,	0.328338393	,	0.331272002	,	0.252478224	,],
[	0.284481314	,	0.092355215	,	0.242807618	,	0.304400768	,],
[	0.377599792	,	0.333310947	,	0.074412005	,	0.289965567	,],
[	0.319124902	,	0.409047575	,	0.308340969	,	0.086458699	,]],
[[	0.087855588	,	0.331368143	,	0.334443252	,	0.251373712	,],
[	0.276203585	,	0.085196844	,	0.240004424	,	0.297700716	,],
[	0.373765479	,	0.33695851	,	0.079293359	,	0.287750151	,],
[	0.318783146	,	0.414440449	,	0.313298377	,	0.090985457	,]],
[[	0.087904034	,	0.333857458	,	0.336980203	,	0.25407218	,],
[	0.277874127	,	0.090188029	,	0.242142618	,	0.299353081	,],
[	0.374552442	,	0.336769472	,	0.076579861	,	0.28864481	,],
[	0.317000693	,	0.412705687	,	0.31186101	,	0.085899906	,]],
[[	0.085504688	,	0.330306244	,	0.333355468	,	0.250344371	,],
[	0.277506541	,	0.085383706	,	0.241187674	,	0.298990229	,],
[	0.372789525	,	0.335360936	,	0.076448206	,	0.2866396	,],
[	0.316791228	,	0.412661287	,	0.311380098	,	0.0872398	,]],
[[	0.090925281	,	0.332934786	,	0.33581024	,	0.252913829	,],
[	0.276248535	,	0.086464013	,	0.240211415	,	0.297746243	,],
[	0.376030442	,	0.339350815	,	0.082971341	,	0.289923251	,],
[	0.320102383	,	0.415594233	,	0.314948607	,	0.088568346	,]],
[[	0.085750741	,	0.331681182	,	0.334838255	,	0.251724584	,],
[	0.279912425	,	0.093957581	,	0.244633031	,	0.301391945	,],
[	0.373247413	,	0.335525344	,	0.07557366	,	0.287149698	,],
[	0.315983673	,	0.411742799	,	0.310691512	,	0.085345638	,]],
[[	0.086726521	,	0.330700447	,	0.333732582	,	0.250653499	,],
[	0.276365442	,	0.084960069	,	0.23999628	,	0.297787711	,],
[	0.373592664	,	0.336515043	,	0.078332923	,	0.287370719	,],
[	0.31821193	,	0.413974547	,	0.31278635	,	0.089913981	,]],
[[	0.088956641	,	0.334996503	,	0.338099418	,	0.255240955	,],
[	0.27726776	,	0.08892701	,	0.241450622	,	0.298626542	,],
[	0.375258378	,	0.337605644	,	0.077274908	,	0.289432588	,],
[	0.317661516	,	0.413342573	,	0.3125321	,	0.086256114	,]],
[[	0.085490532	,	0.330649478	,	0.33354757	,	0.250571909	,],
[	0.27858672	,	0.086059112	,	0.242083382	,	0.299976734	,],
[	0.372668151	,	0.335135608	,	0.075798157	,	0.28641712	,],
[	0.316083339	,	0.411890726	,	0.310710257	,	0.086174944	,]],
[[	0.089722093	,	0.332277549	,	0.335196628	,	0.252171445	,],
[	0.276070266	,	0.085759282	,	0.239925581	,	0.297455997	,],
[	0.375285386	,	0.338466427	,	0.081529388	,	0.289039206	,],
[	0.320205776	,	0.415970723	,	0.314781802	,	0.093150598	,]],
[[	0.086809136	,	0.33271918	,	0.335875005	,	0.252784363	,],
[	0.27905321	,	0.092370267	,	0.243417053	,	0.300432989	,],
[	0.373408849	,	0.335754202	,	0.075753999	,	0.28740046	,],
[	0.31663472	,	0.41225063	,	0.311307555	,	0.085662856	,]],
[[	0.087718052	,	0.338301829	,	0.339341056	,	0.252372614	,],
[	0.272674409	,	0.087016348	,	0.241436053	,	0.294976215	,],
[	0.370819407	,	0.340041261	,	0.078983946	,	0.285171697	,],
[	0.317549474	,	0.420208467	,	0.316747302	,	0.089842332	,]],
[[	0.089034412	,	0.341939836	,	0.343107029	,	0.256242022	,],
[	0.273267178	,	0.090610549	,	0.242562795	,	0.295558094	,],
[	0.371828338	,	0.340270866	,	0.076482192	,	0.286253158	,],
[	0.317574082	,	0.420174352	,	0.317037019	,	0.086930233	,]],
[[	0.085912142	,	0.337692827	,	0.33865018	,	0.25181384	,],
[	0.274895312	,	0.088220612	,	0.243567063	,	0.297237168	,],
[	0.369715626	,	0.338450312	,	0.075869984	,	0.283942988	,],
[	0.315867611	,	0.418502973	,	0.315249292	,	0.08666433	,]],
[[	0.090841702	,	0.340050554	,	0.340972383	,	0.254108028	,],
[	0.272187702	,	0.087566826	,	0.241236175	,	0.294564547	,],
[	0.372643391	,	0.342205164	,	0.082217861	,	0.286989126	,],
[	0.319785062	,	0.422438451	,	0.31902285	,	0.093222976	,]],
[[	0.086878884	,	0.339643125	,	0.340799779	,	0.253850121	,],
[	0.275043187	,	0.094115895	,	0.244709876	,	0.297291894	,],
[	0.370280507	,	0.338875483	,	0.075398677	,	0.284683306	,],
[	0.316278866	,	0.418738389	,	0.315569545	,	0.085996672	,]],
[[	0.087117165	,	0.338101192	,	0.339214998	,	0.252169262	,],
[	0.273105687	,	0.086912344	,	0.241705435	,	0.295410732	,],
[	0.370560055	,	0.339476603	,	0.077789414	,	0.284653632	,],
[	0.317068342	,	0.419698018	,	0.316216856	,	0.088752462	,]],
[[	0.089959214	,	0.342818592	,	0.34384185	,	0.257126797	,],
[	0.273009843	,	0.089674625	,	0.242113363	,	0.295201772	,],
[	0.372908465	,	0.34121457	,	0.077306824	,	0.28734146	,],
[	0.318622544	,	0.421073422	,	0.318099587	,	0.087858001	,]],
[[	0.085912835	,	0.338065558	,	0.339093159	,	0.252186196	,],
[	0.275802662	,	0.088868216	,	0.244380687	,	0.298039548	,],
[	0.369584362	,	0.338290357	,	0.075331337	,	0.283793034	,],
[	0.315612901	,	0.418368849	,	0.315011231	,	0.08608938	,]],
[[	0.089773039	,	0.339474218	,	0.340371847	,	0.253433103	,],
[	0.272472227	,	0.087260765	,	0.241240002	,	0.294773575	,],
[	0.371965554	,	0.341287219	,	0.080864946	,	0.28618347	,],
[	0.318925623	,	0.42162925	,	0.318102135	,	0.091927711	,]],
[[	0.087585515	,	0.340497343	,	0.341591894	,	0.254681312	,],
[	0.274366803	,	0.092631255	,	0.243719196	,	0.296499264	,],
[	0.370730747	,	0.339256838	,	0.075514221	,	0.285090748	,],
[	0.316910758	,	0.419378204	,	0.316119791	,	0.086294326	,]],
[[	0.087541918	,	0.342927398	,	0.342154424	,	0.252502435	,],
[	0.267663937	,	0.087133659	,	0.24043497	,	0.291512639	,],
[	0.36530835	,	0.340188704	,	0.07668464	,	0.28039688	,],
[	0.315906675	,	0.423639642	,	0.318943727	,	0.089152335	,]],
[[	0.088319449	,	0.34604946	,	0.345255693	,	0.255909903	,],
[	0.267183341	,	0.089344142	,	0.240400567	,	0.291032348	,],
[	0.36838112	,	0.342572449	,	0.077058699	,	0.283760086	,],
[	0.316230693	,	0.423765571	,	0.319705068	,	0.086708491	,]],
[[	0.085662134	,	0.342370434	,	0.341573893	,	0.251966994	,],
[	0.270635909	,	0.089186936	,	0.243188684	,	0.294408782	,],
[	0.364531126	,	0.339014147	,	0.074259547	,	0.279615583	,],
[	0.314478863	,	0.422198654	,	0.317710174	,	0.086100067	,]],
[[	0.090869383	,	0.344787643	,	0.343794276	,	0.254295734	,],
[	0.266701704	,	0.08715422	,	0.239749561	,	0.290601937	,],
[	0.367262731	,	0.342420868	,	0.080010612	,	0.282260005	,],
[	0.317837093	,	0.42549254	,	0.320971394	,	0.092296374	,]],
[[	0.086137788	,	0.343696686	,	0.343069681	,	0.253400437	,],
[	0.268395053	,	0.092287173	,	0.242001072	,	0.29234105	,],
[	0.366212312	,	0.340466401	,	0.075082553	,	0.281375736	,],
[	0.315017632	,	0.422452299	,	0.318109921	,	0.085644901	,]],
[[	0.086805854	,	0.342555772	,	0.341833905	,	0.252009735	,],
[	0.268489025	,	0.087623199	,	0.241223911	,	0.292261345	,],
[	0.364908065	,	0.33967617	,	0.075706495	,	0.279973407	,],
[	0.31516429	,	0.422788504	,	0.318257816	,	0.087939812	,]],
[[	0.089308816	,	0.34709587	,	0.346338649	,	0.2568295	,],
[	0.26674275	,	0.088338597	,	0.239849737	,	0.290516725	,],
[	0.369195138	,	0.344799285	,	0.083615379	,	0.284249579	,],
[	0.317448076	,	0.424810242	,	0.320776453	,	0.087776738	,]],
[[	0.085732943	,	0.342667521	,	0.341823946	,	0.252262748	,],
[	0.270959945	,	0.096370878	,	0.244644475	,	0.294481429	,],
[	0.364894163	,	0.339281207	,	0.0742702	,	0.279899189	,],
[	0.314055531	,	0.421773088	,	0.31727102	,	0.085432181	,]],
[[	0.089285854	,	0.34385353	,	0.342924046	,	0.253289164	,],
[	0.267068272	,	0.087086631	,	0.239889069	,	0.290804537	,],
[	0.366079812	,	0.341121347	,	0.07828053	,	0.281022146	,],
[	0.317417778	,	0.425007886	,	0.320487316	,	0.091341842	,]],
[[	0.087086146	,	0.344767591	,	0.343976641	,	0.254301086	,],
[	0.26787802	,	0.091140166	,	0.241334185	,	0.291716745	,],
[	0.366916525	,	0.341122587	,	0.07548121	,	0.282090764	,],
[	0.31542071	,	0.422893052	,	0.318565319	,	0.085880189	,]],
[[	0.08773378	,	0.350066741	,	0.348353958	,	0.253481336	,],
[	0.263152536	,	0.088301111	,	0.242044756	,	0.287912758	,],
[	0.361873516	,	0.343329388	,	0.077213171	,	0.277277346	,],
[	0.314748441	,	0.429031346	,	0.324098771	,	0.089018987	,]],
[[	0.08903573	,	0.353321057	,	0.351767649	,	0.257112915	,],
[	0.261642776	,	0.089281071	,	0.240796428	,	0.286373767	,],
[	0.365944414	,	0.348031851	,	0.084587676	,	0.281422479	,],
[	0.315361261	,	0.429463877	,	0.324910658	,	0.086889473	,]],
[[	0.085653597	,	0.349006912	,	0.347473945	,	0.252613611	,],
[	0.265481871	,	0.097077188	,	0.245352381	,	0.290069211	,],
[	0.361783971	,	0.342850884	,	0.075744926	,	0.277193891	,],
[	0.313199895	,	0.427521099	,	0.322655506	,	0.085998425	,]],
[[	0.090911085	,	0.351545171	,	0.349844997	,	0.255015019	,],
[	0.26170308	,	0.087747844	,	0.240753797	,	0.286551145	,],
[	0.363075134	,	0.34484566	,	0.079622264	,	0.278377128	,],
[	0.316782678	,	0.431089536	,	0.326152198	,	0.092284879	,]],
[[	0.086603881	,	0.350968506	,	0.349435025	,	0.254523326	,],
[	0.262710019	,	0.092041449	,	0.242217578	,	0.287492764	,],
[	0.363998299	,	0.344946448	,	0.077106894	,	0.279613548	,],
[	0.31381163	,	0.427884525	,	0.323135712	,	0.085489999	,]],
[[	0.086888297	,	0.349520199	,	0.347972071	,	0.252961734	,],
[	0.263963899	,	0.088842132	,	0.242582785	,	0.288635614	,],
[	0.361778729	,	0.342998657	,	0.076615626	,	0.277172273	,],
[	0.314252677	,	0.428438314	,	0.323447794	,	0.087916445	,]],
[[	0.089968558	,	0.354342334	,	0.352651313	,	0.258125942	,],
[	0.261654537	,	0.08877188	,	0.240706941	,	0.286337653	,],
[	0.365200158	,	0.347297107	,	0.083289354	,	0.280584164	,],
[	0.316340881	,	0.430311343	,	0.325806956	,	0.087801213	,]],
[[	0.085956648	,	0.349631903	,	0.347936487	,	0.253179212	,],
[	0.264777666	,	0.095592103	,	0.244438433	,	0.289328371	,],
[	0.362174617	,	0.343196652	,	0.075768757	,	0.277649105	,],
[	0.313035628	,	0.427296201	,	0.32246042	,	0.085472135	,]],
[[	0.089425119	,	0.350600803	,	0.348963151	,	0.254042919	,],
[	0.262026428	,	0.087612448	,	0.240968856	,	0.286797311	,],
[	0.362713945	,	0.344270731	,	0.07868524	,	0.277982489	,],
[	0.31601457	,	0.430206838	,	0.325063612	,	0.090808466	,]],
[[	0.087314146	,	0.351704207	,	0.350230742	,	0.255433611	,],
[	0.262234429	,	0.090669457	,	0.241552893	,	0.286923937	,],
[	0.364993839	,	0.345810497	,	0.07800863	,	0.280502114	,],
[	0.314527907	,	0.428505575	,	0.323909963	,	0.086120508	,]],
[[	0.087320437	,	0.354821132	,	0.352641493	,	0.253083442	,],
[	0.257892511	,	0.088661641	,	0.242560698	,	0.283448142	,],
[	0.357682878	,	0.345091723	,	0.07748097	,	0.273625288	,],
[	0.313110588	,	0.432603331	,	0.326991384	,	0.088307842	,]],
[[	0.088529706	,	0.358120148	,	0.356003769	,	0.256781937	,],
[	0.255707486	,	0.08880093	,	0.240743822	,	0.281201913	,],
[	0.361840651	,	0.348667905	,	0.079290474	,	0.278087144	,],
[	0.314149379	,	0.433515735	,	0.32819649	,	0.086676046	,]],
[[	0.085503573	,	0.354135196	,	0.35198366	,	0.252604828	,],
[	0.259094658	,	0.096185718	,	0.244748858	,	0.284512877	,],
[	0.357501505	,	0.3445578	,	0.075770497	,	0.273373816	,],
[	0.311307048	,	0.430973889	,	0.325387854	,	0.085177491	,]],
[[	0.090413615	,	0.356522968	,	0.354138136	,	0.254773971	,],
[	0.256068623	,	0.087718536	,	0.240954857	,	0.281698486	,],
[	0.358889693	,	0.346573891	,	0.079988433	,	0.274747852	,],
[	0.314968379	,	0.434513408	,	0.328802223	,	0.091347038	,]],
[[	0.085938653	,	0.355541078	,	0.353473516	,	0.254017602	,],
[	0.256083251	,	0.090886477	,	0.241532303	,	0.28169013	,],
[	0.359328742	,	0.346257506	,	0.076884231	,	0.275470611	,],
[	0.312263632	,	0.431530027	,	0.326101468	,	0.084985422	,]],
[[	0.086450076	,	0.354456057	,	0.352298241	,	0.252682624	,],
[	0.258630026	,	0.089318106	,	0.243281755	,	0.284105059	,],
[	0.357632188	,	0.344951748	,	0.076915842	,	0.273502697	,],
[	0.312367918	,	0.431943619	,	0.326123288	,	0.087026697	,]],
[[	0.089833216	,	0.359439453	,	0.357181299	,	0.258050879	,],
[	0.255215187	,	0.087975233	,	0.240225415	,	0.280811432	,],
[	0.360954333	,	0.348953319	,	0.083537295	,	0.276772378	,],
[	0.315114053	,	0.434344759	,	0.329160382	,	0.087447879	,]],
[[	0.085515791	,	0.354601344	,	0.352300575	,	0.252943363	,],
[	0.25823664	,	0.094509389	,	0.243744301	,	0.283596902	,],
[	0.357837096	,	0.344817913	,	0.075827895	,	0.273763172	,],
[	0.311733755	,	0.431348159	,	0.325742133	,	0.085096946	,]],
[[	0.088939275	,	0.355515444	,	0.353267994	,	0.253872165	,],
[	0.256552142	,	0.087715085	,	0.241211461	,	0.281974012	,],
[	0.358160382	,	0.345757016	,	0.078770045	,	0.273941203	,],
[	0.314464935	,	0.434022321	,	0.328338762	,	0.090179155	,]],
[[	0.087253966	,	0.356788104	,	0.354703562	,	0.255262672	,],
[	0.256002864	,	0.090052675	,	0.241189859	,	0.281508766	,],
[	0.360332302	,	0.347095118	,	0.077749088	,	0.276392546	,],
[	0.313087019	,	0.432353577	,	0.326990094	,	0.085610519	,]],
[[	0.087509979	,	0.36090962	,	0.357843679	,	0.253854677	,],
[	0.254295322	,	0.090237385	,	0.244734665	,	0.280981842	,],
[	0.355016322	,	0.348894231	,	0.078262427	,	0.271099356	,],
[	0.313329735	,	0.439453839	,	0.33256291	,	0.089189812	,]],
[[	0.087945904	,	0.363675399	,	0.360459564	,	0.256860299	,],
[	0.251378214	,	0.089472949	,	0.242279121	,	0.278087285	,],
[	0.358855012	,	0.353467511	,	0.085813813	,	0.274974123	,],
[	0.313439806	,	0.439299522	,	0.332876288	,	0.08652691	,]],
[[	0.085496946	,	0.360190554	,	0.35703459	,	0.253128048	,],
[	0.254036134	,	0.095906961	,	0.245571974	,	0.280633061	,],
[	0.354792631	,	0.348329402	,	0.076706159	,	0.270952956	,],
[	0.31134807	,	0.437611268	,	0.33079967	,	0.085888575	,]],
[[	0.090942021	,	0.362751585	,	0.359405451	,	0.255540598	,],
[	0.252072096	,	0.088638314	,	0.242695989	,	0.278706559	,],
[	0.356285374	,	0.350412733	,	0.080838756	,	0.272286775	,],
[	0.315616286	,	0.44184697	,	0.33499878	,	0.092796101	,]],
[[	0.085615833	,	0.361421162	,	0.358361679	,	0.254391815	,],
[	0.251763522	,	0.091366964	,	0.242944865	,	0.278443424	,],
[	0.356989027	,	0.350311925	,	0.078074933	,	0.273281558	,],
[	0.311891611	,	0.437879442	,	0.331252915	,	0.085345517	,]],
[[	0.086746031	,	0.36057041	,	0.35753119	,	0.253363141	,],
[	0.255262338	,	0.09102736	,	0.245561323	,	0.281705555	,],
[	0.35484646	,	0.348549873	,	0.077656495	,	0.270840308	,],
[	0.312693482	,	0.438859521	,	0.331920561	,	0.088082727	,]],
[[	0.089136106	,	0.364882082	,	0.361649465	,	0.258085777	,],
[	0.251478846	,	0.089136426	,	0.242173776	,	0.278011286	,],
[	0.358087298	,	0.352583769	,	0.084318998	,	0.274050394	,],
[	0.314318382	,	0.440176976	,	0.333739307	,	0.087258525	,]],
[[	0.085300355	,	0.360408874	,	0.357141718	,	0.253294137	,],
[	0.253458607	,	0.094627896	,	0.244731145	,	0.279988675	,],
[	0.355599247	,	0.348919845	,	0.077020729	,	0.271619111	,],
[	0.31134243	,	0.437427439	,	0.330764902	,	0.085447355	,]],
[[	0.089450567	,	0.362035944	,	0.358683303	,	0.254829009	,],
[	0.253035104	,	0.089318475	,	0.243496701	,	0.27956486	,],
[	0.355699073	,	0.349706023	,	0.079747302	,	0.271659393	,],
[	0.31462388	,	0.440785686	,	0.333865543	,	0.091182964	,]],
[[	0.086711453	,	0.36247929	,	0.359286856	,	0.255409647	,],
[	0.251553436	,	0.090454421	,	0.242629472	,	0.278143213	,],
[	0.357676487	,	0.350867618	,	0.078697119	,	0.273959483	,],
[	0.312531479	,	0.438487664	,	0.331867517	,	0.085753493	,]],
[[	0.101317534	,	0.232928651	,	0.261651529	,	0.27091758	,],
[	0.387225058	,	0.080490699	,	0.250104544	,	0.396301036	,],
[	0.460961012	,	0.30406942	,	0.074777424	,	0.366371884	,],
[	0.357936786	,	0.33133991	,	0.246769581	,	0.099804193	,]],
[[	0.096112555	,	0.226494707	,	0.255307509	,	0.264963366	,],
[	0.388657084	,	0.084423089	,	0.251705349	,	0.397776876	,],
[	0.463664511	,	0.307196131	,	0.075834449	,	0.369599169	,],
[	0.354855735	,	0.328768151	,	0.244330273	,	0.093270632	,]],
[[	0.098763702	,	0.226451794	,	0.255368614	,	0.265399566	,],
[	0.38939082	,	0.082032501	,	0.252524319	,	0.398490845	,],
[	0.460091958	,	0.303247475	,	0.072787042	,	0.36555675	,],
[	0.355394766	,	0.328858517	,	0.244240365	,	0.095517164	,]],
[[	0.098046022	,	0.229715914	,	0.2584688	,	0.267819143	,],
[	0.387432161	,	0.081642277	,	0.250395587	,	0.396536736	,],
[	0.462207046	,	0.305386839	,	0.077130823	,	0.367486627	,],
[	0.357541094	,	0.331691333	,	0.247354446	,	0.095470022	,]],
[[	0.095905331	,	0.225711165	,	0.254723677	,	0.264216964	,],
[	0.390534243	,	0.087531046	,	0.253625706	,	0.399509163	,],
[	0.461578699	,	0.304850475	,	0.07362517	,	0.367262749	,],
[	0.35456207	,	0.328382638	,	0.243865155	,	0.093462132	,]],
[[	0.101873578	,	0.228353269	,	0.257228947	,	0.267300789	,],
[	0.387801068	,	0.08072465	,	0.250649714	,	0.39687408	,],
[	0.460368778	,	0.303463345	,	0.073687479	,	0.365667976	,],
[	0.357020516	,	0.330464646	,	0.245949471	,	0.098351564	,]],
[[	0.096471463	,	0.227294065	,	0.255909483	,	0.265578505	,],
[	0.388140835	,	0.08325552	,	0.251044677	,	0.397091515	,],
[	0.464572669	,	0.308091419	,	0.081015092	,	0.369953801	,],
[	0.355643119	,	0.329494478	,	0.244986293	,	0.0937524	,]],
[[	0.097820154	,	0.22625162	,	0.255004712	,	0.264949655	,],
[	0.390450501	,	0.083028331	,	0.253400791	,	0.399377571	,],
[	0.460401957	,	0.303502926	,	0.072759561	,	0.365752817	,],
[	0.355078249	,	0.328437022	,	0.243941357	,	0.094719366	,]],
[[	0.099240911	,	0.230774935	,	0.259487045	,	0.26880331	,],
[	0.387046343	,	0.080815436	,	0.249979301	,	0.396110778	,],
[	0.46166523	,	0.304844304	,	0.076076583	,	0.366883952	,],
[	0.358610685	,	0.332816016	,	0.248477153	,	0.096559838	,]],
[[	0.096292669	,	0.2263404	,	0.255211824	,	0.264791775	,],
[	0.389816891	,	0.086288855	,	0.252816272	,	0.398794109	,],
[	0.462266079	,	0.305594909	,	0.07429542	,	0.367889409	,],
[	0.354459651	,	0.328300271	,	0.243759688	,	0.092963328	,]],
[[	0.097660229	,	0.234059549	,	0.26097231	,	0.26097495	,],
[	0.374085514	,	0.087260151	,	0.249518683	,	0.384392309	,],
[	0.445910716	,	0.302949555	,	0.072921492	,	0.354388621	,],
[	0.344542919	,	0.330966427	,	0.246766656	,	0.092091069	,]],
[[	0.093232315	,	0.233612116	,	0.260362776	,	0.259906253	,],
[	0.371383271	,	0.08022248	,	0.246531162	,	0.381720965	,],
[	0.444262871	,	0.301116592	,	0.072315047	,	0.352315961	,],
[	0.348232736	,	0.334225527	,	0.250047961	,	0.09820674	,]],
[[	0.094302719	,	0.232338471	,	0.259298969	,	0.259114631	,],
[	0.371704171	,	0.082843416	,	0.246822192	,	0.381971251	,],
[	0.44755028	,	0.304888243	,	0.078607209	,	0.35581671	,],
[	0.345094841	,	0.331540329	,	0.247352533	,	0.091528527	,]],
[[	0.096488572	,	0.237501896	,	0.264354771	,	0.263562792	,],
[	0.374013457	,	0.082486115	,	0.249224634	,	0.384225046	,],
[	0.444280314	,	0.301189496	,	0.071549399	,	0.352546514	,],
[	0.345651038	,	0.331711345	,	0.247468343	,	0.093878416	,]],
[[	0.092340436	,	0.232238777	,	0.259210727	,	0.258586319	,],
[	0.371122386	,	0.080863197	,	0.246206511	,	0.381348741	,],
[	0.445224777	,	0.302052547	,	0.074239295	,	0.353248279	,],
[	0.347134696	,	0.3338599	,	0.249955861	,	0.093323031	,]],
[[	0.09669078	,	0.23353496	,	0.260539264	,	0.260325586	,],
[	0.373373396	,	0.085848431	,	0.248590968	,	0.383611909	,],
[	0.446523088	,	0.303650949	,	0.073457445	,	0.354935325	,],
[	0.34441714	,	0.330873325	,	0.246741777	,	0.091511059	,]],
[[	0.094129109	,	0.234664131	,	0.261424833	,	0.260887426	,],
[	0.372136162	,	0.080871371	,	0.247098307	,	0.382180133	,],
[	0.444054002	,	0.300869145	,	0.071759774	,	0.352141406	,],
[	0.347409752	,	0.333452447	,	0.249238052	,	0.096823044	,]],
[[	0.093502711	,	0.232245413	,	0.259015485	,	0.258754435	,],
[	0.371369016	,	0.082055648	,	0.246447093	,	0.381595026	,],
[	0.447104917	,	0.304198367	,	0.077477274	,	0.35513672	,],
[	0.345401235	,	0.331919603	,	0.24786348	,	0.091790134	,]],
[[	0.097696778	,	0.238977247	,	0.265628797	,	0.264859137	,],
[	0.375101425	,	0.083497943	,	0.250187424	,	0.385248784	,],
[	0.44498448	,	0.301851881	,	0.071949783	,	0.353082691	,],
[	0.345299594	,	0.331438639	,	0.247191493	,	0.093130388	,]],
[[	0.092665302	,	0.232728214	,	0.259639201	,	0.25901643	,],
[	0.371134043	,	0.08034147	,	0.246094388	,	0.381245357	,],
[	0.444688092	,	0.301563402	,	0.07335128	,	0.352756158	,],
[	0.348513936	,	0.335231442	,	0.251376848	,	0.094452377	,]],
[[	0.095393348	,	0.242696252	,	0.267252453	,	0.257626719	,],
[	0.35696052	,	0.082953007	,	0.244599635	,	0.37005027	,],
[	0.434275309	,	0.303989121	,	0.078587408	,	0.345856649	,],
[	0.336127378	,	0.335436302	,	0.251927191	,	0.090462986	,]],
[[	0.091847253	,	0.242656517	,	0.267274562	,	0.25727935	,],
[	0.358992467	,	0.082170494	,	0.246370319	,	0.371877241	,],
[	0.431636268	,	0.300768449	,	0.072099556	,	0.343093995	,],
[	0.337575988	,	0.336422973	,	0.25298526	,	0.093951395	,]],
[[	0.092054019	,	0.240871967	,	0.265483911	,	0.255737354	,],
[	0.355881565	,	0.080357337	,	0.243362038	,	0.368911268	,],
[	0.432321034	,	0.301668605	,	0.07470187	,	0.343706935	,],
[	0.338131556	,	0.337428629	,	0.25397546	,	0.091785131	,]],
[[	0.095329903	,	0.246812471	,	0.271371808	,	0.261202923	,],
[	0.358317499	,	0.085757926	,	0.246067872	,	0.371286717	,],
[	0.433779615	,	0.30309178	,	0.073940277	,	0.345485422	,],
[	0.336044495	,	0.335094912	,	0.251579796	,	0.091045503	,]],
[[	0.090705081	,	0.241047188	,	0.265622073	,	0.255626061	,],
[	0.357164659	,	0.08065931	,	0.244542237	,	0.370092798	,],
[	0.430877061	,	0.300003959	,	0.071929658	,	0.342254705	,],
[	0.340046272	,	0.338620021	,	0.255108626	,	0.097512183	,]],
[[	0.094117598	,	0.241915725	,	0.266555601	,	0.256867357	,],
[	0.356500293	,	0.081960826	,	0.24399798	,	0.369493621	,],
[	0.433756602	,	0.303304606	,	0.077596644	,	0.345208973	,],
[	0.336464839	,	0.335694043	,	0.252349331	,	0.090389555	,]],
[[	0.092887572	,	0.244020764	,	0.268468631	,	0.258393055	,],
[	0.360104521	,	0.083219904	,	0.247534762	,	0.372993726	,],
[	0.432045427	,	0.30129345	,	0.072314213	,	0.343545394	,],
[	0.337084799	,	0.335992338	,	0.252421823	,	0.092813137	,]],
[[	0.091438055	,	0.240784348	,	0.265328018	,	0.255579826	,],
[	0.356170479	,	0.080281828	,	0.243520907	,	0.369070562	,],
[	0.431786129	,	0.301061086	,	0.073705527	,	0.343051387	,],
[	0.339005656	,	0.338209916	,	0.254992758	,	0.092519667	,]],
[[	0.096757483	,	0.248396957	,	0.272868294	,	0.262603106	,],
[	0.358048564	,	0.084845245	,	0.245514289	,	0.370883033	,],
[	0.434869132	,	0.304249362	,	0.074941946	,	0.346526629	,],
[	0.335988676	,	0.334854516	,	0.251432674	,	0.090480001	,]],
[[	0.091333244	,	0.24192659	,	0.266656444	,	0.256451995	,],
[	0.357680272	,	0.080890276	,	0.245070739	,	0.370622526	,],
[	0.431215001	,	0.30044034	,	0.07194825	,	0.342509209	,],
[	0.339235717	,	0.337947512	,	0.254399606	,	0.096168455	,]],
[[	0.093063249	,	0.250268943	,	0.274174778	,	0.255516354	,],
[	0.343992047	,	0.079563973	,	0.242067405	,	0.358899752	,],
[	0.421110588	,	0.30148657	,	0.07432829	,	0.33424371	,],
[	0.330015772	,	0.341098765	,	0.257191631	,	0.088568665	,]],
[[	0.090927322	,	0.251410909	,	0.275242152	,	0.256278921	,],
[	0.346788327	,	0.085244854	,	0.245167524	,	0.361649167	,],
[	0.423011661	,	0.30331777	,	0.073899469	,	0.336355974	,],
[	0.329906722	,	0.340806762	,	0.256817125	,	0.090059782	,]],
[[	0.089994318	,	0.248739686	,	0.272698496	,	0.253795376	,],
[	0.345326988	,	0.079898754	,	0.243322444	,	0.360086168	,],
[	0.419883486	,	0.300102387	,	0.071735196	,	0.332998567	,],
[	0.333594167	,	0.344669276	,	0.260937461	,	0.091913398	,]],
[[	0.094605034	,	0.255750098	,	0.279551998	,	0.260436233	,],
[	0.344840799	,	0.081428927	,	0.242904886	,	0.359651237	,],
[	0.422732402	,	0.303409571	,	0.077286156	,	0.335850118	,],
[	0.329573076	,	0.340441247	,	0.256517267	,	0.088559591	,]],
[[	0.089344779	,	0.249598579	,	0.273424183	,	0.254412519	,],
[	0.348745504	,	0.082958518	,	0.246793057	,	0.363386025	,],
[	0.421179444	,	0.30131633	,	0.072125051	,	0.334241943	,],
[	0.331582138	,	0.342198357	,	0.258083678	,	0.092680635	,]],
[[	0.091729526	,	0.249542851	,	0.273353077	,	0.254594962	,],
[	0.344224262	,	0.079402658	,	0.242212062	,	0.359084644	,],
[	0.42082625	,	0.300979777	,	0.073403676	,	0.333694698	,],
[	0.331250485	,	0.342359782	,	0.258538075	,	0.089606308	,]],
[[	0.091759677	,	0.252568895	,	0.276283826	,	0.257300321	,],
[	0.346359466	,	0.084238295	,	0.244669058	,	0.361113792	,],
[	0.424040692	,	0.304442011	,	0.075003469	,	0.337496924	,],
[	0.329700364	,	0.340603617	,	0.25647574	,	0.089380481	,]],
[[	0.089509032	,	0.248757159	,	0.272507791	,	0.253748393	,],
[	0.346366142	,	0.080765293	,	0.244242028	,	0.360959084	,],
[	0.420211426	,	0.300419348	,	0.071678408	,	0.333234213	,],
[	0.333695455	,	0.344264092	,	0.260300685	,	0.096114027	,]],
[[	0.09488666	,	0.251109638	,	0.274771378	,	0.256328764	,],
[	0.344458256	,	0.080464392	,	0.242331367	,	0.359046813	,],
[	0.422204539	,	0.302665283	,	0.076025951	,	0.335131023	,],
[	0.329776421	,	0.34071299	,	0.256719311	,	0.08848383	,]],
[[	0.089847237	,	0.250240521	,	0.274055452	,	0.255010088	,],
[	0.348321121	,	0.08763133	,	0.246757145	,	0.362988263	,],
[	0.42191369	,	0.302146417	,	0.072913687	,	0.335072713	,],
[	0.33094298	,	0.341589259	,	0.257521943	,	0.091536619	,]],
[[	0.091861167	,	0.261699386	,	0.283286178	,	0.254816135	,],
[	0.334680732	,	0.080403622	,	0.242767827	,	0.350959789	,],
[	0.412088342	,	0.303444686	,	0.072801019	,	0.325515753	,],
[	0.327977373	,	0.350948293	,	0.265966655	,	0.088222178	,]],
[[	0.090341634	,	0.263439879	,	0.285080365	,	0.256135522	,],
[	0.335244539	,	0.08345579	,	0.243634727	,	0.351608491	,],
[	0.415970457	,	0.307763602	,	0.07971145	,	0.329440728	,],
[	0.326294758	,	0.349110002	,	0.263952825	,	0.087933865	,]],
[[	0.089223142	,	0.260508809	,	0.282147665	,	0.253485033	,],
[	0.336777898	,	0.081783951	,	0.24481059	,	0.35301458	,],
[	0.412099259	,	0.303430787	,	0.071790936	,	0.325681113	,],
[	0.330589646	,	0.353046118	,	0.267946994	,	0.094920286	,]],
[[	0.094259464	,	0.267664392	,	0.289287874	,	0.260350779	,],
[	0.334312717	,	0.080865131	,	0.242393163	,	0.350596169	,],
[	0.413178019	,	0.3047556	,	0.075041933	,	0.326609273	,],
[	0.32663697	,	0.349435859	,	0.264328396	,	0.087227528	,]],
[[	0.088708751	,	0.261409276	,	0.283098365	,	0.254043643	,],
[	0.337070625	,	0.086646803	,	0.245345993	,	0.353181401	,],
[	0.414164246	,	0.305499621	,	0.073315173	,	0.327796601	,],
[	0.327643104	,	0.350114479	,	0.264928112	,	0.090186888	,]],
[[	0.090958923	,	0.261267939	,	0.28295359	,	0.254241378	,],
[	0.335121955	,	0.080501121	,	0.243208203	,	0.351435449	,],
[	0.411976541	,	0.303348281	,	0.072268812	,	0.325320571	,],
[	0.328945471	,	0.351868165	,	0.266967678	,	0.088999625	,]],
[[	0.091266133	,	0.264477835	,	0.286020711	,	0.257147013	,],
[	0.334961053	,	0.082531341	,	0.24319152	,	0.351099167	,],
[	0.415017092	,	0.306831235	,	0.078069457	,	0.328473775	,],
[	0.326690878	,	0.349450696	,	0.264350356	,	0.087829213	,]],
[[	0.088741068	,	0.26051901	,	0.282140384	,	0.253383869	,],
[	0.337986529	,	0.082742878	,	0.245878946	,	0.354090996	,],
[	0.412736001	,	0.304003108	,	0.072176284	,	0.326195949	,],
[	0.329720432	,	0.352169062	,	0.267089977	,	0.093448848	,]],
[[	0.094439917	,	0.263153855	,	0.284561644	,	0.25616982	,],
[	0.334468937	,	0.080559358	,	0.242405336	,	0.350528882	,],
[	0.412614269	,	0.304054889	,	0.073877833	,	0.325949648	,],
[	0.327058764	,	0.349719837	,	0.264828438	,	0.087477352	,]],
[[	0.089295748	,	0.262318577	,	0.283955922	,	0.254942227	,],
[	0.336582823	,	0.08548612	,	0.244761119	,	0.352655114	,],
[	0.415368881	,	0.306792267	,	0.074656287	,	0.329063563	,],
[	0.32730774	,	0.349827298	,	0.264680165	,	0.089266357	,]],
[[	0.09072736	,	0.270991592	,	0.290749066	,	0.254165072	,],
[	0.327135979	,	0.082736541	,	0.245177099	,	0.345361749	,],
[	0.403115497	,	0.305745294	,	0.071910413	,	0.319319989	,],
[	0.326433633	,	0.360020296	,	0.274200695	,	0.095143414	,]],
[[	0.090131671	,	0.273328496	,	0.292968674	,	0.256225375	,],
[	0.323887945	,	0.080876851	,	0.241998652	,	0.342170974	,],
[	0.403357197	,	0.306171301	,	0.074059945	,	0.319329592	,],
[	0.322706498	,	0.356979948	,	0.271135594	,	0.087821072	,]],
[[	0.088384818	,	0.270172699	,	0.289839475	,	0.25324107	,],
[	0.325876398	,	0.085939318	,	0.244313584	,	0.344204713	,],
[	0.405594709	,	0.308358827	,	0.073973015	,	0.321993256	,],
[	0.323534148	,	0.357618179	,	0.271739914	,	0.090480403	,]],
[[	0.094017808	,	0.272752512	,	0.292340226	,	0.255997733	,],
[	0.325106821	,	0.081040477	,	0.242950084	,	0.343229261	,],
[	0.402622739	,	0.305289242	,	0.071919885	,	0.318766202	,],
[	0.325139253	,	0.359202716	,	0.273634494	,	0.08974371	,]],
[[	0.088418782	,	0.271480858	,	0.291152142	,	0.254238663	,],
[	0.32422467	,	0.082186304	,	0.242359849	,	0.34247752	,],
[	0.405015793	,	0.308102371	,	0.076911149	,	0.321119773	,],
[	0.322437944	,	0.356360995	,	0.270488714	,	0.088046657	,]],
[[	0.089954606	,	0.270775408	,	0.29048518	,	0.253831111	,],
[	0.328288786	,	0.083821629	,	0.246305774	,	0.346397529	,],
[	0.403477369	,	0.305999476	,	0.072121515	,	0.3197126	,],
[	0.325670043	,	0.359323637	,	0.273506526	,	0.093729637	,]],
[[	0.091412837	,	0.27465059	,	0.294280829	,	0.257461095	,],
[	0.324011948	,	0.080685016	,	0.242021593	,	0.342147073	,],
[	0.403030531	,	0.305669097	,	0.073137455	,	0.318980436	,],
[	0.323152923	,	0.357282399	,	0.271624147	,	0.088050793	,]],
[[	0.088088513	,	0.27027336	,	0.289919332	,	0.253228846	,],
[	0.325439807	,	0.084851191	,	0.24389887	,	0.343707496	,],
[	0.406515316	,	0.30917514	,	0.074870279	,	0.322890131	,],
[	0.32293242	,	0.356944382	,	0.271135256	,	0.089524682	,]],
[[	0.092366836	,	0.271827047	,	0.291290269	,	0.254925026	,],
[	0.325775015	,	0.081502271	,	0.243643094	,	0.343863174	,],
[	0.402822185	,	0.305480668	,	0.071861908	,	0.318903746	,],
[	0.326221474	,	0.360189933	,	0.274620926	,	0.090627506	,]],
[[	0.089351403	,	0.272428623	,	0.292002322	,	0.255277145	,],
[	0.323951474	,	0.081468598	,	0.241863144	,	0.342018575	,],
[	0.404300671	,	0.307183565	,	0.075579697	,	0.32025163	,],
[	0.32254977	,	0.356475213	,	0.270676073	,	0.087859284	,]],
[[	0.08943695	,	0.281110516	,	0.29861982	,	0.252720737	,],
[	0.315933471	,	0.088471434	,	0.244902712	,	0.336565509	,],
[	0.394365314	,	0.30908797	,	0.072506885	,	0.311634477	,],
[	0.322753129	,	0.368841599	,	0.28183246	,	0.09483797	,]],
[[	0.089475014	,	0.283849843	,	0.3012647	,	0.255220947	,],
[	0.313097543	,	0.081008371	,	0.241545482	,	0.333835577	,],
[	0.393153595	,	0.308065385	,	0.072842513	,	0.310181472	,],
[	0.318304745	,	0.365020307	,	0.278041737	,	0.086658976	,]],
[[	0.087378165	,	0.280305163	,	0.297757252	,	0.251775502	,],
[	0.313361709	,	0.083855636	,	0.242075254	,	0.334017946	,],
[	0.397345528	,	0.312719442	,	0.080142155	,	0.314300337	,],
[	0.319584652	,	0.365913961	,	0.279021086	,	0.089982837	,]],
[[	0.092420093	,	0.282534319	,	0.299885407	,	0.254238531	,],
[	0.315504272	,	0.082731653	,	0.243903456	,	0.336175402	,],
[	0.393290246	,	0.307973494	,	0.071729444	,	0.310294937	,],
[	0.320412869	,	0.366915732	,	0.280148974	,	0.088235284	,]],
[[	0.087484505	,	0.281849935	,	0.299355614	,	0.2531111	,],
[	0.312756952	,	0.081451332	,	0.241112477	,	0.333379439	,],
[	0.394598152	,	0.309653015	,	0.075483357	,	0.311679391	,],
[	0.318278071	,	0.364585403	,	0.277630557	,	0.087203492	,]],
[[	0.088653973	,	0.280825139	,	0.298282468	,	0.252230389	,],
[	0.315142724	,	0.087093297	,	0.243992991	,	0.335724025	,],
[	0.395235656	,	0.309847844	,	0.073084103	,	0.312395737	,],
[	0.321915694	,	0.368084456	,	0.281047317	,	0.093359921	,]],
[[	0.090525475	,	0.285102509	,	0.302413897	,	0.256337755	,],
[	0.313420947	,	0.080988534	,	0.241777533	,	0.334026537	,],
[	0.393121338	,	0.307931797	,	0.072360186	,	0.310008657	,],
[	0.318928856	,	0.365436165	,	0.278648923	,	0.086980622	,]],
[[	0.087096968	,	0.2804599	,	0.29790995	,	0.251882055	,],
[	0.31313645	,	0.083002	,	0.241699274	,	0.33376843	,],
[	0.396562523	,	0.311863689	,	0.078790523	,	0.313502779	,],
[	0.318947414	,	0.365391229	,	0.278313784	,	0.08875869	,]],
[[	0.091279014	,	0.281999522	,	0.299324116	,	0.253498965	,],
[	0.316451029	,	0.083563216	,	0.24471999	,	0.33689563	,],
[	0.39387438	,	0.30858652	,	0.07211007	,	0.310819363	,],
[	0.321775066	,	0.36821258	,	0.281415508	,	0.089356913	,]],
[[	0.088250129	,	0.282511384	,	0.300006308	,	0.25385122	,],
[	0.31279125	,	0.081219631	,	0.241125563	,	0.333390433	,],
[	0.393918551	,	0.308837763	,	0.074280602	,	0.310781111	,],
[	0.318465812	,	0.364837419	,	0.277819658	,	0.087044096	,]],
[[	0.087969888	,	0.290031958	,	0.306591534	,	0.25271837	,],
[	0.303720669	,	0.084411374	,	0.242526347	,	0.325552523	,],
[	0.388387821	,	0.314806561	,	0.080540769	,	0.306945502	,],
[	0.316403017	,	0.373859067	,	0.28612515	,	0.090199575	,]],
[[	0.089280817	,	0.293670158	,	0.310195776	,	0.256281478	,],
[	0.304573152	,	0.081789943	,	0.242960323	,	0.326516349	,],
[	0.384602723	,	0.310208419	,	0.072446174	,	0.303086644	,],
[	0.316006595	,	0.373759256	,	0.286221694	,	0.087021918	,]],
[[	0.08606366	,	0.289263528	,	0.305741658	,	0.251875722	,],
[	0.302188179	,	0.081171486	,	0.240874649	,	0.324143885	,],
[	0.385760561	,	0.311739727	,	0.075773679	,	0.304124584	,],
[	0.314778239	,	0.372333183	,	0.284665809	,	0.08714136	,]],
[[	0.090787123	,	0.291362354	,	0.307768619	,	0.254078125	,],
[	0.305377402	,	0.087690766	,	0.24446461	,	0.327348208	,],
[	0.386499675	,	0.312124693	,	0.073849165	,	0.305066243	,],
[	0.318636535	,	0.376126835	,	0.288433382	,	0.093737033	,]],
[[	0.086988088	,	0.29123049	,	0.307846304	,	0.253801663	,],
[	0.30301147	,	0.080681263	,	0.241252357	,	0.324776059	,],
[	0.38450794	,	0.310283249	,	0.073114419	,	0.302892997	,],
[	0.314970486	,	0.372521986	,	0.284849869	,	0.08626598	,]],
[[	0.086951426	,	0.289533433	,	0.306112064	,	0.252075546	,],
[	0.302889985	,	0.083093995	,	0.241612786	,	0.324764316	,],
[	0.387737468	,	0.313859759	,	0.079179964	,	0.306108641	,],
[	0.315772702	,	0.373180975	,	0.28544958	,	0.089124112	,]],
[[	0.090589026	,	0.295016971	,	0.311478845	,	0.257578669	,],
[	0.305668663	,	0.082702127	,	0.244133165	,	0.327368675	,],
[	0.385120171	,	0.310675281	,	0.072684641	,	0.303457561	,],
[	0.31709239	,	0.374750908	,	0.287208893	,	0.087878932	,]],
[[	0.086186515	,	0.289845159	,	0.30630711	,	0.252329539	,],
[	0.302371499	,	0.080937619	,	0.240862851	,	0.324221455	,],
[	0.385078853	,	0.310953986	,	0.074716649	,	0.303301005	,],
[	0.314513763	,	0.372197491	,	0.284546741	,	0.086624922	,]],
[[	0.089412018	,	0.290609311	,	0.306972465	,	0.253306014	,],
[	0.304798864	,	0.086297312	,	0.243597254	,	0.326503352	,],
[	0.387684759	,	0.313314218	,	0.074938348	,	0.306305196	,],
[	0.317834081	,	0.3753589	,	0.287509915	,	0.092381291	,]],
[[	0.087642204	,	0.292000356	,	0.308523112	,	0.254463819	,],
[	0.303839168	,	0.081268919	,	0.242067911	,	0.325505308	,],
[	0.384444027	,	0.310062243	,	0.072776808	,	0.302802762	,],
[	0.315210164	,	0.372825501	,	0.285120807	,	0.086247601	,]],
[[	0.08733768	,	0.30066178	,	0.314205354	,	0.250622656	,],
[	0.291319913	,	0.083954435	,	0.24072492	,	0.314417608	,],
[	0.378657517	,	0.316715365	,	0.078675101	,	0.297732801	,],
[	0.313296774	,	0.382730002	,	0.292819488	,	0.090403454	,]],
[[	0.08838797	,	0.303924656	,	0.317415913	,	0.253878702	,],
[	0.295498056	,	0.08544389	,	0.244620102	,	0.318434914	,],
[	0.376592938	,	0.314104895	,	0.07282014	,	0.295737615	,],
[	0.312402676	,	0.382077198	,	0.292427585	,	0.086591555	,]],
[[	0.085694314	,	0.300133951	,	0.313631364	,	0.250017073	,],
[	0.291071922	,	0.08220137	,	0.24028922	,	0.314142676	,],
[	0.376504989	,	0.314148606	,	0.07454472	,	0.295442774	,],
[	0.311357414	,	0.380989346	,	0.291120518	,	0.08704163	,]],
[[	0.090489062	,	0.30220632	,	0.315667377	,	0.252308186	,],
[	0.292650402	,	0.08672681	,	0.242228612	,	0.315696179	,],
[	0.379251299	,	0.316779848	,	0.075012263	,	0.298571446	,],
[	0.315559722	,	0.385058503	,	0.295179566	,	0.093879922	,]],
[[	0.086509295	,	0.302113079	,	0.315679559	,	0.251898719	,],
[	0.293103242	,	0.083278351	,	0.242086022	,	0.315997595	,],
[	0.375877158	,	0.313421702	,	0.07271206	,	0.294941189	,],
[	0.311443273	,	0.38091139	,	0.29117561	,	0.085926898	,]],
[[	0.086565227	,	0.300146264	,	0.313844222	,	0.250050154	,],
[	0.291386222	,	0.083548203	,	0.240598712	,	0.314314438	,],
[	0.377692117	,	0.315751992	,	0.077246365	,	0.296776504	,],
[	0.312591541	,	0.382037687	,	0.292027636	,	0.089187312	,]],
[[	0.090056556	,	0.305671608	,	0.31904665	,	0.255578528	,],
[	0.294575912	,	0.090129504	,	0.244261333	,	0.317379472	,],
[	0.377338131	,	0.314694115	,	0.073223033	,	0.296393499	,],
[	0.313359744	,	0.382955236	,	0.293431594	,	0.087452136	,]],
[[	0.085718288	,	0.300514005	,	0.313995225	,	0.250368214	,],
[	0.291512548	,	0.082216131	,	0.240581508	,	0.31444927	,],
[	0.376209727	,	0.313819763	,	0.073803917	,	0.295140116	,],
[	0.311009441	,	0.380635279	,	0.290747295	,	0.086189874	,]],
[[	0.089130338	,	0.301416953	,	0.314807361	,	0.251362215	,],
[	0.292049477	,	0.085489908	,	0.241538308	,	0.31504236	,],
[	0.380218969	,	0.318441381	,	0.081111207	,	0.299185426	,],
[	0.314522125	,	0.383981028	,	0.294093705	,	0.0921961	,]],
[[	0.087297382	,	0.302762031	,	0.316283763	,	0.252573776	,],
[	0.293957058	,	0.084045957	,	0.243021562	,	0.316895682	,],
[	0.375803962	,	0.313313725	,	0.072325833	,	0.294851717	,],
[	0.311753371	,	0.381278311	,	0.291555118	,	0.086024601	,]],
[[	0.085100205	,	0.308870798	,	0.321449671	,	0.250660577	,],
[	0.280133284	,	0.082772635	,	0.240818847	,	0.306383957	,],
[	0.365159972	,	0.315122593	,	0.073022847	,	0.286818229	,],
[	0.306606938	,	0.387523279	,	0.297312798	,	0.086218737	,]],
[[	0.088323812	,	0.313657769	,	0.326337792	,	0.255530492	,],
[	0.280510423	,	0.085620433	,	0.24154766	,	0.306707427	,],
[	0.368156969	,	0.318660924	,	0.0789915	,	0.289790357	,],
[	0.310989111	,	0.39166837	,	0.301453887	,	0.092914383	,]],
[[	0.084111393	,	0.308869604	,	0.321488608	,	0.250598293	,],
[	0.282847709	,	0.084680232	,	0.243543448	,	0.30907529	,],
[	0.365690343	,	0.315448051	,	0.072490459	,	0.287360355	,],
[	0.306817822	,	0.387815053	,	0.297686289	,	0.085313486	,]],
[[	0.088204264	,	0.310481135	,	0.323069891	,	0.252265861	,],
[	0.279619499	,	0.083016033	,	0.240426988	,	0.305819375	,],
[	0.365922333	,	0.316142508	,	0.074863905	,	0.287539149	,],
[	0.307993775	,	0.38896634	,	0.298645006	,	0.088266973	,]],
[[	0.085470717	,	0.310955601	,	0.323605544	,	0.252656097	,],
[	0.281983178	,	0.088528214	,	0.243168321	,	0.308097256	,],
[	0.368744219	,	0.31843585	,	0.075173307	,	0.290573478	,],
[	0.309527614	,	0.390242889	,	0.300215789	,	0.087276694	,]],
[[	0.08477771	,	0.308873141	,	0.321586819	,	0.250533416	,],
[	0.28074532	,	0.083060134	,	0.241367244	,	0.306925681	,],
[	0.365052189	,	0.314989955	,	0.072595164	,	0.286634505	,],
[	0.306759765	,	0.38764739	,	0.297431731	,	0.085926559	,]],
[[	0.089640605	,	0.315062653	,	0.327644634	,	0.256874909	,],
[	0.280095611	,	0.084602642	,	0.241019517	,	0.306216827	,],
[	0.367205082	,	0.317612271	,	0.077403657	,	0.2887608	,],
[	0.310151142	,	0.390884717	,	0.300547607	,	0.0915149	,]],
[[	0.08440584	,	0.309303236	,	0.321881246	,	0.250981958	,],
[	0.283838384	,	0.085581405	,	0.244458024	,	0.309988721	,],
[	0.366601521	,	0.31630628	,	0.073252516	,	0.288271768	,],
[	0.307427277	,	0.38837145	,	0.298358347	,	0.085631872	,]],
[[	0.086941713	,	0.309734088	,	0.322354439	,	0.251491067	,],
[	0.279736395	,	0.082709642	,	0.240361309	,	0.305864878	,],
[	0.365483659	,	0.315489776	,	0.073896348	,	0.286947682	,],
[	0.30730769	,	0.388376941	,	0.298061178	,	0.087218642	,]],
[[	0.086690775	,	0.312099969	,	0.324757096	,	0.253716558	,],
[	0.281727609	,	0.087579589	,	0.242807595	,	0.307777577	,],
[	0.369773385	,	0.31938984	,	0.076136126	,	0.291605585	,],
[	0.310517898	,	0.391247718	,	0.301275613	,	0.088317146	,]],
[[	0.085315536	,	0.319889573	,	0.330922312	,	0.25111399	,],
[	0.270242465	,	0.08333979	,	0.240803828	,	0.297898079	,],
[	0.358573057	,	0.320497807	,	0.076048514	,	0.280737136	,],
[	0.304733673	,	0.397009671	,	0.305622398	,	0.086963327	,]],
[[	0.087854576	,	0.324443284	,	0.335232015	,	0.255653201	,],
[	0.270998911	,	0.08686374	,	0.241921637	,	0.298563074	,],
[	0.360779255	,	0.322108879	,	0.075391424	,	0.283116899	,],
[	0.306369959	,	0.398697379	,	0.307608539	,	0.086069488	,]],
[[	0.084227763	,	0.319943998	,	0.330834757	,	0.251033884	,],
[	0.2722844	,	0.084353303	,	0.242661922	,	0.299859481	,],
[	0.357482987	,	0.319010979	,	0.073424422	,	0.279540678	,],
[	0.303589861	,	0.396038792	,	0.304637628	,	0.084302841	,]],
[[	0.088187667	,	0.321382635	,	0.332137484	,	0.252595813	,],
[	0.269770531	,	0.083843236	,	0.240379583	,	0.297446663	,],
[	0.36007892	,	0.322203671	,	0.078821735	,	0.282137199	,],
[	0.30658659	,	0.398886948	,	0.307507704	,	0.089911256	,]],
[[	0.085474786	,	0.322115917	,	0.333063352	,	0.253256773	,],
[	0.272828716	,	0.09027729	,	0.243907551	,	0.300280309	,],
[	0.358854935	,	0.320240808	,	0.073795141	,	0.281009809	,],
[	0.304643815	,	0.396862266	,	0.305567847	,	0.084430261	,]],
[[	0.084873593	,	0.319990121	,	0.330865128	,	0.251021543	,],
[	0.270688672	,	0.083349076	,	0.241048432	,	0.298264645	,],
[	0.357993873	,	0.319683785	,	0.075035805	,	0.280113101	,],
[	0.30422284	,	0.396548723	,	0.304948277	,	0.085824157	,]],
[[	0.089197362	,	0.325883778	,	0.336673583	,	0.257124262	,],
[	0.270938538	,	0.086092774	,	0.241606723	,	0.298442024	,],
[	0.361829657	,	0.323012386	,	0.07642443	,	0.28411374	,],
[	0.307327056	,	0.399608825	,	0.308552867	,	0.087021733	,]],
[[	0.084197672	,	0.320234957	,	0.331036255	,	0.251386506	,],
[	0.273358065	,	0.085097216	,	0.243543247	,	0.300936427	,],
[	0.357685485	,	0.319134675	,	0.07316621	,	0.279722295	,],
[	0.303452917	,	0.395956543	,	0.304628559	,	0.083981128	,]],
[[	0.087188472	,	0.320836516	,	0.331615651	,	0.251925883	,],
[	0.269829081	,	0.083502927	,	0.240357348	,	0.297475703	,],
[	0.359313228	,	0.321348115	,	0.077589849	,	0.281358921	,],
[	0.306055454	,	0.398480771	,	0.306947754	,	0.088694122	,]],
[[	0.086246563	,	0.322889832	,	0.333755586	,	0.254046623	,],
[	0.272291013	,	0.089035015	,	0.243288933	,	0.299746571	,],
[	0.359784648	,	0.321074327	,	0.074461277	,	0.281906008	,],
[	0.305332475	,	0.397546891	,	0.30625147	,	0.085088298	,]],
[[	0.085444929	,	0.332172251	,	0.3417709	,	0.251969437	,],
[	0.261243954	,	0.084615492	,	0.243827708	,	0.291173368	,],
[	0.350539896	,	0.324397577	,	0.075052122	,	0.274084284	,],
[	0.302407106	,	0.406029169	,	0.313533475	,	0.08695203	,]],
[[	0.087293696	,	0.335819428	,	0.345408441	,	0.255907846	,],
[	0.25956224	,	0.085333198	,	0.242572335	,	0.289560594	,],
[	0.353625524	,	0.328176866	,	0.081364506	,	0.277271201	,],
[	0.305427424	,	0.408673143	,	0.316718639	,	0.087685217	,]],
[[	0.083866282	,	0.331599104	,	0.341172291	,	0.251565326	,],
[	0.264131499	,	0.093316453	,	0.247623709	,	0.293857109	,],
[	0.351285284	,	0.324793539	,	0.074473327	,	0.274809999	,],
[	0.301853772	,	0.405449918	,	0.313022961	,	0.08496511	,]],
[[	0.088378918	,	0.333570286	,	0.343036396	,	0.253447285	,],
[	0.259964024	,	0.084048773	,	0.242700449	,	0.289885782	,],
[	0.351674015	,	0.325658099	,	0.077195101	,	0.274982252	,],
[	0.304163628	,	0.407661573	,	0.315266158	,	0.089760201	,]],
[[	0.084858965	,	0.333458715	,	0.343122097	,	0.253274399	,],
[	0.260743781	,	0.087896794	,	0.244074918	,	0.290692841	,],
[	0.353983147	,	0.327351679	,	0.076725507	,	0.277588948	,],
[	0.30327786	,	0.406588138	,	0.314373969	,	0.08551961	,]],
[[	0.084730579	,	0.331681245	,	0.341409734	,	0.251515252	,],
[	0.262350393	,	0.085342921	,	0.244704321	,	0.292052371	,],
[	0.350569289	,	0.324313242	,	0.074589239	,	0.274094753	,],
[	0.302378211	,	0.405880782	,	0.313436535	,	0.086262104	,]],
[[	0.088606186	,	0.337232159	,	0.346682067	,	0.25721162	,],
[	0.25984108	,	0.084981146	,	0.242677419	,	0.289745938	,],
[	0.352716743	,	0.327147616	,	0.079772718	,	0.276189537	,],
[	0.306519937	,	0.409770862	,	0.317735517	,	0.088718067	,]],
[[	0.084064066	,	0.332049469	,	0.341568676	,	0.251882449	,],
[	0.263105208	,	0.091795287	,	0.246350173	,	0.292808239	,],
[	0.352094309	,	0.32550665	,	0.075104965	,	0.27546475	,],
[	0.301993074	,	0.405502721	,	0.313191981	,	0.084979867	,]],
[[	0.087359197	,	0.333055898	,	0.342528766	,	0.252817783	,],
[	0.260486562	,	0.084154606	,	0.243108797	,	0.290359868	,],
[	0.350860781	,	0.324861197	,	0.076022956	,	0.274236757	,],
[	0.303600632	,	0.40707135	,	0.314642682	,	0.088651863	,]],
[[	0.085926633	,	0.33460996	,	0.344210556	,	0.254516419	,],
[	0.260438962	,	0.087010388	,	0.243505757	,	0.290258646	,],
[	0.354995551	,	0.328259737	,	0.077603405	,	0.278535623	,],
[	0.304353574	,	0.407598709	,	0.31542818	,	0.086575795	,]],
[[	0.084594599	,	0.340454417	,	0.349228519	,	0.251680444	,],
[	0.251490805	,	0.08591883	,	0.245156188	,	0.283123113	,],
[	0.340380072	,	0.325532825	,	0.07444508	,	0.26523242	,],
[	0.298675181	,	0.413217823	,	0.320107094	,	0.085017513	,]],
[[	0.086826629	,	0.344463057	,	0.353112061	,	0.255877816	,],
[	0.248519377	,	0.085034397	,	0.242530984	,	0.280297179	,],
[	0.342899929	,	0.328772676	,	0.080027897	,	0.267796451	,],
[	0.302203695	,	0.416470846	,	0.323806616	,	0.086313127	,]],
[[	0.083298007	,	0.340086368	,	0.34884191	,	0.25144893	,],
[	0.251455318	,	0.09149292	,	0.246048881	,	0.283053763	,],
[	0.341634136	,	0.326570938	,	0.074544147	,	0.266478477	,],
[	0.298106405	,	0.41268589	,	0.319636823	,	0.083183565	,]],
[[	0.087782507	,	0.34198511	,	0.350514418	,	0.253235041	,],
[	0.249418928	,	0.084469421	,	0.243258178	,	0.281134803	,],
[	0.34088849	,	0.326380373	,	0.075923925	,	0.265661732	,],
[	0.300108547	,	0.414582255	,	0.321582731	,	0.087443657	,]],
[[	0.084219665	,	0.341916715	,	0.350656624	,	0.253228757	,],
[	0.248972731	,	0.08677669	,	0.243338936	,	0.280633835	,],
[	0.345251427	,	0.329992632	,	0.077541756	,	0.270210126	,],
[	0.300059277	,	0.414200105	,	0.321397224	,	0.084348912	,]],
[[	0.083929489	,	0.34002815	,	0.348798717	,	0.251277149	,],
[	0.252348	,	0.086746295	,	0.245960045	,	0.283941782	,],
[	0.34071388	,	0.325750888	,	0.074142297	,	0.265509827	,],
[	0.298191638	,	0.412678993	,	0.319616703	,	0.084066653	,]],
[[	0.088025878	,	0.345736135	,	0.354284448	,	0.25720069	,],
[	0.248728399	,	0.084604834	,	0.242568773	,	0.280272251	,],
[	0.342326975	,	0.327899385	,	0.078595194	,	0.26696141	,],
[	0.30242156	,	0.416720772	,	0.323579475	,	0.090975152	,]],
[[	0.083512945	,	0.340594477	,	0.34924364	,	0.251879858	,],
[	0.250493119	,	0.089915834	,	0.245029528	,	0.282119107	,],
[	0.342600232	,	0.327412111	,	0.07514153	,	0.26735689	,],
[	0.298412816	,	0.412883564	,	0.319945124	,	0.083144869	,]],
[[	0.086493483	,	0.341360676	,	0.350007697	,	0.252545186	,],
[	0.250312669	,	0.084960989	,	0.243908053	,	0.281858603	,],
[	0.340797051	,	0.326083575	,	0.075340678	,	0.265496897	,],
[	0.299279601	,	0.41383389	,	0.320684731	,	0.086153551	,]],
[[	0.085108261	,	0.342758606	,	0.351575878	,	0.254126491	,],
[	0.248657763	,	0.085946029	,	0.24273228	,	0.28034717	,],
[	0.344392484	,	0.330373375	,	0.082235791	,	0.269139291	,],
[	0.30099427	,	0.415219715	,	0.322328201	,	0.085159353	,]],
[[	0.099276641	,	0.226823317	,	0.260703749	,	0.277342624	,],
[	0.387843247	,	0.081699047	,	0.253317063	,	0.40465348	,],
[	0.459662192	,	0.300048803	,	0.076814982	,	0.373195755	,],
[	0.350640092	,	0.319941508	,	0.239522241	,	0.099336531	,]],
[[	0.094724069	,	0.22104408	,	0.255033039	,	0.271960183	,],
[	0.390663388	,	0.087625005	,	0.256603664	,	0.407441957	,],
[	0.460150169	,	0.300522111	,	0.074482736	,	0.374042627	,],
[	0.348470443	,	0.318190427	,	0.237757649	,	0.093586907	,]],
[[	0.097935961	,	0.22134293	,	0.255383282	,	0.272657389	,],
[	0.388466162	,	0.081272978	,	0.253893202	,	0.405212741	,],
[	0.458266914	,	0.2984387	,	0.073625948	,	0.37181569	,],
[	0.348354978	,	0.317686248	,	0.237149638	,	0.095332743	,]],
[[	0.09649523	,	0.223823911	,	0.257723388	,	0.274451013	,],
[	0.388248543	,	0.083251998	,	0.253882068	,	0.405066948	,],
[	0.461518958	,	0.302076781	,	0.079984949	,	0.375026956	,],
[	0.351283212	,	0.321267734	,	0.241002959	,	0.096028333	,]],
[[	0.094757353	,	0.22015111	,	0.254299879	,	0.271248106	,],
[	0.391018737	,	0.083346244	,	0.256520848	,	0.407767843	,],
[	0.458979028	,	0.299165689	,	0.073458878	,	0.372595871	,],
[	0.34789918	,	0.31761311	,	0.237149419	,	0.093687366	,]],
[[	0.100851774	,	0.228490952	,	0.262219353	,	0.278853685	,],
[	0.387910271	,	0.08135035	,	0.253210874	,	0.404600972	,],
[	0.458946985	,	0.299234587	,	0.075542772	,	0.372453806	,],
[	0.349774566	,	0.319056316	,	0.238635509	,	0.097796589	,]],
[[	0.095094371	,	0.221658685	,	0.255689089	,	0.272492431	,],
[	0.38995543	,	0.086269686	,	0.255625325	,	0.406549766	,],
[	0.46117666	,	0.301502634	,	0.075336517	,	0.374955903	,],
[	0.349202925	,	0.318952902	,	0.238505281	,	0.094244676	,]],
[[	0.096927195	,	0.221034189	,	0.255017185	,	0.27223343	,],
[	0.388898915	,	0.081428345	,	0.254382472	,	0.405677303	,],
[	0.458180434	,	0.298320383	,	0.073224151	,	0.371610294	,],
[	0.348178001	,	0.317525264	,	0.237055357	,	0.094675298	,]],
[[	0.097809998	,	0.225274212	,	0.259154578	,	0.275797317	,],
[	0.388183205	,	0.082642854	,	0.253658947	,	0.404870819	,],
[	0.460776913	,	0.301116271	,	0.078551082	,	0.374241001	,],
[	0.352293595	,	0.322333375	,	0.242116745	,	0.097076916	,]],
[[	0.095132438	,	0.220823344	,	0.254911807	,	0.271792934	,],
[	0.392068746	,	0.089854373	,	0.258090822	,	0.408654606	,],
[	0.459016815	,	0.299187956	,	0.073388657	,	0.372649421	,],
[	0.347836473	,	0.317541512	,	0.236976712	,	0.093263349	,]],
[[	0.097655891	,	0.227039508	,	0.259637536	,	0.271111113	,],
[	0.371813111	,	0.07816366	,	0.24669501	,	0.389514229	,],
[	0.440951731	,	0.290664072	,	0.07095722	,	0.358024373	,],
[	0.335371425	,	0.311939498	,	0.23449411	,	0.092411307	,]],
[[	0.091842822	,	0.220288879	,	0.253067569	,	0.264687108	,],
[	0.372024652	,	0.080446358	,	0.24698577	,	0.389696552	,],
[	0.443486955	,	0.293644634	,	0.076466931	,	0.360613265	,],
[	0.338529309	,	0.315626425	,	0.238404665	,	0.09325873	,]],
[[	0.093650735	,	0.219427096	,	0.252228898	,	0.264163013	,],
[	0.374684537	,	0.080431569	,	0.249653108	,	0.392313688	,],
[	0.441860845	,	0.291606237	,	0.071170012	,	0.359037005	,],
[	0.334715528	,	0.311476815	,	0.23418435	,	0.090470353	,]],
[[	0.094226504	,	0.223667359	,	0.256265005	,	0.267758423	,],
[	0.371124667	,	0.078063158	,	0.246008293	,	0.388810781	,],
[	0.441275703	,	0.29115042	,	0.072413797	,	0.358287678	,],
[	0.337145339	,	0.313381889	,	0.236048521	,	0.095044151	,]],
[[	0.091667247	,	0.219365461	,	0.252110122	,	0.26372664	,],
[	0.373337641	,	0.083163181	,	0.248452109	,	0.390844969	,],
[	0.44475287	,	0.294749106	,	0.073798572	,	0.362123559	,],
[	0.336364758	,	0.313327085	,	0.235982913	,	0.09116601	,]],
[[	0.096615678	,	0.221169899	,	0.253967213	,	0.266038704	,],
[	0.372609104	,	0.078703948	,	0.247422202	,	0.390100262	,],
[	0.440914558	,	0.29059269	,	0.070646105	,	0.357933401	,],
[	0.33509678	,	0.311803085	,	0.234313473	,	0.091562215	,]],
[[	0.092341195	,	0.221109883	,	0.253652683	,	0.265244009	,],
[	0.371289501	,	0.079305848	,	0.246129803	,	0.388849274	,],
[	0.442728091	,	0.29276113	,	0.075139987	,	0.359735058	,],
[	0.339050381	,	0.315234762	,	0.238041794	,	0.098218066	,]],
[[	0.092979778	,	0.219408674	,	0.252144579	,	0.264018503	,],
[	0.375530398	,	0.086975227	,	0.250927731	,	0.392923461	,],
[	0.442584444	,	0.292357793	,	0.071722665	,	0.359710966	,],
[	0.334956903	,	0.311731368	,	0.234190464	,	0.0902953	,]],
[[	0.095528529	,	0.224952984	,	0.257564709	,	0.268896807	,],
[	0.371445317	,	0.078077078	,	0.246236774	,	0.38906346	,],
[	0.441167123	,	0.290995266	,	0.071749425	,	0.35812253	,],
[	0.336405124	,	0.312764721	,	0.23530723	,	0.093776933	,]],
[[	0.091701667	,	0.219700355	,	0.252432055	,	0.264124589	,],
[	0.372432475	,	0.081621776	,	0.247389652	,	0.389936372	,],
[	0.444575822	,	0.294909691	,	0.07851229	,	0.361668637	,],
[	0.337182045	,	0.314202888	,	0.236852852	,	0.091853725	,]],
[[	0.096127056	,	0.225597295	,	0.258185111	,	0.2656841	,],
[	0.372054597	,	0.085178446	,	0.250736789	,	0.389585595	,],
[	0.440844673	,	0.293488233	,	0.07263109	,	0.357600082	,],
[	0.333920532	,	0.315020227	,	0.237534926	,	0.089734908	,]],
[[	0.092516229	,	0.225895887	,	0.25826531	,	0.265458613	,],
[	0.36871309	,	0.07753724	,	0.247003609	,	0.386346929	,],
[	0.43889822	,	0.291376336	,	0.071811003	,	0.355391589	,],
[	0.336311688	,	0.317087058	,	0.239624224	,	0.094413213	,]],
[[	0.092836863	,	0.224032848	,	0.256508248	,	0.263936826	,],
[	0.36928169	,	0.080464273	,	0.247746871	,	0.386808572	,],
[	0.442071625	,	0.294909659	,	0.077825197	,	0.358605011	,],
[	0.335482395	,	0.316677241	,	0.239219015	,	0.090503191	,]],
[[	0.095672807	,	0.229835326	,	0.262171297	,	0.269052499	,],
[	0.371154896	,	0.079429467	,	0.249533603	,	0.388807747	,],
[	0.439431159	,	0.291982241	,	0.071402454	,	0.356018818	,],
[	0.334452188	,	0.315286533	,	0.237637782	,	0.090924704	,]],
[[	0.091429824	,	0.22425374	,	0.256805441	,	0.263859528	,],
[	0.368464077	,	0.078146438	,	0.246812804	,	0.386073662	,],
[	0.439473758	,	0.29202377	,	0.073368876	,	0.355972682	,],
[	0.33892392	,	0.320243595	,	0.242962162	,	0.09368041	,]],
[[	0.095093729	,	0.225108512	,	0.257729932	,	0.26511256	,],
[	0.370895943	,	0.08338857	,	0.249492655	,	0.388466243	,],
[	0.441629041	,	0.29425555	,	0.073287862	,	0.358352051	,],
[	0.33437543	,	0.315456834	,	0.237947701	,	0.089778422	,]],
[[	0.092976352	,	0.226705571	,	0.259033722	,	0.266110665	,],
[	0.369369152	,	0.077946296	,	0.247552115	,	0.386892268	,],
[	0.438926425	,	0.291294134	,	0.07138866	,	0.355309706	,],
[	0.335848764	,	0.316604752	,	0.239029053	,	0.093342178	,]],
[[	0.091978575	,	0.223754882	,	0.256159281	,	0.263549594	,],
[	0.368873151	,	0.079487246	,	0.247210327	,	0.386453407	,],
[	0.441119682	,	0.293925656	,	0.076345414	,	0.357634741	,],
[	0.336433789	,	0.317625289	,	0.240129793	,	0.091097752	,]],
[[	0.097280971	,	0.231492755	,	0.263811692	,	0.270545009	,],
[	0.372321103	,	0.080616991	,	0.250758825	,	0.389959856	,],
[	0.439909185	,	0.29238104	,	0.071687188	,	0.356458518	,],
[	0.334254492	,	0.315144813	,	0.23748027	,	0.090329112	,]],
[[	0.091857111	,	0.224949755	,	0.257408135	,	0.264409991	,],
[	0.368387909	,	0.077520189	,	0.246653313	,	0.385970514	,],
[	0.439186058	,	0.291631623	,	0.072670008	,	0.355452821	,],
[	0.338036526	,	0.318501974	,	0.240978877	,	0.096698419	,]],
[[	0.094978012	,	0.228620503	,	0.260603431	,	0.263672119	,],
[	0.36136549	,	0.081564622	,	0.246761771	,	0.380976244	,],
[	0.433973339	,	0.294246793	,	0.079259523	,	0.353044461	,],
[	0.328719059	,	0.316722241	,	0.239996595	,	0.090006794	,]],
[[	0.090242728	,	0.227983415	,	0.259760396	,	0.262536015	,],
[	0.361144446	,	0.077777897	,	0.246247693	,	0.380747534	,],
[	0.429911134	,	0.289769097	,	0.070914123	,	0.349057489	,],
[	0.329946109	,	0.317556761	,	0.240961861	,	0.09349475	,]],
[[	0.091216808	,	0.226523958	,	0.258473518	,	0.261518456	,],
[	0.35948949	,	0.077970125	,	0.244685387	,	0.379248912	,],
[	0.431092255	,	0.291010822	,	0.074304239	,	0.350117989	,],
[	0.330515617	,	0.318660576	,	0.242050356	,	0.091190972	,]],
[[	0.093470642	,	0.231969496	,	0.263686907	,	0.266241577	,],
[	0.363287448	,	0.084964952	,	0.248862583	,	0.382820943	,],
[	0.431709161	,	0.291601765	,	0.072305094	,	0.350955029	,],
[	0.328564201	,	0.316245461	,	0.239573127	,	0.090530866	,]],
[[	0.089307208	,	0.226432764	,	0.258342186	,	0.261077769	,],
[	0.359959682	,	0.077090263	,	0.24510548	,	0.37961628	,],
[	0.429453368	,	0.289241776	,	0.071225333	,	0.34852298	,],
[	0.332437828	,	0.319800265	,	0.243120972	,	0.096948236	,]],
[[	0.093657949	,	0.227814575	,	0.259712457	,	0.26274707	,],
[	0.360575339	,	0.080212718	,	0.245962874	,	0.380229269	,],
[	0.433154703	,	0.293295721	,	0.07785788	,	0.352079347	,],
[	0.328782482	,	0.316827005	,	0.240210069	,	0.089888403	,]],
[[	0.09095721	,	0.228970086	,	0.260620616	,	0.263348774	,],
[	0.361775171	,	0.07833322	,	0.246889511	,	0.38134005	,],
[	0.430534967	,	0.290253808	,	0.071256306	,	0.349590288	,],
[	0.329648539	,	0.317336539	,	0.240695721	,	0.092489855	,]],
[[	0.090437102	,	0.226320072	,	0.258232509	,	0.261137809	,],
[	0.359340375	,	0.077364984	,	0.244456885	,	0.378969465	,],
[	0.430904289	,	0.290696078	,	0.073604334	,	0.349698889	,],
[	0.331742569	,	0.319783556	,	0.243277001	,	0.09218846	,]],
[[	0.094844571	,	0.233384581	,	0.264940802	,	0.267440781	,],
[	0.362534264	,	0.083646621	,	0.248047189	,	0.382100353	,],
[	0.432704545	,	0.292637012	,	0.073127573	,	0.351974581	,],
[	0.328465315	,	0.316289043	,	0.239559466	,	0.09013069	,]],
[[	0.089797733	,	0.227208035	,	0.258983532	,	0.261640029	,],
[	0.360375891	,	0.077173861	,	0.245371218	,	0.379859468	,],
[	0.430058315	,	0.289759016	,	0.071500688	,	0.349018354	,],
[	0.331551885	,	0.319034552	,	0.242339224	,	0.095653001	,]],
[[	0.092699496	,	0.231580603	,	0.262804077	,	0.260906758	,],
[	0.352988838	,	0.080853872	,	0.244595571	,	0.373842408	,],
[	0.425684024	,	0.292796098	,	0.076677844	,	0.34592185	,],
[	0.325195691	,	0.319049598	,	0.242736856	,	0.088930088	,]],
[[	0.089329272	,	0.231866312	,	0.262890219	,	0.260729415	,],
[	0.353941752	,	0.078505231	,	0.245271251	,	0.374764609	,],
[	0.422799114	,	0.289608636	,	0.069931092	,	0.34325885	,],
[	0.324828769	,	0.318323108	,	0.24191136	,	0.090272165	,]],
[[	0.089327281	,	0.229772646	,	0.260925181	,	0.258928201	,],
[	0.351732166	,	0.077886617	,	0.243029905	,	0.372530259	,],
[	0.423147317	,	0.289968331	,	0.072222348	,	0.343368976	,],
[	0.328226468	,	0.321893068	,	0.245695148	,	0.091423204	,]],
[[	0.093022896	,	0.236178172	,	0.267129085	,	0.26461164	,],
[	0.354961244	,	0.084231701	,	0.246712949	,	0.375710758	,],
[	0.425064356	,	0.291892937	,	0.071895069	,	0.345697958	,],
[	0.324470569	,	0.317922957	,	0.241579157	,	0.088492338	,]],
[[	0.088389289	,	0.230287749	,	0.261483604	,	0.259245158	,],
[	0.352316979	,	0.077424207	,	0.243616369	,	0.373149305	,],
[	0.422194497	,	0.288879915	,	0.069976077	,	0.342430586	,],
[	0.32687253	,	0.320114122	,	0.243646992	,	0.093243207	,]],
[[	0.091865334	,	0.231064356	,	0.262259881	,	0.260264909	,],
[	0.352701505	,	0.080103924	,	0.244231018	,	0.373660329	,],
[	0.424896522	,	0.291912809	,	0.07521664	,	0.345070282	,],
[	0.32595279	,	0.319828621	,	0.24356499	,	0.089509411	,]],
[[	0.09020979	,	0.232979183	,	0.26389472	,	0.261695386	,],
[	0.355195165	,	0.079577997	,	0.246449204	,	0.375918692	,],
[	0.423255066	,	0.28997554	,	0.070176919	,	0.343618996	,],
[	0.324603998	,	0.318165591	,	0.241759895	,	0.089604955	,]],
[[	0.08895515	,	0.229898089	,	0.26098395	,	0.258890464	,],
[	0.35169564	,	0.077577418	,	0.243014591	,	0.372464626	,],
[	0.422717437	,	0.289502293	,	0.071272278	,	0.342836347	,],
[	0.329168111	,	0.323014419	,	0.24677124	,	0.092421511	,]],
[[	0.09423746	,	0.237440723	,	0.268431935	,	0.265922533	,],
[	0.354098135	,	0.082839104	,	0.245813882	,	0.374920257	,],
[	0.426143572	,	0.293071825	,	0.072935624	,	0.346722973	,],
[	0.324722623	,	0.318307824	,	0.241912924	,	0.08853244	,]],
[[	0.088727306	,	0.230878142	,	0.261960784	,	0.259671295	,],
[	0.352754578	,	0.077535871	,	0.24408969	,	0.373552365	,],
[	0.422216394	,	0.288941299	,	0.069697987	,	0.34248723	,],
[	0.326282754	,	0.3195678	,	0.243091686	,	0.092163397	,]],
[[	0.091701031	,	0.234882611	,	0.26487135	,	0.257946025	,],
[	0.344737654	,	0.078738477	,	0.241857651	,	0.365963274	,],
[	0.416238675	,	0.289446329	,	0.073489306	,	0.337931565	,],
[	0.320452287	,	0.320683358	,	0.244822155	,	0.088115171	,]],
[[	0.089155738	,	0.235783605	,	0.26562082	,	0.258356889	,],
[	0.348640841	,	0.080098067	,	0.245555837	,	0.369788249	,],
[	0.416563606	,	0.289630762	,	0.070738535	,	0.338617725	,],
[	0.319570191	,	0.319456371	,	0.243452648	,	0.088728104	,]],
[[	0.088664948	,	0.233464684	,	0.263240729	,	0.256162501	,],
[	0.344592897	,	0.07718113	,	0.241538059	,	0.365730895	,],
[	0.41439347	,	0.287499278	,	0.069946469	,	0.336102788	,],
[	0.324095092	,	0.324255903	,	0.248386357	,	0.091245934	,]],
[[	0.093001605	,	0.240237453	,	0.269991487	,	0.262455215	,],
[	0.345923407	,	0.08124709	,	0.243227902	,	0.367065366	,],
[	0.418181881	,	0.291734849	,	0.07688588	,	0.339951349	,],
[	0.319436516	,	0.319484807	,	0.243432348	,	0.087434913	,]],
[[	0.087996712	,	0.234114175	,	0.264088382	,	0.256743118	,],
[	0.346340429	,	0.07810094	,	0.243301915	,	0.367477546	,],
[	0.415010942	,	0.287989231	,	0.069541036	,	0.336752441	,],
[	0.321286258	,	0.32096992	,	0.244914042	,	0.091405147	,]],
[[	0.090507759	,	0.234230061	,	0.264246214	,	0.257110935	,],
[	0.344309691	,	0.07781202	,	0.24129887	,	0.365424471	,],
[	0.415458784	,	0.288556774	,	0.072170325	,	0.337060123	,],
[	0.321230602	,	0.321459419	,	0.24554948	,	0.088641854	,]],
[[	0.090198871	,	0.23701976	,	0.266638487	,	0.259414169	,],
[	0.348009134	,	0.084902882	,	0.245629287	,	0.369080622	,],
[	0.41732776	,	0.290395207	,	0.071461735	,	0.339279639	,],
[	0.31964707	,	0.319588906	,	0.243584247	,	0.088418567	,]],
[[	0.088068988	,	0.233278964	,	0.263166138	,	0.255998705	,],
[	0.34493234	,	0.07726248	,	0.241868379	,	0.366064936	,],
[	0.414793389	,	0.287708765	,	0.07001559	,	0.336375967	,],
[	0.323474517	,	0.323118886	,	0.247177241	,	0.094851684	,]],
[[	0.094001424	,	0.236065905	,	0.26586447	,	0.259040519	,],
[	0.345280482	,	0.0800369	,	0.242517553	,	0.366350924	,],
[	0.417257823	,	0.290528748	,	0.075245502	,	0.338857694	,],
[	0.319834354	,	0.319878546	,	0.243827345	,	0.087673437	,]],
[[	0.088245594	,	0.234805135	,	0.264654945	,	0.25728321	,],
[	0.34716753	,	0.078798211	,	0.244085216	,	0.368183773	,],
[	0.415663328	,	0.288579063	,	0.069981453	,	0.337355268	,],
[	0.320430586	,	0.320169136	,	0.244189951	,	0.090224241	,]],
[[	0.090212461	,	0.238209446	,	0.267597332	,	0.25708283	,],
[	0.338780798	,	0.077585306	,	0.240981423	,	0.36152566	,],
[	0.41071123	,	0.288861802	,	0.07202131	,	0.333128859	,],
[	0.318959585	,	0.324672672	,	0.248533869	,	0.08856781	,]],
[[	0.088520255	,	0.239871169	,	0.269046618	,	0.25820029	,],
[	0.342042865	,	0.084060662	,	0.244710001	,	0.364619433	,],
[	0.413188233	,	0.2913183	,	0.072092939	,	0.336020078	,],
[	0.316759755	,	0.322292178	,	0.246152379	,	0.087815277	,]],
[[	0.087585892	,	0.237099639	,	0.26634375	,	0.255735934	,],
[	0.339436228	,	0.077038203	,	0.241438769	,	0.362049532	,],
[	0.409965883	,	0.287945877	,	0.069827294	,	0.332500714	,],
[	0.320207574	,	0.325436624	,	0.249337694	,	0.093887013	,]],
[[	0.092435618	,	0.244366267	,	0.273508348	,	0.262509113	,],
[	0.339515402	,	0.079470869	,	0.241857887	,	0.362202614	,],
[	0.412413073	,	0.290699106	,	0.07496079	,	0.334830132	,],
[	0.317490998	,	0.322802457	,	0.246704955	,	0.087431997	,]],
[[	0.086947101	,	0.237929056	,	0.26714186	,	0.256316551	,],
[	0.341958207	,	0.078990063	,	0.244045767	,	0.364502581	,],
[	0.411159986	,	0.289174642	,	0.070248698	,	0.333675975	,],
[	0.317879362	,	0.323006096	,	0.246825199	,	0.089765944	,]],
[[	0.089202796	,	0.237630054	,	0.26705049	,	0.256386632	,],
[	0.338698854	,	0.077175866	,	0.240881239	,	0.36139924	,],
[	0.410490582	,	0.288489109	,	0.071243474	,	0.33279124	,],
[	0.319848384	,	0.325555363	,	0.249590566	,	0.089404488	,]],
[[	0.089261857	,	0.240953647	,	0.269959341	,	0.259066039	,],
[	0.341355405	,	0.082671127	,	0.243915308	,	0.363900744	,],
[	0.414344123	,	0.292500925	,	0.07324485	,	0.337111401	,],
[	0.316818357	,	0.322261674	,	0.246151544	,	0.087490699	,]],
[[	0.087018158	,	0.237002781	,	0.266198302	,	0.255565121	,],
[	0.340042521	,	0.07729414	,	0.241909118	,	0.362491564	,],
[	0.410320814	,	0.288229137	,	0.069647603	,	0.332739363	,],
[	0.319512493	,	0.324721641	,	0.248628714	,	0.092701146	,]],
[[	0.092513487	,	0.239436413	,	0.268647969	,	0.258261583	,],
[	0.339026495	,	0.078653898	,	0.241369674	,	0.361686736	,],
[	0.41186834	,	0.290020109	,	0.073789395	,	0.334087519	,],
[	0.318043291	,	0.323520057	,	0.247427909	,	0.087809415	,]],
[[	0.087692749	,	0.238957907	,	0.268181107	,	0.257258409	,],
[	0.343256584	,	0.080150175	,	0.245242867	,	0.365714905	,],
[	0.412130616	,	0.290205812	,	0.071126713	,	0.3346976	,],
[	0.317527256	,	0.32278418	,	0.246584655	,	0.089085287	,]],
[[	0.088973516	,	0.243309455	,	0.271666547	,	0.256007738	,],
[	0.333120357	,	0.076963409	,	0.241096135	,	0.356627679	,],
[	0.404629139	,	0.288652205	,	0.070363548	,	0.32776517	,],
[	0.317974005	,	0.329543305	,	0.253508636	,	0.089493853	,]],
[[	0.08941239	,	0.246689969	,	0.274855027	,	0.259019277	,],
[	0.334414051	,	0.08106476	,	0.242774606	,	0.357860462	,],
[	0.408629973	,	0.293202535	,	0.077795946	,	0.331931552	,],
[	0.314509417	,	0.32592562	,	0.249729995	,	0.087024187	,]],
[[	0.086926994	,	0.242602779	,	0.270911609	,	0.255145105	,],
[	0.33490957	,	0.077865317	,	0.242790295	,	0.358343131	,],
[	0.404344678	,	0.288341842	,	0.06916672	,	0.327754353	,],
[	0.316856109	,	0.327931812	,	0.251782199	,	0.091554204	,]],
[[	0.092382703	,	0.245061603	,	0.273351035	,	0.257929673	,],
[	0.332894867	,	0.077763835	,	0.240937956	,	0.356347361	,],
[	0.40576881	,	0.290096436	,	0.072887117	,	0.329081561	,],
[	0.315907145	,	0.327330676	,	0.251106251	,	0.087598878	,]],
[[	0.087078369	,	0.244077559	,	0.272368116	,	0.256521767	,],
[	0.336766126	,	0.08470141	,	0.245219562	,	0.359984156	,],
[	0.40626969	,	0.290305641	,	0.070456014	,	0.329827838	,],
[	0.314926692	,	0.325956439	,	0.249789235	,	0.088043754	,]],
[[	0.088367407	,	0.243164953	,	0.271492735	,	0.255717297	,],
[	0.33334697	,	0.077009297	,	0.241322075	,	0.356839129	,],
[	0.404469485	,	0.288436098	,	0.069883664	,	0.327647393	,],
[	0.319085568	,	0.33072562	,	0.254710831	,	0.090428635	,]],
[[	0.090478474	,	0.247923248	,	0.276003778	,	0.260058746	,],
[	0.333689068	,	0.079671068	,	0.241833365	,	0.357185228	,],
[	0.407782602	,	0.292303444	,	0.076281897	,	0.330967829	,],
[	0.314800267	,	0.326174566	,	0.249990741	,	0.086841531	,]],
[[	0.086949003	,	0.243062282	,	0.271351419	,	0.255518166	,],
[	0.33597056	,	0.078709942	,	0.243719759	,	0.35908195	,],
[	0.404725072	,	0.288723877	,	0.069213672	,	0.328055945	,],
[	0.316079054	,	0.327164015	,	0.250943198	,	0.090173278	,]],
[[	0.091110795	,	0.244506806	,	0.272796646	,	0.257149286	,],
[	0.332950301	,	0.077414483	,	0.240820592	,	0.356306058	,],
[	0.405270841	,	0.289480616	,	0.071838691	,	0.328439853	,],
[	0.316544988	,	0.327832759	,	0.251695402	,	0.087930327	,]],
[[	0.087936341	,	0.245112629	,	0.273418669	,	0.257439938	,],
[	0.335648472	,	0.083145289	,	0.244012081	,	0.358922235	,],
[	0.407480921	,	0.291510897	,	0.071494051	,	0.330893134	,],
[	0.314579756	,	0.325724734	,	0.249510031	,	0.087282824	,]],
[[	0.090152019	,	0.249083576	,	0.2779991	,	0.257049337	,],
[	0.329186905	,	0.07704338	,	0.241592175	,	0.353458337	,],
[	0.401231526	,	0.289602467	,	0.070309609	,	0.32484442	,],
[	0.316016968	,	0.33361089	,	0.257374371	,	0.089034003	,]],
[[	0.088048915	,	0.25048442	,	0.27929207	,	0.258058892	,],
[	0.329991186	,	0.08050704	,	0.242898742	,	0.354339708	,],
[	0.404389761	,	0.293141031	,	0.07624613	,	0.327966108	,],
[	0.313199863	,	0.330709554	,	0.254245261	,	0.087170535	,]],
[[	0.087292704	,	0.247807714	,	0.276581996	,	0.255502	,],
[	0.331404638	,	0.078466627	,	0.243754743	,	0.355553003	,],
[	0.402180678	,	0.290361351	,	0.070200738	,	0.325728376	,],
[	0.316070677	,	0.333288639	,	0.256854807	,	0.092357439	,]],
[[	0.092001189	,	0.254884955	,	0.283589387	,	0.262312801	,],
[	0.328796893	,	0.077651167	,	0.241372864	,	0.353137683	,],
[	0.402200728	,	0.290543551	,	0.072015713	,	0.325577047	,],
[	0.314156479	,	0.331517683	,	0.255193529	,	0.087299834	,]],
[[	0.086682577	,	0.248681716	,	0.277466546	,	0.25624034	,],
[	0.332179292	,	0.084129925	,	0.245104792	,	0.356355113	,],
[	0.404993839	,	0.293309195	,	0.072606793	,	0.328793657	,],
[	0.313954812	,	0.331258484	,	0.254734374	,	0.088634511	,]],
[[	0.089105644	,	0.24864922	,	0.277490962	,	0.256511827	,],
[	0.329696378	,	0.077217182	,	0.242137028	,	0.353884688	,],
[	0.401702263	,	0.289806394	,	0.070238887	,	0.325107485	,],
[	0.317511101	,	0.335233182	,	0.25886162	,	0.090296585	,]],
[[	0.089369223	,	0.251823581	,	0.280479433	,	0.259273974	,],
[	0.329743122	,	0.079627811	,	0.242377371	,	0.353968796	,],
[	0.403781647	,	0.292479487	,	0.075060702	,	0.327251002	,],
[	0.313078273	,	0.330612222	,	0.254139792	,	0.086754358	,]],
[[	0.086846631	,	0.247870422	,	0.276618996	,	0.255537916	,],
[	0.33267298	,	0.079542694	,	0.245018165	,	0.356709205	,],
[	0.402873978	,	0.291136533	,	0.070634832	,	0.32645948	,],
[	0.315174179	,	0.33243924	,	0.255880955	,	0.090932708	,]],
[[	0.0921791	,	0.250213639	,	0.278937381	,	0.258168058	,],
[	0.329121056	,	0.077375687	,	0.241524363	,	0.353256362	,],
[	0.40204909	,	0.290161254	,	0.071438375	,	0.325271604	,],
[	0.314917703	,	0.332439381	,	0.255903756	,	0.08779631	,]],
[[	0.087172826	,	0.249376253	,	0.2781214	,	0.256869879	,],
[	0.331419892	,	0.082649885	,	0.244268041	,	0.355429988	,],
[	0.406019141	,	0.294890109	,	0.078824714	,	0.32949385	,],
[	0.313462873	,	0.330817928	,	0.254272664	,	0.087960726	,]],
[[	0.088779309	,	0.252410226	,	0.27987024	,	0.255170044	,],
[	0.324322494	,	0.077231444	,	0.241231179	,	0.348819416	,],
[	0.396270388	,	0.290410172	,	0.070197409	,	0.320312419	,],
[	0.314500601	,	0.336853799	,	0.259958163	,	0.088606786	,]],
[[	0.087679239	,	0.254536058	,	0.281811424	,	0.257029325	,],
[	0.323912566	,	0.079102205	,	0.240964952	,	0.348346046	,],
[	0.399068414	,	0.293620949	,	0.075791026	,	0.323102368	,],
[	0.310824436	,	0.33319886	,	0.256138023	,	0.085946242	,]],
[[	0.086190205	,	0.251332183	,	0.278709059	,	0.253928914	,],
[	0.327452291	,	0.079828997	,	0.244327749	,	0.351837125	,],
[	0.397137129	,	0.291229225	,	0.070027581	,	0.321247058	,],
[	0.313487947	,	0.335529971	,	0.25852481	,	0.090809107	,]],
[[	0.091678106	,	0.259072556	,	0.286375828	,	0.261335481	,],
[	0.323392795	,	0.077045748	,	0.240342631	,	0.347828995	,],
[	0.396821712	,	0.291150048	,	0.071679143	,	0.320868323	,],
[	0.312329964	,	0.334538617	,	0.257551418	,	0.086393318	,]],
[[	0.086060937	,	0.252521536	,	0.279866658	,	0.254951155	,],
[	0.32567799	,	0.08212427	,	0.242946209	,	0.349997391	,],
[	0.399809597	,	0.294009918	,	0.072396546	,	0.324134561	,],
[	0.311603277	,	0.333715572	,	0.25656875	,	0.087304598	,]],
[[	0.087866999	,	0.252077754	,	0.2795589	,	0.254681932	,],
[	0.324933712	,	0.07761521	,	0.241776024	,	0.34938637	,],
[	0.396198921	,	0.29020703	,	0.069648511	,	0.320181817	,],
[	0.315581863	,	0.338015506	,	0.261082034	,	0.089378268	,]],
[[	0.088894845	,	0.255868081	,	0.283050908	,	0.258105792	,],
[	0.323776851	,	0.078373999	,	0.240780531	,	0.348154665	,],
[	0.398504177	,	0.292875894	,	0.074696315	,	0.322444912	,],
[	0.311222917	,	0.333549365	,	0.2565356	,	0.085899691	,]],
[[	0.085949398	,	0.251550675	,	0.27881429	,	0.253995902	,],
[	0.32754564	,	0.08563157	,	0.245014586	,	0.351671679	,],
[	0.397755744	,	0.291782406	,	0.070432857	,	0.321897323	,],
[	0.312858157	,	0.334831081	,	0.257826155	,	0.089556086	,]],
[[	0.090863564	,	0.253526877	,	0.280859919	,	0.25626816	,],
[	0.323751203	,	0.077103978	,	0.24054405	,	0.347971473	,],
[	0.396692002	,	0.290799468	,	0.071119554	,	0.320623568	,],
[	0.313110912	,	0.335299456	,	0.258370455	,	0.087059482	,]],
[[	0.086551734	,	0.253351886	,	0.280596921	,	0.255637415	,],
[	0.324918624	,	0.08081247	,	0.241943591	,	0.349102828	,],
[	0.400613452	,	0.295319845	,	0.078248131	,	0.324558103	,],
[	0.311186976	,	0.333366055	,	0.256293564	,	0.086579176	,]],
[[	0.087963811	,	0.257029125	,	0.283672502	,	0.25470035	,],
[	0.320718263	,	0.078771787	,	0.242949792	,	0.346055528	,],
[	0.391729022	,	0.291211767	,	0.070014701	,	0.316970308	,],
[	0.312193648	,	0.340017505	,	0.262949624	,	0.093055708	,]],
[[	0.086775882	,	0.258985769	,	0.285507127	,	0.256303956	,],
[	0.318150278	,	0.07787557	,	0.240657619	,	0.343598322	,],
[	0.391389957	,	0.290994946	,	0.071722311	,	0.316555873	,],
[	0.308326484	,	0.336704444	,	0.259563533	,	0.085643021	,]],
[[	0.085331803	,	0.255886084	,	0.28242685	,	0.25327995	,],
[	0.321117891	,	0.083946576	,	0.244141221	,	0.346551639	,],
[	0.39462282	,	0.294111984	,	0.072560045	,	0.320020582	,],
[	0.309044467	,	0.3372243	,	0.260063984	,	0.088211244	,]],
[[	0.090842435	,	0.263521053	,	0.290086772	,	0.260832835	,],
[	0.318937223	,	0.077470104	,	0.241244507	,	0.344298682	,],
[	0.390718527	,	0.290110254	,	0.069524231	,	0.315816415	,],
[	0.311041224	,	0.339347341	,	0.262261297	,	0.087716045	,]],
[[	0.085220547	,	0.257240218	,	0.283736795	,	0.25450177	,],
[	0.31896233	,	0.079744792	,	0.241603233	,	0.344381838	,],
[	0.393112834	,	0.292916964	,	0.074641605	,	0.318225786	,],
[	0.308233596	,	0.336288323	,	0.259074925	,	0.085808925	,]],
[[	0.087051353	,	0.256707979	,	0.283346291	,	0.254137976	,],
[	0.322140772	,	0.08013772	,	0.244421729	,	0.34746973	,],
[	0.39208034	,	0.291498196	,	0.070232009	,	0.317396688	,],
[	0.311233534	,	0.339133223	,	0.261951704	,	0.091460159	,]],
[[	0.087753873	,	0.260151734	,	0.286542041	,	0.257337155	,],
[	0.318170405	,	0.077398454	,	0.240569046	,	0.343590769	,],
[	0.390976073	,	0.290642146	,	0.070766298	,	0.31610967	,],
[	0.309058665	,	0.337431777	,	0.260311012	,	0.085985929	,]],
[[	0.084904968	,	0.255977455	,	0.28247386	,	0.253368171	,],
[	0.320206971	,	0.082308883	,	0.242985384	,	0.345497569	,],
[	0.395065336	,	0.29528742	,	0.078114463	,	0.320253879	,],
[	0.308643735	,	0.336858297	,	0.25965111	,	0.087367321	,]],
[[	0.090067114	,	0.258157406	,	0.284613827	,	0.255713701	,],
[	0.319772829	,	0.077957085	,	0.242060441	,	0.345108648	,],
[	0.390964501	,	0.290341882	,	0.069546071	,	0.316124828	,],
[	0.312004669	,	0.340354899	,	0.263283855	,	0.088560636	,]],
[[	0.085535116	,	0.257726523	,	0.284234339	,	0.254946645	,],
[	0.318582574	,	0.078829892	,	0.241113748	,	0.343928592	,],
[	0.39194757	,	0.291751004	,	0.073009482	,	0.316971	,],
[	0.308307553	,	0.336613218	,	0.259321391	,	0.085808975	,]],
[[	0.087282638	,	0.261377787	,	0.287114561	,	0.252466131	,],
[	0.31651794	,	0.086155788	,	0.243863837	,	0.341191725	,],
[	0.386991159	,	0.292650177	,	0.070399529	,	0.311895803	,],
[	0.309423339	,	0.342930272	,	0.265427189	,	0.090889861	,]],
[[	0.086975416	,	0.263998907	,	0.289499163	,	0.25481811	,],
[	0.312152305	,	0.07709762	,	0.238841346	,	0.337025842	,],
[	0.385304396	,	0.291066228	,	0.070218397	,	0.309982277	,],
[	0.307152743	,	0.341114819	,	0.263551149	,	0.085246323	,]],
[[	0.084649431	,	0.260233246	,	0.285809572	,	0.251212961	,],
[	0.313341392	,	0.080881022	,	0.240396819	,	0.338201094	,],
[	0.389201125	,	0.295510735	,	0.077409992	,	0.31389264	,],
[	0.306908363	,	0.340632454	,	0.263080145	,	0.08694643	,]],
[[	0.090354349	,	0.262996197	,	0.288514168	,	0.254143372	,],
[	0.314410124	,	0.078465491	,	0.24098622	,	0.339239263	,],
[	0.385639127	,	0.291281117	,	0.069424143	,	0.310429101	,],
[	0.309824378	,	0.343627226	,	0.266214892	,	0.087617215	,]],
[[	0.085108898	,	0.261810663	,	0.287462329	,	0.25266717	,],
[	0.312188612	,	0.077940552	,	0.238946508	,	0.337051767	,],
[	0.386491919	,	0.292477837	,	0.072586905	,	0.311170504	,],
[	0.306011852	,	0.339833576	,	0.26215272	,	0.084646669	,]],
[[	0.086223539	,	0.260803957	,	0.286388261	,	0.251795766	,],
[	0.315738348	,	0.084734596	,	0.242922729	,	0.340337529	,],
[	0.387984999	,	0.293674282	,	0.071180034	,	0.31290068	,],
[	0.308709066	,	0.342228218	,	0.264636267	,	0.089540445	,]],
[[	0.088068461	,	0.265132513	,	0.290624828	,	0.255885895	,],
[	0.31262596	,	0.077104649	,	0.239258363	,	0.337459911	,],
[	0.38521194	,	0.290893691	,	0.069784421	,	0.309860211	,],
[	0.307787177	,	0.341563802	,	0.264161962	,	0.08570471	,]],
[[	0.08483629	,	0.26081101	,	0.286251904	,	0.251639785	,],
[	0.312990412	,	0.079945129	,	0.239820482	,	0.337762244	,],
[	0.388626239	,	0.294782981	,	0.076100604	,	0.313278581	,],
[	0.306386415	,	0.340104766	,	0.262538694	,	0.08578132	,]],
[[	0.089076263	,	0.262332469	,	0.287853312	,	0.253349822	,],
[	0.315252143	,	0.079157304	,	0.241802576	,	0.33989145	,],
[	0.386132251	,	0.291702892	,	0.069701516	,	0.310850054	,],
[	0.310973898	,	0.344456142	,	0.267017477	,	0.093113749	,]],
[[	0.085786945	,	0.262679948	,	0.288247969	,	0.253385092	,],
[	0.312038119	,	0.077411578	,	0.238697582	,	0.336782284	,],
[	0.385969243	,	0.291838479	,	0.071522006	,	0.310569811	,],
[	0.306655961	,	0.340416574	,	0.262737395	,	0.084922877	,]],
[[	0.086564688	,	0.265603237	,	0.290645182	,	0.252738966	,],
[	0.310370861	,	0.084697478	,	0.243402215	,	0.336760011	,],
[	0.384893161	,	0.295001078	,	0.07224367	,	0.310653314	,],
[	0.307094432	,	0.345704196	,	0.268129936	,	0.08928924	,]],
[[	0.087304841	,	0.268910852	,	0.293795832	,	0.255877475	,],
[	0.307516413	,	0.077420069	,	0.239834629	,	0.334059692	,],
[	0.381555756	,	0.291718464	,	0.069906135	,	0.307102753	,],
[	0.305819084	,	0.344724267	,	0.267290511	,	0.084978718	,]],
[[	0.084533783	,	0.264828696	,	0.289839917	,	0.251951522	,],
[	0.307926939	,	0.080088539	,	0.240525916	,	0.334415663	,],
[	0.383722424	,	0.294388663	,	0.074967746	,	0.309273451	,],
[	0.304787904	,	0.343715519	,	0.26611741	,	0.085572395	,]],
[[	0.089814725	,	0.26740066	,	0.292315134	,	0.254628251	,],
[	0.310107289	,	0.079078652	,	0.242116302	,	0.336341636	,],
[	0.382539868	,	0.292694277	,	0.070162006	,	0.308290696	,],
[	0.309490262	,	0.348015719	,	0.270478761	,	0.09288584	,]],
[[	0.085023121	,	0.266477791	,	0.291400793	,	0.253382084	,],
[	0.307120465	,	0.077648541	,	0.239462889	,	0.333486387	,],
[	0.381835274	,	0.292147425	,	0.071144673	,	0.307312192	,],
[	0.304795692	,	0.343602823	,	0.265952259	,	0.084453053	,]],
[[	0.085780282	,	0.265272409	,	0.290314623	,	0.252331215	,],
[	0.309615829	,	0.083318222	,	0.24243238	,	0.33592966	,],
[	0.385855549	,	0.296074983	,	0.073219612	,	0.311620961	,],
[	0.306281548	,	0.344816346	,	0.267192545	,	0.087973014	,]],
[[	0.088680572	,	0.270330151	,	0.295125546	,	0.25720133	,],
[	0.308234234	,	0.077600991	,	0.240372748	,	0.334597027	,],
[	0.381669598	,	0.291700344	,	0.069816211	,	0.307121228	,],
[	0.306871688	,	0.345770321	,	0.2682491	,	0.085797019	,]],
[[	0.084578531	,	0.265384481	,	0.290298422	,	0.252363483	,],
[	0.307665907	,	0.079278208	,	0.240110402	,	0.334011025	,],
[	0.382861672	,	0.293338141	,	0.073538721	,	0.308253325	,],
[	0.30437748	,	0.343225614	,	0.265545646	,	0.084715369	,]],
[[	0.088399327	,	0.266639705	,	0.291496917	,	0.253675293	,],
[	0.311503096	,	0.080263788	,	0.243607464	,	0.337625629	,],
[	0.383316467	,	0.293428174	,	0.070769876	,	0.308989539	,],
[	0.308470947	,	0.346937358	,	0.269509433	,	0.091315134	,]],
[[	0.085741867	,	0.267342234	,	0.292224956	,	0.25422628	,],
[	0.307190248	,	0.077479984	,	0.239321088	,	0.333533396	,],
[	0.38150428	,	0.29174055	,	0.070342085	,	0.306935524	,],
[	0.305291083	,	0.34409326	,	0.266509135	,	0.084595516	,]],
[[	0.086720165	,	0.271598174	,	0.29607815	,	0.252616439	,],
[	0.305780635	,	0.084728725	,	0.244115042	,	0.332874693	,],
[	0.381013009	,	0.297119924	,	0.073250174	,	0.307385304	,],
[	0.305531773	,	0.349876678	,	0.271750967	,	0.089214827	,]],
[[	0.086542844	,	0.274198316	,	0.298521784	,	0.255175164	,],
[	0.303141864	,	0.077825146	,	0.240757474	,	0.330306068	,],
[	0.377392489	,	0.293567662	,	0.070643493	,	0.303489014	,],
[	0.304414645	,	0.349065885	,	0.271000225	,	0.085228851	,]],
[[	0.084556738	,	0.270764749	,	0.295078165	,	0.251802247	,],
[	0.302941648	,	0.080055512	,	0.240954226	,	0.330128984	,],
[	0.379877757	,	0.296454257	,	0.075922052	,	0.305892519	,],
[	0.303476895	,	0.347940217	,	0.269863757	,	0.085659414	,]],
[[	0.090307615	,	0.273577922	,	0.297998392	,	0.254760728	,],
[	0.306280068	,	0.080389516	,	0.243782876	,	0.333371414	,],
[	0.378760975	,	0.294881517	,	0.07122789	,	0.305008194	,],
[	0.30820011	,	0.352697629	,	0.274726612	,	0.088558713	,]],
[[	0.084518895	,	0.271985888	,	0.296401627	,	0.252930406	,],
[	0.302629246	,	0.078048934	,	0.240266399	,	0.329744235	,],
[	0.377905318	,	0.294153385	,	0.072038996	,	0.303873395	,],
[	0.303371083	,	0.347843772	,	0.269623305	,	0.084323976	,]],
[[	0.085980969	,	0.271298805	,	0.295775866	,	0.252225957	,],
[	0.304734992	,	0.083138633	,	0.242898551	,	0.331784423	,],
[	0.38197109	,	0.298065978	,	0.07415768	,	0.308271404	,],
[	0.30514714	,	0.349499701	,	0.271199543	,	0.088176625	,]],
[[	0.087898418	,	0.275645643	,	0.299921746	,	0.256497302	,],
[	0.304154655	,	0.078548863	,	0.241706986	,	0.331292044	,],
[	0.377455742	,	0.293645558	,	0.070526818	,	0.303602029	,],
[	0.305365148	,	0.349989993	,	0.271875573	,	0.08583382	,]],
[[	0.084256554	,	0.270993057	,	0.295337889	,	0.251926519	,],
[	0.30292773	,	0.079390058	,	0.240784577	,	0.330006877	,],
[	0.379060242	,	0.295523288	,	0.074577001	,	0.305013901	,],
[	0.303169197	,	0.347651993	,	0.269422639	,	0.084920542	,]],
[[	0.088771541	,	0.272710043	,	0.297066299	,	0.25371165	,],
[	0.307192768	,	0.087027573	,	0.245584733	,	0.334071917	,],
[	0.379551023	,	0.295608665	,	0.071718445	,	0.305594221	,],
[	0.307219535	,	0.351439077	,	0.273228074	,	0.091470044	,]],
[[	0.085286527	,	0.272818735	,	0.297096366	,	0.253625686	,],
[	0.302790382	,	0.077915164	,	0.240416327	,	0.329888224	,],
[	0.377664953	,	0.293816617	,	0.071450712	,	0.303647919	,],
[	0.303839064	,	0.348377177	,	0.270207065	,	0.084752665	,]],
[[	0.086066597	,	0.27642804	,	0.300963148	,	0.252964046	,],
[	0.299309216	,	0.082479215	,	0.242946028	,	0.32778496	,],
[	0.377717818	,	0.29985093	,	0.079268306	,	0.304427186	,],
[	0.303298245	,	0.353970731	,	0.276250992	,	0.087556009	,]],
[[	0.086511192	,	0.279478048	,	0.303826159	,	0.255925751	,],
[	0.299965775	,	0.079192646	,	0.243049017	,	0.3284538	,],
[	0.373948025	,	0.295325925	,	0.070831169	,	0.300689291	,],
[	0.304307308	,	0.355269865	,	0.277688954	,	0.086262113	,]],
[[	0.084151079	,	0.275847216	,	0.300341462	,	0.25233308	,],
[	0.298102778	,	0.079343565	,	0.241482074	,	0.326589549	,],
[	0.374792517	,	0.296564583	,	0.074197536	,	0.301469583	,],
[	0.301959451	,	0.352864936	,	0.274948121	,	0.08494582	,]],
[[	0.089675245	,	0.278354671	,	0.302757573	,	0.254896583	,],
[	0.301364559	,	0.085923053	,	0.245291663	,	0.329749509	,],
[	0.376199839	,	0.297473217	,	0.0726259	,	0.303054897	,],
[	0.305220958	,	0.355888202	,	0.278239619	,	0.090730384	,]],
[[	0.084264654	,	0.276917969	,	0.301433683	,	0.253374603	,],
[	0.298487669	,	0.07816034	,	0.241413306	,	0.326864226	,],
[	0.373848465	,	0.295292548	,	0.071543422	,	0.300456809	,],
[	0.302681547	,	0.353497398	,	0.275834894	,	0.084759415	,]],
[[	0.085253646	,	0.276109337	,	0.300593265	,	0.252427042	,],
[	0.298916623	,	0.081482886	,	0.242382309	,	0.327258412	,],
[	0.376799329	,	0.298709059	,	0.077581713	,	0.303314573	,],
[	0.302828744	,	0.353562551	,	0.275721917	,	0.086621569	,]],
[[	0.087732934	,	0.280856364	,	0.305069725	,	0.257184632	,],
[	0.300978159	,	0.080077165	,	0.244005939	,	0.329398749	,],
[	0.374167424	,	0.295468755	,	0.070883207	,	0.300922839	,],
[	0.305461023	,	0.356395401	,	0.278906883	,	0.087119447	,]],
[[	0.08404917	,	0.276073868	,	0.300449672	,	0.252415343	,],
[	0.29801356	,	0.078791874	,	0.241244144	,	0.326458619	,],
[	0.37433068	,	0.295958011	,	0.07318667	,	0.300874992	,],
[	0.301791008	,	0.35266184	,	0.2749038	,	0.084364199	,]],
[[	0.088046987	,	0.277394338	,	0.301839483	,	0.253900689	,],
[	0.30094201	,	0.084903305	,	0.244719006	,	0.329276847	,],
[	0.377173543	,	0.298458054	,	0.07329681	,	0.303965929	,],
[	0.304532215	,	0.355127328	,	0.277511424	,	0.089372692	,]],
[[	0.085144898	,	0.278101167	,	0.302570487	,	0.25445823	,],
[	0.299034449	,	0.078510585	,	0.242010035	,	0.327419681	,],
[	0.373548611	,	0.295013931	,	0.070958097	,	0.300195355	,],
[	0.303315744	,	0.354173112	,	0.276425233	,	0.085059708	,]],
[[	0.084507198	,	0.280113265	,	0.304550497	,	0.250989397	,],
[	0.292086532	,	0.081203712	,	0.241885663	,	0.321462525	,],
[	0.369584581	,	0.297885261	,	0.076511075	,	0.297843128	,],
[	0.30006414	,	0.3563559	,	0.278615288	,	0.087345819	,]],
[[	0.085600784	,	0.283750504	,	0.307978561	,	0.254548125	,],
[	0.294302539	,	0.080172395	,	0.243672787	,	0.323549038	,],
[	0.368364972	,	0.295846929	,	0.071216145	,	0.296654254	,],
[	0.300400969	,	0.356956792	,	0.279427267	,	0.085089854	,]],
[[	0.082459108	,	0.279362911	,	0.303630503	,	0.250124228	,],
[	0.291147304	,	0.078491317	,	0.240629411	,	0.320402792	,],
[	0.367421683	,	0.295421899	,	0.072480617	,	0.295671956	,],
[	0.298323635	,	0.354964529	,	0.277187661	,	0.084229557	,]],
[[	0.087711755	,	0.281778597	,	0.306076516	,	0.252718713	,],
[	0.293772341	,	0.084383559	,	0.243817633	,	0.322959105	,],
[	0.371393972	,	0.298908943	,	0.073790322	,	0.299679048	,],
[	0.302028915	,	0.358432847	,	0.280731878	,	0.090518224	,]],
[[	0.083300978	,	0.281207178	,	0.305588707	,	0.251981497	,],
[	0.292554025	,	0.078773539	,	0.241829563	,	0.32169959	,],
[	0.367291825	,	0.294839233	,	0.07076406	,	0.295458192	,],
[	0.298982612	,	0.355410307	,	0.27776699	,	0.0839394	,]],
[[	0.083559118	,	0.279770383	,	0.304163022	,	0.250517814	,],
[	0.291867597	,	0.080480179	,	0.241519087	,	0.320988492	,],
[	0.368900717	,	0.297061142	,	0.07518876	,	0.297040425	,],
[	0.299347898	,	0.355672555	,	0.277958826	,	0.086178957	,]],
[[	0.086849464	,	0.28497196	,	0.309190991	,	0.255797464	,],
[	0.295722886	,	0.081281185	,	0.244901507	,	0.324842821	,],
[	0.368933775	,	0.296574713	,	0.071707161	,	0.297204886	,],
[	0.301588472	,	0.358060033	,	0.280519215	,	0.0859953	,]],
[[	0.082402523	,	0.279697473	,	0.303944278	,	0.250454485	,],
[	0.291581384	,	0.078581285	,	0.240975338	,	0.320875396	,],
[	0.367710796	,	0.295485961	,	0.072089881	,	0.295832857	,],
[	0.298401135	,	0.35496432	,	0.277255775	,	0.083891742	,]],
[[	0.086371547	,	0.281045664	,	0.305437492	,	0.25183525	,],
[	0.293063998	,	0.083046293	,	0.242966608	,	0.32217744	,],
[	0.371138541	,	0.299499285	,	0.078836032	,	0.299204208	,],
[	0.301175496	,	0.357565613	,	0.27988939	,	0.089186264	,]],
[[	0.084164838	,	0.282232479	,	0.306604752	,	0.253033594	,],
[	0.293384131	,	0.079360998	,	0.242580693	,	0.322496159	,],
[	0.367502156	,	0.295044102	,	0.070621277	,	0.295669311	,],
[	0.299508278	,	0.356057385	,	0.278298031	,	0.084188534	,]],
[[	0.084449567	,	0.285341301	,	0.308617617	,	0.251431874	,],
[	0.286887971	,	0.080158012	,	0.241440116	,	0.317392726	,],
[	0.364174931	,	0.298432421	,	0.074839147	,	0.293244501	,],
[	0.297555105	,	0.359739976	,	0.28158295	,	0.085742567	,]],
[[	0.08636674	,	0.289539629	,	0.312675262	,	0.255471395	,],
[	0.290619848	,	0.08749864	,	0.245812587	,	0.321010796	,],
[	0.36450109	,	0.298292819	,	0.07174157	,	0.29383172	,],
[	0.300431004	,	0.362784466	,	0.284724131	,	0.086275833	,]],
[[	0.082559447	,	0.28473231	,	0.307911214	,	0.25086547	,],
[	0.286771465	,	0.078658855	,	0.241235673	,	0.317372444	,],
[	0.362875665	,	0.296806049	,	0.071672577	,	0.291907361	,],
[	0.296811988	,	0.359113632	,	0.280907847	,	0.083646944	,]],
[[	0.087514651	,	0.286979003	,	0.310002226	,	0.252974614	,],
[	0.287979093	,	0.082723402	,	0.242821804	,	0.318452247	,],
[	0.3660998	,	0.300731018	,	0.078304874	,	0.295210568	,],
[	0.29951816	,	0.361784139	,	0.283532643	,	0.088747588	,]],
[[	0.083688445	,	0.286777764	,	0.310039398	,	0.252737115	,],
[	0.289005728	,	0.079736441	,	0.243104287	,	0.319464121	,],
[	0.363062545	,	0.296849446	,	0.070730114	,	0.292243713	,],
[	0.298192185	,	0.360521003	,	0.282198513	,	0.084180246	,]],
[[	0.083768007	,	0.284999613	,	0.308335541	,	0.251018936	,],
[	0.286663404	,	0.079581529	,	0.241079056	,	0.317139281	,],
[	0.363523521	,	0.29773036	,	0.073695934	,	0.292639469	,],
[	0.297351562	,	0.359654487	,	0.281306813	,	0.084986825	,]],
[[	0.087403348	,	0.290630205	,	0.313726977	,	0.256632193	,],
[	0.289768817	,	0.085999511	,	0.244751356	,	0.320125877	,],
[	0.365555084	,	0.299179798	,	0.072672399	,	0.294809522	,],
[	0.301394437	,	0.363754503	,	0.285720244	,	0.087190766	,]],
[[	0.082796993	,	0.28527888	,	0.308460994	,	0.251231392	,],
[	0.2870767	,	0.078523298	,	0.241332196	,	0.317597206	,],
[	0.362885501	,	0.296654706	,	0.07122933	,	0.29178418	,],
[	0.297134303	,	0.359602943	,	0.281225211	,	0.083688878	,]],
[[	0.086348497	,	0.286375528	,	0.309481281	,	0.252373227	,],
[	0.287369205	,	0.081497716	,	0.242058684	,	0.317820574	,],
[	0.365288192	,	0.299704065	,	0.076837245	,	0.294312767	,],
[	0.298658937	,	0.3609466	,	0.282675656	,	0.087428498	,]],
[[	0.084753576	,	0.287915824	,	0.311058037	,	0.253826408	,],
[	0.290024429	,	0.080742054	,	0.244099435	,	0.320374459	,],
[	0.363582778	,	0.297325302	,	0.070973741	,	0.292685189	,],
[	0.299088749	,	0.361284737	,	0.283091042	,	0.084852661	,]],
[[	0.086263205	,	0.296083911	,	0.318142238	,	0.254807204	,],
[	0.288030812	,	0.082526097	,	0.246560358	,	0.318509683	,],
[	0.369484709	,	0.307993129	,	0.078176941	,	0.29550197	,],
[	0.304864594	,	0.374084663	,	0.29262983	,	0.088437194	,]],
[[	0.088027859	,	0.30010754	,	0.321955784	,	0.258831579	,],
[	0.292219206	,	0.09008488	,	0.251250625	,	0.322537359	,],
[	0.368779689	,	0.30664237	,	0.073760593	,	0.294959295	,],
[	0.305349439	,	0.374644747	,	0.29345413	,	0.086423966	,]],
[[	0.084797144	,	0.295855759	,	0.317848191	,	0.254472258	,],
[	0.28791153	,	0.080920239	,	0.246214315	,	0.318402004	,],
[	0.36795171	,	0.306062408	,	0.074834572	,	0.293857068	,],
[	0.302976916	,	0.372376047	,	0.29094983	,	0.085336967	,]],
[[	0.089258337	,	0.29765703	,	0.319628565	,	0.256461025	,],
[	0.28919221	,	0.084917812	,	0.247958176	,	0.31952975	,],
[	0.371831975	,	0.310526047	,	0.082118467	,	0.297854185	,],
[	0.306891508	,	0.376152241	,	0.294691391	,	0.091752056	,]],
[[	0.085822205	,	0.297786477	,	0.319826851	,	0.256402398	,],
[	0.290470675	,	0.082622284	,	0.248536888	,	0.32080012	,],
[	0.367963756	,	0.305864609	,	0.073409601	,	0.293957813	,],
[	0.30389294	,	0.373069688	,	0.291732349	,	0.085243957	,]],
[[	0.085738461	,	0.296088603	,	0.318042258	,	0.254591715	,],
[	0.287813115	,	0.081859203	,	0.246207397	,	0.318158906	,],
[	0.368916435	,	0.307291027	,	0.077017539	,	0.294904339	,],
[	0.304147196	,	0.373376336	,	0.291935347	,	0.087245797	,]],
[[	0.089373105	,	0.301491751	,	0.323310041	,	0.26014351	,],
[	0.291252221	,	0.088436357	,	0.250133097	,	0.321416508	,],
[	0.369499906	,	0.307294895	,	0.074239352	,	0.29565691	,],
[	0.306571813	,	0.375883507	,	0.29463994	,	0.087432228	,]],
[[	0.08480078	,	0.296244794	,	0.318232485	,	0.254838734	,],
[	0.288475909	,	0.08111485	,	0.246638663	,	0.318884869	,],
[	0.367609843	,	0.30562568	,	0.073993351	,	0.293573118	,],
[	0.303115973	,	0.372629625	,	0.29113188	,	0.085071438	,]],
[[	0.087935226	,	0.296918316	,	0.318837944	,	0.255581283	,],
[	0.288749777	,	0.083877233	,	0.247332088	,	0.319134117	,],
[	0.370999014	,	0.309601177	,	0.080529945	,	0.29696464	,],
[	0.30594341	,	0.375120607	,	0.293671589	,	0.090226461	,]],
[[	0.086591957	,	0.298591558	,	0.320555367	,	0.257109392	,],
[	0.291166254	,	0.083096823	,	0.249224036	,	0.321421575	,],
[	0.368123983	,	0.306003943	,	0.073312183	,	0.29416373	,],
[	0.304562703	,	0.373835285	,	0.292450607	,	0.085743	,]],
[[	0.086977899	,	0.302818993	,	0.324190655	,	0.256253773	,],
[	0.283643366	,	0.082394382	,	0.24685355	,	0.314491469	,],
[	0.366456601	,	0.309879457	,	0.077755646	,	0.292202033	,],
[	0.304045183	,	0.378929351	,	0.29689755	,	0.088584777	,]],
[[	0.087295168	,	0.305498109	,	0.326813009	,	0.258919889	,],
[	0.287858733	,	0.089998986	,	0.251538042	,	0.318482656	,],
[	0.366687851	,	0.309566488	,	0.074806619	,	0.292731781	,],
[	0.304670384	,	0.379782293	,	0.297824714	,	0.086398993	,]],
[[	0.08474554	,	0.301797985	,	0.323136452	,	0.25531717	,],
[	0.283869055	,	0.081232272	,	0.246907514	,	0.314852119	,],
[	0.364700591	,	0.307872733	,	0.074360059	,	0.290526523	,],
[	0.302301084	,	0.377407936	,	0.295398371	,	0.085405381	,]],
[[	0.090214424	,	0.304281787	,	0.325723798	,	0.257829709	,],
[	0.284862192	,	0.084976776	,	0.248386744	,	0.315776864	,],
[	0.368206975	,	0.311925636	,	0.081021089	,	0.294044379	,],
[	0.306146156	,	0.381112809	,	0.299146391	,	0.09184011	,]],
[[	0.085125999	,	0.303262816	,	0.324554358	,	0.256681429	,],
[	0.286281378	,	0.082708251	,	0.249056807	,	0.317008428	,],
[	0.365289176	,	0.308299972	,	0.073786417	,	0.29115333	,],
[	0.303023718	,	0.377889996	,	0.295870319	,	0.084971493	,]],
[[	0.086012858	,	0.302173456	,	0.323607356	,	0.255621948	,],
[	0.283518763	,	0.081862092	,	0.2466984	,	0.314458536	,],
[	0.36569634	,	0.309170121	,	0.076589547	,	0.29146417	,],
[	0.303441318	,	0.378317357	,	0.296247203	,	0.087383917	,]],
[[	0.088345576	,	0.306657863	,	0.327781303	,	0.260086533	,],
[	0.287037726	,	0.088569393	,	0.250638239	,	0.317697608	,],
[	0.367592944	,	0.310424571	,	0.075585315	,	0.293450776	,],
[	0.30538247	,	0.380441905	,	0.29852427	,	0.08707313	,]],
[[	0.084701681	,	0.30209773	,	0.323289636	,	0.255474709	,],
[	0.284271293	,	0.081266123	,	0.247231909	,	0.315095149	,],
[	0.36472162	,	0.307801169	,	0.073998694	,	0.290521824	,],
[	0.302185268	,	0.377320579	,	0.295233898	,	0.084823047	,]],
[[	0.088871207	,	0.303702031	,	0.324963138	,	0.257198637	,],
[	0.28433758	,	0.083853272	,	0.247701385	,	0.31518321	,],
[	0.367639636	,	0.311207725	,	0.079788628	,	0.293335838	,],
[	0.305480979	,	0.380419535	,	0.298353433	,	0.090599759	,]],
[[	0.085891083	,	0.304053487	,	0.325499335	,	0.257511771	,],
[	0.287566178	,	0.083777906	,	0.250087277	,	0.318175726	,],
[	0.365871574	,	0.3086909	,	0.073946559	,	0.2916804	,],
[	0.303523186	,	0.378451428	,	0.296445477	,	0.085297002	,]],
[[	0.085597153	,	0.308120033	,	0.328493064	,	0.256036987	,],
[	0.280018306	,	0.082690836	,	0.248039046	,	0.311510997	,],
[	0.36114662	,	0.309588203	,	0.07386894	,	0.286824523	,],
[	0.302407736	,	0.382352028	,	0.299517598	,	0.085097318	,]],
[[	0.087905782	,	0.312296827	,	0.332440112	,	0.260246398	,],
[	0.280783385	,	0.086096125	,	0.249203849	,	0.312215754	,],
[	0.363657946	,	0.312664861	,	0.079128872	,	0.289382139	,],
[	0.305555327	,	0.385407106	,	0.302474052	,	0.090551167	,]],
[[	0.084356129	,	0.307751188	,	0.328079436	,	0.255679889	,],
[	0.282460059	,	0.084497748	,	0.250372039	,	0.314054588	,],
[	0.362570499	,	0.310910718	,	0.074382001	,	0.288487487	,],
[	0.303362811	,	0.383413179	,	0.300594945	,	0.085227491	,]],
[[	0.088268226	,	0.309444391	,	0.329704347	,	0.257287628	,],
[	0.279678672	,	0.083447858	,	0.247860901	,	0.311261662	,],
[	0.361445773	,	0.31014292	,	0.075139547	,	0.28725769	,],
[	0.302690493	,	0.3828357	,	0.299936828	,	0.086315416	,]],
[[	0.085085772	,	0.309685132	,	0.330005285	,	0.257525981	,],
[	0.282260963	,	0.08911915	,	0.25096765	,	0.313638294	,],
[	0.365760496	,	0.314080878	,	0.077190694	,	0.291815786	,],
[	0.30653617	,	0.386236287	,	0.303557948	,	0.088135046	,]],
[[	0.084945445	,	0.307748324	,	0.328119637	,	0.255613738	,],
[	0.280227521	,	0.082615727	,	0.248048594	,	0.31162941	,],
[	0.361528687	,	0.309957986	,	0.073881491	,	0.287262945	,],
[	0.302376724	,	0.382322177	,	0.299351081	,	0.084642358	,]],
[[	0.089239883	,	0.31365801	,	0.333790469	,	0.261564187	,],
[	0.280195405	,	0.084978593	,	0.248447769	,	0.311583874	,],
[	0.362971625	,	0.311910337	,	0.077881256	,	0.288653276	,],
[	0.304777702	,	0.384556825	,	0.301626091	,	0.089206971	,]],
[[	0.084207425	,	0.308127309	,	0.328321681	,	0.255977653	,],
[	0.28354569	,	0.08540805	,	0.251459382	,	0.314933546	,],
[	0.363005787	,	0.311296064	,	0.074614862	,	0.288845103	,],
[	0.303830185	,	0.383875983	,	0.301120756	,	0.085572663	,]],
[[	0.087191694	,	0.308866138	,	0.329031509	,	0.256657317	,],
[	0.279712821	,	0.082943027	,	0.247772402	,	0.311208935	,],
[	0.361633538	,	0.310201805	,	0.074708638	,	0.287244189	,],
[	0.302366051	,	0.382419264	,	0.29950159	,	0.085390183	,]],
[[	0.086353773	,	0.310825275	,	0.331111212	,	0.258758214	,],
[	0.281985275	,	0.08809744	,	0.250437128	,	0.313361804	,],
[	0.364937426	,	0.314118187	,	0.081191092	,	0.290547454	,],
[	0.306995703	,	0.386872682	,	0.303952077	,	0.092513393	,]],
[[	0.085747248	,	0.313492753	,	0.333185482	,	0.256176094	,],
[	0.275422743	,	0.082701502	,	0.248550673	,	0.307847269	,],
[	0.357181868	,	0.311434173	,	0.074151234	,	0.283479616	,],
[	0.30047125	,	0.386287984	,	0.303119048	,	0.084264083	,]],
[[	0.087938864	,	0.317496739	,	0.337077061	,	0.260369964	,],
[	0.275388479	,	0.085229149	,	0.248981004	,	0.307769273	,],
[	0.359098712	,	0.31382312	,	0.078795743	,	0.285350036	,],
[	0.304230856	,	0.389834546	,	0.306758536	,	0.090441807	,]],
[[	0.084292279	,	0.312973553	,	0.332653023	,	0.255765317	,],
[	0.278231798	,	0.084711841	,	0.25124195	,	0.310612298	,],
[	0.358665562	,	0.312722786	,	0.074858805	,	0.285034032	,],
[	0.301317713	,	0.387026879	,	0.304049534	,	0.08427042	,]],
[[	0.088398642	,	0.314667654	,	0.334320784	,	0.257438167	,],
[	0.274676181	,	0.082771356	,	0.247931401	,	0.30704635	,],
[	0.35721864	,	0.311636381	,	0.075060799	,	0.283401469	,],
[	0.301325336	,	0.387150038	,	0.303986144	,	0.086059985	,]],
[[	0.085170926	,	0.314918882	,	0.334651135	,	0.257633048	,],
[	0.277058527	,	0.08831508	,	0.250871351	,	0.309376509	,],
[	0.361257513	,	0.316340918	,	0.082438589	,	0.287470075	,],
[	0.304285151	,	0.389815444	,	0.306902624	,	0.086889042	,]],
[[	0.084814586	,	0.312841663	,	0.332630025	,	0.255513156	,],
[	0.276102211	,	0.083098252	,	0.249158694	,	0.308357602	,],
[	0.357308201	,	0.311466353	,	0.073940242	,	0.283674379	,],
[	0.30052986	,	0.386277088	,	0.303076915	,	0.083943708	,]],
[[	0.089362571	,	0.318888727	,	0.338453439	,	0.261692272	,],
[	0.275188981	,	0.084399209	,	0.248594143	,	0.307492611	,],
[	0.358290567	,	0.312952029	,	0.077485723	,	0.284501194	,],
[	0.303378082	,	0.388949111	,	0.305869568	,	0.089084897	,]],
[[	0.084164205	,	0.313329492	,	0.332920801	,	0.255961054	,],
[	0.279648442	,	0.092465117	,	0.253625108	,	0.311698293	,],
[	0.35938313	,	0.313351363	,	0.075270988	,	0.285806532	,],
[	0.301787133	,	0.387522617	,	0.304538452	,	0.084527738	,]],
[[	0.087286422	,	0.314124695	,	0.333746384	,	0.256774049	,],
[	0.274981145	,	0.082686397	,	0.24817241	,	0.307305473	,],
[	0.357002547	,	0.311310597	,	0.074551384	,	0.283267711	,],
[	0.301144895	,	0.3869371	,	0.303766876	,	0.085293958	,]],
[[	0.086318256	,	0.315913258	,	0.335632176	,	0.258631614	,],
[	0.27642453	,	0.08693586	,	0.250042186	,	0.308592168	,],
[	0.360362655	,	0.315306853	,	0.080896148	,	0.286530182	,],
[	0.305223342	,	0.390708165	,	0.307910399	,	0.087713932	,]],
[[	0.086308912	,	0.318989168	,	0.338372556	,	0.256434282	,],
[	0.26983885	,	0.082425062	,	0.248564358	,	0.30321607	,],
[	0.353656435	,	0.314006378	,	0.075900446	,	0.280866502	,],
[	0.300632156	,	0.39238305	,	0.308719081	,	0.086580861	,]],
[[	0.086900443	,	0.321811513	,	0.341128326	,	0.259399904	,],
[	0.270996173	,	0.08659684	,	0.250214865	,	0.304331629	,],
[	0.357667309	,	0.31877712	,	0.083393251	,	0.284994748	,],
[	0.304080754	,	0.395683965	,	0.312284494	,	0.087928786	,]],
[[	0.084376406	,	0.318157437	,	0.337537882	,	0.255668263	,],
[	0.271841209	,	0.083455856	,	0.250430376	,	0.305260739	,],
[	0.353522765	,	0.313552572	,	0.074164123	,	0.280723325	,],
[	0.300153951	,	0.392011299	,	0.308407136	,	0.084930848	,]],
[[	0.089441937	,	0.320687178	,	0.339925315	,	0.258127204	,],
[	0.269563139	,	0.083261786	,	0.248550538	,	0.303004434	,],
[	0.354952805	,	0.315419002	,	0.078373327	,	0.282024272	,],
[	0.302256328	,	0.394110935	,	0.310365223	,	0.089403537	,]],
[[	0.084703102	,	0.319686897	,	0.339086447	,	0.257154053	,],
[	0.272790056	,	0.090078619	,	0.252467572	,	0.306099426	,],
[	0.355498367	,	0.315471565	,	0.075391639	,	0.282864903	,],
[	0.301673183	,	0.393278933	,	0.309739697	,	0.085601622	,]],
[[	0.085249151	,	0.318300874	,	0.337816401	,	0.25580749	,],
[	0.270139521	,	0.082508879	,	0.248778189	,	0.303471961	,],
[	0.353192813	,	0.31350228	,	0.074764902	,	0.280413951	,],
[	0.300309638	,	0.392048709	,	0.30819372	,	0.085683291	,]],
[[	0.088559961	,	0.323526154	,	0.342819503	,	0.261082722	,],
[	0.270521878	,	0.085533633	,	0.249666334	,	0.303824673	,],
[	0.356758089	,	0.317709983	,	0.081786281	,	0.283966006	,],
[	0.304758218	,	0.396214743	,	0.312583413	,	0.092835629	,]],
[[	0.084315244	,	0.318553785	,	0.337930399	,	0.256099552	,],
[	0.272892603	,	0.084300507	,	0.251402553	,	0.306212357	,],
[	0.353968445	,	0.313896264	,	0.074190896	,	0.281096265	,],
[	0.300360974	,	0.392178245	,	0.308528594	,	0.084798103	,]],
[[	0.088174107	,	0.320019193	,	0.339372091	,	0.257519164	,],
[	0.269788268	,	0.083104576	,	0.248658563	,	0.303217506	,],
[	0.354359579	,	0.314808607	,	0.077340878	,	0.281437536	,],
[	0.30156913	,	0.393256818	,	0.309561342	,	0.087985022	,]],
[[	0.08559723	,	0.320617059	,	0.340008052	,	0.258025855	,],
[	0.272179521	,	0.088760579	,	0.251634741	,	0.305401275	,],
[	0.356185773	,	0.316034162	,	0.075881644	,	0.283537013	,],
[	0.302681187	,	0.394207085	,	0.310690884	,	0.086464878	,]],
[[	0.086522044	,	0.324993802	,	0.343870849	,	0.257150759	,],
[	0.265787698	,	0.083367617	,	0.250068602	,	0.299909744	,],
[	0.350595452	,	0.31644547	,	0.076265655	,	0.277863204	,],
[	0.300060269	,	0.397529445	,	0.313078357	,	0.086765171	,]],
[[	0.087199566	,	0.327953334	,	0.346715008	,	0.260208245	,],
[	0.266822301	,	0.087310732	,	0.251663624	,	0.300830349	,],
[	0.354451115	,	0.321118542	,	0.083710708	,	0.281893295	,],
[	0.302653913	,	0.399869827	,	0.315933702	,	0.08706428	,]],
[[	0.084708341	,	0.324279092	,	0.343110645	,	0.256385321	,],
[	0.268384974	,	0.085074719	,	0.252461168	,	0.30235943	,],
[	0.350719079	,	0.316217083	,	0.074913764	,	0.278020445	,],
[	0.299210735	,	0.396661065	,	0.312459835	,	0.084682128	,]],
[[	0.089711737	,	0.326580721	,	0.345428408	,	0.258730732	,],
[	0.265695375	,	0.084165046	,	0.250095783	,	0.299590434	,],
[	0.351727637	,	0.317877228	,	0.078765569	,	0.278957933	,],
[	0.302206635	,	0.399681367	,	0.315397234	,	0.089859626	,]],
[[	0.085145998	,	0.325874515	,	0.344819122	,	0.257885464	,],
[	0.268668919	,	0.090585818	,	0.253699053	,	0.302516356	,],
[	0.352776086	,	0.318131689	,	0.076097722	,	0.280055115	,],
[	0.300609868	,	0.39777858	,	0.313574721	,	0.085191178	,]],
[[	0.085649626	,	0.324474596	,	0.343376223	,	0.256516893	,],
[	0.266201811	,	0.08351532	,	0.250415565	,	0.300285233	,],
[	0.35024971	,	0.315982152	,	0.075368957	,	0.277401729	,],
[	0.299713421	,	0.397125203	,	0.312817767	,	0.085937121	,]],
[[	0.088298512	,	0.329145363	,	0.347788779	,	0.261295434	,],
[	0.26650446	,	0.086327848	,	0.251161085	,	0.300298718	,],
[	0.353834366	,	0.320275699	,	0.082277237	,	0.281044674	,],
[	0.303962005	,	0.401200206	,	0.317210727	,	0.088324761	,]],
[[	0.084489083	,	0.32455941	,	0.343362248	,	0.256628735	,],
[	0.269433914	,	0.085848264	,	0.253374806	,	0.303210401	,],
[	0.351004661	,	0.316533123	,	0.07498693	,	0.278324014	,],
[	0.299455481	,	0.396876503	,	0.312615662	,	0.084549845	,]],
[[	0.088484513	,	0.326081859	,	0.34481684	,	0.25809758	,],
[	0.265701261	,	0.083829849	,	0.250069552	,	0.299679689	,],
[	0.351497009	,	0.317543454	,	0.077792031	,	0.278650014	,],
[	0.30112419	,	0.398650391	,	0.314290142	,	0.088435736	,]],
[[	0.085797667	,	0.326606736	,	0.345479324	,	0.258669601	,],
[	0.268157639	,	0.089443055	,	0.253104357	,	0.301921131	,],
[	0.353391665	,	0.318753967	,	0.076645433	,	0.280764886	,],
[	0.301341885	,	0.398561581	,	0.314455188	,	0.085796049	,]],
[[	0.085863016	,	0.329529317	,	0.346691537	,	0.25696556	,],
[	0.26192316	,	0.083618405	,	0.250220907	,	0.2966652	,],
[	0.346872	,	0.317893713	,	0.075334339	,	0.274420331	,],
[	0.298640315	,	0.400936795	,	0.315464017	,	0.086188459	,]],
[[	0.087733862	,	0.333268992	,	0.350292502	,	0.261004747	,],
[	0.261159382	,	0.085357446	,	0.250006277	,	0.296018453	,],
[	0.349948644	,	0.321562051	,	0.081599579	,	0.277518146	,],
[	0.302411753	,	0.40427091	,	0.319319946	,	0.087833759	,]],
[[	0.08415657	,	0.328809635	,	0.345999683	,	0.256416508	,],
[	0.26537426	,	0.086364924	,	0.253541369	,	0.300019444	,],
[	0.34746295	,	0.318097477	,	0.074666744	,	0.27507027	,],
[	0.298011548	,	0.40022311	,	0.314999961	,	0.084423513	,]],
[[	0.0888247	,	0.330905473	,	0.347977172	,	0.258429576	,],
[	0.260998988	,	0.083519808	,	0.249492067	,	0.295769644	,],
[	0.347489442	,	0.318647006	,	0.077053571	,	0.275006608	,],
[	0.299963353	,	0.402211358	,	0.316864836	,	0.088528615	,]],
[[	0.085164681	,	0.330819018	,	0.348002658	,	0.258322973	,],
[	0.26257121	,	0.08812702	,	0.251620129	,	0.297251726	,],
[	0.35026917	,	0.320754308	,	0.076830429	,	0.27796729	,],
[	0.299939249	,	0.401832479	,	0.316646995	,	0.085545587	,]],
[[	0.085165697	,	0.329187361	,	0.346362453	,	0.256564723	,],
[	0.262832635	,	0.084220355	,	0.251010294	,	0.297599471	,],
[	0.346764599	,	0.317671512	,	0.074733363	,	0.274315837	,],
[	0.298307471	,	0.40056877	,	0.315093973	,	0.085469426	,]],
[[	0.088991651	,	0.334636091	,	0.351604377	,	0.262255562	,],
[	0.260935443	,	0.084601947	,	0.249584829	,	0.295682362	,],
[	0.348783389	,	0.320343126	,	0.079806392	,	0.276312542	,],
[	0.302849828	,	0.404829761	,	0.319516038	,	0.092570664	,]],
[[	0.084425237	,	0.329527884	,	0.346471324	,	0.256909768	,],
[	0.26493535	,	0.092035868	,	0.2541262	,	0.299424771	,],
[	0.348373861	,	0.318974641	,	0.075272947	,	0.275903433	,],
[	0.298275694	,	0.400640697	,	0.315199454	,	0.08442234	,]],
[[	0.087679198	,	0.330395755	,	0.347560823	,	0.257862972	,],
[	0.261568025	,	0.083637931	,	0.249886047	,	0.296236215	,],
[	0.347015531	,	0.318109046	,	0.076131728	,	0.274363358	,],
[	0.299504989	,	0.401752842	,	0.31633987	,	0.087609935	,]],
[[	0.086204411	,	0.331765671	,	0.349003606	,	0.259399989	,],
[	0.261894172	,	0.086882204	,	0.250786616	,	0.296564841	,],
[	0.35110999	,	0.322964973	,	0.083747607	,	0.27877661	,],
[	0.300726675	,	0.402731735	,	0.317568394	,	0.086246738	,]],
[[	0.084843658	,	0.334242145	,	0.352645351	,	0.257230501	,],
[	0.258354741	,	0.08516507	,	0.254876234	,	0.295165947	,],
[	0.342172849	,	0.319239958	,	0.075887844	,	0.270990196	,],
[	0.296893618	,	0.405305942	,	0.321022177	,	0.085320687	,]],
[[	0.086952253	,	0.338202612	,	0.35649201	,	0.261448281	,],
[	0.255886084	,	0.084789262	,	0.252811759	,	0.292767661	,],
[	0.34380864	,	0.321482882	,	0.080417363	,	0.272542261	,],
[	0.301093652	,	0.409354292	,	0.325125488	,	0.092235732	,]],
[[	0.083453854	,	0.333864157	,	0.352136908	,	0.256847972	,],
[	0.259151681	,	0.091551932	,	0.256672323	,	0.295857283	,],
[	0.343485869	,	0.320251839	,	0.076282387	,	0.272289342	,],
[	0.297090937	,	0.405549009	,	0.321256685	,	0.084490757	,]],
[[	0.087812806	,	0.335764383	,	0.354006877	,	0.258700077	,],
[	0.256492527	,	0.084064738	,	0.253247257	,	0.293407558	,],
[	0.342307487	,	0.319467608	,	0.076942558	,	0.270820632	,],
[	0.298090812	,	0.406600635	,	0.322295818	,	0.087561728	,]],
[[	0.084458573	,	0.335752018	,	0.354146487	,	0.258750616	,],
[	0.25657399	,	0.086845303	,	0.253851735	,	0.293483806	,],
[	0.345922511	,	0.323838099	,	0.083996608	,	0.27444597	,],
[	0.299374567	,	0.407507722	,	0.323336597	,	0.086105139	,]],
[[	0.084029408	,	0.333719133	,	0.35221392	,	0.256651336	,],
[	0.259213885	,	0.085799063	,	0.255612102	,	0.295901139	,],
[	0.342552335	,	0.319510201	,	0.075861177	,	0.271235139	,],
[	0.296807394	,	0.405110745	,	0.320713304	,	0.084818838	,]],
[[	0.088324774	,	0.339677508	,	0.35780979	,	0.262803789	,],
[	0.255999163	,	0.084490029	,	0.252824973	,	0.292756304	,],
[	0.343260997	,	0.320799322	,	0.07923819	,	0.27187705	,],
[	0.300231264	,	0.408552745	,	0.324155894	,	0.090779725	,]],
[[	0.083641732	,	0.334208173	,	0.352536566	,	0.257243603	,],
[	0.258587848	,	0.090368394	,	0.256002139	,	0.295233786	,],
[	0.344878588	,	0.321607866	,	0.077479228	,	0.27357744	,],
[	0.297430712	,	0.405813163	,	0.321596471	,	0.084671155	,]],
[[	0.086591409	,	0.335034245	,	0.353338956	,	0.257941715	,],
[	0.257285905	,	0.084352393	,	0.253837984	,	0.294112846	,],
[	0.341955192	,	0.31905113	,	0.076187788	,	0.270534521	,],
[	0.297512693	,	0.405988985	,	0.321579827	,	0.086368198	,]],
[[	0.085399649	,	0.336657764	,	0.355042988	,	0.259722583	,],
[	0.256306584	,	0.085882287	,	0.253317485	,	0.293003928	,],
[	0.345157133	,	0.322991233	,	0.082469148	,	0.273747767	,],
[	0.300474684	,	0.408519932	,	0.324446702	,	0.087197064	,]],
[[	0.085559949	,	0.34111938	,	0.358076433	,	0.258714017	,],
[	0.255755797	,	0.086974326	,	0.256023048	,	0.292911864	,],
[	0.340766578	,	0.323277127	,	0.076151351	,	0.269116988	,],
[	0.298158267	,	0.412342391	,	0.326111014	,	0.085573817	,]],
[[	0.087552944	,	0.345054613	,	0.361909871	,	0.262843506	,],
[	0.252478889	,	0.085765098	,	0.253234573	,	0.289663668	,],
[	0.341917968	,	0.32504969	,	0.080113975	,	0.270239682	,],
[	0.302151914	,	0.416200901	,	0.329986242	,	0.092112319	,]],
[[	0.083915715	,	0.340520033	,	0.357366588	,	0.258203677	,],
[	0.255440371	,	0.092018432	,	0.256790827	,	0.292320416	,],
[	0.342802116	,	0.325134123	,	0.077240245	,	0.271267854	,],
[	0.298644591	,	0.412744053	,	0.326533898	,	0.084990272	,]],
[[	0.088414171	,	0.342567673	,	0.359346222	,	0.260077613	,],
[	0.253736416	,	0.085608274	,	0.254245029	,	0.290862731	,],
[	0.340581857	,	0.323295702	,	0.076899514	,	0.268856485	,],
[	0.299238561	,	0.413615232	,	0.327292239	,	0.087618977	,]],
[[	0.084858693	,	0.342448065	,	0.359417844	,	0.260188281	,],
[	0.252978403	,	0.087535702	,	0.254102921	,	0.290274239	,],
[	0.343911999	,	0.327336686	,	0.083579064	,	0.272172112	,],
[	0.301142626	,	0.414933215	,	0.32893023	,	0.086912783	,]],
[[	0.084777108	,	0.340729367	,	0.357680015	,	0.258314891	,],
[	0.256654538	,	0.087635804	,	0.256872898	,	0.293591666	,],
[	0.34130925	,	0.323719969	,	0.076353538	,	0.269649898	,],
[	0.298267136	,	0.412417101	,	0.3261168	,	0.08516673	,]],
[[	0.088982117	,	0.346499968	,	0.363220818	,	0.264334796	,],
[	0.252717911	,	0.085429115	,	0.253337515	,	0.289734861	,],
[	0.341446512	,	0.324416926	,	0.078991105	,	0.269583684	,],
[	0.301264903	,	0.415300831	,	0.32907441	,	0.090682873	,]],
[[	0.084020918	,	0.341051837	,	0.357882389	,	0.25860272	,],
[	0.254592113	,	0.09062617	,	0.255876795	,	0.291523429	,],
[	0.343705003	,	0.325928323	,	0.078107118	,	0.272093546	,],
[	0.298794483	,	0.41295984	,	0.326791843	,	0.084998754	,]],
[[	0.087031144	,	0.341771294	,	0.358562582	,	0.259253996	,],
[	0.254550063	,	0.08593866	,	0.254828654	,	0.291454199	,],
[	0.340816075	,	0.32335003	,	0.076486674	,	0.268984242	,],
[	0.298688475	,	0.41290879	,	0.326613381	,	0.086569879	,]],
[[	0.085942312	,	0.343452842	,	0.360404544	,	0.261182136	,],
[	0.252704501	,	0.086668721	,	0.253610325	,	0.28977475	,],
[	0.343122248	,	0.326508234	,	0.082271437	,	0.271311105	,],
[	0.302165183	,	0.415901196	,	0.329954202	,	0.087891473	,]],
[[	0.085419773	,	0.346290948	,	0.363044619	,	0.259025554	,],
[	0.251645155	,	0.087507401	,	0.257661544	,	0.289871175	,],
[	0.336812137	,	0.325184287	,	0.07638189	,	0.265659737	,],
[	0.297264755	,	0.416790266	,	0.330644547	,	0.085925611	,]],
[[	0.087172185	,	0.350037474	,	0.366590711	,	0.263025304	,],
[	0.248426997	,	0.086301836	,	0.254927996	,	0.286753508	,],
[	0.337813582	,	0.326944976	,	0.080397188	,	0.266567738	,],
[	0.301668689	,	0.421029096	,	0.334981489	,	0.092939726	,]],
[[	0.083723736	,	0.345656548	,	0.362446822	,	0.258484477	,],
[	0.251208754	,	0.092508013	,	0.258500261	,	0.289374779	,],
[	0.338819569	,	0.326943651	,	0.077458993	,	0.267624937	,],
[	0.297367431	,	0.416911774	,	0.330795547	,	0.084817523	,]],
[[	0.088427731	,	0.347758851	,	0.364383146	,	0.260460305	,],
[	0.249386067	,	0.085907447	,	0.255646684	,	0.28766548	,],
[	0.336539167	,	0.325073843	,	0.077073562	,	0.265298407	,],
[	0.298575982	,	0.418106483	,	0.331993795	,	0.088153545	,]],
[[	0.084529421	,	0.347386542	,	0.364170137	,	0.26032865	,],
[	0.248677211	,	0.087937438	,	0.255527415	,	0.286966677	,],
[	0.339805167	,	0.329186029	,	0.083901109	,	0.268608372	,],
[	0.299705057	,	0.41888198	,	0.332908956	,	0.086525683	,]],
[[	0.084507308	,	0.345731149	,	0.362537968	,	0.258498496	,],
[	0.252906914	,	0.088588791	,	0.258854855	,	0.291005099	,],
[	0.336979797	,	0.325321971	,	0.07618341	,	0.265761354	,],
[	0.297337416	,	0.416733446	,	0.330598657	,	0.085313897	,]],
[[	0.088405399	,	0.351208015	,	0.367722801	,	0.264227286	,],
[	0.248784442	,	0.086168462	,	0.25517275	,	0.286990458	,],
[	0.337467028	,	0.326357995	,	0.079398964	,	0.266211098	,],
[	0.30055173	,	0.419901058	,	0.333754811	,	0.091312211	,]],
[[	0.083792191	,	0.345982009	,	0.362599547	,	0.258859354	,],
[	0.250192581	,	0.090960706	,	0.25717754	,	0.288264039	,],
[	0.339517029	,	0.32753534	,	0.078070404	,	0.268368948	,],
[	0.297886826	,	0.417355671	,	0.331252653	,	0.085165888	,]],
[[	0.08713894	,	0.347039163	,	0.36373658	,	0.259797831	,],
[	0.250223386	,	0.086435312	,	0.256379927	,	0.288517701	,],
[	0.336452739	,	0.324954839	,	0.076527878	,	0.265164612	,],
[	0.297985973	,	0.41743444	,	0.331369262	,	0.087047311	,]],
[[	0.08585115	,	0.348495161	,	0.365182918	,	0.261367892	,],
[	0.248474041	,	0.087074127	,	0.255125307	,	0.28663058	,],
[	0.339217185	,	0.328455588	,	0.082570478	,	0.267941627	,],
[	0.300486754	,	0.419621823	,	0.333735423	,	0.087206224	,]],
[[	0.098644095	,	0.225274434	,	0.264359487	,	0.279878537	,],
[	0.375837302	,	0.079176697	,	0.25538391	,	0.40329385	,],
[	0.442562708	,	0.290603621	,	0.071567271	,	0.366926863	,],
[	0.332330953	,	0.309787493	,	0.236145426	,	0.092545743	,]],
[[	0.092905439	,	0.223561796	,	0.262472542	,	0.277521588	,],
[	0.373839336	,	0.078864745	,	0.253336852	,	0.401343097	,],
[	0.44355776	,	0.291596214	,	0.07453783	,	0.367858247	,],
[	0.336431529	,	0.313584149	,	0.240129652	,	0.099354969	,]],
[[	0.094496238	,	0.222819693	,	0.261865473	,	0.277169363	,],
[	0.377573748	,	0.085669882	,	0.257602713	,	0.405200182	,],
[	0.444735583	,	0.292937345	,	0.07355412	,	0.369213877	,],
[	0.332787772	,	0.31022531	,	0.236681073	,	0.091911396	,]],
[[	0.095321521	,	0.226897335	,	0.265659427	,	0.280466068	,],
[	0.374393238	,	0.078237802	,	0.253862997	,	0.401899069	,],
[	0.442105328	,	0.290004894	,	0.07169747	,	0.366343333	,],
[	0.333547627	,	0.310635982	,	0.237133559	,	0.094631909	,]],
[[	0.092389087	,	0.222430047	,	0.261473211	,	0.276447239	,],
[	0.375109409	,	0.081273215	,	0.254784955	,	0.402740663	,],
[	0.445332949	,	0.293573632	,	0.077666075	,	0.369625352	,],
[	0.334864283	,	0.31258035	,	0.239151745	,	0.093442278	,]],
[[	0.097186113	,	0.224295345	,	0.263466114	,	0.278795107	,],
[	0.376820582	,	0.080110196	,	0.256171884	,	0.404107668	,],
[	0.443081731	,	0.291021398	,	0.071875785	,	0.367476651	,],
[	0.332406368	,	0.309891186	,	0.236369465	,	0.092120195	,]],
[[	0.093322042	,	0.22425958	,	0.263019546	,	0.27806938	,],
[	0.374089356	,	0.078712197	,	0.25351862	,	0.401515154	,],
[	0.442991895	,	0.29091757	,	0.073454965	,	0.367235272	,],
[	0.335694935	,	0.312790102	,	0.239271922	,	0.097797255	,]],
[[	0.093646449	,	0.22262901	,	0.261557824	,	0.276833448	,],
[	0.376607725	,	0.084213702	,	0.25649126	,	0.404282611	,],
[	0.445935185	,	0.294048983	,	0.074553525	,	0.370350574	,],
[	0.333398806	,	0.310838461	,	0.237246992	,	0.09222203	,]],
[[	0.096633312	,	0.228239942	,	0.266989701	,	0.281715169	,],
[	0.37518594	,	0.078703297	,	0.254488797	,	0.402555416	,],
[	0.442745462	,	0.290494896	,	0.071821567	,	0.366906561	,],
[	0.333045723	,	0.310253682	,	0.23662759	,	0.093586521	,]],
[[	0.092248892	,	0.222563036	,	0.261567511	,	0.276488267	,],
[	0.374624489	,	0.080264686	,	0.254218644	,	0.402127277	,],
[	0.444862693	,	0.293016992	,	0.076474113	,	0.369044745	,],
[	0.336200514	,	0.314012159	,	0.240520636	,	0.094646908	,]],
[[	0.09383034	,	0.230690472	,	0.26854757	,	0.273830732	,],
[	0.359678161	,	0.08088428	,	0.251636373	,	0.388976486	,],
[	0.428223801	,	0.289340158	,	0.075088737	,	0.355408947	,],
[	0.323895015	,	0.314325879	,	0.241779334	,	0.091472129	,]],
[[	0.090776133	,	0.231232948	,	0.268805654	,	0.273724689	,],
[	0.361293751	,	0.079473351	,	0.252930974	,	0.390301415	,],
[	0.427540787	,	0.288424667	,	0.071119645	,	0.354722542	,],
[	0.322659897	,	0.312889583	,	0.240193785	,	0.091569589	,]],
[[	0.090743334	,	0.229019443	,	0.266827904	,	0.272032813	,],
[	0.358371431	,	0.078062357	,	0.250148356	,	0.38767734	,],
[	0.426441592	,	0.287223414	,	0.071467652	,	0.35347902	,],
[	0.326837243	,	0.316640214	,	0.244118935	,	0.09838171	,]],
[[	0.094358968	,	0.235232448	,	0.272883093	,	0.277471963	,],
[	0.361593673	,	0.084083664	,	0.253584414	,	0.390825935	,],
[	0.430790954	,	0.291889946	,	0.074462026	,	0.358104868	,],
[	0.32260827	,	0.312819822	,	0.240196295	,	0.090553374	,]],
[[	0.089463359	,	0.229316809	,	0.26714414	,	0.272035794	,],
[	0.359322668	,	0.077836528	,	0.250925557	,	0.388383913	,],
[	0.426399321	,	0.287049485	,	0.070280397	,	0.353352245	,],
[	0.324264301	,	0.314099682	,	0.241337197	,	0.093933947	,]],
[[	0.092940009	,	0.230158963	,	0.268159042	,	0.273260283	,],
[	0.35924072	,	0.079808493	,	0.251029503	,	0.388470993	,],
[	0.428139592	,	0.289037792	,	0.074320526	,	0.355097848	,],
[	0.324831553	,	0.315265741	,	0.2427225	,	0.092239912	,]],
[[	0.091409471	,	0.231978498	,	0.26949163	,	0.274465049	,],
[	0.362373148	,	0.080529484	,	0.253938695	,	0.391284056	,],
[	0.428229212	,	0.289107857	,	0.071824281	,	0.355410135	,],
[	0.322503929	,	0.312655687	,	0.239969031	,	0.09099794	,]],
[[	0.089913036	,	0.228797449	,	0.266504894	,	0.271577745	,],
[	0.358641635	,	0.077930188	,	0.250280072	,	0.387832711	,],
[	0.426552904	,	0.287359008	,	0.071314664	,	0.3535668	,],
[	0.326095937	,	0.315970423	,	0.243323665	,	0.096880544	,]],
[[	0.095814186	,	0.236700507	,	0.274205303	,	0.278864428	,],
[	0.360442411	,	0.082402031	,	0.252428506	,	0.389663935	,],
[	0.430349891	,	0.291657391	,	0.078030502	,	0.357544573	,],
[	0.322920961	,	0.313227696	,	0.240500229	,	0.090562429	,]],
[[	0.089677781	,	0.229853699	,	0.26741826	,	0.272412876	,],
[	0.360160259	,	0.078457322	,	0.251787196	,	0.389258711	,],
[	0.426874106	,	0.287653517	,	0.070607495	,	0.353922852	,],
[	0.323296906	,	0.313240417	,	0.240475631	,	0.092563523	,]],
[[	0.091453674	,	0.237200013	,	0.273201919	,	0.268723577	,],
[	0.344088258	,	0.077292651	,	0.247422503	,	0.374411011	,],
[	0.413517669	,	0.285793472	,	0.07204244	,	0.342416534	,],
[	0.317290232	,	0.318744674	,	0.24783071	,	0.090042511	,]],
[[	0.089261622	,	0.238375035	,	0.274121452	,	0.269353484	,],
[	0.348004642	,	0.084343003	,	0.251835558	,	0.378438253	,],
[	0.415610071	,	0.287852495	,	0.071870782	,	0.344809481	,],
[	0.31479663	,	0.316141246	,	0.24511338	,	0.088648879	,]],
[[	0.088365756	,	0.235827569	,	0.271732345	,	0.267156016	,],
[	0.344785074	,	0.076742867	,	0.247837356	,	0.37493653	,],
[	0.412726385	,	0.284812008	,	0.069862982	,	0.341653652	,],
[	0.317470604	,	0.318517403	,	0.247700242	,	0.093861119	,]],
[[	0.09304264	,	0.242816683	,	0.278388064	,	0.27344485	,],
[	0.34544193	,	0.0797206	,	0.248791838	,	0.375819034	,],
[	0.415539916	,	0.288121935	,	0.075433343	,	0.344608219	,],
[	0.315448217	,	0.316885069	,	0.245787657	,	0.088389585	,]],
[[	0.08762723	,	0.236457902	,	0.2722966	,	0.267526583	,],
[	0.347381816	,	0.078625675	,	0.250377961	,	0.377463704	,],
[	0.413854262	,	0.285912447	,	0.0701279	,	0.342747703	,],
[	0.315216405	,	0.316479156	,	0.245255701	,	0.089824181	,]],
[[	0.090470649	,	0.236781819	,	0.272725499	,	0.26811882	,],
[	0.344130061	,	0.076977567	,	0.247382332	,	0.374520299	,],
[	0.413167632	,	0.285387231	,	0.071279859	,	0.342084095	,],
[	0.318466484	,	0.320065424	,	0.249149332	,	0.091058828	,]],
[[	0.090139268	,	0.23942431	,	0.275110578	,	0.270231258	,],
[	0.347170576	,	0.082818578	,	0.250784844	,	0.377469817	,],
[	0.416998713	,	0.289193698	,	0.073032244	,	0.34610454	,],
[	0.314497348	,	0.315889428	,	0.244757483	,	0.087990144	,]],
[[	0.087992699	,	0.235686376	,	0.271550458	,	0.2669363	,],
[	0.345455491	,	0.077188366	,	0.248471514	,	0.375563304	,],
[	0.413331503	,	0.285302214	,	0.069984151	,	0.342086193	,],
[	0.316483034	,	0.317541988	,	0.246539104	,	0.092272438	,]],
[[	0.093592232	,	0.238456571	,	0.274313781	,	0.269978998	,],
[	0.34482794	,	0.078668043	,	0.248042114	,	0.375119349	,],
[	0.414598882	,	0.286968273	,	0.073980196	,	0.343537017	,],
[	0.316077861	,	0.317447379	,	0.246441449	,	0.08892606	,]],
[[	0.088017721	,	0.237013367	,	0.27275292	,	0.267908859	,],
[	0.348330866	,	0.079634244	,	0.251444886	,	0.378391767	,],
[	0.414274052	,	0.286367796	,	0.070503098	,	0.343145263	,],
[	0.314635759	,	0.31585922	,	0.24483943	,	0.088859158	,]],
[[	0.089366194	,	0.245999276	,	0.280992294	,	0.266630566	,],
[	0.333083944	,	0.076012977	,	0.247092128	,	0.365338011	,],
[	0.402698176	,	0.285494372	,	0.069682413	,	0.333019604	,],
[	0.314260521	,	0.327395801	,	0.256282036	,	0.090750264	,]],
[[	0.087127568	,	0.246797849	,	0.281732473	,	0.267050296	,],
[	0.334005527	,	0.079288	,	0.248240367	,	0.366291704	,],
[	0.405163722	,	0.288376143	,	0.074939931	,	0.335596506	,],
[	0.309596193	,	0.322606954	,	0.251380731	,	0.0869403	,]],
[[	0.086079846	,	0.244124525	,	0.279080123	,	0.264690115	,],
[	0.335788571	,	0.077882077	,	0.249452185	,	0.367741064	,],
[	0.403813755	,	0.286661616	,	0.070117577	,	0.334214591	,],
[	0.311606524	,	0.324325539	,	0.253171594	,	0.091059707	,]],
[[	0.091405632	,	0.251695401	,	0.286421378	,	0.271542275	,],
[	0.332738003	,	0.076391024	,	0.246632074	,	0.36494503	,],
[	0.402990006	,	0.285947195	,	0.07104706	,	0.333396437	,],
[	0.311545396	,	0.324492245	,	0.253220896	,	0.088166503	,]],
[[	0.085751971	,	0.245217083	,	0.280129186	,	0.265465132	,],
[	0.335844033	,	0.082431882	,	0.250181122	,	0.368048388	,],
[	0.406665733	,	0.289558575	,	0.072602742	,	0.337154622	,],
[	0.309861396	,	0.322676589	,	0.251229909	,	0.087743628	,]],
[[	0.088274502	,	0.245364509	,	0.280419681	,	0.26592572	,],
[	0.333837187	,	0.076314971	,	0.247619889	,	0.365903634	,],
[	0.402959962	,	0.285665057	,	0.069677236	,	0.333199862	,],
[	0.314000886	,	0.326428517	,	0.255157362	,	0.094454061	,]],
[[	0.088263406	,	0.248089929	,	0.282948697	,	0.268200434	,],
[	0.33362536	,	0.078267049	,	0.247738836	,	0.36580043	,],
[	0.404542778	,	0.287630565	,	0.073677062	,	0.334759089	,],
[	0.310028357	,	0.323005263	,	0.251738837	,	0.08694266	,]],
[[	0.085685326	,	0.244170225	,	0.279057155	,	0.264535738	,],
[	0.336855534	,	0.078905108	,	0.250632065	,	0.368754956	,],
[	0.404774273	,	0.287523815	,	0.070815735	,	0.335000438	,],
[	0.310649748	,	0.323461981	,	0.252197266	,	0.089603824	,]],
[[	0.091613084	,	0.247198631	,	0.282080946	,	0.267808006	,],
[	0.332964607	,	0.076269522	,	0.246800516	,	0.365022584	,],
[	0.403056588	,	0.285863243	,	0.070500019	,	0.333242807	,],
[	0.312565195	,	0.325558022	,	0.254385881	,	0.089026464	,]],
[[	0.086076507	,	0.245646709	,	0.280642611	,	0.265860514	,],
[	0.33501939	,	0.080945635	,	0.249307726	,	0.367126268	,],
[	0.4066871	,	0.290083288	,	0.077330719	,	0.337149472	,],
[	0.309817434	,	0.322687742	,	0.251213255	,	0.087345036	,]],
[[	0.087819419	,	0.254356478	,	0.288002524	,	0.264451036	,],
[	0.324003869	,	0.077574768	,	0.248232993	,	0.357562057	,],
[	0.39319416	,	0.286895614	,	0.069724722	,	0.324563802	,],
[	0.30969814	,	0.333583571	,	0.261910192	,	0.093754692	,]],
[[	0.086442205	,	0.255852593	,	0.289341017	,	0.26566657	,],
[	0.322374992	,	0.077924479	,	0.246958795	,	0.356099427	,],
[	0.393626165	,	0.287613727	,	0.072144159	,	0.325008117	,],
[	0.305687356	,	0.330043435	,	0.25829941	,	0.086363005	,]],
[[	0.085123575	,	0.252995724	,	0.286640833	,	0.26310183	,],
[	0.326585005	,	0.085342492	,	0.251644396	,	0.360163057	,],
[	0.3957637	,	0.289552607	,	0.071781815	,	0.327208052	,],
[	0.306537537	,	0.33073175	,	0.259046552	,	0.089050861	,]],
[[	0.090892443	,	0.260626707	,	0.294094129	,	0.27035437	,],
[	0.322587299	,	0.07671203	,	0.246837326	,	0.356214627	,],
[	0.392591316	,	0.286379908	,	0.069748218	,	0.323892017	,],
[	0.308300779	,	0.332573172	,	0.260873212	,	0.088346476	,]],
[[	0.085009967	,	0.254121489	,	0.287722648	,	0.263969661	,],
[	0.323678355	,	0.080368807	,	0.248300511	,	0.357295308	,],
[	0.395422706	,	0.289701429	,	0.075453826	,	0.326836956	,],
[	0.305505331	,	0.329683847	,	0.257929962	,	0.086722909	,]],
[[	0.086791338	,	0.253722941	,	0.287346113	,	0.263718576	,],
[	0.324679883	,	0.077961054	,	0.248842476	,	0.358192731	,],
[	0.393843311	,	0.287446016	,	0.070072618	,	0.325195865	,],
[	0.308604259	,	0.332467944	,	0.260753105	,	0.092000241	,]],
[[	0.08762746	,	0.257222331	,	0.290514489	,	0.266868195	,],
[	0.322052751	,	0.076942823	,	0.246419335	,	0.355719401	,],
[	0.393331662	,	0.287262118	,	0.071506569	,	0.324628086	,],
[	0.306418461	,	0.330907862	,	0.259093655	,	0.086653601	,]],
[[	0.084771982	,	0.253050088	,	0.286574524	,	0.262980013	,],
[	0.325683316	,	0.083538473	,	0.25047743	,	0.359167125	,],
[	0.396921598	,	0.290659649	,	0.072939156	,	0.328334321	,],
[	0.306125802	,	0.33023496	,	0.258488258	,	0.088108002	,]],
[[	0.09019047	,	0.255682515	,	0.289263222	,	0.265952904	,],
[	0.3229305	,	0.07672812	,	0.247024753	,	0.356396656	,],
[	0.392679891	,	0.286380084	,	0.069538688	,	0.323976708	,],
[	0.309330206	,	0.333573	,	0.261876831	,	0.089232473	,]],
[[	0.085578047	,	0.254830457	,	0.288342893	,	0.264568735	,],
[	0.323332903	,	0.079343522	,	0.247804047	,	0.356810206	,],
[	0.394758763	,	0.288849689	,	0.07419337	,	0.326132914	,],
[	0.305472316	,	0.329648626	,	0.257849753	,	0.086211491	,]],
[[	0.085538073	,	0.261577161	,	0.293679712	,	0.260709578	,],
[	0.313655887	,	0.083948452	,	0.249292711	,	0.348261057	,],
[	0.385596465	,	0.292880947	,	0.078832271	,	0.318983996	,],
[	0.301587401	,	0.336917996	,	0.265267434	,	0.088108361	,]],
[[	0.08564162	,	0.264220191	,	0.296123398	,	0.263041875	,],
[	0.311295008	,	0.077336037	,	0.24610697	,	0.345818753	,],
[	0.381290756	,	0.287696234	,	0.070158191	,	0.314582133	,],
[	0.303327576	,	0.338943976	,	0.26746236	,	0.087574187	,]],
[[	0.083637847	,	0.260948481	,	0.2928821	,	0.259870256	,],
[	0.310765902	,	0.079121287	,	0.245989314	,	0.345365882	,],
[	0.382653908	,	0.289413957	,	0.073897912	,	0.315848233	,],
[	0.300181575	,	0.335799131	,	0.264142046	,	0.08553386	,]],
[[	0.08886189	,	0.26330905	,	0.29529598	,	0.262560954	,],
[	0.314184139	,	0.079768431	,	0.249005666	,	0.348629129	,],
[	0.383116765	,	0.289536085	,	0.071198841	,	0.316343098	,],
[	0.303609503	,	0.338887146	,	0.267307398	,	0.091269757	,]],
[[	0.083688415	,	0.262044114	,	0.294084331	,	0.260912099	,],
[	0.310574945	,	0.077346049	,	0.245496177	,	0.345026213	,],
[	0.381399657	,	0.287954607	,	0.070997042	,	0.314623658	,],
[	0.301448673	,	0.336924121	,	0.265269497	,	0.085729756	,]],
[[	0.084663619	,	0.261099944	,	0.29323847	,	0.26013536	,],
[	0.312584262	,	0.082196473	,	0.247995392	,	0.347099315	,],
[	0.384550785	,	0.291682172	,	0.077163022	,	0.317880685	,],
[	0.300681339	,	0.33608408	,	0.264343325	,	0.086840529	,]],
[[	0.086919994	,	0.265635232	,	0.297392767	,	0.264336505	,],
[	0.311711241	,	0.077626772	,	0.246529856	,	0.346120832	,],
[	0.3818692	,	0.288236673	,	0.07042038	,	0.315052227	,],
[	0.30429375	,	0.339871012	,	0.268345273	,	0.088464224	,]],
[[	0.083410542	,	0.261175592	,	0.293110834	,	0.260001535	,],
[	0.310788637	,	0.078477309	,	0.24581115	,	0.345255971	,],
[	0.382175298	,	0.288842277	,	0.072810147	,	0.315274409	,],
[	0.300179566	,	0.335704035	,	0.26409197	,	0.085167653	,]],
[[	0.087641063	,	0.262711694	,	0.294666294	,	0.261742432	,],
[	0.315176758	,	0.08069138	,	0.249968519	,	0.349475971	,],
[	0.38384108	,	0.290265996	,	0.0719264	,	0.317200376	,],
[	0.302502859	,	0.337805392	,	0.266169603	,	0.089550805	,]],
[[	0.084543412	,	0.263097955	,	0.295067908	,	0.261803788	,],
[	0.310718635	,	0.077150221	,	0.245562206	,	0.34513504	,],
[	0.381191455	,	0.287784377	,	0.070616011	,	0.314412351	,],
[	0.301940643	,	0.337471795	,	0.265826988	,	0.086120679	,]],
[[	0.084132249	,	0.270300182	,	0.301612715	,	0.259265865	,],
[	0.301674339	,	0.080919054	,	0.247521859	,	0.337565065	,],
[	0.373663067	,	0.291335135	,	0.074850723	,	0.308216101	,],
[	0.29673852	,	0.343829289	,	0.271224971	,	0.086490063	,]],
[[	0.085438666	,	0.273962715	,	0.305060519	,	0.262674769	,],
[	0.303012946	,	0.078952222	,	0.248384052	,	0.338813439	,],
[	0.373512632	,	0.290588002	,	0.071179119	,	0.308032834	,],
[	0.300262202	,	0.347540631	,	0.275063887	,	0.088176505	,]],
[[	0.082384292	,	0.26970185	,	0.300934691	,	0.258582973	,],
[	0.300474206	,	0.078135301	,	0.24617882	,	0.336489366	,],
[	0.371790922	,	0.289188995	,	0.071207427	,	0.306237576	,],
[	0.296034854	,	0.343408482	,	0.270773521	,	0.084694985	,]],
[[	0.08767216	,	0.27229091	,	0.303504449	,	0.261278006	,],
[	0.303548911	,	0.084154916	,	0.249783189	,	0.3394701	,],
[	0.37545698	,	0.293471121	,	0.078082855	,	0.310055355	,],
[	0.298406615	,	0.345463743	,	0.272880959	,	0.089095542	,]],
[[	0.082996002	,	0.271356905	,	0.302749303	,	0.260095618	,],
[	0.301233853	,	0.077525296	,	0.246627455	,	0.336952325	,],
[	0.372400557	,	0.289581314	,	0.070550253	,	0.306803265	,],
[	0.298240281	,	0.345434897	,	0.272751621	,	0.086179271	,]],
[[	0.083505233	,	0.269930529	,	0.301270426	,	0.258784464	,],
[	0.30138607	,	0.080012779	,	0.247082856	,	0.337216759	,],
[	0.373119204	,	0.290866413	,	0.073701717	,	0.307577802	,],
[	0.296566498	,	0.343748354	,	0.271014823	,	0.085856292	,]],
[[	0.086714575	,	0.275197495	,	0.306265985	,	0.263995139	,],
[	0.303982471	,	0.079710749	,	0.249242946	,	0.339606377	,],
[	0.37457325	,	0.291666877	,	0.071988535	,	0.308978694	,],
[	0.301053558	,	0.347859276	,	0.275353532	,	0.092803838	,]],
[[	0.082182085	,	0.269902831	,	0.301111396	,	0.258765719	,],
[	0.300312601	,	0.077426585	,	0.245769768	,	0.336116023	,],
[	0.371864277	,	0.289273275	,	0.070794494	,	0.306277158	,],
[	0.296615278	,	0.343825403	,	0.271221558	,	0.08494024	,]],
[[	0.086264937	,	0.271461347	,	0.30270156	,	0.260444408	,],
[	0.302640554	,	0.082636122	,	0.248621882	,	0.338325179	,],
[	0.375061098	,	0.29286283	,	0.076913648	,	0.309467647	,],
[	0.297772807	,	0.344821276	,	0.27213745	,	0.087896895	,]],
[[	0.083829242	,	0.272276127	,	0.303380214	,	0.260920971	,],
[	0.30136476	,	0.077495053	,	0.246713942	,	0.337219156	,],
[	0.372974718	,	0.290102244	,	0.070785093	,	0.307347304	,],
[	0.29905741	,	0.346185636	,	0.273542502	,	0.086897736	,]],
[[	0.083849977	,	0.280009594	,	0.309425001	,	0.256858391	,],
[	0.290368605	,	0.07866724	,	0.246258606	,	0.326052325	,],
[	0.36414173	,	0.292302531	,	0.073179249	,	0.298032806	,],
[	0.293918747	,	0.351861594	,	0.278433626	,	0.084937916	,]],
[[	0.085227037	,	0.283651998	,	0.312986142	,	0.260372945	,],
[	0.294729566	,	0.080466168	,	0.250280248	,	0.330272445	,],
[	0.366433379	,	0.294116106	,	0.07288779	,	0.30055323	,],
[	0.298562985	,	0.356532526	,	0.283367634	,	0.08769434	,]],
[[	0.082127065	,	0.279510347	,	0.308870066	,	0.256327264	,],
[	0.290198578	,	0.07707329	,	0.245861626	,	0.325814181	,],
[	0.363114666	,	0.291003814	,	0.070736663	,	0.297101148	,],
[	0.293584887	,	0.351644836	,	0.278269884	,	0.083395333	,]],
[[	0.087272252	,	0.281981472	,	0.311318622	,	0.258930149	,],
[	0.29147649	,	0.081050128	,	0.247600363	,	0.327160652	,],
[	0.365908072	,	0.294313845	,	0.076373601	,	0.299816171	,],
[	0.295145005	,	0.353115733	,	0.279710557	,	0.087021334	,]],
[[	0.082932875	,	0.28118184	,	0.310658587	,	0.257939025	,],
[	0.292437654	,	0.078419298	,	0.247821777	,	0.327864831	,],
[	0.364396872	,	0.292212536	,	0.071177022	,	0.29844256	,],
[	0.296102854	,	0.353922493	,	0.280538188	,	0.085181808	,]],
[[	0.083172946	,	0.279728628	,	0.309180825	,	0.256545706	,],
[	0.290117614	,	0.077816573	,	0.245835922	,	0.325643353	,],
[	0.363904457	,	0.292009067	,	0.072535124	,	0.29781704	,],
[	0.293674046	,	0.351459499	,	0.278041988	,	0.084101289	,]],
[[	0.086505609	,	0.285117385	,	0.314253827	,	0.261663361	,],
[	0.293858619	,	0.084781347	,	0.250138869	,	0.329239085	,],
[	0.367621962	,	0.295370673	,	0.074072796	,	0.301687972	,],
[	0.297936107	,	0.355543687	,	0.282033133	,	0.090602408	,]],
[[	0.081956503	,	0.279612341	,	0.308984882	,	0.256348768	,],
[	0.290566274	,	0.076958568	,	0.246152559	,	0.32606469	,],
[	0.36351063	,	0.291304843	,	0.070734176	,	0.297295805	,],
[	0.294156859	,	0.352153982	,	0.278682	,	0.083567226	,]],
[[	0.086075063	,	0.281348165	,	0.31070011	,	0.258199487	,],
[	0.290939933	,	0.079915949	,	0.246881966	,	0.326479296	,],
[	0.365212556	,	0.293530163	,	0.075090627	,	0.299103891	,],
[	0.294742428	,	0.352628416	,	0.279196036	,	0.085973514	,]],
[[	0.083591939	,	0.281984918	,	0.311360028	,	0.258707989	,],
[	0.293544007	,	0.079344425	,	0.248836418	,	0.328876541	,],
[	0.365053628	,	0.292778277	,	0.07160851	,	0.299030749	,],
[	0.296830684	,	0.354669619	,	0.281318527	,	0.085781804	,]],
[[	0.08346699	,	0.290791537	,	0.319999515	,	0.25732854	,],
[	0.282019685	,	0.079339914	,	0.248220356	,	0.319331591	,],
[	0.356519324	,	0.294961483	,	0.072595842	,	0.290595089	,],
[	0.292094661	,	0.360787165	,	0.287532675	,	0.083738686	,]],
[[	0.086012528	,	0.295362846	,	0.324348504	,	0.261864776	,],
[	0.284671988	,	0.085014868	,	0.251401737	,	0.321849283	,],
[	0.360408453	,	0.299517946	,	0.07989864	,	0.294568961	,],
[	0.29621978	,	0.364691612	,	0.291514744	,	0.090238006	,]],
[[	0.082217975	,	0.290517657	,	0.31963409	,	0.257132809	,],
[	0.283479591	,	0.07967774	,	0.249478694	,	0.320693399	,],
[	0.356346862	,	0.294535845	,	0.071032683	,	0.290317082	,],
[	0.292334822	,	0.361101191	,	0.287915737	,	0.082930405	,]],
[[	0.086323546	,	0.292274926	,	0.321386649	,	0.258927305	,],
[	0.282573981	,	0.081042926	,	0.24910306	,	0.319915188	,],
[	0.357879218	,	0.296429371	,	0.075198695	,	0.291877938	,],
[	0.293223993	,	0.361933661	,	0.288683276	,	0.085635131	,]],
[[	0.083245921	,	0.292553463	,	0.321575305	,	0.25897653	,],
[	0.28663112	,	0.082132582	,	0.252452926	,	0.323653744	,],
[	0.358206921	,	0.296181556	,	0.072191591	,	0.292261172	,],
[	0.295179305	,	0.363671092	,	0.290625672	,	0.085113845	,]],
[[	0.082869013	,	0.29057653	,	0.319735165	,	0.257152929	,],
[	0.282161107	,	0.079071057	,	0.248265964	,	0.319435396	,],
[	0.356223505	,	0.294487583	,	0.07191061	,	0.290220517	,],
[	0.291919051	,	0.36051021	,	0.287181925	,	0.083009154	,]],
[[	0.087077584	,	0.296519326	,	0.325366654	,	0.262984391	,],
[	0.28423421	,	0.083990979	,	0.250893341	,	0.321383312	,],
[	0.359661291	,	0.298601658	,	0.078343895	,	0.293626062	,],
[	0.295571789	,	0.364045446	,	0.290796468	,	0.089056187	,]],
[[	0.082092045	,	0.290838239	,	0.319803888	,	0.257331777	,],
[	0.283950683	,	0.079703532	,	0.249848411	,	0.320964165	,],
[	0.356930403	,	0.294932103	,	0.071237034	,	0.290813844	,],
[	0.292801802	,	0.361497272	,	0.288416882	,	0.083164334	,]],
[[	0.085384784	,	0.291889768	,	0.32086188	,	0.258411256	,],
[	0.28248339	,	0.080293401	,	0.2486335	,	0.319490895	,],
[	0.35708257	,	0.29565709	,	0.073888584	,	0.290970086	,],
[	0.292660733	,	0.361332084	,	0.288007784	,	0.084666227	,]],
[[	0.084137407	,	0.293409902	,	0.322465474	,	0.259820476	,],
[	0.286430567	,	0.087769948	,	0.253347317	,	0.323617541	,],
[	0.359327261	,	0.297307053	,	0.073147154	,	0.293343768	,],
[	0.296146065	,	0.364678227	,	0.291543295	,	0.085931499	,]],
[[	0.082903647	,	0.300903209	,	0.329637853	,	0.257659225	,],
[	0.273478086	,	0.079188889	,	0.250249015	,	0.312661138	,],
[	0.349342588	,	0.298275086	,	0.072142571	,	0.284334954	,],
[	0.290888985	,	0.370997542	,	0.297022664	,	0.084121132	,]],
[[	0.085523262	,	0.305530265	,	0.333919456	,	0.262178334	,],
[	0.274253439	,	0.082618979	,	0.251609221	,	0.313429826	,],
[	0.350982497	,	0.3004679	,	0.076704325	,	0.285940518	,],
[	0.293779294	,	0.373923092	,	0.299806096	,	0.089298662	,]],
[[	0.081725271	,	0.300856009	,	0.32938869	,	0.25749391	,],
[	0.275720793	,	0.080683812	,	0.252348424	,	0.314874352	,],
[	0.351036305	,	0.299776019	,	0.073009442	,	0.285980225	,],
[	0.292055596	,	0.37225572	,	0.29826356	,	0.084547779	,]],
[[	0.086105963	,	0.302684579	,	0.331227747	,	0.259337065	,],
[	0.273004759	,	0.079686084	,	0.249996504	,	0.312138844	,],
[	0.349216021	,	0.298377138	,	0.073129851	,	0.284153303	,],
[	0.29117381	,	0.371485173	,	0.297406859	,	0.085203222	,]],
[[	0.082849098	,	0.302880075	,	0.331404123	,	0.259421477	,],
[	0.276138485	,	0.086007118	,	0.253639758	,	0.315188007	,],
[	0.353546591	,	0.303189655	,	0.080441098	,	0.288276138	,],
[	0.295669029	,	0.375670335	,	0.301673431	,	0.087875098	,]],
[[	0.082204187	,	0.300572764	,	0.329264777	,	0.257211067	,],
[	0.274066316	,	0.079449122	,	0.250645418	,	0.313000065	,],
[	0.349445228	,	0.298327267	,	0.071943005	,	0.284253355	,],
[	0.290836218	,	0.371045599	,	0.296901391	,	0.083650015	,]],
[[	0.086680989	,	0.306790343	,	0.335124694	,	0.263407901	,],
[	0.273796435	,	0.081678495	,	0.250960952	,	0.312915337	,],
[	0.350418298	,	0.299762952	,	0.075574577	,	0.285227666	,],
[	0.292875236	,	0.372940584	,	0.298794362	,	0.087842382	,]],
[[	0.081864872	,	0.301317949	,	0.329693466	,	0.257820334	,],
[	0.27674796	,	0.081508185	,	0.253273556	,	0.315707562	,],
[	0.351874255	,	0.300457762	,	0.073551448	,	0.286760218	,],
[	0.292865798	,	0.373032434	,	0.299065388	,	0.085125679	,]],
[[	0.084765802	,	0.301986149	,	0.330522921	,	0.258595107	,],
[	0.273346583	,	0.079436103	,	0.250165947	,	0.312316367	,],
[	0.349171521	,	0.29816391	,	0.072528594	,	0.283883701	,],
[	0.291064497	,	0.371265158	,	0.297104239	,	0.084527468	,]],
[[	0.083700047	,	0.30379682	,	0.332320102	,	0.260283767	,],
[	0.275317665	,	0.084704438	,	0.252849638	,	0.314424286	,],
[	0.352371323	,	0.301965643	,	0.078723941	,	0.287130678	,],
[	0.295501567	,	0.37550491	,	0.30143756	,	0.091619857	,]],
[[	0.082099298	,	0.311835373	,	0.340126731	,	0.259224298	,],
[	0.266743956	,	0.081616926	,	0.255531591	,	0.308783732	,],
[	0.342276491	,	0.302370886	,	0.073451221	,	0.278859937	,],
[	0.289327473	,	0.380538423	,	0.306368124	,	0.084090579	,]],
[[	0.085487596	,	0.316846248	,	0.344910863	,	0.264311994	,],
[	0.265030249	,	0.08189648	,	0.254271532	,	0.307180762	,],
[	0.342078668	,	0.30274304	,	0.075373596	,	0.278554542	,],
[	0.291880169	,	0.383006646	,	0.308858013	,	0.088718329	,]],
[[	0.081157412	,	0.311948066	,	0.340095894	,	0.259314046	,],
[	0.268963172	,	0.089279148	,	0.258713219	,	0.310944567	,],
[	0.34535343	,	0.305200684	,	0.075803993	,	0.281846731	,],
[	0.291139606	,	0.382322843	,	0.308222624	,	0.085124678	,]],
[[	0.084715204	,	0.313265622	,	0.341356953	,	0.260612993	,],
[	0.265203367	,	0.080841806	,	0.254030409	,	0.307249474	,],
[	0.341371165	,	0.30158359	,	0.073164922	,	0.277786698	,],
[	0.289756785	,	0.381043532	,	0.306794532	,	0.085024939	,]],
[[	0.082563808	,	0.31403217	,	0.342215023	,	0.261351646	,],
[	0.265987423	,	0.084199227	,	0.255417116	,	0.30805012	,],
[	0.343748341	,	0.304577499	,	0.078288228	,	0.280115029	,],
[	0.294108729	,	0.385212598	,	0.311115055	,	0.092114065	,]],
[[	0.081521838	,	0.311592751	,	0.339912253	,	0.258908115	,],
[	0.267677079	,	0.082211216	,	0.256177539	,	0.309496333	,],
[	0.342870717	,	0.30291981	,	0.073801252	,	0.27940185	,],
[	0.289813201	,	0.380835635	,	0.306696011	,	0.084144382	,]],
[[	0.08675698	,	0.318134633	,	0.3461681	,	0.265592941	,],
[	0.264971231	,	0.081357292	,	0.253910581	,	0.306865179	,],
[	0.341633873	,	0.302078285	,	0.074320996	,	0.278023058	,],
[	0.291098554	,	0.382273057	,	0.308044418	,	0.087406303	,]],
[[	0.081144451	,	0.312249568	,	0.340401651	,	0.259619394	,],
[	0.268257911	,	0.087859135	,	0.257807754	,	0.310143651	,],
[	0.345610759	,	0.306621477	,	0.081571631	,	0.282004508	,],
[	0.291742932	,	0.382768906	,	0.308716681	,	0.085559539	,]],
[[	0.083670127	,	0.312630759	,	0.340834471	,	0.259899772	,],
[	0.266134677	,	0.081157892	,	0.254771564	,	0.308003443	,],
[	0.341660705	,	0.301824817	,	0.073087213	,	0.278047284	,],
[	0.289766422	,	0.380924633	,	0.306738677	,	0.084608433	,]],
[[	0.08339974	,	0.314979218	,	0.343125744	,	0.262310177	,],
[	0.265792077	,	0.083403586	,	0.255027399	,	0.307718594	,],
[	0.342801529	,	0.303469585	,	0.076696481	,	0.279111173	,],
[	0.293489916	,	0.384581909	,	0.310475557	,	0.090917813	,]],
[[	0.082595288	,	0.322736267	,	0.35043656	,	0.260024045	,],
[	0.258875366	,	0.082503804	,	0.257479516	,	0.30142634	,],
[	0.336222766	,	0.306691163	,	0.074545594	,	0.272002724	,],
[	0.287988347	,	0.389291552	,	0.314711136	,	0.082938706	,]],
[[	0.085072243	,	0.327105076	,	0.354543617	,	0.264401407	,],
[	0.256023872	,	0.081463426	,	0.255078762	,	0.298675955	,],
[	0.33500442	,	0.306044145	,	0.075491721	,	0.270630496	,],
[	0.290352077	,	0.391549568	,	0.316996047	,	0.087247954	,]],
[[	0.081352458	,	0.322553611	,	0.350113713	,	0.259780269	,],
[	0.258962211	,	0.087535987	,	0.258555804	,	0.301479702	,],
[	0.338843802	,	0.310528357	,	0.082724045	,	0.274619189	,],
[	0.289565217	,	0.39079432	,	0.316335696	,	0.083808347	,]],
[[	0.085321449	,	0.324298257	,	0.351846003	,	0.261508189	,],
[	0.256749456	,	0.080843125	,	0.255575491	,	0.299370674	,],
[	0.334941968	,	0.305611804	,	0.073977727	,	0.27052252	,],
[	0.28825017	,	0.389707823	,	0.315106983	,	0.083787469	,]],
[[	0.08239471	,	0.324389417	,	0.351954083	,	0.261717354	,],
[	0.256030215	,	0.082704541	,	0.255330735	,	0.298770157	,],
[	0.336286503	,	0.30743709	,	0.077775359	,	0.271790981	,],
[	0.292335401	,	0.3936255	,	0.319080819	,	0.090591291	,]],
[[	0.081777391	,	0.322374971	,	0.350008285	,	0.259462351	,],
[	0.260058578	,	0.083431404	,	0.258601764	,	0.302529424	,],
[	0.336901576	,	0.307401206	,	0.075177079	,	0.272631524	,],
[	0.288068722	,	0.389304196	,	0.314607718	,	0.082651912	,]],
[[	0.086414167	,	0.328391188	,	0.355723399	,	0.265691868	,],
[	0.256012057	,	0.08091335	,	0.25493709	,	0.298547091	,],
[	0.334660168	,	0.305520696	,	0.074547147	,	0.270219109	,],
[	0.289498733	,	0.390831569	,	0.316119696	,	0.086002831	,]],
[[	0.081187461	,	0.322719893	,	0.350220263	,	0.259973087	,],
[	0.258349619	,	0.08628532	,	0.257850705	,	0.300818988	,],
[	0.337909299	,	0.309409551	,	0.08094562	,	0.273443638	,],
[	0.290577116	,	0.391796518	,	0.317335167	,	0.084593084	,]],
[[	0.083950946	,	0.323369167	,	0.351012691	,	0.260548396	,],
[	0.257580593	,	0.081243719	,	0.256171131	,	0.300028116	,],
[	0.33520706	,	0.305747591	,	0.073861955	,	0.270707647	,],
[	0.288038808	,	0.389421213	,	0.314774179	,	0.083211729	,]],
[[	0.083469962	,	0.32546723	,	0.353036203	,	0.262695349	,],
[	0.256190173	,	0.082275018	,	0.255360147	,	0.298746065	,],
[	0.335569567	,	0.306641993	,	0.076596941	,	0.2710211	,],
[	0.291787076	,	0.393097626	,	0.318527922	,	0.089363342	,]],
[[	0.082796077	,	0.332877272	,	0.359489596	,	0.25916135	,],
[	0.251521837	,	0.084157823	,	0.260557485	,	0.294544389	,],
[	0.329263272	,	0.311136015	,	0.074708984	,	0.265019712	,],
[	0.287244278	,	0.399731714	,	0.323359449	,	0.082919934	,]],
[[	0.085117365	,	0.337023914	,	0.3633145	,	0.263527223	,],
[	0.247787203	,	0.082203772	,	0.257287461	,	0.290906717	,],
[	0.327785268	,	0.31007735	,	0.075212595	,	0.263419146	,],
[	0.290206784	,	0.402694667	,	0.326294432	,	0.088331856	,]],
[[	0.081634501	,	0.33279342	,	0.359278053	,	0.259120651	,],
[	0.250089702	,	0.087976953	,	0.260361298	,	0.293151702	,],
[	0.331667112	,	0.31478623	,	0.08275051	,	0.267409198	,],
[	0.288549834	,	0.400897941	,	0.324691444	,	0.083417068	,]],
[[	0.085609256	,	0.334416242	,	0.360844477	,	0.260637569	,],
[	0.249090609	,	0.082401338	,	0.258314641	,	0.292218099	,],
[	0.327969379	,	0.309871734	,	0.073895309	,	0.263585614	,],
[	0.28781786	,	0.400376014	,	0.324054623	,	0.084297591	,]],
[[	0.082486782	,	0.33455386	,	0.360983309	,	0.260896711	,],
[	0.247793065	,	0.083489839	,	0.257668415	,	0.290979236	,],
[	0.328968883	,	0.311503896	,	0.077609622	,	0.264523111	,],
[	0.291879611	,	0.403858348	,	0.327807433	,	0.086451442	,]],
[[	0.082104487	,	0.332549861	,	0.359152062	,	0.258838755	,],
[	0.252188541	,	0.091505538	,	0.262773711	,	0.295247555	,],
[	0.330193188	,	0.311967902	,	0.075451483	,	0.265860484	,],
[	0.287395396	,	0.3997659	,	0.323368622	,	0.082755671	,]],
[[	0.086504519	,	0.338406407	,	0.364634208	,	0.264971626	,],
[	0.247955423	,	0.081971602	,	0.257265437	,	0.290947101	,],
[	0.327553464	,	0.309758194	,	0.074574746	,	0.263048294	,],
[	0.289307115	,	0.401774182	,	0.325348567	,	0.086882181	,]],
[[	0.08136266	,	0.332894176	,	0.359163368	,	0.259185747	,],
[	0.249198759	,	0.086312121	,	0.259268169	,	0.292233718	,],
[	0.330915035	,	0.313754932	,	0.080953758	,	0.266423524	,],
[	0.289426194	,	0.401843962	,	0.325564911	,	0.084087886	,]],
[[	0.084376117	,	0.333775031	,	0.360193977	,	0.26000262	,],
[	0.250174903	,	0.083127553	,	0.25930656	,	0.293156086	,],
[	0.328510633	,	0.310337086	,	0.074105843	,	0.264049058	,],
[	0.287431144	,	0.399994465	,	0.323661462	,	0.083513287	,]],
[[	0.083401568	,	0.335347783	,	0.361691409	,	0.261625765	,],
[	0.24754896	,	0.082760337	,	0.257323578	,	0.290723924	,],
[	0.328679601	,	0.311060763	,	0.076664985	,	0.264072838	,],
[	0.29189571	,	0.404376471	,	0.328027876	,	0.090623964	,]],
[[	0.096985435	,	0.225398826	,	0.26837837	,	0.291990939	,],
[	0.3745054	,	0.077973078	,	0.256504468	,	0.411563156	,],
[	0.439366533	,	0.284954536	,	0.070809201	,	0.373720662	,],
[	0.324731814	,	0.29951949	,	0.229295052	,	0.093789601	,]],
[[	0.091140285	,	0.218574802	,	0.261846293	,	0.285710504	,],
[	0.374506086	,	0.080006602	,	0.256677767	,	0.411600116	,],
[	0.441355328	,	0.287235431	,	0.075351268	,	0.375791878	,],
[	0.327809638	,	0.302234835	,	0.232141389	,	0.099573469	,]],
[[	0.093242086	,	0.218143036	,	0.261568085	,	0.2857928	,],
[	0.377701818	,	0.080738746	,	0.259660802	,	0.414579115	,],
[	0.440664121	,	0.286366313	,	0.071561034	,	0.374984588	,],
[	0.324983676	,	0.299813919	,	0.229483692	,	0.093027377	,]],
[[	0.093846223	,	0.222184195	,	0.265312053	,	0.288887319	,],
[	0.373911191	,	0.078082134	,	0.255983139	,	0.411019721	,],
[	0.439760219	,	0.285345257	,	0.071990697	,	0.373955571	,],
[	0.325377504	,	0.299875306	,	0.229626758	,	0.095337868	,]],
[[	0.091098097	,	0.21779861	,	0.26116944	,	0.285036207	,],
[	0.375906658	,	0.08268536	,	0.258375855	,	0.413032623	,],
[	0.443419268	,	0.289503837	,	0.078860155	,	0.377885529	,],
[	0.327474424	,	0.302468148	,	0.232311674	,	0.094881318	,]],
[[	0.096280559	,	0.219832985	,	0.263417511	,	0.28758195	,],
[	0.375334464	,	0.078643024	,	0.257177182	,	0.4121277	,],
[	0.439707546	,	0.285256777	,	0.070821043	,	0.373911642	,],
[	0.324345959	,	0.299220715	,	0.228948672	,	0.093161105	,]],
[[	0.091636025	,	0.219196524	,	0.26246309	,	0.286264901	,],
[	0.374199828	,	0.07929343	,	0.256262647	,	0.411254122	,],
[	0.440611713	,	0.286394846	,	0.07405616	,	0.375018701	,],
[	0.326994295	,	0.301521899	,	0.231314995	,	0.098091902	,]],
[[	0.092430793	,	0.217863481	,	0.261215334	,	0.285387886	,],
[	0.377998058	,	0.08628792	,	0.260625918	,	0.415063158	,],
[	0.441747403	,	0.287458402	,	0.07243342	,	0.375984229	,],
[	0.325368258	,	0.300225084	,	0.229957547	,	0.093174831	,]],
[[	0.094602078	,	0.222930682	,	0.266178206	,	0.289604974	,],
[	0.374262569	,	0.078000641	,	0.256187734	,	0.41111864	,],
[	0.439576584	,	0.285111554	,	0.071363505	,	0.373744054	,],
[	0.324863357	,	0.299461021	,	0.229137345	,	0.094395503	,]],
[[	0.091080061	,	0.21817062	,	0.26152418	,	0.285245493	,],
[	0.375306606	,	0.081627791	,	0.257564326	,	0.412346666	,],
[	0.442679784	,	0.288729667	,	0.077546536	,	0.377073532	,],
[	0.328455687	,	0.303522383	,	0.233405219	,	0.095753964	,]],
[[	0.095227674	,	0.223244417	,	0.266117841	,	0.285152454	,],
[	0.368401061	,	0.080228799	,	0.257724332	,	0.406022916	,],
[	0.431069729	,	0.28413298	,	0.071507212	,	0.367454787	,],
[	0.317637683	,	0.299608939	,	0.230586513	,	0.091815618	,]],
[[	0.090035267	,	0.222126135	,	0.264999203	,	0.283313724	,],
[	0.364397761	,	0.077291324	,	0.253676812	,	0.402269265	,],
[	0.429774237	,	0.282959304	,	0.071767665	,	0.366195868	,],
[	0.31917472	,	0.300957039	,	0.232000669	,	0.09571795	,]],
[[	0.091514128	,	0.221289839	,	0.264192824	,	0.282904188	,],
[	0.366404262	,	0.081991627	,	0.256203111	,	0.4043461	,],
[	0.433624018	,	0.287347329	,	0.078653886	,	0.370229174	,],
[	0.319402567	,	0.301439518	,	0.232552593	,	0.093080037	,]],
[[	0.093027454	,	0.225949395	,	0.268663584	,	0.28670376	,],
[	0.366157528	,	0.078213898	,	0.255500272	,	0.403821605	,],
[	0.429844588	,	0.282846865	,	0.07052623	,	0.366218994	,],
[	0.317690752	,	0.299482411	,	0.230469943	,	0.092584304	,]],
[[	0.08957311	,	0.221167032	,	0.264032352	,	0.282378692	,],
[	0.364640482	,	0.07849193	,	0.254209028	,	0.402481702	,],
[	0.430709387	,	0.284033221	,	0.073740789	,	0.367144488	,],
[	0.321515652	,	0.30296503	,	0.234008718	,	0.099118071	,]],
[[	0.094255208	,	0.222907756	,	0.265858585	,	0.284618997	,],
[	0.368457715	,	0.085490147	,	0.258432146	,	0.406310041	,],
[	0.43187638	,	0.285084797	,	0.072266414	,	0.368298614	,],
[	0.317749891	,	0.299802025	,	0.230810775	,	0.09168824	,]],
[[	0.090885177	,	0.223223149	,	0.265948304	,	0.284246151	,],
[	0.365057535	,	0.077643633	,	0.254325259	,	0.40272606	,],
[	0.429797445	,	0.282947669	,	0.071407174	,	0.366176943	,],
[	0.318376686	,	0.300117244	,	0.231183363	,	0.094390389	,]],
[[	0.090743579	,	0.221025585	,	0.26389287	,	0.282569261	,],
[	0.365676365	,	0.080728667	,	0.255331016	,	0.403485026	,],
[	0.432795231	,	0.286245934	,	0.07719334	,	0.369263771	,],
[	0.320626169	,	0.302620615	,	0.23360814	,	0.093833243	,]],
[[	0.094136115	,	0.227178418	,	0.269887408	,	0.287812056	,],
[	0.367099498	,	0.078943653	,	0.25635642	,	0.404587148	,],
[	0.430136964	,	0.283132508	,	0.070742133	,	0.366439044	,],
[	0.317414864	,	0.299289098	,	0.230135163	,	0.091960479	,]],
[[	0.089586374	,	0.221420197	,	0.264308163	,	0.282552735	,],
[	0.36473305	,	0.078088333	,	0.254084791	,	0.402423306	,],
[	0.430664643	,	0.283740757	,	0.073152774	,	0.366989205	,],
[	0.320623023	,	0.30213532	,	0.233226402	,	0.097647829	,]],
[[	0.092798803	,	0.225053187	,	0.266929588	,	0.280177962	,],
[	0.357855075	,	0.081307954	,	0.253253685	,	0.395874885	,],
[	0.424062037	,	0.283586029	,	0.076723578	,	0.361002794	,],
[	0.312883783	,	0.300651681	,	0.232969353	,	0.090790777	,]],
[[	0.088761626	,	0.224757036	,	0.266419051	,	0.279102908	,],
[	0.357244051	,	0.077100464	,	0.252002681	,	0.394867978	,],
[	0.420852416	,	0.279884745	,	0.069404366	,	0.357745203	,],
[	0.312548748	,	0.29996407	,	0.232256134	,	0.091943775	,]],
[[	0.089183935	,	0.223137936	,	0.264938317	,	0.27791468	,],
[	0.355935433	,	0.077589471	,	0.251012255	,	0.39390866	,],
[	0.421356181	,	0.280591304	,	0.072030152	,	0.358131413	,],
[	0.316547028	,	0.30414313	,	0.236566824	,	0.09376494	,]],
[[	0.092417705	,	0.229146159	,	0.270658727	,	0.283133021	,],
[	0.360197317	,	0.085056432	,	0.255728494	,	0.398134741	,],
[	0.423147753	,	0.282334206	,	0.071277687	,	0.360048789	,],
[	0.31220868	,	0.299623063	,	0.2318408	,	0.090441947	,]],
[[	0.087892727	,	0.223276422	,	0.265091056	,	0.277835357	,],
[	0.356048264	,	0.076544879	,	0.251059071	,	0.393898963	,],
[	0.420401211	,	0.279275352	,	0.06943221	,	0.357080235	,],
[	0.314239461	,	0.301342291	,	0.233589095	,	0.094525066	,]],
[[	0.091390046	,	0.224120968	,	0.266045349	,	0.279073078	,],
[	0.35728692	,	0.080187716	,	0.252518792	,	0.395197059	,],
[	0.423493884	,	0.282801291	,	0.075526199	,	0.360283999	,],
[	0.313776468	,	0.30139626	,	0.233742463	,	0.091237314	,]],
[[	0.089548743	,	0.225761954	,	0.267201708	,	0.280031106	,],
[	0.358102673	,	0.077821078	,	0.252983151	,	0.395666962	,],
[	0.421028288	,	0.280079176	,	0.069379744	,	0.357798736	,],
[	0.312060508	,	0.299504986	,	0.231762282	,	0.090898443	,]],
[[	0.088397324	,	0.222800746	,	0.264455947	,	0.277535304	,],
[	0.355868089	,	0.0771688	,	0.250768996	,	0.393673185	,],
[	0.420977343	,	0.280084229	,	0.071140071	,	0.357756076	,],
[	0.316204345	,	0.303231941	,	0.235649536	,	0.097561702	,]],
[[	0.093308561	,	0.230025474	,	0.27145261	,	0.283913028	,],
[	0.35905877	,	0.083367635	,	0.254472087	,	0.396917898	,],
[	0.424363238	,	0.283525309	,	0.072370903	,	0.361148333	,],
[	0.312407366	,	0.299874079	,	0.232050359	,	0.090189217	,]],
[[	0.088027973	,	0.22382382	,	0.265502385	,	0.27821382	,],
[	0.356213813	,	0.076366935	,	0.251092099	,	0.393905686	,],
[	0.420271296	,	0.279331843	,	0.069197578	,	0.35701039	,],
[	0.313071994	,	0.300202092	,	0.232482912	,	0.092904212	,]],
[[	0.090777558	,	0.227393578	,	0.26900665	,	0.27731291	,],
[	0.348072796	,	0.077309045	,	0.250047619	,	0.387592712	,],
[	0.413271144	,	0.278997444	,	0.071147984	,	0.351478926	,],
[	0.311421508	,	0.304695607	,	0.237827	,	0.092099119	,]],
[[	0.088369875	,	0.228215332	,	0.269650951	,	0.277467349	,],
[	0.352057375	,	0.084290892	,	0.25450849	,	0.391569942	,],
[	0.415558035	,	0.28138497	,	0.071162489	,	0.353937407	,],
[	0.307637324	,	0.300724577	,	0.233697187	,	0.089108362	,]],
[[	0.087960449	,	0.226024883	,	0.267616632	,	0.275723896	,],
[	0.348096159	,	0.075935601	,	0.249799878	,	0.387410992	,],
[	0.41256687	,	0.27811829	,	0.069062434	,	0.350791625	,],
[	0.309843878	,	0.30269259	,	0.235688241	,	0.093652054	,]],
[[	0.0921294	,	0.232665364	,	0.273991587	,	0.281611054	,],
[	0.34947543	,	0.079730066	,	0.2514463	,	0.388747908	,],
[	0.415011104	,	0.280868379	,	0.074064218	,	0.353282837	,],
[	0.308995379	,	0.302077458	,	0.235077613	,	0.089776579	,]],
[[	0.087108747	,	0.22662209	,	0.2681421	,	0.275989832	,],
[	0.350115263	,	0.077245288	,	0.251751322	,	0.389214282	,],
[	0.413356519	,	0.278958163	,	0.069218941	,	0.35161591	,],
[	0.308061995	,	0.300858812	,	0.233771348	,	0.090260274	,]],
[[	0.089667808	,	0.226745591	,	0.268424613	,	0.276562229	,],
[	0.347831088	,	0.076551518	,	0.249695452	,	0.387235374	,],
[	0.412727697	,	0.278317697	,	0.070144661	,	0.350812266	,],
[	0.312414667	,	0.304833239	,	0.237995848	,	0.097226391	,]],
[[	0.089287362	,	0.229465635	,	0.270795476	,	0.278599876	,],
[	0.351074933	,	0.082658004	,	0.253306984	,	0.390388543	,],
[	0.416528758	,	0.282362896	,	0.072196514	,	0.354886877	,],
[	0.307773622	,	0.300783949	,	0.233850239	,	0.088902757	,]],
[[	0.087174098	,	0.22555525	,	0.267051218	,	0.275102596	,],
[	0.348607901	,	0.076165468	,	0.250192366	,	0.387835776	,],
[	0.412779153	,	0.278214949	,	0.068921855	,	0.350820905	,],
[	0.309068568	,	0.301812427	,	0.23484293	,	0.092219751	,]],
[[	0.093081666	,	0.228683147	,	0.2701865	,	0.278650077	,],
[	0.348680668	,	0.078388426	,	0.250602794	,	0.387978574	,],
[	0.414329911	,	0.280164323	,	0.073016602	,	0.352547835	,],
[	0.310175837	,	0.303108497	,	0.236178576	,	0.090635806	,]],
[[	0.087261531	,	0.227015102	,	0.268395878	,	0.276193503	,],
[	0.351147203	,	0.078248906	,	0.252866496	,	0.390214586	,],
[	0.414185908	,	0.279887908	,	0.06989327	,	0.352424058	,],
[	0.307361809	,	0.300260474	,	0.233114791	,	0.089194612	,]],
[[	0.089080967	,	0.22998263	,	0.270142627	,	0.273627969	,],
[	0.341125152	,	0.076085148	,	0.248272791	,	0.379899879	,],
[	0.406239276	,	0.277681913	,	0.070198942	,	0.345032093	,],
[	0.308787515	,	0.307307146	,	0.240572148	,	0.091995528	,]],
[[	0.087317825	,	0.231653789	,	0.271499873	,	0.274497792	,],
[	0.343974533	,	0.081519463	,	0.251479133	,	0.382728518	,],
[	0.410255183	,	0.282306288	,	0.077372917	,	0.34916752	,],
[	0.304123391	,	0.302483105	,	0.23556938	,	0.08811914	,]],
[[	0.086189159	,	0.228531504	,	0.268628191	,	0.271947149	,],
[	0.34207338	,	0.075721863	,	0.248766114	,	0.380542355	,],
[	0.406305923	,	0.277604196	,	0.069270078	,	0.345113846	,],
[	0.305966379	,	0.30398937	,	0.237077636	,	0.091905732	,]],
[[	0.09147516	,	0.236278645	,	0.276046421	,	0.278865235	,],
[	0.341796008	,	0.077498005	,	0.248871843	,	0.380481105	,],
[	0.407629089	,	0.279164501	,	0.072683831	,	0.346343259	,],
[	0.306164617	,	0.30449512	,	0.237567893	,	0.08937202	,]],
[[	0.085477789	,	0.22940946	,	0.269400987	,	0.272356756	,],
[	0.345058797	,	0.078200412	,	0.25192283	,	0.38338316	,],
[	0.408230667	,	0.279726357	,	0.070638474	,	0.347066323	,],
[	0.304420044	,	0.302556703	,	0.235574993	,	0.088833855	,]],
[[	0.088002377	,	0.229477155	,	0.269588712	,	0.272887476	,],
[	0.341245956	,	0.075685437	,	0.248173813	,	0.379841891	,],
[	0.406063023	,	0.277378446	,	0.069632452	,	0.344647687	,],
[	0.308133342	,	0.305898292	,	0.239049747	,	0.095241942	,]],
[[	0.087944066	,	0.232350707	,	0.272116031	,	0.275113418	,],
[	0.343055409	,	0.080009825	,	0.250337574	,	0.38170643	,],
[	0.409306773	,	0.281072109	,	0.075671263	,	0.34812748	,],
[	0.304522044	,	0.302858982	,	0.235864518	,	0.088030009	,]],
[[	0.085637358	,	0.228506	,	0.26847727	,	0.271803514	,],
[	0.342800763	,	0.07638246	,	0.249595065	,	0.381223577	,],
[	0.407010152	,	0.278240376	,	0.069538484	,	0.345646412	,],
[	0.305332018	,	0.303363816	,	0.236346842	,	0.090757654	,]],
[[	0.091297593	,	0.231239076	,	0.271369908	,	0.27491745	,],
[	0.341378837	,	0.076698414	,	0.248246765	,	0.379901752	,],
[	0.40692616	,	0.278399559	,	0.071392499	,	0.3455797	,],
[	0.307050662	,	0.305370014	,	0.238514759	,	0.090087939	,]],
[[	0.085984317	,	0.229852857	,	0.269918096	,	0.272859384	,],
[	0.345339336	,	0.083725946	,	0.25279537	,	0.383908512	,],
[	0.409459674	,	0.280899411	,	0.071685803	,	0.348109933	,],
[	0.304127511	,	0.30221721	,	0.235282683	,	0.088264348	,]],
[[	0.089037933	,	0.234202071	,	0.274639554	,	0.273149179	,],
[	0.336885184	,	0.076233952	,	0.248347247	,	0.376253576	,],
[	0.400929117	,	0.277442408	,	0.069673728	,	0.340645133	,],
[	0.305399275	,	0.309820514	,	0.244061723	,	0.091028458	,]],
[[	0.087384711	,	0.235751313	,	0.276016573	,	0.274085505	,],
[	0.339316961	,	0.081540181	,	0.25115093	,	0.378705217	,],
[	0.404451102	,	0.28144399	,	0.076266547	,	0.344409383	,],
[	0.30060652	,	0.304810925	,	0.238891469	,	0.087039231	,]],
[[	0.086300249	,	0.232918214	,	0.273336032	,	0.271643009	,],
[	0.338109519	,	0.076564067	,	0.249283706	,	0.377174126	,],
[	0.401172093	,	0.277487129	,	0.06869791	,	0.34086854	,],
[	0.302692306	,	0.306617097	,	0.240752636	,	0.091241049	,]],
[[	0.091451005	,	0.240333007	,	0.280499725	,	0.2783859	,],
[	0.337314746	,	0.077731288	,	0.24877113	,	0.376584585	,],
[	0.401965758	,	0.278548448	,	0.071793816	,	0.341695679	,],
[	0.302873346	,	0.30704823	,	0.241175296	,	0.088540331	,]],
[[	0.085670677	,	0.233714094	,	0.274133318	,	0.272200619	,],
[	0.341253909	,	0.079186033	,	0.252520879	,	0.380306896	,],
[	0.403459935	,	0.279853903	,	0.070638433	,	0.343165976	,],
[	0.301236789	,	0.305218433	,	0.239241954	,	0.088237717	,]],
[[	0.088072245	,	0.233575286	,	0.274125441	,	0.272584777	,],
[	0.336887674	,	0.075871284	,	0.248220128	,	0.376147232	,],
[	0.400879529	,	0.277288297	,	0.069203518	,	0.340447093	,],
[	0.304746544	,	0.308460581	,	0.242545229	,	0.094645184	,]],
[[	0.088058888	,	0.236649193	,	0.276722916	,	0.274866244	,],
[	0.338582101	,	0.080132202	,	0.250231049	,	0.377861353	,],
[	0.403892368	,	0.280740128	,	0.075028721	,	0.343564874	,],
[	0.301295963	,	0.305559751	,	0.239567511	,	0.087237546	,]],
[[	0.085729598	,	0.232689271	,	0.272998078	,	0.271383703	,],
[	0.338524597	,	0.076814309	,	0.249691227	,	0.377527498	,],
[	0.401917104	,	0.278344476	,	0.069245011	,	0.341618783	,],
[	0.301945258	,	0.305986042	,	0.240019522	,	0.089977618	,]],
[[	0.091619314	,	0.235609102	,	0.276053371	,	0.27466942	,],
[	0.336931735	,	0.07692131	,	0.248334754	,	0.376261894	,],
[	0.40140315	,	0.278064594	,	0.070753919	,	0.34110085	,],
[	0.303736128	,	0.30799448	,	0.242086384	,	0.089243372	,]],
[[	0.086112214	,	0.234218915	,	0.274524382	,	0.272562253	,],
[	0.34055429	,	0.083626317	,	0.252312276	,	0.379799467	,],
[	0.404814731	,	0.281283163	,	0.071823013	,	0.344451304	,],
[	0.300798695	,	0.304771594	,	0.238774038	,	0.087421443	,]],
[[	0.08771599	,	0.237416826	,	0.277511571	,	0.270890525	,],
[	0.329422451	,	0.074920847	,	0.247152127	,	0.369611687	,],
[	0.393839994	,	0.27606825	,	0.068876031	,	0.334302234	,],
[	0.301662391	,	0.310975349	,	0.246095073	,	0.093526439	,]],
[[	0.085624691	,	0.238489085	,	0.278284848	,	0.27149757	,],
[	0.330753088	,	0.078854593	,	0.248993688	,	0.371082649	,],
[	0.396258113	,	0.278953624	,	0.074213861	,	0.336884896	,],
[	0.298108544	,	0.308054579	,	0.242980274	,	0.086474289	,]],
[[	0.084701104	,	0.235873136	,	0.275830579	,	0.269243764	,],
[	0.331740374	,	0.076444027	,	0.249282422	,	0.371667598	,],
[	0.394641726	,	0.276820791	,	0.068701168	,	0.335071633	,],
[	0.298729088	,	0.308439926	,	0.243347683	,	0.089047658	,]],
[[	0.089685876	,	0.242938126	,	0.28274145	,	0.275702908	,],
[	0.329474789	,	0.075825037	,	0.247289357	,	0.369666663	,],
[	0.394294292	,	0.276671846	,	0.070285229	,	0.334719485	,],
[	0.301231962	,	0.310954442	,	0.245971503	,	0.088913153	,]],
[[	0.084410137	,	0.236835478	,	0.276806916	,	0.269879728	,],
[	0.332851336	,	0.082248037	,	0.251164837	,	0.373022186	,],
[	0.397572014	,	0.279890255	,	0.071375764	,	0.338036887	,],
[	0.297784745	,	0.307546488	,	0.242368118	,	0.086658849	,]],
[[	0.086763846	,	0.236780625	,	0.276944763	,	0.270387443	,],
[	0.329934896	,	0.075255026	,	0.24767157	,	0.369988805	,],
[	0.393764132	,	0.275965194	,	0.06855774	,	0.334069723	,],
[	0.300740184	,	0.310167354	,	0.245112364	,	0.092034874	,]],
[[	0.086778439	,	0.239832421	,	0.279574133	,	0.272667882	,],
[	0.330002287	,	0.077483188	,	0.248013404	,	0.370198235	,],
[	0.395900428	,	0.278414459	,	0.07320302	,	0.3362762	,],
[	0.298688876	,	0.308692947	,	0.243620386	,	0.086683048	,]],
[[	0.084415464	,	0.235961967	,	0.275934055	,	0.269248322	,],
[	0.332560746	,	0.077228332	,	0.250113558	,	0.372482429	,],
[	0.395302889	,	0.277522988	,	0.069270996	,	0.335669013	,],
[	0.298544947	,	0.308177728	,	0.243078447	,	0.088206262	,]],
[[	0.090046783	,	0.238712369	,	0.278658159	,	0.272246013	,],
[	0.329368491	,	0.075265275	,	0.246935228	,	0.36946572	,],
[	0.393855502	,	0.276232504	,	0.06960385	,	0.334305729	,],
[	0.30220394	,	0.31210109	,	0.247018742	,	0.089792824	,]],
[[	0.084770824	,	0.237350279	,	0.277265637	,	0.270349593	,],
[	0.331904672	,	0.08066657	,	0.24996296	,	0.372035425	,],
[	0.398037985	,	0.280909908	,	0.076844618	,	0.338580963	,],
[	0.298026918	,	0.307815293	,	0.24258282	,	0.086506695	,]],
[[	0.086445648	,	0.24125831	,	0.280511536	,	0.267752516	,],
[	0.323924497	,	0.075132312	,	0.246829382	,	0.36393132	,],
[	0.388143891	,	0.27617772	,	0.067686619	,	0.328379525	,],
[	0.299393875	,	0.3148365	,	0.248623582	,	0.092328241	,]],
[[	0.086164876	,	0.243813118	,	0.282664842	,	0.269678929	,],
[	0.323756177	,	0.077166989	,	0.246882555	,	0.363880191	,],
[	0.389797375	,	0.278076682	,	0.071693574	,	0.330023706	,],
[	0.296585054	,	0.312272965	,	0.245872882	,	0.086081557	,]],
[[	0.084483869	,	0.2406175	,	0.279774395	,	0.266952336	,],
[	0.326637219	,	0.077282303	,	0.249317282	,	0.366465186	,],
[	0.390332647	,	0.278337094	,	0.069157097	,	0.33069186	,],
[	0.296722464	,	0.312413225	,	0.245952025	,	0.087910613	,]],
[[	0.089847208	,	0.243174493	,	0.282193913	,	0.269758913	,],
[	0.323298436	,	0.075045313	,	0.246095812	,	0.36328544	,],
[	0.388269177	,	0.276399204	,	0.06861873	,	0.328556715	,],
[	0.299903839	,	0.31553357	,	0.249209707	,	0.088667782	,]],
[[	0.084474879	,	0.241933367	,	0.280984208	,	0.267841544	,],
[	0.325675416	,	0.08039719	,	0.248969161	,	0.365711837	,],
[	0.391693402	,	0.280344028	,	0.075169818	,	0.332123378	,],
[	0.296363406	,	0.311915491	,	0.245485414	,	0.086228917	,]],
[[	0.085445431	,	0.240713338	,	0.279890135	,	0.267019814	,],
[	0.324325104	,	0.075258108	,	0.247092424	,	0.364232348	,],
[	0.388542335	,	0.276505108	,	0.067783859	,	0.328768856	,],
[	0.298605497	,	0.313906241	,	0.247637242	,	0.090894106	,]],
[[	0.087008145	,	0.244842707	,	0.28369928	,	0.270631005	,],
[	0.323508041	,	0.076394313	,	0.24659886	,	0.36360761	,],
[	0.389004428	,	0.27712364	,	0.070310537	,	0.329210762	,],
[	0.297376761	,	0.31309993	,	0.246724016	,	0.086507856	,]],
[[	0.083887001	,	0.24040368	,	0.279467046	,	0.266611809	,],
[	0.327813877	,	0.084027072	,	0.251449197	,	0.367807989	,],
[	0.391089635	,	0.279209887	,	0.06994987	,	0.331440612	,],
[	0.296607388	,	0.312098088	,	0.245758045	,	0.087433089	,]],
[[	0.088550574	,	0.242621392	,	0.281671382	,	0.26904323	,],
[	0.32341147	,	0.07481551	,	0.246132594	,	0.36327789	,],
[	0.387968607	,	0.275975762	,	0.067842431	,	0.32810093	,],
[	0.300717425	,	0.316444978	,	0.25010301	,	0.089628604	,]],
[[	0.084726879	,	0.242281834	,	0.281250309	,	0.268196989	,],
[	0.324817821	,	0.078857642	,	0.247970683	,	0.364720975	,],
[	0.391041344	,	0.279477398	,	0.07371281	,	0.331441328	,],
[	0.296335972	,	0.311943413	,	0.245424578	,	0.085942879	,]],
[[	0.086500379	,	0.246197804	,	0.285106516	,	0.268191359	,],
[	0.321470331	,	0.076066585	,	0.248733178	,	0.361865002	,],
[	0.38515977	,	0.276984137	,	0.067835033	,	0.325531641	,],
[	0.298278198	,	0.318173944	,	0.252721796	,	0.091884587	,]],
[[	0.084931883	,	0.247646773	,	0.28632793	,	0.269176108	,],
[	0.319917359	,	0.076480599	,	0.247463596	,	0.360464902	,],
[	0.385153403	,	0.277199346	,	0.069817434	,	0.325536488	,],
[	0.295820648	,	0.316093902	,	0.250495125	,	0.085960689	,]],
[[	0.084150809	,	0.245229447	,	0.283970706	,	0.267052146	,],
[	0.324171879	,	0.083946157	,	0.252299157	,	0.364691851	,],
[	0.388078385	,	0.280011599	,	0.070559931	,	0.328572994	,],
[	0.295727938	,	0.315777792	,	0.250240558	,	0.087673696	,]],
[[	0.089232988	,	0.25220679	,	0.29074567	,	0.273513957	,],
[	0.3203979	,	0.075485037	,	0.247543917	,	0.360690696	,],
[	0.384589358	,	0.276438531	,	0.067889564	,	0.324923161	,],
[	0.298935459	,	0.319058745	,	0.253584528	,	0.088685289	,]],
[[	0.083684251	,	0.246050997	,	0.284776785	,	0.267541307	,],
[	0.321337207	,	0.078966806	,	0.24898561	,	0.361785388	,],
[	0.386616726	,	0.278838736	,	0.072672656	,	0.327120692	,],
[	0.295425898	,	0.315343168	,	0.249846348	,	0.086112736	,]],
[[	0.0857961	,	0.245879303	,	0.284746767	,	0.267713446	,],
[	0.322351886	,	0.076752265	,	0.249465968	,	0.362529813	,],
[	0.385801006	,	0.27764978	,	0.06836581	,	0.326132489	,],
[	0.2973192	,	0.317111602	,	0.251687795	,	0.090352788	,]],
[[	0.086145209	,	0.248864322	,	0.287385893	,	0.270295703	,],
[	0.320051995	,	0.075945644	,	0.247401959	,	0.360388247	,],
[	0.384810176	,	0.276743412	,	0.06906287	,	0.325192105	,],
[	0.296530299	,	0.316803773	,	0.251288842	,	0.086472301	,]],
[[	0.083623702	,	0.245015026	,	0.283726925	,	0.266623446	,],
[	0.323327824	,	0.082355941	,	0.251223238	,	0.363700683	,],
[	0.389042855	,	0.280974021	,	0.071485527	,	0.329525793	,],
[	0.295454562	,	0.31550326	,	0.249960263	,	0.086922931	,]],
[[	0.088940663	,	0.247633844	,	0.286493156	,	0.269703623	,],
[	0.32082904	,	0.075630634	,	0.247842444	,	0.361051879	,],
[	0.384663112	,	0.276414739	,	0.067616279	,	0.324976406	,],
[	0.299664471	,	0.319443885	,	0.254089587	,	0.093981887	,]],
[[	0.084033686	,	0.246610764	,	0.285250762	,	0.26806147	,],
[	0.320816031	,	0.077976367	,	0.248299511	,	0.361195716	,],
[	0.386112929	,	0.278250589	,	0.07164027	,	0.32647895	,],
[	0.295316	,	0.315371904	,	0.249796435	,	0.085654496	,]],
[[	0.085827612	,	0.250123455	,	0.288835952	,	0.266982282	,],
[	0.318003782	,	0.077392867	,	0.24986672	,	0.358663518	,],
[	0.382160492	,	0.279459275	,	0.069246288	,	0.323010376	,],
[	0.294970267	,	0.320353209	,	0.254460282	,	0.089352442	,]],
[[	0.084993533	,	0.252220389	,	0.290468635	,	0.268461646	,],
[	0.315058563	,	0.075635542	,	0.246941536	,	0.355695032	,],
[	0.380350699	,	0.277692636	,	0.068837386	,	0.321185987	,],
[	0.294371173	,	0.320154177	,	0.254229498	,	0.085793705	,]],
[[	0.083319501	,	0.249007227	,	0.287555016	,	0.265672605	,],
[	0.31741736	,	0.080982112	,	0.2499505	,	0.358237582	,],
[	0.383593017	,	0.281454092	,	0.075218748	,	0.324424169	,],
[	0.292843906	,	0.318580216	,	0.252637748	,	0.085852626	,]],
[[	0.089353932	,	0.252113389	,	0.290544439	,	0.268989794	,],
[	0.316184652	,	0.075779763	,	0.24794742	,	0.356676739	,],
[	0.380748557	,	0.277899918	,	0.067990746	,	0.321496202	,],
[	0.297489563	,	0.322875306	,	0.257093433	,	0.093088709	,]],
[[	0.083106352	,	0.250120281	,	0.288533004	,	0.266452345	,],
[	0.315246332	,	0.076947734	,	0.247290184	,	0.355963044	,],
[	0.381236366	,	0.278694335	,	0.070904622	,	0.322048673	,],
[	0.293360689	,	0.318895272	,	0.252880391	,	0.084958059	,]],
[[	0.084792492	,	0.249553837	,	0.288116075	,	0.266140979	,],
[	0.319264476	,	0.078457592	,	0.251010157	,	0.359603123	,],
[	0.383194704	,	0.280415176	,	0.070074664	,	0.323880089	,],
[	0.29415775	,	0.319647741	,	0.253647895	,	0.088083643	,]],
[[	0.086391286	,	0.253779934	,	0.291974945	,	0.269927342	,],
[	0.314888039	,	0.075159302	,	0.246768542	,	0.355523382	,],
[	0.380295229	,	0.277607779	,	0.068351998	,	0.32101647	,],
[	0.295213026	,	0.320975871	,	0.255063001	,	0.086373998	,]],
[[	0.082912373	,	0.248898981	,	0.287344563	,	0.265465303	,],
[	0.316564748	,	0.079618203	,	0.248986148	,	0.357358627	,],
[	0.382960069	,	0.280638478	,	0.073880162	,	0.32374506	,],
[	0.292709602	,	0.318269937	,	0.252270268	,	0.085223006	,]],
[[	0.087919246	,	0.251189751	,	0.289764319	,	0.268087297	,],
[	0.316772655	,	0.076277932	,	0.248302458	,	0.35716713	,],
[	0.381235467	,	0.278344311	,	0.06829031	,	0.321811394	,],
[	0.296361249	,	0.32172574	,	0.25579261	,	0.091320545	,]],
[[	0.083831067	,	0.250854395	,	0.289248806	,	0.267082168	,],
[	0.31504104	,	0.076205994	,	0.246979387	,	0.35551636	,],
[	0.380921033	,	0.278233453	,	0.069824003	,	0.321500265	,],
[	0.293588165	,	0.31925171	,	0.253231338	,	0.085173114	,]],
[[	0.084664935	,	0.253669106	,	0.291387847	,	0.265233978	,],
[	0.31320641	,	0.083203855	,	0.251044445	,	0.354061704	,],
[	0.37882573	,	0.28092871	,	0.07140407	,	0.319544599	,],
[	0.291720413	,	0.322155194	,	0.256751358	,	0.087121691	,]],
[[	0.08492028	,	0.256540363	,	0.293924813	,	0.267718	,],
[	0.30959085	,	0.075192465	,	0.246684277	,	0.350405336	,],
[	0.374795214	,	0.276929157	,	0.068296536	,	0.315482566	,],
[	0.294352658	,	0.325064193	,	0.259701382	,	0.087382334	,]],
[[	0.082496671	,	0.25283312	,	0.290435561	,	0.264283992	,],
[	0.310405652	,	0.078469886	,	0.248018813	,	0.35131566	,],
[	0.376825523	,	0.27941777	,	0.073033094	,	0.31754707	,],
[	0.290647297	,	0.321415727	,	0.255863281	,	0.084827287	,]],
[[	0.088172385	,	0.255727074	,	0.293335368	,	0.267369886	,],
[	0.311831088	,	0.07670792	,	0.248691852	,	0.352422957	,],
[	0.3763809	,	0.278644111	,	0.069342253	,	0.31720875	,],
[	0.293607257	,	0.324005834	,	0.258683744	,	0.090019616	,]],
[[	0.082755162	,	0.254228255	,	0.291845159	,	0.265382397	,],
[	0.309324056	,	0.07569295	,	0.246517878	,	0.350150019	,],
[	0.37492461	,	0.277228539	,	0.069416187	,	0.315657313	,],
[	0.292439325	,	0.323030327	,	0.25760526	,	0.085625183	,]],
[[	0.083889801	,	0.253204511	,	0.29100313	,	0.264782502	,],
[	0.312015811	,	0.081498654	,	0.249707566	,	0.352834006	,],
[	0.379184876	,	0.282113275	,	0.076843804	,	0.320080856	,],
[	0.291399202	,	0.321831874	,	0.256376838	,	0.086342829	,]],
[[	0.086193195	,	0.257992791	,	0.295287458	,	0.269035201	,],
[	0.310023517	,	0.075217105	,	0.247100874	,	0.35071906	,],
[	0.375177143	,	0.277318516	,	0.068335156	,	0.315750148	,],
[	0.295192193	,	0.325881937	,	0.26063497	,	0.088160158	,]],
[[	0.082399415	,	0.253181096	,	0.290533504	,	0.264380579	,],
[	0.31002955	,	0.077503878	,	0.24739349	,	0.350875335	,],
[	0.376073348	,	0.278627332	,	0.07160086	,	0.316730084	,],
[	0.290903678	,	0.321559271	,	0.256041908	,	0.084643978	,]],
[[	0.086540602	,	0.254631509	,	0.292203177	,	0.266213556	,],
[	0.313188366	,	0.077818919	,	0.249888953	,	0.353612398	,],
[	0.377420297	,	0.27953617	,	0.069997409	,	0.318099623	,],
[	0.292629819	,	0.323172972	,	0.257783854	,	0.088703823	,]],
[[	0.083506634	,	0.254974747	,	0.292499458	,	0.266121435	,],
[	0.309354505	,	0.075305502	,	0.246434506	,	0.350014125	,],
[	0.374941501	,	0.277100618	,	0.068921695	,	0.3155412	,],
[	0.293037633	,	0.323594213	,	0.258073782	,	0.086110414	,]],
[[	0.083326285	,	0.256642388	,	0.294067043	,	0.263025769	,],
[	0.304384683	,	0.079408282	,	0.247805627	,	0.346102386	,],
[	0.37053526	,	0.279238806	,	0.072832827	,	0.312283433	,],
[	0.288019808	,	0.323569466	,	0.258675647	,	0.084451726	,]],
[[	0.084937315	,	0.260432477	,	0.297492547	,	0.26642706	,],
[	0.305071008	,	0.076591423	,	0.247937392	,	0.346530946	,],
[	0.370083018	,	0.278438429	,	0.069092866	,	0.311891217	,],
[	0.292474864	,	0.327678456	,	0.26293153	,	0.091526271	,]],
[[	0.081524803	,	0.256041185	,	0.293321935	,	0.262301128	,],
[	0.303015124	,	0.07627478	,	0.246188308	,	0.344712116	,],
[	0.368546596	,	0.276947465	,	0.069225758	,	0.310264905	,],
[	0.288357984	,	0.323913817	,	0.259087371	,	0.083828359	,]],
[[	0.086331821	,	0.258026124	,	0.295333715	,	0.264555137	,],
[	0.306775135	,	0.083181141	,	0.250577296	,	0.348471545	,],
[	0.372667171	,	0.281588283	,	0.076452334	,	0.314432395	,],
[	0.289508955	,	0.324973586	,	0.260037864	,	0.086803459	,]],
[[	0.082362273	,	0.257802308	,	0.295033831	,	0.263881979	,],
[	0.30371678	,	0.075619433	,	0.246468254	,	0.345129193	,],
[	0.368652544	,	0.27697887	,	0.068132867	,	0.310375771	,],
[	0.291129197	,	0.32664768	,	0.261755629	,	0.085842612	,]],
[[	0.082616139	,	0.25624382	,	0.293624791	,	0.262598098	,],
[	0.303890375	,	0.078338557	,	0.247127922	,	0.345467061	,],
[	0.369786539	,	0.278403566	,	0.071556748	,	0.311389432	,],
[	0.288213575	,	0.323635724	,	0.258700604	,	0.08413287	,]],
[[	0.08600182	,	0.261560854	,	0.298616233	,	0.267567524	,],
[	0.306131895	,	0.077355584	,	0.248924282	,	0.347428554	,],
[	0.370836831	,	0.279094504	,	0.069609017	,	0.312553713	,],
[	0.291700786	,	0.3267466	,	0.261939708	,	0.090063471	,]],
[[	0.08157708	,	0.256305097	,	0.293501046	,	0.262395924	,],
[	0.302958632	,	0.07565848	,	0.246004984	,	0.344526382	,],
[	0.368382688	,	0.276783859	,	0.068581384	,	0.310040138	,],
[	0.288869702	,	0.324436041	,	0.259619795	,	0.083985317	,]],
[[	0.085051542	,	0.257466761	,	0.294719866	,	0.263906401	,],
[	0.305591929	,	0.081432963	,	0.249211624	,	0.347283431	,],
[	0.371954025	,	0.280845509	,	0.075134714	,	0.313729343	,],
[	0.288880875	,	0.324430883	,	0.259497047	,	0.085764725	,]],
[[	0.083526761	,	0.258908312	,	0.296195183	,	0.264906757	,],
[	0.30418019	,	0.075815523	,	0.246881186	,	0.345483635	,],
[	0.369186673	,	0.277326529	,	0.068254011	,	0.310766232	,],
[	0.291774538	,	0.327339882	,	0.262537475	,	0.086492007	,]],
[[	0.082158828	,	0.259538554	,	0.296204044	,	0.26074293	,],
[	0.297802814	,	0.077623978	,	0.245608806	,	0.339591698	,],
[	0.363763536	,	0.277456897	,	0.070267026	,	0.305993699	,],
[	0.285488712	,	0.326158466	,	0.261240311	,	0.083974747	,]],
[[	0.083173017	,	0.262876572	,	0.299115366	,	0.263663933	,],
[	0.299777927	,	0.076350601	,	0.246993534	,	0.34125994	,],
[	0.365686743	,	0.279021102	,	0.069399876	,	0.307842981	,],
[	0.289457634	,	0.329987454	,	0.265106385	,	0.090588874	,]],
[[	0.080181976	,	0.258745614	,	0.295205661	,	0.259790831	,],
[	0.296877427	,	0.075130595	,	0.244431319	,	0.338626191	,],
[	0.362565447	,	0.275995316	,	0.06744675	,	0.304664165	,],
[	0.285421216	,	0.326368744	,	0.261473361	,	0.082840606	,]],
[[	0.085613663	,	0.261478059	,	0.298017309	,	0.262794258	,],
[	0.299872633	,	0.081177133	,	0.248047389	,	0.341615523	,],
[	0.365648373	,	0.279553661	,	0.073537984	,	0.307787008	,],
[	0.286673835	,	0.327540926	,	0.262552206	,	0.086177358	,]],
[[	0.081195685	,	0.260978035	,	0.297447075	,	0.261796116	,],
[	0.297917856	,	0.07500936	,	0.245152738	,	0.339389538	,],
[	0.363708446	,	0.277051406	,	0.067680758	,	0.305750871	,],
[	0.287983568	,	0.328729411	,	0.263719339	,	0.08481121	,]],
[[	0.081261615	,	0.259094254	,	0.295652502	,	0.260132548	,],
[	0.297534895	,	0.076912337	,	0.245227317	,	0.339138685	,],
[	0.36326676	,	0.276801839	,	0.069274062	,	0.305296768	,],
[	0.285016908	,	0.325783178	,	0.260796875	,	0.083141255	,]],
[[	0.084919173	,	0.264764597	,	0.300898455	,	0.265587529	,],
[	0.300710096	,	0.077170897	,	0.247807737	,	0.34207412	,],
[	0.366536769	,	0.279808817	,	0.070094284	,	0.308651668	,],
[	0.288548979	,	0.329047061	,	0.264188327	,	0.089142946	,]],
[[	0.080547732	,	0.259435304	,	0.295852026	,	0.260365007	,],
[	0.296756044	,	0.074498337	,	0.244152443	,	0.33839444	,],
[	0.362680692	,	0.275967639	,	0.067257682	,	0.30470068	,],
[	0.285993723	,	0.326805754	,	0.261851744	,	0.083188313	,]],
[[	0.084098456	,	0.260574654	,	0.297081258	,	0.261800742	,],
[	0.299106487	,	0.079686803	,	0.247027928	,	0.340780658	,],
[	0.364893356	,	0.278614569	,	0.072183827	,	0.306938127	,],
[	0.285825557	,	0.326593148	,	0.261581494	,	0.084778158	,]],
[[	0.081846563	,	0.261483049	,	0.297887545	,	0.262331607	,],
[	0.29874976	,	0.075472589	,	0.245847565	,	0.340081565	,],
[	0.364377465	,	0.277646588	,	0.068138311	,	0.306481571	,],
[	0.289063071	,	0.329874074	,	0.264888306	,	0.085829811	,]],
[[	0.082221902	,	0.263795591	,	0.300746236	,	0.260252319	,],
[	0.293764523	,	0.077787783	,	0.246703592	,	0.335802373	,],
[	0.359549868	,	0.27819218	,	0.071396161	,	0.301605733	,],
[	0.283799952	,	0.329430631	,	0.264773402	,	0.082839436	,]],
[[	0.083290446	,	0.267264956	,	0.303912111	,	0.263413502	,],
[	0.296684704	,	0.077741753	,	0.249058312	,	0.338422202	,],
[	0.360806848	,	0.278804307	,	0.069436142	,	0.302637884	,],
[	0.287912147	,	0.3334056	,	0.268744963	,	0.089464561	,]],
[[	0.08028154	,	0.263225174	,	0.299993142	,	0.259490992	,],
[	0.292990222	,	0.075472022	,	0.245635407	,	0.334917989	,],
[	0.358361295	,	0.276679884	,	0.068527428	,	0.300225268	,],
[	0.284450116	,	0.330371834	,	0.265669284	,	0.082514739	,]],
[[	0.085329229	,	0.265536287	,	0.302356482	,	0.262096256	,],
[	0.295148449	,	0.080502546	,	0.248331371	,	0.337026918	,],
[	0.361530602	,	0.280370213	,	0.074751552	,	0.303454286	,],
[	0.285026882	,	0.330761774	,	0.266070789	,	0.085097151	,]],
[[	0.080914543	,	0.264791675	,	0.301613195	,	0.260913319	,],
[	0.294597234	,	0.075957637	,	0.246843116	,	0.336141148	,],
[	0.359038908	,	0.277193589	,	0.068061265	,	0.300934076	,],
[	0.287343764	,	0.33298652	,	0.268305234	,	0.084776834	,]],
[[	0.081256695	,	0.26338079	,	0.300285175	,	0.259690571	,],
[	0.293315455	,	0.076846905	,	0.245942632	,	0.335100856	,],
[	0.35906971	,	0.277531031	,	0.070313946	,	0.300888812	,],
[	0.284074057	,	0.329684721	,	0.264912443	,	0.082664953	,]],
[[	0.084133544	,	0.268311268	,	0.304800845	,	0.26430823	,],
[	0.297521098	,	0.084509948	,	0.25087893	,	0.339322301	,],
[	0.361979542	,	0.279984747	,	0.07052559	,	0.303815154	,],
[	0.287037641	,	0.332592439	,	0.267890298	,	0.088077949	,]],
[[	0.080128928	,	0.263368736	,	0.300092822	,	0.259583254	,],
[	0.293109988	,	0.075138052	,	0.245646686	,	0.334990042	,],
[	0.358426176	,	0.276555825	,	0.068098004	,	0.300130077	,],
[	0.285070108	,	0.330897459	,	0.266218437	,	0.082800935	,]],
[[	0.084113351	,	0.264904833	,	0.301601968	,	0.261286327	,],
[	0.294692973	,	0.079458644	,	0.247661774	,	0.336589129	,],
[	0.36062025	,	0.279285364	,	0.073221454	,	0.302364382	,],
[	0.28444729	,	0.330043127	,	0.265378816	,	0.083868195	,]],
[[	0.081443781	,	0.265410152	,	0.30212304	,	0.261440009	,],
[	0.295119413	,	0.076366842	,	0.247368309	,	0.336698327	,],
[	0.359561888	,	0.277666155	,	0.068492985	,	0.30134948	,],
[	0.288384732	,	0.33399435	,	0.269376629	,	0.085698036	,]],
[[	0.081467379	,	0.266761829	,	0.30235554	,	0.25728136	,],
[	0.286579512	,	0.077030793	,	0.244862793	,	0.32839978	,],
[	0.352357586	,	0.276633072	,	0.069158037	,	0.294563886	,],
[	0.280654621	,	0.331206712	,	0.266605326	,	0.082254035	,]],
[[	0.082483974	,	0.270345932	,	0.30548287	,	0.260485293	,],
[	0.290375754	,	0.078000135	,	0.248060816	,	0.331798949	,],
[	0.355149055	,	0.278965115	,	0.069327767	,	0.297316301	,],
[	0.284435696	,	0.334758278	,	0.270231868	,	0.088835339	,]],
[[	0.079667392	,	0.26620158	,	0.301564139	,	0.256574596	,],
[	0.285947321	,	0.074998841	,	0.243990801	,	0.327690557	,],
[	0.351679125	,	0.275698143	,	0.067130508	,	0.29381619	,],
[	0.281049426	,	0.331795488	,	0.267256205	,	0.081934608	,]],
[[	0.084889677	,	0.268648981	,	0.304039078	,	0.259153151	,],
[	0.288111547	,	0.079882765	,	0.246678632	,	0.329841566	,],
[	0.353950526	,	0.278498095	,	0.072099935	,	0.296211166	,],
[	0.281416002	,	0.331976146	,	0.267460106	,	0.084123587	,]],
[[	0.080128584	,	0.267845471	,	0.303285384	,	0.258039837	,],
[	0.288086821	,	0.076071591	,	0.245803128	,	0.329560269	,],
[	0.353156778	,	0.277090878	,	0.067674418	,	0.295372111	,],
[	0.284227787	,	0.334727452	,	0.27021485	,	0.084371019	,]],
[[	0.080553825	,	0.266365762	,	0.30187928	,	0.256763597	,],
[	0.286225643	,	0.076228355	,	0.244328712	,	0.327855506	,],
[	0.352120615	,	0.276323133	,	0.068548751	,	0.294291383	,],
[	0.280658794	,	0.331174336	,	0.266554167	,	0.081985152	,]],
[[	0.083791505	,	0.271662761	,	0.306780983	,	0.261767394	,],
[	0.290385495	,	0.083767361	,	0.249127328	,	0.331973841	,],
[	0.356462985	,	0.281264063	,	0.076157836	,	0.29868124	,],
[	0.28359881	,	0.333955064	,	0.269408453	,	0.087333914	,]],
[[	0.079679989	,	0.266487953	,	0.301805885	,	0.256818766	,],
[	0.286540629	,	0.075135576	,	0.244423023	,	0.328164205	,],
[	0.352120229	,	0.275955198	,	0.067036688	,	0.294193075	,],
[	0.281627344	,	0.332272247	,	0.267655171	,	0.082070761	,]],
[[	0.083453401	,	0.267874983	,	0.303254154	,	0.258328559	,],
[	0.287440317	,	0.07877254	,	0.24588516	,	0.329173794	,],
[	0.35347644	,	0.277819938	,	0.07109699	,	0.295521272	,],
[	0.281154852	,	0.331697901	,	0.267051899	,	0.083400227	,]],
[[	0.081097003	,	0.268824403	,	0.30414823	,	0.25897629	,],
[	0.288817158	,	0.076529003	,	0.246372854	,	0.33009337	,],
[	0.354122834	,	0.27792875	,	0.068336502	,	0.296248037	,],
[	0.285228194	,	0.335795547	,	0.271188152	,	0.085323276	,]],
[[	0.080188646	,	0.27032876	,	0.305557289	,	0.255619822	,],
[	0.280225664	,	0.074933178	,	0.243173078	,	0.322189656	,],
[	0.346589982	,	0.275791932	,	0.067409659	,	0.288907394	,],
[	0.278765508	,	0.334565072	,	0.269993267	,	0.080705785	,]],
[[	0.083087611	,	0.275061833	,	0.309980202	,	0.260126662	,],
[	0.282656098	,	0.080309085	,	0.246146458	,	0.324689261	,],
[	0.349173606	,	0.278913034	,	0.072817171	,	0.291512963	,],
[	0.2810818	,	0.336570294	,	0.272113306	,	0.085366158	,]],
[[	0.078796329	,	0.270014112	,	0.305057879	,	0.255200067	,],
[	0.281697459	,	0.075272086	,	0.244458074	,	0.323513277	,],
[	0.34795966	,	0.27695603	,	0.067769806	,	0.290265498	,],
[	0.280149691	,	0.335906697	,	0.271606309	,	0.081334156	,]],
[[	0.083175458	,	0.271856301	,	0.306975618	,	0.257286616	,],
[	0.280599999	,	0.076420579	,	0.243723634	,	0.322574943	,],
[	0.347194957	,	0.276576485	,	0.068925348	,	0.289312797	,],
[	0.278965449	,	0.334699753	,	0.27027961	,	0.081666144	,]],
[[	0.080237477	,	0.272359932	,	0.307449973	,	0.257382296	,],
[	0.284830546	,	0.08409684	,	0.24866572	,	0.32679436	,],
[	0.351267838	,	0.280250187	,	0.070666714	,	0.293562446	,],
[	0.283452317	,	0.338953312	,	0.274648467	,	0.08902735	,]],
[[	0.079574541	,	0.269948119	,	0.30516105	,	0.255205948	,],
[	0.280579513	,	0.07487001	,	0.243332855	,	0.322321867	,],
[	0.346979767	,	0.27607189	,	0.067326785	,	0.289145555	,],
[	0.279183511	,	0.334765821	,	0.270279408	,	0.080643734	,]],
[[	0.084480936	,	0.276500858	,	0.311246816	,	0.26148096	,],
[	0.282108619	,	0.079242089	,	0.24543129	,	0.324023148	,],
[	0.348267522	,	0.277820499	,	0.071304088	,	0.290478041	,],
[	0.280515539	,	0.336108519	,	0.271597425	,	0.084312975	,]],
[[	0.079223991	,	0.270732016	,	0.305639406	,	0.255765633	,],
[	0.282570038	,	0.075992118	,	0.245151885	,	0.324202504	,],
[	0.348819873	,	0.277807246	,	0.068396257	,	0.291043228	,],
[	0.281120644	,	0.336824868	,	0.272400452	,	0.08203472	,]],
[[	0.082000392	,	0.271265829	,	0.306333448	,	0.25662632	,],
[	0.280328084	,	0.075672469	,	0.243361845	,	0.322245517	,],
[	0.347045606	,	0.27626774	,	0.068252662	,	0.289124059	,],
[	0.278563415	,	0.334280083	,	0.269838105	,	0.081054678	,]],
[[	0.08149587	,	0.273538084	,	0.308574608	,	0.258554453	,],
[	0.283856351	,	0.082481582	,	0.24754569	,	0.325716383	,],
[	0.350525056	,	0.28040535	,	0.074953339	,	0.292742537	,],
[	0.282698541	,	0.338171119	,	0.27381989	,	0.087626686	,]],
[[	0.080115561	,	0.275588911	,	0.31140321	,	0.255976432	,],
[	0.275642277	,	0.074534253	,	0.245026601	,	0.319076093	,],
[	0.342338261	,	0.277294985	,	0.06832048	,	0.285904356	,],
[	0.277604306	,	0.338977419	,	0.27486891	,	0.081255585	,]],
[[	0.082298819	,	0.279757174	,	0.31526311	,	0.259974685	,],
[	0.277799117	,	0.079706941	,	0.247673171	,	0.321219831	,],
[	0.344870623	,	0.280472056	,	0.07386689	,	0.288609257	,],
[	0.280106141	,	0.341355581	,	0.277257753	,	0.085971379	,]],
[[	0.078499411	,	0.275041805	,	0.310648432	,	0.255365803	,],
[	0.277217065	,	0.075029535	,	0.246270565	,	0.320446912	,],
[	0.343617423	,	0.278385664	,	0.068692251	,	0.287233291	,],
[	0.279080476	,	0.340542747	,	0.276468792	,	0.082056752	,]],
[[	0.083227342	,	0.277201757	,	0.312833356	,	0.257697083	,],
[	0.275828238	,	0.075824987	,	0.245358432	,	0.31930547	,],
[	0.342487295	,	0.277693267	,	0.069662022	,	0.28612153	,],
[	0.278009627	,	0.339459452	,	0.275394227	,	0.082522675	,]],
[[	0.079628958	,	0.277167783	,	0.312785557	,	0.257382893	,],
[	0.279972196	,	0.083507671	,	0.250238925	,	0.323371147	,],
[	0.346573223	,	0.281262684	,	0.071303625	,	0.290204332	,],
[	0.282378961	,	0.343507215	,	0.279580525	,	0.089583688	,]],
[[	0.079325947	,	0.275225592	,	0.311013988	,	0.255549491	,],
[	0.276003943	,	0.074612347	,	0.245180874	,	0.319359155	,],
[	0.342434485	,	0.277267252	,	0.067946415	,	0.28588727	,],
[	0.278137938	,	0.339600708	,	0.275314202	,	0.081353867	,]],
[[	0.084013799	,	0.281499367	,	0.316882381	,	0.261625216	,],
[	0.277131937	,	0.078482111	,	0.246929048	,	0.320479833	,],
[	0.343871802	,	0.279211262	,	0.072262372	,	0.287397496	,],
[	0.279498994	,	0.340827251	,	0.276634284	,	0.08498513	,]],
[[	0.078875766	,	0.275766918	,	0.311301169	,	0.255992535	,],
[	0.278167805	,	0.075860263	,	0.247284843	,	0.321363966	,],
[	0.344062118	,	0.278877297	,	0.069035506	,	0.287618806	,],
[	0.28009861	,	0.341417549	,	0.277259948	,	0.082717655	,]],
[[	0.081974519	,	0.276622201	,	0.312189817	,	0.256906722	,],
[	0.275605412	,	0.075124911	,	0.24498145	,	0.318859735	,],
[	0.34213702	,	0.277300017	,	0.068890425	,	0.285700202	,],
[	0.277892841	,	0.339266421	,	0.275096569	,	0.081871804	,]],
[[	0.080738167	,	0.278226907	,	0.313832101	,	0.258391419	,],
[	0.2791263	,	0.081843426	,	0.249227796	,	0.32248272	,],
[	0.346245714	,	0.281945483	,	0.076082928	,	0.289918013	,],
[	0.281563273	,	0.34274473	,	0.278767847	,	0.088072501	,]],
[[	0.081427556	,	0.285555784	,	0.322192112	,	0.260888372	,],
[	0.277747141	,	0.077074017	,	0.2518539	,	0.322368986	,],
[	0.346491662	,	0.285433799	,	0.07047626	,	0.288367805	,],
[	0.283789178	,	0.352084581	,	0.286733973	,	0.083292066	,]],
[[	0.084786447	,	0.290714168	,	0.327116877	,	0.265906676	,],
[	0.279371635	,	0.081358749	,	0.253987415	,	0.324038801	,],
[	0.348845652	,	0.28851375	,	0.075781043	,	0.290941512	,],
[	0.285629679	,	0.353908498	,	0.288589051	,	0.087458365	,]],
[[	0.080557052	,	0.285753108	,	0.322268889	,	0.260981839	,],
[	0.27979682	,	0.078273132	,	0.253813518	,	0.324336357	,],
[	0.347613621	,	0.286438409	,	0.070806835	,	0.289595547	,],
[	0.285411862	,	0.353778307	,	0.288502247	,	0.084189269	,]],
[[	0.083960682	,	0.286777019	,	0.323295307	,	0.262117491	,],
[	0.277623609	,	0.077947704	,	0.251893472	,	0.322246433	,],
[	0.346493064	,	0.285803105	,	0.07163912	,	0.288486378	,],
[	0.283692298	,	0.352055887	,	0.286709777	,	0.083884646	,]],
[[	0.081949425	,	0.287967911	,	0.324467475	,	0.263088919	,],
[	0.280893695	,	0.084460686	,	0.255879588	,	0.325598859	,],
[	0.350725732	,	0.28951238	,	0.073480289	,	0.292857763	,],
[	0.287796151	,	0.355960442	,	0.290758525	,	0.090803255	,]],
[[	0.080888271	,	0.285355699	,	0.322079273	,	0.260687268	,],
[	0.278089763	,	0.077060431	,	0.252144921	,	0.322705674	,],
[	0.34639855	,	0.285352722	,	0.070125854	,	0.288376814	,],
[	0.284309253	,	0.352477148	,	0.287131931	,	0.083313623	,]],
[[	0.086200675	,	0.292183776	,	0.328438649	,	0.267266749	,],
[	0.27854663	,	0.080064556	,	0.253052301	,	0.323158662	,],
[	0.347991441	,	0.287460324	,	0.074288729	,	0.289913905	,],
[	0.28503381	,	0.353170199	,	0.287868403	,	0.086149748	,]],
[[	0.080653907	,	0.286103286	,	0.322523517	,	0.261317214	,],
[	0.280851529	,	0.079012849	,	0.254759519	,	0.325394273	,],
[	0.348101802	,	0.286969485	,	0.071133387	,	0.290065993	,],
[	0.286203902	,	0.354493456	,	0.28909697	,	0.084693991	,]],
[[	0.082930323	,	0.286351119	,	0.322892335	,	0.26159556	,],
[	0.277815856	,	0.077626852	,	0.25200678	,	0.322381844	,],
[	0.346552745	,	0.285666438	,	0.071122335	,	0.288430037	,],
[	0.2837167	,	0.352111739	,	0.286716385	,	0.083541152	,]],
[[	0.082950171	,	0.288996593	,	0.325387748	,	0.264093504	,],
[	0.280185965	,	0.08313512	,	0.254990378	,	0.324844429	,],
[	0.350361768	,	0.290124527	,	0.078137687	,	0.29235005	,],
[	0.286898792	,	0.355005851	,	0.289718911	,	0.089223437	,]],
[[	0.081176511	,	0.291674288	,	0.327789158	,	0.261299394	,],
[	0.275543315	,	0.077816077	,	0.254669255	,	0.320471302	,],
[	0.344623129	,	0.288218571	,	0.071150543	,	0.286043076	,],
[	0.284615131	,	0.358533904	,	0.292324172	,	0.083037032	,]],
[[	0.085113479	,	0.29716026	,	0.33302618	,	0.266759892	,],
[	0.27557998	,	0.080361855	,	0.255218448	,	0.320654123	,],
[	0.345756224	,	0.28970906	,	0.074758227	,	0.287179416	,],
[	0.286464082	,	0.360344223	,	0.294116362	,	0.086939696	,]],
[[	0.080401525	,	0.291837331	,	0.327789542	,	0.26138124	,],
[	0.278025889	,	0.079547176	,	0.2570367	,	0.322862212	,],
[	0.34661776	,	0.289934739	,	0.072340976	,	0.287966851	,],
[	0.28620896	,	0.360163548	,	0.294066092	,	0.084016705	,]],
[[	0.083635455	,	0.292833236	,	0.328837184	,	0.262493807	,],
[	0.274999009	,	0.078023928	,	0.254314521	,	0.319952915	,],
[	0.344340518	,	0.288082399	,	0.071648069	,	0.285607591	,],
[	0.284528316	,	0.358593736	,	0.292477037	,	0.08368866	,]],
[[	0.082159937	,	0.294269274	,	0.330195766	,	0.263692534	,],
[	0.277148012	,	0.083341957	,	0.257156768	,	0.322159837	,],
[	0.348004979	,	0.292187891	,	0.078369193	,	0.289332363	,],
[	0.28827616	,	0.362164878	,	0.296202029	,	0.090122208	,]],
[[	0.080829219	,	0.291626819	,	0.327722239	,	0.261122577	,],
[	0.276204416	,	0.078130794	,	0.255146751	,	0.320953315	,],
[	0.344916726	,	0.288336473	,	0.071131832	,	0.286254178	,],
[	0.285025588	,	0.358885198	,	0.292649732	,	0.083008525	,]],
[[	0.086378076	,	0.298628539	,	0.334329889	,	0.268062516	,],
[	0.275427954	,	0.079714004	,	0.254863149	,	0.320351413	,],
[	0.345215518	,	0.289134411	,	0.073742462	,	0.286551596	,],
[	0.285762722	,	0.359559411	,	0.293343202	,	0.085703611	,]],
[[	0.080765178	,	0.292381703	,	0.328375273	,	0.261915939	,],
[	0.279465639	,	0.080774014	,	0.258340481	,	0.324076548	,],
[	0.347615924	,	0.290947221	,	0.073189914	,	0.288939121	,],
[	0.287175052	,	0.361080435	,	0.295060793	,	0.084739077	,]],
[[	0.082682793	,	0.292293231	,	0.328341932	,	0.261915669	,],
[	0.27485637	,	0.077621316	,	0.254157174	,	0.319800662	,],
[	0.344557141	,	0.288097441	,	0.071533362	,	0.285686276	,],
[	0.284678742	,	0.358665443	,	0.292479297	,	0.08329289	,]],
[[	0.083187927	,	0.295275014	,	0.331330936	,	0.264784786	,],
[	0.276602415	,	0.082127863	,	0.256411219	,	0.321549356	,],
[	0.346948208	,	0.291148413	,	0.076793573	,	0.2882873	,],
[	0.287928622	,	0.361693082	,	0.295595764	,	0.088905796	,]],
[[	0.080917027	,	0.297231131	,	0.332905967	,	0.26122294	,],
[	0.272188071	,	0.07864238	,	0.256149095	,	0.316906435	,],
[	0.342049259	,	0.290722987	,	0.072154999	,	0.283303158	,],
[	0.284504796	,	0.36373679	,	0.297485333	,	0.08298106	,]],
[[	0.084650541	,	0.302423751	,	0.337887981	,	0.266448562	,],
[	0.271481601	,	0.08018562	,	0.256004204	,	0.316294329	,],
[	0.341717215	,	0.290778921	,	0.073980713	,	0.282921178	,],
[	0.28532995	,	0.364597668	,	0.298314246	,	0.085610797	,]],
[[	0.080333841	,	0.297523101	,	0.333121806	,	0.261472437	,],
[	0.275365068	,	0.081350073	,	0.259232885	,	0.319997857	,],
[	0.345115267	,	0.293547055	,	0.07469378	,	0.286406281	,],
[	0.287187594	,	0.366446876	,	0.30037665	,	0.085167814	,]],
[[	0.083451242	,	0.298385452	,	0.333945176	,	0.262583079	,],
[	0.270967505	,	0.078274305	,	0.255209952	,	0.315825885	,],
[	0.341309751	,	0.290023688	,	0.072041894	,	0.282449015	,],
[	0.283933894	,	0.363302544	,	0.297018595	,	0.08297867	,]],
[[	0.082015656	,	0.299846082	,	0.335383996	,	0.263824191	,],
[	0.272231696	,	0.082358149	,	0.256994552	,	0.317096892	,],
[	0.343810866	,	0.292938359	,	0.077227983	,	0.284792981	,],
[	0.287015213	,	0.366188215	,	0.300122093	,	0.088404364	,]],
[[	0.080513742	,	0.297095285	,	0.332803632	,	0.261117329	,],
[	0.272900413	,	0.079146061	,	0.256816212	,	0.317589409	,],
[	0.342654922	,	0.291316341	,	0.072587651	,	0.283926182	,],
[	0.285347875	,	0.36447802	,	0.29832779	,	0.083500918	,]],
[[	0.086255422	,	0.304043992	,	0.33940165	,	0.268042853	,],
[	0.271060425	,	0.079383983	,	0.255426989	,	0.31582008	,],
[	0.34156681	,	0.29051567	,	0.07335026	,	0.282693563	,],
[	0.284885011	,	0.364070396	,	0.297781904	,	0.084660466	,]],
[[	0.080593148	,	0.298045774	,	0.333524672	,	0.262021861	,],
[	0.275027659	,	0.086558929	,	0.259962354	,	0.319662546	,],
[	0.345588662	,	0.295159893	,	0.080715061	,	0.286794155	,],
[	0.28829113	,	0.367428731	,	0.301360055	,	0.086115917	,]],
[[	0.08245939	,	0.29780314	,	0.333487898	,	0.261892673	,],
[	0.271335545	,	0.078208076	,	0.25547843	,	0.316084357	,],
[	0.341434893	,	0.290046511	,	0.071745021	,	0.282527463	,],
[	0.284334909	,	0.363693123	,	0.297445163	,	0.083147989	,]],
[[	0.082923787	,	0.300693937	,	0.336307494	,	0.26465174	,],
[	0.271892725	,	0.081393294	,	0.256564067	,	0.31673106	,],
[	0.342769483	,	0.291765935	,	0.075592567	,	0.283774125	,],
[	0.286388323	,	0.365541988	,	0.299369096	,	0.087175447	,]],
[[	0.081161775	,	0.302589863	,	0.338182792	,	0.261494543	,],
[	0.268953893	,	0.080050576	,	0.258575841	,	0.314917155	,],
[	0.339218701	,	0.292720144	,	0.072595241	,	0.28099686	,],
[	0.283933717	,	0.367782112	,	0.301656552	,	0.083208462	,]],
[[	0.085342869	,	0.308122179	,	0.343483208	,	0.267096654	,],
[	0.266966149	,	0.079986393	,	0.257097599	,	0.313143759	,],
[	0.337630658	,	0.291590864	,	0.072884927	,	0.279414922	,],
[	0.284115838	,	0.36797905	,	0.301739659	,	0.085112948	,]],
[[	0.080717011	,	0.302958903	,	0.338431828	,	0.261836987	,],
[	0.270689138	,	0.086958589	,	0.261394483	,	0.316789101	,],
[	0.341504009	,	0.295990401	,	0.079967292	,	0.283373092	,],
[	0.286613473	,	0.370306947	,	0.304427045	,	0.085466502	,]],
[[	0.083737052	,	0.303809487	,	0.339369501	,	0.262815322	,],
[	0.267302584	,	0.079049015	,	0.257258442	,	0.313538786	,],
[	0.337836003	,	0.291285963	,	0.071615025	,	0.279418505	,],
[	0.283429305	,	0.367397798	,	0.301214227	,	0.083265883	,]],
[[	0.082734308	,	0.305791573	,	0.341171426	,	0.264537979	,],
[	0.267603065	,	0.082014546	,	0.258097445	,	0.313886548	,],
[	0.339090932	,	0.293112329	,	0.07546361	,	0.280706571	,],
[	0.285945277	,	0.369907587	,	0.3037732	,	0.08811666	,]],
[[	0.080955511	,	0.302606186	,	0.338124466	,	0.261481718	,],
[	0.269716098	,	0.080636705	,	0.259236542	,	0.31560466	,],
[	0.339784157	,	0.293206683	,	0.073132805	,	0.281594052	,],
[	0.284447145	,	0.368270321	,	0.302067706	,	0.083556377	,]],
[[	0.086884741	,	0.309574009	,	0.344763519	,	0.268462334	,],
[	0.267124869	,	0.079647745	,	0.257054122	,	0.313175814	,],
[	0.33738548	,	0.291116298	,	0.072127774	,	0.278954282	,],
[	0.284091565	,	0.367880517	,	0.301640871	,	0.084604195	,]],
[[	0.080953228	,	0.303451235	,	0.338771076	,	0.262270819	,],
[	0.269858068	,	0.085456451	,	0.260466333	,	0.315989567	,],
[	0.340567608	,	0.294961432	,	0.078351029	,	0.282250319	,],
[	0.28800012	,	0.371650541	,	0.305637246	,	0.086611716	,]],
[[	0.082869618	,	0.30353851	,	0.338965677	,	0.262310941	,],
[	0.267728097	,	0.079137848	,	0.257463002	,	0.313745104	,],
[	0.338144002	,	0.291611215	,	0.071740954	,	0.279780773	,],
[	0.283252863	,	0.367195142	,	0.300991627	,	0.082725302	,]],
[[	0.083698669	,	0.306604635	,	0.341962909	,	0.265389401	,],
[	0.267547419	,	0.081287515	,	0.257773467	,	0.313629649	,],
[	0.338321366	,	0.29224568	,	0.074165041	,	0.279949603	,],
[	0.285374798	,	0.36930383	,	0.303126712	,	0.086978476	,]],
[[	0.08065241	,	0.307745127	,	0.343256851	,	0.261868949	,],
[	0.265909123	,	0.081248598	,	0.260464666	,	0.312725247	,],
[	0.336444111	,	0.295583377	,	0.073692392	,	0.278417481	,],
[	0.284204436	,	0.374171342	,	0.307451215	,	0.083781327	,]],
[[	0.084335854	,	0.312887605	,	0.348090079	,	0.266954698	,],
[	0.262492765	,	0.079475231	,	0.257526259	,	0.309573239	,],
[	0.334573771	,	0.29392124	,	0.073302882	,	0.276240926	,],
[	0.284223168	,	0.374259776	,	0.307565556	,	0.085612974	,]],
[[	0.079459521	,	0.307326822	,	0.3427648	,	0.26142183	,],
[	0.265501407	,	0.085772157	,	0.261142174	,	0.312534518	,],
[	0.338263676	,	0.298239029	,	0.080185798	,	0.280050901	,],
[	0.286903743	,	0.376786481	,	0.310279374	,	0.086061917	,]],
[[	0.083280587	,	0.308972871	,	0.344429841	,	0.263095122	,],
[	0.263479752	,	0.079218651	,	0.25822	,	0.310375884	,],
[	0.334949156	,	0.294151779	,	0.072647648	,	0.276635583	,],
[	0.283343686	,	0.373406942	,	0.306723218	,	0.0834889	,]],
[[	0.081365475	,	0.310166535	,	0.345465392	,	0.264087443	,],
[	0.263104344	,	0.081363335	,	0.258454493	,	0.310200508	,],
[	0.335503867	,	0.295068537	,	0.075347818	,	0.277010378	,],
[	0.285785594	,	0.375826919	,	0.309222037	,	0.088233245	,]],
[[	0.079902559	,	0.307211965	,	0.342768808	,	0.261302697	,],
[	0.266742048	,	0.081930403	,	0.261182624	,	0.313422932	,],
[	0.337677568	,	0.296573892	,	0.07470121	,	0.279290735	,],
[	0.284886753	,	0.374751581	,	0.308080693	,	0.084184193	,]],
[[	0.085698965	,	0.314259357	,	0.349318733	,	0.268350776	,],
[	0.262799117	,	0.079374824	,	0.257638721	,	0.309688574	,],
[	0.334429916	,	0.29371953	,	0.072992461	,	0.276011123	,],
[	0.284009432	,	0.374083639	,	0.30733056	,	0.084896734	,]],
[[	0.080090419	,	0.308295993	,	0.343610873	,	0.262303764	,],
[	0.26471713	,	0.084443135	,	0.26025385	,	0.311683875	,],
[	0.337227681	,	0.297050801	,	0.078483455	,	0.278791872	,],
[	0.288266159	,	0.377939368	,	0.311492745	,	0.087324203	,]],
[[	0.081991366	,	0.308301356	,	0.343697903	,	0.262354915	,],
[	0.264659341	,	0.080262729	,	0.25934035	,	0.311405075	,],
[	0.335387203	,	0.294481659	,	0.072804835	,	0.276998702	,],
[	0.283875679	,	0.37384155	,	0.307148736	,	0.083690737	,]],
[[	0.082349692	,	0.311038882	,	0.346340389	,	0.264952045	,],
[	0.262696778	,	0.080537308	,	0.257904409	,	0.309708272	,],
[	0.335175321	,	0.294614227	,	0.074414389	,	0.276663095	,],
[	0.285254553	,	0.375357733	,	0.308637611	,	0.087107093	,]],
[[	0.080494798	,	0.311957852	,	0.347423039	,	0.261612733	,],
[	0.263713656	,	0.082735642	,	0.263169444	,	0.310494253	,],
[	0.33416162	,	0.29751325	,	0.074546612	,	0.275860336	,],
[	0.284356866	,	0.378414538	,	0.311779596	,	0.084367224	,]],
[[	0.08563148	,	0.318245478	,	0.353314705	,	0.267926876	,],
[	0.25946364	,	0.079868271	,	0.259418246	,	0.306509616	,],
[	0.330898061	,	0.29461422	,	0.072553475	,	0.272543656	,],
[	0.283981677	,	0.378226337	,	0.311540584	,	0.085626805	,]],
[[	0.080329555	,	0.312512381	,	0.34778805	,	0.262147884	,],
[	0.261332603	,	0.084832794	,	0.261885848	,	0.308442465	,],
[	0.333394057	,	0.297676649	,	0.077958156	,	0.275103823	,],
[	0.287550289	,	0.381459987	,	0.314975502	,	0.087284057	,]],
[[	0.082766504	,	0.312896743	,	0.348298956	,	0.262612501	,],
[	0.260983777	,	0.080477108	,	0.260710101	,	0.3079022	,],
[	0.332144135	,	0.295524222	,	0.072872511	,	0.273754328	,],
[	0.283312849	,	0.3776649	,	0.310940397	,	0.083935737	,]],
[[	0.082508939	,	0.315281314	,	0.350569531	,	0.264823898	,],
[	0.259339418	,	0.081073327	,	0.259642851	,	0.306444316	,],
[	0.331576399	,	0.295360546	,	0.074162116	,	0.273084958	,],
[	0.285147157	,	0.379408368	,	0.31267513	,	0.087773129	,]],
[[	0.080264304	,	0.312070258	,	0.347461904	,	0.261564464	,],
[	0.263719351	,	0.088604792	,	0.26459817	,	0.310726675	,],
[	0.335221488	,	0.298592501	,	0.075667805	,	0.276987192	,],
[	0.285280413	,	0.379189433	,	0.312484339	,	0.085135556	,]],
[[	0.085967963	,	0.315085984	,	0.350613031	,	0.264764887	,],
[	0.259581255	,	0.079898767	,	0.259471609	,	0.306624896	,],
[	0.331093739	,	0.294744482	,	0.072356913	,	0.272656392	,],
[	0.283463557	,	0.377592771	,	0.310869582	,	0.084670862	,]],
[[	0.080729702	,	0.313209836	,	0.348392094	,	0.262746591	,],
[	0.260661481	,	0.083623459	,	0.261102329	,	0.307708016	,],
[	0.332475626	,	0.296617895	,	0.076551504	,	0.274082942	,],
[	0.287871564	,	0.382007224	,	0.315316378	,	0.091724491	,]],
[[	0.081984872	,	0.312792186	,	0.348114448	,	0.262340368	,],
[	0.262051326	,	0.081356801	,	0.261648586	,	0.308832811	,],
[	0.332875349	,	0.296225565	,	0.073389714	,	0.274526005	,],
[	0.283535668	,	0.377720366	,	0.311024458	,	0.08391809	,]],
[[	0.083682782	,	0.316531884	,	0.351670048	,	0.266024206	,],
[	0.25922361	,	0.080475017	,	0.25936628	,	0.30628979	,],
[	0.331193177	,	0.294959669	,	0.073250607	,	0.272681467	,],
[	0.28454074	,	0.378740926	,	0.312025359	,	0.086765008	,]],
[[	0.080724719	,	0.317091588	,	0.352678198	,	0.262018305	,],
[	0.258401349	,	0.081959139	,	0.264285899	,	0.305921942	,],
[	0.330484613	,	0.299312577	,	0.075178738	,	0.271659517	,],
[	0.283411573	,	0.383425882	,	0.316008323	,	0.083298438	,]],
[[	0.084295662	,	0.322283971	,	0.357545625	,	0.26725675	,],
[	0.254943824	,	0.080114145	,	0.261186306	,	0.302572688	,],
[	0.328266816	,	0.297477015	,	0.074530487	,	0.269290804	,],
[	0.284256596	,	0.384412215	,	0.31702728	,	0.086103944	,]],
[[	0.079997346	,	0.317330353	,	0.35268634	,	0.262250354	,],
[	0.257299196	,	0.08569342	,	0.264267118	,	0.304991614	,],
[	0.331311562	,	0.301126589	,	0.080808284	,	0.272440089	,],
[	0.285699689	,	0.385729845	,	0.318408554	,	0.085177455	,]],
[[	0.083386969	,	0.318411102	,	0.353857096	,	0.263316494	,],
[	0.256198304	,	0.080213497	,	0.262165572	,	0.30373118	,],
[	0.328964537	,	0.297949636	,	0.074009464	,	0.269950989	,],
[	0.28314559	,	0.383285744	,	0.315865415	,	0.083680622	,]],
[[	0.081303084	,	0.319466411	,	0.354786698	,	0.264290889	,],
[	0.255125554	,	0.081668591	,	0.261774616	,	0.302792979	,],
[	0.32915161	,	0.298615509	,	0.076476417	,	0.270083717	,],
[	0.285951461	,	0.386130827	,	0.318793739	,	0.088945372	,]],
[[	0.080023442	,	0.31688911	,	0.352509152	,	0.261701433	,],
[	0.259591006	,	0.089613847	,	0.266903822	,	0.30723699	,],
[	0.331695053	,	0.300464083	,	0.076122133	,	0.27270705	,],
[	0.283986531	,	0.383863518	,	0.316474163	,	0.083618965	,]],
[[	0.085927979	,	0.32387489	,	0.358987776	,	0.26880439	,],
[	0.255092634	,	0.079999906	,	0.261260108	,	0.302575761	,],
[	0.328339608	,	0.297437464	,	0.074168104	,	0.269337859	,],
[	0.283723004	,	0.383800515	,	0.316410379	,	0.085055382	,]],
[[	0.080053144	,	0.317559665	,	0.352850006	,	0.262447509	,],
[	0.256667222	,	0.084581619	,	0.263410175	,	0.304231097	,],
[	0.330485616	,	0.300270713	,	0.079391316	,	0.271578002	,],
[	0.286907887	,	0.386776136	,	0.319552862	,	0.086163146	,]],
[[	0.08234438	,	0.317881422	,	0.353291777	,	0.262738783	,],
[	0.257091298	,	0.080803684	,	0.262912258	,	0.304428808	,],
[	0.32960276	,	0.298539452	,	0.074388989	,	0.270541088	,],
[	0.283129897	,	0.383268875	,	0.315762224	,	0.083203654	,]],
[[	0.082390018	,	0.320515353	,	0.355800177	,	0.265324203	,],
[	0.254860554	,	0.08070042	,	0.261322211	,	0.302468621	,],
[	0.328477999	,	0.297818792	,	0.075526659	,	0.269347908	,],
[	0.285063826	,	0.385126023	,	0.317765519	,	0.087478584	,]],
[[	0.080145998	,	0.321220111	,	0.356524419	,	0.261034726	,],
[	0.254816816	,	0.083112675	,	0.265232541	,	0.302515338	,],
[	0.326719907	,	0.301133001	,	0.075620351	,	0.268244287	,],
[	0.282746767	,	0.387582492	,	0.320004134	,	0.082816685	,]],
[[	0.084224199	,	0.32664897	,	0.361591635	,	0.266660519	,],
[	0.250293875	,	0.079998594	,	0.261112423	,	0.298146785	,],
[	0.323463015	,	0.298278823	,	0.073926589	,	0.26481352	,],
[	0.283513709	,	0.388291985	,	0.320769542	,	0.085349402	,]],
[[	0.079529307	,	0.321330747	,	0.356415227	,	0.261276377	,],
[	0.251997571	,	0.084738437	,	0.26357839	,	0.299878085	,],
[	0.326141697	,	0.301700246	,	0.079694394	,	0.267661583	,],
[	0.285227805	,	0.389897643	,	0.322452223	,	0.084768542	,]],
[[	0.082712437	,	0.322280972	,	0.357537081	,	0.262184176	,],
[	0.252076187	,	0.080821986	,	0.26276186	,	0.299802307	,],
[	0.32459909	,	0.29904328	,	0.073859261	,	0.265906187	,],
[	0.282332947	,	0.387235259	,	0.319720576	,	0.083090811	,]],
[[	0.081152937	,	0.323540378	,	0.358703775	,	0.2634156	,],
[	0.250094307	,	0.081033172	,	0.261312931	,	0.298022493	,],
[	0.324046458	,	0.299048198	,	0.075367844	,	0.265299073	,],
[	0.285207096	,	0.390178002	,	0.32265139	,	0.088321426	,]],
[[	0.079587896	,	0.320794406	,	0.356111551	,	0.260655136	,],
[	0.253886241	,	0.088174064	,	0.265869844	,	0.301826974	,],
[	0.327804261	,	0.302123675	,	0.076558641	,	0.269121808	,],
[	0.283527561	,	0.388145069	,	0.320601568	,	0.083299771	,]],
[[	0.085745714	,	0.32811441	,	0.36296701	,	0.267976383	,],
[	0.250837445	,	0.080184996	,	0.261594987	,	0.298564107	,],
[	0.323274745	,	0.297934363	,	0.073414286	,	0.264638669	,],
[	0.283015414	,	0.387887422	,	0.320282889	,	0.084578586	,]],
[[	0.079778804	,	0.321762548	,	0.356895632	,	0.26169377	,],
[	0.251229984	,	0.083440847	,	0.262619348	,	0.299119002	,],
[	0.325400653	,	0.300667595	,	0.078279039	,	0.266710645	,],
[	0.286296556	,	0.390888097	,	0.323510094	,	0.085756398	,]],
[[	0.081826478	,	0.321872329	,	0.357051628	,	0.261819113	,],
[	0.253208845	,	0.081639501	,	0.263816817	,	0.300902497	,],
[	0.325415563	,	0.299772513	,	0.074583118	,	0.266781438	,],
[	0.282420045	,	0.387200119	,	0.319713887	,	0.082776087	,]],
[[	0.082506116	,	0.325054419	,	0.360125527	,	0.264974081	,],
[	0.250149181	,	0.080596186	,	0.261203774	,	0.298006624	,],
[	0.323689401	,	0.298614701	,	0.074629464	,	0.264972742	,],
[	0.28453133	,	0.389456247	,	0.321858562	,	0.087104848	,]],
[[	0.080990864	,	0.327161976	,	0.361494849	,	0.261710093	,],
[	0.249795592	,	0.089573742	,	0.267147948	,	0.298564244	,],
[	0.324229948	,	0.303941212	,	0.076578792	,	0.265951588	,],
[	0.281815107	,	0.391975871	,	0.324362316	,	0.083433222	,]],
[[	0.084965123	,	0.332694159	,	0.366751839	,	0.267327281	,],
[	0.246797739	,	0.081547634	,	0.262978173	,	0.29550227	,],
[	0.320158099	,	0.300420457	,	0.073857166	,	0.261855248	,],
[	0.282466947	,	0.392652969	,	0.325110225	,	0.086040232	,]],
[[	0.080317227	,	0.327312904	,	0.3615696	,	0.26190204	,],
[	0.247240006	,	0.084662177	,	0.263986116	,	0.295872493	,],
[	0.322251528	,	0.302956037	,	0.078811628	,	0.263837338	,],
[	0.284352056	,	0.394340431	,	0.326915022	,	0.085375933	,]],
[[	0.083329892	,	0.328291771	,	0.362561062	,	0.262865466	,],
[	0.249087397	,	0.082925935	,	0.264979509	,	0.297672526	,],
[	0.322091522	,	0.301932915	,	0.074842746	,	0.26371386	,],
[	0.281089222	,	0.391362929	,	0.323889288	,	0.083324478	,]],
[[	0.081914044	,	0.329841733	,	0.3640369	,	0.264341986	,],
[	0.246034543	,	0.081845928	,	0.262480547	,	0.29477337	,],
[	0.320773169	,	0.301107106	,	0.075454349	,	0.262237971	,],
[	0.284438023	,	0.39464154	,	0.327218768	,	0.089060605	,]],
[[	0.080427302	,	0.326944094	,	0.361366396	,	0.261438689	,],
[	0.249062349	,	0.088083725	,	0.266191753	,	0.297766904	,],
[	0.32442707	,	0.305484984	,	0.082617795	,	0.266069758	,],
[	0.282342521	,	0.392339789	,	0.324784395	,	0.083653777	,]],
[[	0.086220847	,	0.333854665	,	0.367728685	,	0.268480398	,],
[	0.247259823	,	0.081653604	,	0.263241451	,	0.295826916	,],
[	0.320601125	,	0.300552621	,	0.073832753	,	0.262076116	,],
[	0.282075939	,	0.392149121	,	0.324546218	,	0.084989965	,]],
[[	0.080618358	,	0.327865552	,	0.36203398	,	0.262383762	,],
[	0.246606389	,	0.083621061	,	0.263248843	,	0.295325674	,],
[	0.321708199	,	0.302320626	,	0.07777277	,	0.263291737	,],
[	0.285217746	,	0.39510536	,	0.327652086	,	0.086081109	,]],
[[	0.082443408	,	0.32784429	,	0.362068041	,	0.262335504	,],
[	0.250453632	,	0.084002884	,	0.266200782	,	0.298863566	,],
[	0.322879363	,	0.302638861	,	0.075253337	,	0.264387385	,],
[	0.28138148	,	0.39158755	,	0.323983348	,	0.08327983	,]],
[[	0.082924927	,	0.330698668	,	0.364751527	,	0.265280443	,],
[	0.246300159	,	0.081622072	,	0.262625716	,	0.295060157	,],
[	0.320414661	,	0.300586053	,	0.074600867	,	0.261889596	,],
[	0.283937748	,	0.394051687	,	0.326550724	,	0.087891967	,]],
[[	0.080620384	,	0.332333401	,	0.366351057	,	0.261229757	,],
[	0.244425682	,	0.088941001	,	0.267521806	,	0.293469624	,],
[	0.320224015	,	0.30573987	,	0.07734353	,	0.261771159	,],
[	0.280887994	,	0.396123906	,	0.328443424	,	0.082535062	,]],
[[	0.083941636	,	0.337314695	,	0.370960696	,	0.266328873	,],
[	0.241972027	,	0.081414794	,	0.263836219	,	0.290876112	,],
[	0.31597019	,	0.301792373	,	0.074230501	,	0.257398774	,],
[	0.281464216	,	0.396911029	,	0.329149907	,	0.085118209	,]],
[[	0.079583823	,	0.332175452	,	0.366036925	,	0.261197361	,],
[	0.242089809	,	0.084326364	,	0.264700573	,	0.291132346	,],
[	0.317613199	,	0.30417483	,	0.078751017	,	0.25913946	,],
[	0.283327981	,	0.398490733	,	0.331010863	,	0.084613302	,]],
[[	0.083441968	,	0.333883987	,	0.367802684	,	0.262789168	,],
[	0.244934726	,	0.083507218	,	0.266496419	,	0.293747302	,],
[	0.31774957	,	0.303368548	,	0.075140871	,	0.259170156	,],
[	0.280329304	,	0.396014967	,	0.328214325	,	0.082748409	,]],
[[	0.081342822	,	0.334627435	,	0.368467229	,	0.263617175	,],
[	0.241108313	,	0.081639049	,	0.263275014	,	0.290097405	,],
[	0.316170051	,	0.302253805	,	0.075402351	,	0.257529762	,],
[	0.283273356	,	0.398762789	,	0.331149488	,	0.088117099	,]],
[[	0.079951539	,	0.332028166	,	0.366011892	,	0.260881732	,],
[	0.243983145	,	0.087639966	,	0.266761448	,	0.292819827	,],
[	0.319525226	,	0.306380745	,	0.082142415	,	0.261012357	,],
[	0.281647938	,	0.396769377	,	0.329115984	,	0.08295655	,]],
[[	0.085382234	,	0.338741837	,	0.372309894	,	0.267748252	,],
[	0.242579628	,	0.081640204	,	0.26419227	,	0.291331286	,],
[	0.316461178	,	0.302251702	,	0.07431949	,	0.257868236	,],
[	0.281366963	,	0.396863539	,	0.328949735	,	0.084500533	,]],
[[	0.079985111	,	0.332953933	,	0.366752492	,	0.261846026	,],
[	0.241797796	,	0.08338144	,	0.264101546	,	0.290673178	,],
[	0.317023323	,	0.303458964	,	0.07749563	,	0.258387312	,],
[	0.284588107	,	0.399688744	,	0.332181829	,	0.085671059	,]],
[[	0.082405613	,	0.333305799	,	0.367302479	,	0.262212196	,],
[	0.245897211	,	0.084257392	,	0.267303737	,	0.294453068	,],
[	0.318788108	,	0.304254571	,	0.075860814	,	0.260205083	,],
[	0.28061602	,	0.396035043	,	0.328337731	,	0.082601175	,]],
[[	0.08224921	,	0.335583696	,	0.369443777	,	0.264569986	,],
[	0.241467656	,	0.081443044	,	0.26350581	,	0.290330935	,],
[	0.315872232	,	0.301786388	,	0.074604209	,	0.257237639	,],
[	0.282596348	,	0.398000913	,	0.330327574	,	0.08675935	,]],
[[	0.094013671	,	0.223849563	,	0.271494507	,	0.2931117	,],
[	0.36511945	,	0.084294353	,	0.261252777	,	0.411770885	,],
[	0.427247066	,	0.281996579	,	0.072496764	,	0.37190188	,],
[	0.310965079	,	0.294459412	,	0.22987465	,	0.092325557	,]],
[[	0.089726577	,	0.223247665	,	0.270632507	,	0.291841867	,],
[	0.361364537	,	0.076418976	,	0.256750645	,	0.407854244	,],
[	0.424007578	,	0.278517693	,	0.070040661	,	0.368513894	,],
[	0.310085453	,	0.293415839	,	0.228704442	,	0.093109628	,]],
[[	0.090469686	,	0.221938934	,	0.26950948	,	0.291035357	,],
[	0.362420351	,	0.079641087	,	0.258158226	,	0.409031627	,],
[	0.42644824	,	0.281344892	,	0.075043199	,	0.371078433	,],
[	0.313955899	,	0.297565279	,	0.233058017	,	0.094987344	,]],
[[	0.093138747	,	0.227396507	,	0.274584104	,	0.295551386	,],
[	0.363759877	,	0.078324623	,	0.259246571	,	0.410171756	,],
[	0.425031825	,	0.279623072	,	0.070430834	,	0.369604164	,],
[	0.309989493	,	0.293339049	,	0.228640684	,	0.091739578	,]],
[[	0.088865907	,	0.22184865	,	0.269429747	,	0.290575954	,],
[	0.360928763	,	0.0767098	,	0.25636375	,	0.407543088	,],
[	0.424141261	,	0.278712149	,	0.071100758	,	0.368660377	,],
[	0.311772307	,	0.294893845	,	0.230226476	,	0.095683443	,]],
[[	0.092602655	,	0.222893646	,	0.270606771	,	0.292135029	,],
[	0.36398422	,	0.082598951	,	0.259858722	,	0.410597249	,],
[	0.4282243	,	0.283056728	,	0.073491104	,	0.372922962	,],
[	0.311563976	,	0.295275056	,	0.230623526	,	0.092664935	,]],
[[	0.090468348	,	0.224183876	,	0.27144018	,	0.292610569	,],
[	0.362231822	,	0.077022315	,	0.257573587	,	0.408602557	,],
[	0.42392314	,	0.278498214	,	0.069696576	,	0.368375953	,],
[	0.309760686	,	0.293074946	,	0.22840798	,	0.092335387	,]],
[[	0.089646651	,	0.221572244	,	0.269171664	,	0.290532379	,],
[	0.361699388	,	0.078525885	,	0.257226062	,	0.408246984	,],
[	0.425528622	,	0.280193416	,	0.073511038	,	0.370017469	,],
[	0.313942245	,	0.297000343	,	0.232455767	,	0.099018344	,]],
[[	0.094424658	,	0.228610674	,	0.275843905	,	0.296798509	,],
[	0.365003124	,	0.079434976	,	0.260465756	,	0.411234399	,],
[	0.425849298	,	0.280503121	,	0.071191103	,	0.370354723	,],
[	0.310342958	,	0.293751014	,	0.228988897	,	0.091766787	,]],
[[	0.089124694	,	0.222341662	,	0.2697736	,	0.290922016	,],
[	0.360974947	,	0.076514466	,	0.256410571	,	0.407575517	,],
[	0.424186049	,	0.278694228	,	0.070631353	,	0.368447947	,],
[	0.310975847	,	0.294115456	,	0.229434301	,	0.094511934	,]],
[[	0.089873611	,	0.229213164	,	0.276564838	,	0.286627843	,],
[	0.345823572	,	0.075549112	,	0.253922734	,	0.393010981	,],
[	0.406697474	,	0.273361692	,	0.06927338	,	0.353426311	,],
[	0.301308161	,	0.295955213	,	0.234350458	,	0.093103556	,]],
[[	0.088753613	,	0.231128557	,	0.278100706	,	0.28776938	,],
[	0.347623524	,	0.079977995	,	0.256160179	,	0.394924386	,],
[	0.409325416	,	0.276250736	,	0.074437273	,	0.356169794	,],
[	0.300334501	,	0.295476334	,	0.23377767	,	0.089108628	,]],
[[	0.087064106	,	0.22782566	,	0.275045437	,	0.285078554	,],
[	0.347082619	,	0.076121278	,	0.255142991	,	0.394089369	,],
[	0.407592212	,	0.274337118	,	0.069522138	,	0.354293578	,],
[	0.29943706	,	0.294288313	,	0.232723035	,	0.089739928	,]],
[[	0.092638398	,	0.235555931	,	0.282286037	,	0.291850036	,],
[	0.345948963	,	0.076432555	,	0.254093821	,	0.393073321	,],
[	0.407076531	,	0.273877067	,	0.070601407	,	0.353796375	,],
[	0.304040087	,	0.298531137	,	0.237106613	,	0.096811169	,]],
[[	0.08684888	,	0.228928702	,	0.276011061	,	0.285768271	,],
[	0.3502161	,	0.083747486	,	0.258774689	,	0.397359043	,],
[	0.410815116	,	0.277750749	,	0.072623674	,	0.357625848	,],
[	0.299578828	,	0.29450123	,	0.2328266	,	0.088647116	,]],
[[	0.088886625	,	0.228709567	,	0.27601622	,	0.285888846	,],
[	0.346019341	,	0.075479399	,	0.254047052	,	0.393122602	,],
[	0.406821287	,	0.273409088	,	0.069061436	,	0.353502592	,],
[	0.300973442	,	0.295566709	,	0.233965688	,	0.092044835	,]],
[[	0.089709339	,	0.232113254	,	0.279043379	,	0.288719585	,],
[	0.347081516	,	0.078774616	,	0.255432352	,	0.394281929	,],
[	0.408623417	,	0.275521958	,	0.073239933	,	0.355362168	,],
[	0.301248865	,	0.296445735	,	0.234819858	,	0.089707751	,]],
[[	0.087084858	,	0.228234685	,	0.275396021	,	0.285333765	,],
[	0.347945805	,	0.076887282	,	0.255985165	,	0.394866286	,],
[	0.408297018	,	0.275177164	,	0.070170683	,	0.35502565	,],
[	0.299322317	,	0.294250266	,	0.232523057	,	0.089097877	,]],
[[	0.09213728	,	0.230461658	,	0.27764637	,	0.287901578	,],
[	0.345731794	,	0.075987823	,	0.253738746	,	0.392748679	,],
[	0.407288556	,	0.273921165	,	0.070297687	,	0.353881282	,],
[	0.303195864	,	0.297670429	,	0.236138995	,	0.095453834	,]],
[[	0.087811319	,	0.230104479	,	0.277167431	,	0.286730052	,],
[	0.349230513	,	0.082151182	,	0.25772605	,	0.396322245	,],
[	0.410644733	,	0.277715754	,	0.076495396	,	0.357339059	,],
[	0.299680097	,	0.294600684	,	0.232923352	,	0.088480256	,]],
[[	0.087712248	,	0.235036329	,	0.282105479	,	0.281545782	,],
[	0.334723729	,	0.074527747	,	0.253808359	,	0.383005606	,],
[	0.397093703	,	0.273266403	,	0.068857713	,	0.343848527	,],
[	0.298174317	,	0.303388301	,	0.242743882	,	0.092554769	,]],
[[	0.086165086	,	0.236549411	,	0.28327812	,	0.282469518	,],
[	0.336263244	,	0.078597475	,	0.255682321	,	0.384466683	,],
[	0.399208122	,	0.275621417	,	0.073386676	,	0.345950679	,],
[	0.295950753	,	0.301692442	,	0.241007997	,	0.087218989	,]],
[[	0.084953631	,	0.23360845	,	0.280544233	,	0.279930573	,],
[	0.33662422	,	0.075580324	,	0.255416711	,	0.384616079	,],
[	0.398511671	,	0.274748281	,	0.069600752	,	0.345260502	,],
[	0.295734555	,	0.301295437	,	0.24048265	,	0.088580652	,]],
[[	0.090437958	,	0.241211314	,	0.28783394	,	0.286871723	,],
[	0.334987772	,	0.075569015	,	0.254104499	,	0.383174034	,],
[	0.397346192	,	0.273526885	,	0.069813782	,	0.343943328	,],
[	0.29948361	,	0.305103037	,	0.244423811	,	0.090330134	,]],
[[	0.084801749	,	0.234888277	,	0.281744551	,	0.280884115	,],
[	0.338789263	,	0.082365497	,	0.25827085	,	0.386925753	,],
[	0.40164445	,	0.278342981	,	0.077273458	,	0.34836332	,],
[	0.295456768	,	0.301093242	,	0.24020677	,	0.087042264	,]],
[[	0.086748934	,	0.234520062	,	0.281547803	,	0.280976066	,],
[	0.335308127	,	0.074804238	,	0.254250742	,	0.383445158	,],
[	0.397414073	,	0.273509706	,	0.068779749	,	0.343944173	,],
[	0.297317748	,	0.30255118	,	0.241846695	,	0.091283806	,]],
[[	0.087178414	,	0.237660751	,	0.284303172	,	0.283453725	,],
[	0.33563889	,	0.077460847	,	0.254992548	,	0.383919501	,],
[	0.398705402	,	0.274962318	,	0.072272687	,	0.34524184	,],
[	0.296897546	,	0.302609982	,	0.241842324	,	0.087731103	,]],
[[	0.084716646	,	0.233702734	,	0.280625199	,	0.279930206	,],
[	0.337859317	,	0.076844802	,	0.25673059	,	0.38580965	,],
[	0.399623792	,	0.275802078	,	0.070535612	,	0.346257935	,],
[	0.295441946	,	0.300931348	,	0.240119069	,	0.087817437	,]],
[[	0.090138969	,	0.236322687	,	0.283336924	,	0.282966639	,],
[	0.335126828	,	0.07539858	,	0.25406391	,	0.383254683	,],
[	0.397273331	,	0.273383181	,	0.069356227	,	0.343817496	,],
[	0.29945398	,	0.304559588	,	0.244080913	,	0.094353069	,]],
[[	0.085429753	,	0.235641229	,	0.282423259	,	0.281514886	,],
[	0.337702719	,	0.08074767	,	0.257170092	,	0.385902777	,],
[	0.40030666	,	0.27688934	,	0.075415665	,	0.346960721	,],
[	0.295524226	,	0.301119013	,	0.24034521	,	0.086953102	,]],
[[	0.085250746	,	0.241277346	,	0.287227118	,	0.275894876	,],
[	0.323935131	,	0.075128503	,	0.254039041	,	0.371987163	,],
[	0.385942068	,	0.27371848	,	0.070505868	,	0.333996753	,],
[	0.289645172	,	0.306150037	,	0.246910653	,	0.087737889	,]],
[[	0.085327221	,	0.244067512	,	0.289638217	,	0.278155959	,],
[	0.321637269	,	0.074132096	,	0.25192484	,	0.369729288	,],
[	0.383458486	,	0.271168614	,	0.069192965	,	0.331276192	,],
[	0.291771637	,	0.308632079	,	0.249374793	,	0.087570239	,]],
[[	0.082914016	,	0.240181897	,	0.286047639	,	0.274680044	,],
[	0.325192929	,	0.080785147	,	0.256043448	,	0.373434782	,],
[	0.386835826	,	0.275002102	,	0.075311611	,	0.334753514	,],
[	0.288605839	,	0.305275461	,	0.245890895	,	0.085532719	,]],
[[	0.088515209	,	0.243056534	,	0.288914085	,	0.277905152	,],
[	0.322259207	,	0.073814902	,	0.252362174	,	0.370301521	,],
[	0.384031032	,	0.271821333	,	0.068939725	,	0.332060143	,],
[	0.291527834	,	0.307846113	,	0.248655581	,	0.090793121	,]],
[[	0.083345354	,	0.241787042	,	0.287469821	,	0.275992675	,],
[	0.322936627	,	0.076573706	,	0.253323292	,	0.37105798	,],
[	0.38442975	,	0.27232902	,	0.071229759	,	0.332374828	,],
[	0.289768332	,	0.306301322	,	0.247044149	,	0.085657438	,]],
[[	0.084519044	,	0.240861548	,	0.286783099	,	0.275415951	,],
[	0.325081247	,	0.075881562	,	0.255089035	,	0.372896489	,],
[	0.386778391	,	0.274479698	,	0.071209555	,	0.334697817	,],
[	0.289102457	,	0.305511554	,	0.246157532	,	0.086757618	,]],
[[	0.086623996	,	0.245474391	,	0.290906587	,	0.279426767	,],
[	0.321516044	,	0.0737676	,	0.251741997	,	0.369633428	,],
[	0.383688195	,	0.271260187	,	0.069026253	,	0.331456901	,],
[	0.292957828	,	0.309766475	,	0.250530427	,	0.08864576	,]],
[[	0.082731729	,	0.240407438	,	0.286096544	,	0.274779465	,],
[	0.324346769	,	0.079334244	,	0.254981446	,	0.372499428	,],
[	0.386181933	,	0.274167289	,	0.074198824	,	0.334091565	,],
[	0.288598772	,	0.305271931	,	0.245896396	,	0.085154127	,]],
[[	0.087295725	,	0.242302309	,	0.288156433	,	0.276960637	,],
[	0.322774698	,	0.074096184	,	0.252879808	,	0.370732207	,],
[	0.384692586	,	0.272338975	,	0.069260566	,	0.332580703	,],
[	0.290725429	,	0.307178697	,	0.247928679	,	0.089450848	,]],
[[	0.084152066	,	0.242752612	,	0.288405057	,	0.276748131	,],
[	0.32226136	,	0.075489588	,	0.252573937	,	0.370373887	,],
[	0.384110633	,	0.271848952	,	0.070430788	,	0.331904868	,],
[	0.290658771	,	0.307271542	,	0.247951023	,	0.086471977	,]],
[[	0.084673339	,	0.25094677	,	0.296298145	,	0.275303168	,],
[	0.318133411	,	0.083373771	,	0.258722544	,	0.367282407	,],
[	0.38117982	,	0.279003282	,	0.078440722	,	0.329506764	,],
[	0.287143331	,	0.314047904	,	0.255202402	,	0.086698383	,]],
[[	0.085362151	,	0.254068148	,	0.299194461	,	0.278144052	,],
[	0.313576281	,	0.074553085	,	0.253533357	,	0.362788372	,],
[	0.376718293	,	0.273799413	,	0.069691364	,	0.324938693	,],
[	0.291479009	,	0.318648538	,	0.259921337	,	0.089127762	,]],
[[	0.082458613	,	0.249986395	,	0.295328126	,	0.274379792	,],
[	0.314771902	,	0.078205559	,	0.255130382	,	0.364114061	,],
[	0.378073203	,	0.275572417	,	0.073499846	,	0.326320565	,],
[	0.287001792	,	0.314042841	,	0.255206609	,	0.085374442	,]],
[[	0.087826736	,	0.25248364	,	0.297933453	,	0.277122665	,],
[	0.316079401	,	0.076248591	,	0.255795987	,	0.365062229	,],
[	0.378756238	,	0.275990534	,	0.071132798	,	0.327034233	,],
[	0.288806406	,	0.315669758	,	0.256791366	,	0.089217714	,]],
[[	0.0831761	,	0.251839	,	0.297100663	,	0.275878567	,],
[	0.31380651	,	0.075454349	,	0.253780366	,	0.362981157	,],
[	0.37657433	,	0.273860199	,	0.070318768	,	0.324740025	,],
[	0.289027539	,	0.315966001	,	0.257213211	,	0.086739585	,]],
[[	0.083692448	,	0.250428182	,	0.295789296	,	0.274682937	,],
[	0.317045953	,	0.081820151	,	0.257592703	,	0.366237259	,],
[	0.380066298	,	0.277803049	,	0.076752528	,	0.328360878	,],
[	0.28687576	,	0.313747148	,	0.254873173	,	0.085827174	,]],
[[	0.086862756	,	0.25576368	,	0.300848643	,	0.279711159	,],
[	0.314290598	,	0.074758927	,	0.254059732	,	0.363352771	,],
[	0.377155839	,	0.274331461	,	0.069910932	,	0.325311996	,],
[	0.291301144	,	0.317909735	,	0.259164309	,	0.092797877	,]],
[[	0.082306319	,	0.25029143	,	0.295468982	,	0.274448151	,],
[	0.314375441	,	0.077310794	,	0.25447259	,	0.363533469	,],
[	0.377601911	,	0.274875981	,	0.072305009	,	0.325588567	,],
[	0.287431733	,	0.314578171	,	0.255737284	,	0.085470147	,]],
[[	0.086309446	,	0.251681695	,	0.296963967	,	0.276179538	,],
[	0.317044724	,	0.077123804	,	0.256757238	,	0.365964499	,],
[	0.379594946	,	0.276805534	,	0.07194157	,	0.327808371	,],
[	0.288028002	,	0.31497165	,	0.256140616	,	0.088078148	,]],
[[	0.083954937	,	0.252612948	,	0.297853223	,	0.276632391	,],
[	0.313720582	,	0.075063576	,	0.253538538	,	0.362708889	,],
[	0.376743149	,	0.273953797	,	0.070080276	,	0.324923863	,],
[	0.290154118	,	0.317114163	,	0.258343651	,	0.087695124	,]],
[[	0.081616438	,	0.256835734	,	0.301236876	,	0.270568952	,],
[	0.302055744	,	0.076435457	,	0.253323053	,	0.351880248	,],
[	0.364156699	,	0.272932471	,	0.070262503	,	0.313751046	,],
[	0.280992635	,	0.318309595	,	0.260242014	,	0.083822856	,]],
[[	0.084498275	,	0.261619059	,	0.305752768	,	0.274992831	,],
[	0.305673797	,	0.077480596	,	0.256447177	,	0.355276426	,],
[	0.368293152	,	0.27687258	,	0.072360956	,	0.317957012	,],
[	0.282833438	,	0.319911446	,	0.261964478	,	0.087703612	,]],
[[	0.080428971	,	0.256560224	,	0.300923798	,	0.270093076	,],
[	0.301822383	,	0.07469831	,	0.252750611	,	0.351480166	,],
[	0.363730051	,	0.272273694	,	0.068527259	,	0.313284637	,],
[	0.28294015	,	0.320396444	,	0.262401942	,	0.085197066	,]],
[[	0.084319804	,	0.258056201	,	0.302379351	,	0.271864524	,],
[	0.303732753	,	0.079363194	,	0.255192738	,	0.353449862	,],
[	0.365634746	,	0.274575389	,	0.072769491	,	0.315165082	,],
[	0.280898064	,	0.318308076	,	0.260206445	,	0.084431974	,]],
[[	0.08224795	,	0.259384373	,	0.303513784	,	0.272580484	,],
[	0.303644525	,	0.075520889	,	0.254369438	,	0.353150661	,],
[	0.365586411	,	0.274144566	,	0.069828833	,	0.315222311	,],
[	0.284654464	,	0.321748335	,	0.263802381	,	0.090785629	,]],
[[	0.081246508	,	0.25672672	,	0.301071335	,	0.270255301	,],
[	0.302092354	,	0.075931795	,	0.253110012	,	0.351745816	,],
[	0.363858335	,	0.272749446	,	0.06945604	,	0.313523088	,],
[	0.281670688	,	0.319031585	,	0.260935786	,	0.084099492	,]],
[[	0.086170186	,	0.263287832	,	0.307268762	,	0.276518023	,],
[	0.306246631	,	0.083180245	,	0.257818723	,	0.355819638	,],
[	0.367660736	,	0.276989996	,	0.0761449	,	0.317203941	,],
[	0.282259337	,	0.319321118	,	0.261334387	,	0.086591905	,]],
[[	0.080739758	,	0.257230727	,	0.301433577	,	0.270607383	,],
[	0.301910245	,	0.074452133	,	0.252824391	,	0.351446296	,],
[	0.36434317	,	0.272949241	,	0.068884452	,	0.31380926	,],
[	0.284019816	,	0.321416802	,	0.263405611	,	0.086101589	,]],
[[	0.083256733	,	0.257556718	,	0.301911404	,	0.271267299	,],
[	0.303129598	,	0.078129422	,	0.254389366	,	0.352832457	,],
[	0.364822502	,	0.273629547	,	0.071381227	,	0.314227691	,],
[	0.28084575	,	0.318277133	,	0.260115766	,	0.083995723	,]],
[[	0.082986959	,	0.260203557	,	0.304323459	,	0.273392992	,],
[	0.304262096	,	0.076071305	,	0.25498313	,	0.353608058	,],
[	0.366492315	,	0.275139581	,	0.070606505	,	0.316120535	,],
[	0.283834669	,	0.320910388	,	0.262922172	,	0.089355303	,]],
[[	0.0793832	,	0.26034938	,	0.303578301	,	0.263898248	,],
[	0.288438587	,	0.074918789	,	0.249248819	,	0.337809734	,],
[	0.348296781	,	0.267974807	,	0.06730475	,	0.298590719	,],
[	0.274044455	,	0.320155835	,	0.262508049	,	0.08176958	,]],
[[	0.082409405	,	0.26532899	,	0.308266175	,	0.268663679	,],
[	0.29183182	,	0.08141233	,	0.253233371	,	0.341120857	,],
[	0.351955988	,	0.272137856	,	0.073870196	,	0.302297095	,],
[	0.275458602	,	0.321424332	,	0.263791527	,	0.085267559	,]],
[[	0.078497961	,	0.260440055	,	0.303538965	,	0.263896239	,],
[	0.288823307	,	0.074027505	,	0.249490466	,	0.33813433	,],
[	0.348944427	,	0.26844404	,	0.066700434	,	0.299289599	,],
[	0.276279927	,	0.32254558	,	0.264903805	,	0.083272194	,]],
[[	0.082200121	,	0.261734251	,	0.304952791	,	0.26546299	,],
[	0.289197027	,	0.076852261	,	0.250296303	,	0.338695279	,],
[	0.349359775	,	0.26918322	,	0.069405872	,	0.299582427	,],
[	0.273800086	,	0.320057421	,	0.262383163	,	0.082246221	,]],
[[	0.079571167	,	0.262613819	,	0.305674972	,	0.265841601	,],
[	0.291583587	,	0.076129951	,	0.251970099	,	0.340619766	,],
[	0.351439413	,	0.270980986	,	0.068699286	,	0.301777673	,],
[	0.277584082	,	0.323477367	,	0.265992136	,	0.088445076	,]],
[[	0.078744149	,	0.260136405	,	0.303326675	,	0.263604898	,],
[	0.288136483	,	0.074291981	,	0.248938651	,	0.33755689	,],
[	0.348481556	,	0.267994096	,	0.066929123	,	0.298622436	,],
[	0.274513792	,	0.320654836	,	0.262901087	,	0.081917848	,]],
[[	0.083702808	,	0.266645372	,	0.309614029	,	0.269914359	,],
[	0.291217261	,	0.08026057	,	0.25246625	,	0.340510897	,],
[	0.351249118	,	0.27133702	,	0.072562557	,	0.301532431	,],
[	0.275060175	,	0.321023053	,	0.263356423	,	0.084328319	,]],
[[	0.078335495	,	0.260612075	,	0.303659453	,	0.264013866	,],
[	0.289231127	,	0.074232171	,	0.24977298	,	0.338383032	,],
[	0.34969274	,	0.269124506	,	0.067183351	,	0.299847823	,],
[	0.277093258	,	0.3232795	,	0.265707287	,	0.084018123	,]],
[[	0.081202075	,	0.26126139	,	0.304324251	,	0.264792712	,],
[	0.288969829	,	0.076147147	,	0.249964145	,	0.338284294	,],
[	0.349006745	,	0.268727575	,	0.068459025	,	0.299115226	,],
[	0.273797255	,	0.320041874	,	0.262345801	,	0.081815294	,]],
[[	0.080317518	,	0.263390646	,	0.306403903	,	0.266661904	,],
[	0.292280478	,	0.076854535	,	0.252729996	,	0.34137141	,],
[	0.352435786	,	0.271906979	,	0.069607665	,	0.302660615	,],
[	0.27685803	,	0.322866121	,	0.265270937	,	0.087231413	,]],
[[	0.076396935	,	0.262568634	,	0.303630347	,	0.25564994	,],
[	0.270626524	,	0.071647798	,	0.242543384	,	0.319400287	,],
[	0.329421651	,	0.261424256	,	0.064939184	,	0.28128892	,],
[	0.262399599	,	0.317199495	,	0.26182421	,	0.078491284	,]],
[[	0.080246367	,	0.268218025	,	0.308995367	,	0.261065586	,],
[	0.271511894	,	0.075124963	,	0.243948365	,	0.320258599	,],
[	0.329689545	,	0.26195693	,	0.067307677	,	0.281456375	,],
[	0.262756552	,	0.31746408	,	0.262103996	,	0.080730034	,]],
[[	0.075593379	,	0.262717693	,	0.303709221	,	0.255762679	,],
[	0.273117441	,	0.073275853	,	0.244844105	,	0.321571544	,],
[	0.332190478	,	0.264061143	,	0.067158045	,	0.284060402	,],
[	0.265458868	,	0.320385745	,	0.264996757	,	0.080984712	,]],
[[	0.078939712	,	0.263850607	,	0.304833699	,	0.257040295	,],
[	0.270310867	,	0.072311671	,	0.242428491	,	0.319105603	,],
[	0.328884712	,	0.26081431	,	0.06498884	,	0.280563803	,],
[	0.262037346	,	0.31692506	,	0.261441164	,	0.078583741	,]],
[[	0.077220796	,	0.265263574	,	0.306194707	,	0.258082247	,],
[	0.27310598	,	0.078249148	,	0.245814283	,	0.321790407	,],
[	0.331403623	,	0.263877389	,	0.070310024	,	0.283205396	,],
[	0.264419043	,	0.319095941	,	0.263734495	,	0.083345381	,]],
[[	0.075911592	,	0.26243843	,	0.303441719	,	0.255373419	,],
[	0.271444147	,	0.072000855	,	0.243101467	,	0.319919734	,],
[	0.330470609	,	0.262238362	,	0.065539236	,	0.282126415	,],
[	0.263023491	,	0.317772694	,	0.262355992	,	0.078791224	,]],
[[	0.081802406	,	0.269626307	,	0.310368434	,	0.262438094	,],
[	0.271275742	,	0.074321406	,	0.243512235	,	0.319849125	,],
[	0.32918806	,	0.26142088	,	0.066302903	,	0.280934706	,],
[	0.262538259	,	0.317153198	,	0.261687286	,	0.079847076	,]],
[[	0.075901311	,	0.263380082	,	0.304198768	,	0.256308052	,],
[	0.274290903	,	0.074399194	,	0.245934822	,	0.322725802	,],
[	0.33336525	,	0.265207905	,	0.068114602	,	0.285165816	,],
[	0.266322731	,	0.321205148	,	0.265820631	,	0.081873535	,]],
[[	0.077743745	,	0.263115868	,	0.304047091	,	0.256171559	,],
[	0.270443121	,	0.071930347	,	0.242449777	,	0.319117098	,],
[	0.329162457	,	0.261154623	,	0.064993911	,	0.280875731	,],
[	0.261970895	,	0.316686258	,	0.261391283	,	0.078305643	,]],
[[	0.078436866	,	0.266304011	,	0.307098482	,	0.259024429	,],
[	0.272662379	,	0.077059195	,	0.245238427	,	0.321326083	,],
[	0.330815358	,	0.263185497	,	0.069227112	,	0.282508681	,],
[	0.263836945	,	0.318498456	,	0.263135641	,	0.082315051	,]],
[[	0.079175971	,	0.281463081	,	0.324438389	,	0.263026882	,],
[	0.275299479	,	0.076570023	,	0.25649466	,	0.325716402	,],
[	0.339275027	,	0.278504991	,	0.070731395	,	0.288175552	,],
[	0.276212474	,	0.344788505	,	0.286354949	,	0.082933827	,]],
[[	0.083926241	,	0.287631903	,	0.330379967	,	0.269121073	,],
[	0.275126526	,	0.078650587	,	0.256889447	,	0.325694405	,],
[	0.338170802	,	0.27786557	,	0.071566108	,	0.287110038	,],
[	0.275979997	,	0.344481297	,	0.286121181	,	0.084474127	,]],
[[	0.078600307	,	0.281698203	,	0.32463921	,	0.263292873	,],
[	0.2779507	,	0.0785975	,	0.258960402	,	0.32821583	,],
[	0.342158693	,	0.281366484	,	0.07326423	,	0.291053979	,],
[	0.279028795	,	0.347517362	,	0.2891425	,	0.085394888	,]],
[[	0.081881942	,	0.282829131	,	0.325798274	,	0.264507143	,],
[	0.274545407	,	0.076515915	,	0.256012639	,	0.325038023	,],
[	0.338111957	,	0.277420058	,	0.070105251	,	0.286869036	,],
[	0.275193815	,	0.343788662	,	0.285474852	,	0.082562985	,]],
[[	0.08100177	,	0.284859455	,	0.327603746	,	0.266175196	,],
[	0.276497525	,	0.081444466	,	0.258444037	,	0.326925382	,],
[	0.339795171	,	0.279574465	,	0.07431046	,	0.288632702	,],
[	0.278103291	,	0.346549132	,	0.288236516	,	0.087565771	,]],
[[	0.078952032	,	0.281591348	,	0.324570234	,	0.263117813	,],
[	0.275900833	,	0.076812479	,	0.2569696	,	0.326121973	,],
[	0.339794671	,	0.278948919	,	0.070914212	,	0.288503239	,],
[	0.27670467	,	0.345159431	,	0.28678696	,	0.083237192	,]],
[[	0.084845995	,	0.2846347	,	0.327764292	,	0.266368525	,],
[	0.274682692	,	0.077856857	,	0.256176354	,	0.325155361	,],
[	0.338110736	,	0.277629596	,	0.071021677	,	0.286923541	,],
[	0.275670424	,	0.344210752	,	0.285796916	,	0.083731103	,]],
[[	0.07923134	,	0.282661737	,	0.32541492	,	0.264150227	,],
[	0.27903483	,	0.079633905	,	0.260004	,	0.329212838	,],
[	0.34220602	,	0.282305534	,	0.078111263	,	0.291119063	,],
[	0.280320202	,	0.348813982	,	0.290514183	,	0.086635752	,]],
[[	0.080891877	,	0.282425588	,	0.32534544	,	0.263996405	,],
[	0.274706979	,	0.076325667	,	0.255943967	,	0.325015811	,],
[	0.338606493	,	0.27780295	,	0.070203063	,	0.287318896	,],
[	0.275604838	,	0.344230488	,	0.285726414	,	0.082485605	,]],
[[	0.081944629	,	0.28582035	,	0.328493338	,	0.267238344	,],
[	0.276050287	,	0.080299007	,	0.257778096	,	0.326445765	,],
[	0.339290198	,	0.278974215	,	0.073199661	,	0.288060274	,],
[	0.277204252	,	0.345595605	,	0.287268031	,	0.086024721	,]],
[[	0.079445265	,	0.294668239	,	0.337363356	,	0.26560834	,],
[	0.271771821	,	0.078854588	,	0.262708351	,	0.323323416	,],
[	0.336773198	,	0.284986493	,	0.072935624	,	0.284881853	,],
[	0.279266646	,	0.358012524	,	0.298944097	,	0.083656476	,]],
[[	0.083878722	,	0.300494542	,	0.342849586	,	0.271367299	,],
[	0.269550245	,	0.078315791	,	0.260856141	,	0.321244262	,],
[	0.33463796	,	0.283101154	,	0.072307306	,	0.282675233	,],
[	0.278381647	,	0.357204105	,	0.298158204	,	0.084264542	,]],
[[	0.078972204	,	0.29498736	,	0.337575833	,	0.266024983	,],
[	0.273555086	,	0.085446415	,	0.265494506	,	0.325170812	,],
[	0.337773898	,	0.286975075	,	0.078456689	,	0.285945423	,],
[	0.282575375	,	0.361266505	,	0.302358908	,	0.086656803	,]],
[[	0.081795585	,	0.295821208	,	0.338522844	,	0.266881933	,],
[	0.270144026	,	0.077778653	,	0.261241251	,	0.321747084	,],
[	0.335055921	,	0.283233567	,	0.071575745	,	0.283033113	,],
[	0.278079584	,	0.357079769	,	0.297976935	,	0.082899186	,]],
[[	0.080850254	,	0.297626138	,	0.340122667	,	0.268457946	,],
[	0.270411061	,	0.080479984	,	0.262097046	,	0.322127715	,],
[	0.335317811	,	0.284026931	,	0.074000611	,	0.283312493	,],
[	0.279380793	,	0.358244226	,	0.299300074	,	0.086417602	,]],
[[	0.079007769	,	0.294587885	,	0.337209656	,	0.265507978	,],
[	0.272890939	,	0.079757516	,	0.263571684	,	0.324260951	,],
[	0.337879305	,	0.286007291	,	0.073811011	,	0.285895588	,],
[	0.280040597	,	0.358738966	,	0.299695063	,	0.084305812	,]],
[[	0.085030394	,	0.297938542	,	0.340639081	,	0.26900959	,],
[	0.269784744	,	0.078075964	,	0.26086873	,	0.321377952	,],
[	0.334370832	,	0.282881456	,	0.071759879	,	0.282444465	,],
[	0.27803881	,	0.356831697	,	0.297864298	,	0.083627457	,]],
[[	0.079018517	,	0.295345526	,	0.337727292	,	0.26624524	,],
[	0.272524397	,	0.083696819	,	0.264272064	,	0.324051421	,],
[	0.336824148	,	0.285822842	,	0.076959221	,	0.284953168	,],
[	0.282379363	,	0.361048638	,	0.302045643	,	0.090422137	,]],
[[	0.080750489	,	0.29516102	,	0.337763806	,	0.266134367	,],
[	0.270751352	,	0.078058075	,	0.261624061	,	0.322269016	,],
[	0.335629546	,	0.283836512	,	0.071903767	,	0.283617709	,],
[	0.278359808	,	0.357193287	,	0.298079839	,	0.082950515	,]],
[[	0.082019871	,	0.298624515	,	0.341127234	,	0.269531945	,],
[	0.270277937	,	0.079706331	,	0.261690626	,	0.321799924	,],
[	0.334886452	,	0.283464903	,	0.073168169	,	0.282818136	,],
[	0.27903964	,	0.357933134	,	0.298869521	,	0.085506743	,]],
[[	0.079157777	,	0.305258424	,	0.34831487	,	0.265331185	,],
[	0.264220465	,	0.080763722	,	0.26679452	,	0.316658327	,],
[	0.331459057	,	0.290291464	,	0.074949148	,	0.278600141	,],
[	0.278338848	,	0.367992051	,	0.308680047	,	0.08296541	,]],
[[	0.083773535	,	0.311057941	,	0.353781379	,	0.271102759	,],
[	0.261330422	,	0.079484686	,	0.264391369	,	0.31391702	,],
[	0.327800512	,	0.286945092	,	0.072440081	,	0.274838606	,],
[	0.277793111	,	0.367493861	,	0.308106749	,	0.084024295	,]],
[[	0.078827199	,	0.3057077	,	0.348650445	,	0.265735228	,],
[	0.264246255	,	0.085368848	,	0.267921184	,	0.316823271	,],
[	0.330323706	,	0.28994094	,	0.077619321	,	0.277430902	,],
[	0.281762996	,	0.3712544	,	0.312042252	,	0.085955138	,]],
[[	0.081986546	,	0.306642497	,	0.349833137	,	0.266774475	,],
[	0.262364448	,	0.079368634	,	0.265167984	,	0.3148402	,],
[	0.329304232	,	0.288154895	,	0.07302823	,	0.27617539	,],
[	0.277310889	,	0.367034493	,	0.307710153	,	0.082342515	,]],
[[	0.08061642	,	0.308083641	,	0.351041272	,	0.268089177	,],
[	0.261670546	,	0.08095208	,	0.265024859	,	0.31434573	,],
[	0.328386068	,	0.287671029	,	0.073999484	,	0.27534158	,],
[	0.279261689	,	0.369008176	,	0.309701968	,	0.086338347	,]],
[[	0.07877189	,	0.305083326	,	0.348244711	,	0.265197068	,],
[	0.265447444	,	0.081949506	,	0.267989013	,	0.317730649	,],
[	0.332601478	,	0.291439716	,	0.076018793	,	0.279674386	,],
[	0.279177625	,	0.368709441	,	0.309345406	,	0.083583325	,]],
[[	0.085172615	,	0.308755627	,	0.352098736	,	0.268907115	,],
[	0.261343627	,	0.079224016	,	0.264343033	,	0.313880789	,],
[	0.328377943	,	0.287366186	,	0.072602104	,	0.27525131	,],
[	0.277528763	,	0.367197309	,	0.307831754	,	0.083230895	,]],
[[	0.079064453	,	0.306113015	,	0.3489512	,	0.266071641	,],
[	0.263352992	,	0.083929102	,	0.2668458	,	0.315884586	,],
[	0.329216165	,	0.288805789	,	0.076196428	,	0.276322	,],
[	0.281996051	,	0.371566207	,	0.312318984	,	0.090247897	,]],
[[	0.080918768	,	0.30630117	,	0.349257037	,	0.266268294	,],
[	0.263123771	,	0.079838978	,	0.265819735	,	0.315542405	,],
[	0.330215281	,	0.289069661	,	0.073698366	,	0.27709592	,],
[	0.277554242	,	0.367330296	,	0.307964343	,	0.082431876	,]],
[[	0.081599059	,	0.309088091	,	0.351977241	,	0.269064104	,],
[	0.26173125	,	0.080582582	,	0.264991787	,	0.314255828	,],
[	0.328057606	,	0.287341763	,	0.073263792	,	0.275005898	,],
[	0.278630815	,	0.368320497	,	0.308972605	,	0.085235224	,]],
[[	0.079820348	,	0.314004691	,	0.357368437	,	0.26528213	,],
[	0.256553395	,	0.081079638	,	0.269594103	,	0.309301441	,],
[	0.324440149	,	0.29302119	,	0.076074956	,	0.271272135	,],
[	0.277263521	,	0.376547403	,	0.317731829	,	0.083140872	,]],
[[	0.082911406	,	0.318659298	,	0.361687207	,	0.270028105	,],
[	0.252945016	,	0.078986587	,	0.266472434	,	0.306033995	,],
[	0.320497711	,	0.289467145	,	0.073345572	,	0.267407174	,],
[	0.277749572	,	0.377068513	,	0.318239023	,	0.085382199	,]],
[[	0.078874569	,	0.314041432	,	0.357255519	,	0.2653489	,],
[	0.255453176	,	0.084450845	,	0.269613877	,	0.3083412	,],
[	0.322424956	,	0.292024066	,	0.078185848	,	0.269437609	,],
[	0.280285615	,	0.379414557	,	0.320777133	,	0.085775481	,]],
[[	0.082757509	,	0.315649351	,	0.359014678	,	0.267057403	,],
[	0.254362665	,	0.079325838	,	0.26761156	,	0.307298748	,],
[	0.32210952	,	0.290824354	,	0.074075704	,	0.268950399	,],
[	0.276501659	,	0.375981907	,	0.317135791	,	0.082938409	,]],
[[	0.080288699	,	0.31616404	,	0.359286755	,	0.267384521	,],
[	0.253033689	,	0.080281735	,	0.266879197	,	0.3061686	,],
[	0.321038118	,	0.290149468	,	0.074763738	,	0.267830225	,],
[	0.279394315	,	0.378746866	,	0.320077613	,	0.088163043	,]],
[[	0.079159339	,	0.313639996	,	0.356942777	,	0.264878519	,],
[	0.2577501	,	0.082015455	,	0.270702094	,	0.310435227	,],
[	0.324626121	,	0.29443796	,	0.081661458	,	0.271611988	,],
[	0.27782538	,	0.377159322	,	0.318342115	,	0.083635434	,]],
[[	0.08445903	,	0.320273538	,	0.363168471	,	0.271602308	,],
[	0.253063968	,	0.078738685	,	0.266469732	,	0.306005711	,],
[	0.320963772	,	0.289770674	,	0.073535922	,	0.267762836	,],
[	0.27746778	,	0.376748969	,	0.317849285	,	0.084632605	,]],
[[	0.079071398	,	0.31445075	,	0.357603963	,	0.265765588	,],
[	0.254853216	,	0.083276612	,	0.268834386	,	0.307742803	,],
[	0.321694992	,	0.291061112	,	0.076842086	,	0.268550914	,],
[	0.281199356	,	0.380351824	,	0.321632071	,	0.086482543	,]],
[[	0.081653699	,	0.315028884	,	0.35836602	,	0.266359898	,],
[	0.255119847	,	0.079829937	,	0.268272746	,	0.307959994	,],
[	0.323043036	,	0.29163966	,	0.074681748	,	0.269781539	,],
[	0.276760788	,	0.37610943	,	0.3172703	,	0.082964276	,]],
[[	0.081405814	,	0.317150685	,	0.360197521	,	0.268400247	,],
[	0.253191898	,	0.079650688	,	0.266784063	,	0.306017293	,],
[	0.320272377	,	0.289331208	,	0.073758815	,	0.267081934	,],
[	0.278768925	,	0.378091738	,	0.319269558	,	0.086877419	,]],
[[	0.080213204	,	0.325649031	,	0.369079191	,	0.26628579	,],
[	0.250256484	,	0.087439096	,	0.275174184	,	0.304318219	,],
[	0.318600859	,	0.297693784	,	0.081428055	,	0.264638549	,],
[	0.279329196	,	0.388397643	,	0.329631904	,	0.083441212	,]],
[[	0.084717313	,	0.331317609	,	0.374310301	,	0.271945742	,],
[	0.248208209	,	0.080872126	,	0.272055681	,	0.302016934	,],
[	0.316706622	,	0.294859714	,	0.075307426	,	0.262595047	,],
[	0.278148052	,	0.387397119	,	0.328477132	,	0.083810208	,]],
[[	0.079583177	,	0.325900753	,	0.369068744	,	0.266471423	,],
[	0.248028467	,	0.083206485	,	0.272442079	,	0.301960098	,],
[	0.316504884	,	0.295306977	,	0.077434518	,	0.262468678	,],
[	0.282863382	,	0.39171787	,	0.333250876	,	0.086792057	,]],
[[	0.082671036	,	0.326883845	,	0.37025561	,	0.267515901	,],
[	0.25101401	,	0.083035196	,	0.274588039	,	0.304738585	,],
[	0.319364154	,	0.297422406	,	0.077519023	,	0.265278493	,],
[	0.278054549	,	0.387189595	,	0.328565956	,	0.082639635	,]],
[[	0.081511338	,	0.328475365	,	0.371671235	,	0.269075502	,],
[	0.247224164	,	0.080833529	,	0.271291004	,	0.301171039	,],
[	0.315733999	,	0.294213149	,	0.075197861	,	0.261635291	,],
[	0.279233898	,	0.388517475	,	0.329683152	,	0.085778183	,]],
[[	0.080004937	,	0.325806409	,	0.36915239	,	0.266320282	,],
[	0.249665454	,	0.086152763	,	0.274361971	,	0.303648051	,],
[	0.317769277	,	0.296786837	,	0.07999678	,	0.263673677	,],
[	0.280435497	,	0.389337459	,	0.330611261	,	0.084298614	,]],
[[	0.086025035	,	0.329121115	,	0.372493341	,	0.269721289	,],
[	0.248815856	,	0.081268599	,	0.272532044	,	0.302638386	,],
[	0.317226845	,	0.295404484	,	0.075593773	,	0.263080215	,],
[	0.278074385	,	0.387176663	,	0.328352146	,	0.08322958	,]],
[[	0.080198421	,	0.326783637	,	0.369888658	,	0.267200334	,],
[	0.247661001	,	0.082235941	,	0.271808126	,	0.301362316	,],
[	0.316057278	,	0.294750338	,	0.07647669	,	0.261957745	,],
[	0.281740953	,	0.390839133	,	0.332095441	,	0.089218808	,]],
[[	0.08164485	,	0.326381873	,	0.369564156	,	0.266941611	,],
[	0.252002554	,	0.084003722	,	0.275477456	,	0.305744193	,],
[	0.320332321	,	0.299685101	,	0.083985992	,	0.266302979	,],
[	0.278572068	,	0.38767731	,	0.328995737	,	0.082712698	,]],
[[	0.082645433	,	0.329525635	,	0.372544218	,	0.270084388	,],
[	0.247922485	,	0.080969767	,	0.271790655	,	0.301619241	,],
[	0.31583151	,	0.29410879	,	0.074861889	,	0.261625283	,],
[	0.278945119	,	0.388258823	,	0.329416881	,	0.085041834	]]],dtype=np.float32)
								
x_train=np.array([[[	0.10962821	,	0.231239038	,	0.250216802	,	0.258478596	,],
[	0.416053368	,	0.09124152	,	0.255613519	,	0.402112363	,],
[	0.491449922	,	0.322576596	,	0.081862485	,	0.374501945	,],
[	0.386704565	,	0.347314133	,	0.251958299	,	0.10224749	,]],
[[	0.106058228	,	0.22602485	,	0.245295958	,	0.253749427	,],
[	0.413201372	,	0.089706536	,	0.252695553	,	0.399326333	,],
[	0.495158212	,	0.326660986	,	0.088634624	,	0.378368622	,],
[	0.388896581	,	0.350290703	,	0.254901865	,	0.10224192	,]],
[[	0.110234557	,	0.227183218	,	0.246573344	,	0.255504034	,],
[	0.415967864	,	0.09512945	,	0.255728787	,	0.402074233	,],
[	0.491443035	,	0.322681193	,	0.080875994	,	0.374522941	,],
[	0.385897997	,	0.346773299	,	0.251389968	,	0.100098941	,]],
[[	0.107167279	,	0.228342353	,	0.247564173	,	0.25579616	,],
[	0.414262088	,	0.089715169	,	0.25359853	,	0.40016542	,],
[	0.49263479	,	0.32390666	,	0.083995584	,	0.375708399	,],
[	0.388625928	,	0.349118627	,	0.253821445	,	0.105237565	,]],
[[	0.106830859	,	0.225746448	,	0.245137354	,	0.253657034	,],
[	0.413535967	,	0.090926585	,	0.252981461	,	0.399597695	,],
[	0.493535987	,	0.32493333	,	0.082450558	,	0.376766919	,],
[	0.386763962	,	0.347798783	,	0.252424322	,	0.100177339	,]],
[[	0.110910128	,	0.232611613	,	0.251536149	,	0.259797055	,],
[	0.416895879	,	0.092149476	,	0.256531236	,	0.402885551	,],
[	0.491350909	,	0.322386441	,	0.081291108	,	0.374232837	,],
[	0.386264382	,	0.346888586	,	0.251404493	,	0.101225803	,]],
[[	0.106167469	,	0.226549916	,	0.245786245	,	0.254179797	,],
[	0.413295597	,	0.089263496	,	0.252776501	,	0.399341196	,],
[	0.494439973	,	0.325898775	,	0.087245211	,	0.377559992	,],
[	0.390032239	,	0.351419236	,	0.256189207	,	0.103223348	,]],
[[	0.109127515	,	0.226707442	,	0.24606056	,	0.254902661	,],
[	0.415427681	,	0.094013187	,	0.254971063	,	0.401425991	,],
[	0.491791185	,	0.323116369	,	0.080851485	,	0.374957374	,],
[	0.385823085	,	0.346816437	,	0.251407966	,	0.099774397	,]],
[[	0.108214502	,	0.229565985	,	0.248769918	,	0.256891234	,],
[	0.414904794	,	0.090183123	,	0.254241881	,	0.400790968	,],
[	0.492314766	,	0.3234639	,	0.083285124	,	0.375251535	,],
[	0.387703066	,	0.34824434	,	0.252818455	,	0.103745165	,]],
[[	0.106485334	,	0.225884514	,	0.245299213	,	0.253649328	,],
[	0.413491879	,	0.090501107	,	0.252825147	,	0.399457873	,],
[	0.494371193	,	0.325850874	,	0.083243738	,	0.377595203	,],
[	0.387629793	,	0.348709395	,	0.253331071	,	0.100840987	,]],
[[	0.108872489	,	0.242517981	,	0.258750393	,	0.256881686	,],
[	0.39704893	,	0.090355109	,	0.249570656	,	0.386126503	,],
[	0.478229421	,	0.323904302	,	0.081874919	,	0.365345321	,],
[	0.375767014	,	0.35114613	,	0.256172984	,	0.098720978	,]],
[[	0.103824279	,	0.23620994	,	0.252598859	,	0.250912481	,],
[	0.399902478	,	0.09084506	,	0.252422344	,	0.388886813	,],
[	0.47562322	,	0.32091675	,	0.079949696	,	0.362450473	,],
[	0.376062934	,	0.351026077	,	0.256037371	,	0.100792177	,]],
[[	0.106287862	,	0.236021522	,	0.252583268	,	0.25118942	,],
[	0.39645998	,	0.088485004	,	0.248866423	,	0.385416683	,],
[	0.477926261	,	0.323482277	,	0.084928097	,	0.36486709	,],
[	0.378374733	,	0.353974131	,	0.259085627	,	0.100833478	,]],
[[	0.105912639	,	0.23929789	,	0.255684381	,	0.253699741	,],
[	0.398473762	,	0.093049109	,	0.251045767	,	0.38737433	,],
[	0.476507893	,	0.322015276	,	0.080177559	,	0.363500485	,],
[	0.375101723	,	0.350353376	,	0.255413127	,	0.098624652	,]],
[[	0.104112318	,	0.235679837	,	0.252179807	,	0.250443938	,],
[	0.397591451	,	0.088810684	,	0.249924867	,	0.38654374	,],
[	0.47596119	,	0.321315914	,	0.081243998	,	0.362675093	,],
[	0.378066427	,	0.352897224	,	0.257998963	,	0.103800339	,]],
[[	0.109359501	,	0.237874824	,	0.25448369	,	0.253160976	,],
[	0.396833107	,	0.089692575	,	0.249122097	,	0.385605861	,],
[	0.479539804	,	0.325187176	,	0.083100408	,	0.366596421	,],
[	0.376303065	,	0.351748454	,	0.256632055	,	0.099029372	,]],
[[	0.104424417	,	0.237174277	,	0.253517883	,	0.251731354	,],
[	0.400620373	,	0.096663087	,	0.253496148	,	0.389589051	,],
[	0.475771245	,	0.321076708	,	0.079800429	,	0.362474802	,],
[	0.375824523	,	0.350790048	,	0.255678355	,	0.100004569	,]],
[[	0.105445893	,	0.235671056	,	0.252293014	,	0.250700056	,],
[	0.396737081	,	0.088295859	,	0.249066249	,	0.385656767	,],
[	0.477209506	,	0.322639782	,	0.083502809	,	0.363942933	,],
[	0.379549758	,	0.355194058	,	0.260217634	,	0.101884241	,]],
[[	0.107106631	,	0.240621781	,	0.256887771	,	0.254889876	,],
[	0.398046519	,	0.09210335	,	0.250497677	,	0.386869824	,],
[	0.477400392	,	0.322846971	,	0.080872476	,	0.364295201	,],
[	0.375284124	,	0.350593007	,	0.255530382	,	0.098443239	,]],
[[	0.103908238	,	0.235745871	,	0.252270349	,	0.25045144	,],
[	0.398757326	,	0.089737916	,	0.251051765	,	0.387552783	,],
[	0.476066315	,	0.321267109	,	0.080745783	,	0.362681464	,],
[	0.377214966	,	0.352079202	,	0.257162892	,	0.102424594	,]],
[[	0.106837111	,	0.245880113	,	0.260003165	,	0.250407641	,],
[	0.381027808	,	0.087776196	,	0.245221056	,	0.373136901	,],
[	0.462266336	,	0.3209679	,	0.082095271	,	0.353297758	,],
[	0.368235441	,	0.356038992	,	0.260720977	,	0.099228967	,]],
[[	0.101405064	,	0.244614495	,	0.258508669	,	0.248552952	,],
[	0.381748961	,	0.090755345	,	0.246130659	,	0.373837802	,],
[	0.462779335	,	0.321558374	,	0.080056043	,	0.354026781	,],
[	0.365024413	,	0.352598581	,	0.257121136	,	0.096978314	,]],
[[	0.102996483	,	0.243823351	,	0.257794555	,	0.248103255	,],
[	0.383458007	,	0.089554543	,	0.247636754	,	0.3754519	,],
[	0.460674561	,	0.319284947	,	0.079175126	,	0.35173458	,],
[	0.368177206	,	0.355286969	,	0.259928916	,	0.102452519	,]],
[[	0.103889502	,	0.247919663	,	0.261844951	,	0.251645652	,],
[	0.38067827	,	0.088271374	,	0.244889376	,	0.3727971	,],
[	0.463977076	,	0.322876517	,	0.085214516	,	0.355135833	,],
[	0.366275493	,	0.354098523	,	0.258595309	,	0.097418764	,]],
[[	0.100922316	,	0.243352956	,	0.257400491	,	0.247444286	,],
[	0.38381572	,	0.094001094	,	0.24832644	,	0.37588769	,],
[	0.461302932	,	0.319917352	,	0.078864433	,	0.352419643	,],
[	0.365812054	,	0.353220248	,	0.257787732	,	0.098524198	,]],
[[	0.105639167	,	0.245219015	,	0.259170213	,	0.249676898	,],
[	0.381530277	,	0.088013711	,	0.245610207	,	0.373463977	,],
[	0.461495447	,	0.32019396	,	0.080884861	,	0.352493134	,],
[	0.369582278	,	0.357392307	,	0.262005955	,	0.100307913	,]],
[[	0.101845019	,	0.24527837	,	0.25916106	,	0.249125464	,],
[	0.381242253	,	0.0896782	,	0.245409118	,	0.373135758	,],
[	0.463841077	,	0.322580241	,	0.081015845	,	0.355006846	,],
[	0.365330523	,	0.352868991	,	0.25738101	,	0.09689679	,]],
[[	0.102213528	,	0.243543201	,	0.257403233	,	0.247729198	,],
[	0.38466106	,	0.090775912	,	0.248911543	,	0.376736531	,],
[	0.460749545	,	0.319256307	,	0.078777601	,	0.351620751	,],
[	0.367363354	,	0.354495277	,	0.2590182	,	0.101010684	,]],
[[	0.105361814	,	0.249543462	,	0.263337341	,	0.253156771	,],
[	0.380660897	,	0.087821194	,	0.244902468	,	0.372775971	,],
[	0.463170898	,	0.322005692	,	0.083778531	,	0.354143485	,],
[	0.366771854	,	0.354577422	,	0.259127181	,	0.097830131	,]],
[[	0.10115374	,	0.243939263	,	0.257805801	,	0.247805374	,],
[	0.382828898	,	0.092590247	,	0.247233078	,	0.374825038	,],
[	0.461947574	,	0.320640931	,	0.07933993	,	0.3529424	,],
[	0.36570951	,	0.353200824	,	0.257618867	,	0.097831067	,]],
[[	0.104357644	,	0.254561108	,	0.265933705	,	0.248586712	,],
[	0.367855519	,	0.088948392	,	0.243237596	,	0.36247122	,],
[	0.446471477	,	0.318730947	,	0.079356064	,	0.340713239	,],
[	0.360016163	,	0.360447519	,	0.26489325	,	0.103877144	,]],
[[	0.099156007	,	0.253443658	,	0.264550979	,	0.246867219	,],
[	0.366178608	,	0.088869921	,	0.241522949	,	0.36081764	,],
[	0.450566249	,	0.323057001	,	0.081755555	,	0.345196668	,],
[	0.355697924	,	0.356553537	,	0.260731516	,	0.095638941	,]],
[[	0.100444876	,	0.252392896	,	0.263581324	,	0.246173732	,],
[	0.369636171	,	0.09546404	,	0.245534257	,	0.36424282	,],
[	0.446489116	,	0.318711059	,	0.078176803	,	0.340755374	,],
[	0.357035485	,	0.357562509	,	0.261832563	,	0.099025903	,]],
[[	0.101970423	,	0.257068015	,	0.268083354	,	0.250189286	,],
[	0.366406098	,	0.087987904	,	0.241758875	,	0.36106769	,],
[	0.447934174	,	0.320275984	,	0.081806617	,	0.342131823	,],
[	0.357461692	,	0.35859079	,	0.262835783	,	0.096843191	,]],
[[	0.098595344	,	0.252261256	,	0.263434932	,	0.245705623	,],
[	0.36708685	,	0.091022303	,	0.242782298	,	0.361805343	,],
[	0.448405474	,	0.320666032	,	0.079374128	,	0.342850447	,],
[	0.355510954	,	0.356385789	,	0.260646067	,	0.096167339	,]],
[[	0.103278948	,	0.253807083	,	0.265142333	,	0.247702465	,],
[	0.368985152	,	0.089875909	,	0.244316179	,	0.363580428	,],
[	0.446238131	,	0.318549439	,	0.078547314	,	0.340471647	,],
[	0.359065605	,	0.359580288	,	0.263938237	,	0.1022823	,]],
[[	0.099809801	,	0.254268227	,	0.265267431	,	0.24755527	,],
[	0.36610203	,	0.088434958	,	0.24146134	,	0.360616799	,],
[	0.449835788	,	0.322502572	,	0.08530417	,	0.344301948	,],
[	0.355891486	,	0.356859602	,	0.261056967	,	0.095629667	,]],
[[	0.099714106	,	0.252154269	,	0.263352297	,	0.245831271	,],
[	0.368744179	,	0.093917522	,	0.244428744	,	0.363265785	,],
[	0.446954691	,	0.319182843	,	0.078332771	,	0.341158686	,],
[	0.356470444	,	0.357123591	,	0.26127375	,	0.098046094	,]],
[[	0.103314148	,	0.258306267	,	0.269398982	,	0.251422863	,],
[	0.367104841	,	0.088480867	,	0.242436445	,	0.361712018	,],
[	0.447215845	,	0.319567436	,	0.080575843	,	0.341425427	,],
[	0.358382706	,	0.359455729	,	0.263740706	,	0.097597451	,]],
[[	0.098829202	,	0.252753627	,	0.263913195	,	0.246100374	,],
[	0.366594221	,	0.089978964	,	0.242096742	,	0.361192465	,],
[	0.449237629	,	0.321621378	,	0.080225256	,	0.343670362	,],
[	0.355541933	,	0.356436802	,	0.26056282	,	0.095725081	,]],
[[	0.10145671	,	0.260531486	,	0.269969989	,	0.24427133	,],
[	0.35185918	,	0.093413403	,	0.240759881	,	0.350353849	,],
[	0.43061611	,	0.315624484	,	0.076607695	,	0.328831447	,],
[	0.346274825	,	0.358410398	,	0.264215807	,	0.097240786	,]],
[[	0.096960269	,	0.25994066	,	0.269183941	,	0.2430785	,],
[	0.348427644	,	0.085530791	,	0.23658601	,	0.346841195	,],
[	0.431792717	,	0.316983866	,	0.080092698	,	0.330046758	,],
[	0.345730899	,	0.358146036	,	0.263948667	,	0.093627114	,]],
[[	0.097669163	,	0.258454469	,	0.267852291	,	0.241933892	,],
[	0.349311488	,	0.088748035	,	0.237797899	,	0.347790383	,],
[	0.432524607	,	0.317624636	,	0.077895637	,	0.330866476	,],
[	0.344397173	,	0.356622986	,	0.262354594	,	0.093740524	,]],
[[	0.099868867	,	0.263647764	,	0.272925874	,	0.246635967	,],
[	0.350993748	,	0.087387355	,	0.239159892	,	0.349332324	,],
[	0.430542125	,	0.315532974	,	0.077189764	,	0.32870891	,],
[	0.34881681	,	0.360562662	,	0.2664332	,	0.100927529	,]],
[[	0.095931208	,	0.258382877	,	0.267796556	,	0.241623311	,],
[	0.348247161	,	0.086086093	,	0.236449094	,	0.346732512	,],
[	0.43341112	,	0.318897032	,	0.083176114	,	0.331747258	,],
[	0.344505006	,	0.356973228	,	0.262747268	,	0.092842516	,]],
[[	0.099934484	,	0.259628333	,	0.269159389	,	0.243236114	,],
[	0.350933644	,	0.091983554	,	0.239713146	,	0.349394223	,],
[	0.430900732	,	0.315843819	,	0.076539996	,	0.329078624	,],
[	0.345701205	,	0.357846036	,	0.263551829	,	0.096028411	,]],
[[	0.09750943	,	0.260617736	,	0.269856737	,	0.243773772	,],
[	0.349054999	,	0.085783435	,	0.237146669	,	0.347276511	,],
[	0.431489912	,	0.316598441	,	0.079243701	,	0.329674446	,],
[	0.346724892	,	0.359116526	,	0.264844568	,	0.094266026	,]],
[[	0.09701183	,	0.258283098	,	0.267598496	,	0.241734562	,],
[	0.348779866	,	0.087776672	,	0.237110323	,	0.347151156	,],
[	0.433506342	,	0.318603827	,	0.078840345	,	0.331841528	,],
[	0.34444434	,	0.356725508	,	0.262278807	,	0.093379201	,]],
[[	0.101374484	,	0.265055872	,	0.274404643	,	0.248098137	,],
[	0.351842937	,	0.088073121	,	0.239956221	,	0.350077058	,],
[	0.430689523	,	0.315637013	,	0.07686986	,	0.328651291	,],
[	0.348129672	,	0.359932163	,	0.265742804	,	0.099536166	,]],
[[	0.096308302	,	0.259130814	,	0.26855407	,	0.242242109	,],
[	0.348067961	,	0.085573025	,	0.236192552	,	0.346355624	,],
[	0.432651216	,	0.317957754	,	0.081875154	,	0.330878388	,],
[	0.345058846	,	0.357470452	,	0.263211582	,	0.09298036	,]],
[[	0.100229774	,	0.273758409	,	0.280933151	,	0.245885683	,],
[	0.342244094	,	0.088499818	,	0.23865363	,	0.341635464	,],
[	0.432162954	,	0.327553501	,	0.081790362	,	0.330382276	,],
[	0.345901979	,	0.371426212	,	0.2732319	,	0.093980392	,]],
[[	0.098013926	,	0.274883892	,	0.281931634	,	0.246563348	,],
[	0.346259099	,	0.095771755	,	0.243113673	,	0.345728877	,],
[	0.427854008	,	0.323083144	,	0.078013617	,	0.325696004	,],
[	0.349857104	,	0.375088418	,	0.27700359	,	0.100483972	,]],
[[	0.097215683	,	0.272331819	,	0.27937549	,	0.244287801	,],
[	0.342313303	,	0.087277001	,	0.238429868	,	0.341689518	,],
[	0.429498256	,	0.324980117	,	0.082208624	,	0.327292505	,],
[	0.346251946	,	0.371776208	,	0.273552016	,	0.093255438	,]],
[[	0.101791047	,	0.279140399	,	0.28610408	,	0.250799111	,],
[	0.343527168	,	0.091013895	,	0.23995548	,	0.342809402	,],
[	0.429623268	,	0.324882677	,	0.079273025	,	0.327522166	,],
[	0.347093784	,	0.372412018	,	0.27426326	,	0.095986616	,]],
[[	0.096652207	,	0.273161227	,	0.280194152	,	0.244767164	,],
[	0.344817579	,	0.089108333	,	0.240935884	,	0.344044757	,],
[	0.427605251	,	0.322927732	,	0.078618873	,	0.325365413	,],
[	0.348356519	,	0.374064127	,	0.275979546	,	0.094803627	,]],
[[	0.099563108	,	0.273504174	,	0.280639265	,	0.245557568	,],
[	0.34217155	,	0.088078863	,	0.238442911	,	0.341518322	,],
[	0.431648271	,	0.327281824	,	0.085647897	,	0.329423369	,],
[	0.345760578	,	0.371410341	,	0.273102686	,	0.093320102	,]],
[[	0.098841889	,	0.275829413	,	0.282768836	,	0.247426857	,],
[	0.345478009	,	0.094270533	,	0.24210845	,	0.344783139	,],
[	0.428254687	,	0.323432322	,	0.078221049	,	0.325988331	,],
[	0.349159875	,	0.374405266	,	0.276142638	,	0.09907784	,]],
[[	0.096978551	,	0.2724426	,	0.279549183	,	0.244265434	,],
[	0.342954264	,	0.087743085	,	0.239058922	,	0.342215188	,],
[	0.428941522	,	0.324276948	,	0.08099321	,	0.326577277	,],
[	0.346505899	,	0.372135579	,	0.273810913	,	0.093216453	,]],
[[	0.102703321	,	0.275051903	,	0.28216422	,	0.247369248	,],
[	0.343145776	,	0.08990806	,	0.239328309	,	0.342290584	,],
[	0.430544302	,	0.325749404	,	0.080157348	,	0.328405654	,],
[	0.34659772	,	0.371971381	,	0.27364861	,	0.09493673	,]],
[[	0.097259559	,	0.273853618	,	0.280896774	,	0.245451556	,],
[	0.345578813	,	0.089785116	,	0.241751078	,	0.344747769	,],
[	0.427593717	,	0.322700967	,	0.078233792	,	0.325195395	,],
[	0.349031432	,	0.374735727	,	0.276672545	,	0.095527268	,]],
[[	0.099228328	,	0.28498358	,	0.289498657	,	0.246337255	,],
[	0.330266169	,	0.088488221	,	0.237827412	,	0.332593179	,],
[	0.420627662	,	0.328917802	,	0.084190556	,	0.320927297	,],
[	0.340627823	,	0.378915146	,	0.278872287	,	0.093134604	,]],
[[	0.096406159	,	0.285571384	,	0.289875724	,	0.246442888	,],
[	0.332089345	,	0.093124453	,	0.240034768	,	0.334443822	,],
[	0.418350887	,	0.326249009	,	0.078154092	,	0.318569198	,],
[	0.344524271	,	0.382461251	,	0.282638199	,	0.099469743	,]],
[[	0.096048198	,	0.283340867	,	0.287803004	,	0.244429933	,],
[	0.331455308	,	0.088723019	,	0.23879557	,	0.33369532	,],
[	0.418460859	,	0.326383377	,	0.080096032	,	0.318565558	,],
[	0.341310067	,	0.379564116	,	0.279539479	,	0.092653401	,]],
[[	0.100159923	,	0.289701078	,	0.29391165	,	0.250419966	,],
[	0.330381073	,	0.089597579	,	0.23797312	,	0.332611511	,],
[	0.421478853	,	0.32933413	,	0.081039213	,	0.321761683	,],
[	0.341659031	,	0.379709105	,	0.279713193	,	0.094916653	,]],
[[	0.095185791	,	0.283792815	,	0.288238971	,	0.244697779	,],
[	0.334577328	,	0.096999663	,	0.242604301	,	0.33672011	,],
[	0.41756013	,	0.32545881	,	0.077886084	,	0.317673099	,],
[	0.343867266	,	0.382162846	,	0.282400855	,	0.094849078	,]],
[[	0.098245283	,	0.284380571	,	0.288893489	,	0.245542851	,],
[	0.330265343	,	0.088144709	,	0.237743821	,	0.332485499	,],
[	0.420031792	,	0.328198769	,	0.082844398	,	0.320106659	,],
[	0.34055008	,	0.378800235	,	0.278856179	,	0.092525089	,]],
[[	0.097400429	,	0.286614375	,	0.290941197	,	0.247446599	,],
[	0.331383993	,	0.091795093	,	0.23920036	,	0.333668082	,],
[	0.419157571	,	0.32700453	,	0.078879863	,	0.319291446	,],
[	0.343741194	,	0.38174798	,	0.281838569	,	0.0981167	,]],
[[	0.095420056	,	0.283187399	,	0.287551871	,	0.244152453	,],
[	0.332381716	,	0.089416448	,	0.239686421	,	0.334435154	,],
[	0.418003461	,	0.32589541	,	0.079109191	,	0.318066963	,],
[	0.341917208	,	0.380049895	,	0.280175321	,	0.093132505	,]],
[[	0.101517004	,	0.291050612	,	0.295411854	,	0.251832712	,],
[	0.330057376	,	0.088879688	,	0.237550863	,	0.332234865	,],
[	0.42227316	,	0.330626809	,	0.086586562	,	0.322405255	,],
[	0.341148073	,	0.379218313	,	0.279193463	,	0.093971498	,]],
[[	0.095701294	,	0.284695846	,	0.289068459	,	0.245380445	,],
[	0.333877753	,	0.095663981	,	0.241873174	,	0.335972067	,],
[	0.417871709	,	0.325725458	,	0.07805551	,	0.317886034	,],
[	0.344972912	,	0.383303919	,	0.283575286	,	0.095900289	,]],
[[	0.102161326	,	0.306900185	,	0.308583566	,	0.255075391	,],
[	0.329833861	,	0.091702729	,	0.244980362	,	0.334172827	,],
[	0.422447042	,	0.341325733	,	0.0848566	,	0.321180194	,],
[	0.346491875	,	0.399264957	,	0.294328348	,	0.094475831	,]],
[[	0.099201719	,	0.307275995	,	0.308810065	,	0.255016138	,],
[	0.330035681	,	0.094094233	,	0.245407933	,	0.334341119	,],
[	0.422581195	,	0.341245408	,	0.082068424	,	0.321482334	,],
[	0.348454389	,	0.401000059	,	0.296158804	,	0.098449713	,]],
[[	0.098984872	,	0.305265032	,	0.306853595	,	0.253211041	,],
[	0.332969082	,	0.09408797	,	0.247939281	,	0.33708585	,],
[	0.420989681	,	0.339643222	,	0.08171686	,	0.319650493	,],
[	0.348018196	,	0.400752072	,	0.295989776	,	0.095348654	,]],
[[	0.102776474	,	0.311216398	,	0.312769038	,	0.258886125	,],
[	0.32920014	,	0.091841567	,	0.24432457	,	0.333468998	,],
[	0.42476397	,	0.34382373	,	0.088469603	,	0.323511098	,],
[	0.346804184	,	0.399445422	,	0.294448372	,	0.095412066	,]],
[[	0.097801135	,	0.305511358	,	0.307128773	,	0.253223149	,],
[	0.331805644	,	0.097076388	,	0.247374776	,	0.336100622	,],
[	0.421102199	,	0.339619341	,	0.080748941	,	0.319845354	,],
[	0.351201797	,	0.403448293	,	0.298605949	,	0.102401042	,]],
[[	0.100992442	,	0.30642006	,	0.308101981	,	0.254397753	,],
[	0.330537913	,	0.092179947	,	0.245666253	,	0.334678107	,],
[	0.422065095	,	0.340712105	,	0.083858636	,	0.32057697	,],
[	0.346711981	,	0.399493746	,	0.294556749	,	0.094376044	,]],
[[	0.100141541	,	0.308181911	,	0.309671138	,	0.255890299	,],
[	0.329681551	,	0.093176598	,	0.245036499	,	0.333978824	,],
[	0.423426186	,	0.341972283	,	0.082696356	,	0.322213635	,],
[	0.347759938	,	0.400430039	,	0.29544299	,	0.097251023	,]],
[[	0.098541252	,	0.305252694	,	0.306771595	,	0.253086455	,],
[	0.333953938	,	0.095153082	,	0.249010447	,	0.337949382	,],
[	0.420696704	,	0.339276329	,	0.081052049	,	0.319288863	,],
[	0.349239127	,	0.401938932	,	0.297139718	,	0.096577465	,]],
[[	0.104297908	,	0.312616293	,	0.314175022	,	0.260290011	,],
[	0.329474811	,	0.091669871	,	0.244466227	,	0.333606809	,],
[	0.423922234	,	0.34280367	,	0.08686099	,	0.322534076	,],
[	0.346382186	,	0.399104648	,	0.29407188	,	0.094604882	,]],
[[	0.098435186	,	0.306287063	,	0.307829938	,	0.253961911	,],
[	0.331001281	,	0.095846647	,	0.246530064	,	0.33528957	,],
[	0.421604046	,	0.340074162	,	0.081124824	,	0.320238982	,],
[	0.349935898	,	0.402349016	,	0.297515128	,	0.100703528	,]],
[[	0.099369329	,	0.316085179	,	0.31518743	,	0.253705314	,],
[	0.31810484	,	0.0930458	,	0.244313474	,	0.325744537	,],
[	0.41077172	,	0.34273548	,	0.08192305	,	0.312738563	,],
[	0.341079821	,	0.407453773	,	0.300408721	,	0.094343306	,]],
[[	0.097574292	,	0.317283172	,	0.316218076	,	0.25458633	,],
[	0.315762648	,	0.092317201	,	0.242269203	,	0.323481222	,],
[	0.414865571	,	0.346789149	,	0.084417824	,	0.317272177	,],
[	0.34274747	,	0.409153379	,	0.301985994	,	0.098097606	,]],
[[	0.096697859	,	0.314773417	,	0.313927035	,	0.252293418	,],
[	0.318945366	,	0.098592189	,	0.245923811	,	0.32671717	,],
[	0.410652265	,	0.342478538	,	0.080733799	,	0.312740314	,],
[	0.342729336	,	0.409145602	,	0.302101478	,	0.095478843	,]],
[[	0.101576174	,	0.321501878	,	0.320462286	,	0.258758514	,],
[	0.31650341	,	0.091795283	,	0.242604054	,	0.324089435	,],
[	0.412109387	,	0.344325445	,	0.084503313	,	0.314148608	,],
[	0.341164299	,	0.40760173	,	0.300323114	,	0.0950891	,]],
[[	0.096132903	,	0.315427482	,	0.314490303	,	0.252738215	,],
[	0.316443318	,	0.094174899	,	0.242902821	,	0.324122815	,],
[	0.412238645	,	0.344020995	,	0.081751137	,	0.314419948	,],
[	0.345161605	,	0.411180843	,	0.304113041	,	0.101487493	,]],
[[	0.098118661	,	0.315283358	,	0.314489875	,	0.252767959	,],
[	0.319013976	,	0.093836502	,	0.245256147	,	0.326648451	,],
[	0.410627208	,	0.34254964	,	0.081402436	,	0.312544925	,],
[	0.341282562	,	0.407877	,	0.300623359	,	0.094347165	,]],
[[	0.098759708	,	0.318491711	,	0.31737686	,	0.255744514	,],
[	0.315857093	,	0.091918638	,	0.242190202	,	0.323490616	,],
[	0.414246557	,	0.346604241	,	0.08797557	,	0.316295912	,],
[	0.342302198	,	0.408656997	,	0.301444547	,	0.097047991	,]],
[[	0.09613343	,	0.314589531	,	0.313628006	,	0.251975884	,],
[	0.318084613	,	0.097157949	,	0.24494956	,	0.325823334	,],
[	0.411231414	,	0.342983197	,	0.081051266	,	0.313199742	,],
[	0.343950323	,	0.41030957	,	0.303150774	,	0.09651909	,]],
[[	0.101548336	,	0.317198349	,	0.316259702	,	0.254878993	,],
[	0.317229427	,	0.092373788	,	0.24335664	,	0.324752613	,],
[	0.411683361	,	0.343676727	,	0.083364678	,	0.313593298	,],
[	0.341320502	,	0.407676002	,	0.300337791	,	0.094776738	,]],
[[	0.096565716	,	0.316091631	,	0.315179996	,	0.253375519	,],
[	0.316374291	,	0.093574044	,	0.242801954	,	0.323912247	,],
[	0.413614466	,	0.3454392	,	0.082991302	,	0.315780385	,],
[	0.344379661	,	0.410495991	,	0.303294242	,	0.100130967	,]],
[[	0.099467017	,	0.33089344	,	0.326664135	,	0.255879492	,],
[	0.311122471	,	0.100848961	,	0.247815089	,	0.320266528	,],
[	0.406151265	,	0.350068779	,	0.082447363	,	0.307852198	,],
[	0.341492459	,	0.420527215	,	0.310728339	,	0.094945454	,]],
[[	0.09817209	,	0.332250175	,	0.327938366	,	0.257125539	,],
[	0.308141008	,	0.093577213	,	0.244129941	,	0.317302354	,],
[	0.408643099	,	0.353016476	,	0.087557579	,	0.310453482	,],
[	0.340845878	,	0.419820989	,	0.30988937	,	0.095695693	,]],
[[	0.096839286	,	0.329580895	,	0.325298034	,	0.254590726	,],
[	0.308280647	,	0.09611434	,	0.244709263	,	0.317420585	,],
[	0.407491461	,	0.351268477	,	0.082857526	,	0.309262901	,],
[	0.344592525	,	0.423362388	,	0.313773131	,	0.097774905	,]],
[[	0.102679409	,	0.336885883	,	0.332529494	,	0.261754031	,],
[	0.310816057	,	0.095467838	,	0.246638462	,	0.31986604	,],
[	0.406516294	,	0.350476078	,	0.083635215	,	0.30820285	,],
[	0.34047224	,	0.419409807	,	0.30943695	,	0.094224632	,]],
[[	0.09659549	,	0.330552117	,	0.326288171	,	0.255250593	,],
[	0.307500886	,	0.093639028	,	0.243507552	,	0.316597373	,],
[	0.410386804	,	0.354079864	,	0.085683389	,	0.312484608	,],
[	0.342369003	,	0.421159837	,	0.311154037	,	0.098116558	,]],
[[	0.098559699	,	0.330379609	,	0.326113389	,	0.255210151	,],
[	0.309944048	,	0.099223722	,	0.24652338	,	0.319076621	,],
[	0.40642748	,	0.350160825	,	0.082294693	,	0.308056353	,],
[	0.342238472	,	0.42121918	,	0.311441402	,	0.09550489	,]],
[[	0.099482593	,	0.333700036	,	0.329246655	,	0.258459337	,],
[	0.3085888	,	0.093803362	,	0.244577315	,	0.317622685	,],
[	0.407931426	,	0.352106425	,	0.086255838	,	0.309655606	,],
[	0.340605272	,	0.419648604	,	0.309678344	,	0.09509134	,]],
[[	0.096619873	,	0.329876124	,	0.325491333	,	0.254617461	,],
[	0.307882777	,	0.095190724	,	0.244193837	,	0.317019895	,],
[	0.408001626	,	0.351756246	,	0.08334426	,	0.30979075	,],
[	0.344605081	,	0.423331693	,	0.313482791	,	0.101517779	,]],
[[	0.101575203	,	0.331960286	,	0.327648435	,	0.257013943	,],
[	0.312071168	,	0.096696598	,	0.247937714	,	0.321019091	,],
[	0.406372279	,	0.350319979	,	0.082985413	,	0.308023369	,],
[	0.340851747	,	0.419816612	,	0.309815007	,	0.094331976	,]],
[[	0.09721065	,	0.331222247	,	0.326837359	,	0.255985177	,],
[	0.30788334	,	0.093604336	,	0.24376924	,	0.316850482	,],
[	0.409939482	,	0.354329003	,	0.089527136	,	0.31166774	,],
[	0.341768961	,	0.420625284	,	0.310662427	,	0.097063729	,]],
[[	0.097557256	,	0.338765541	,	0.333758266	,	0.255184339	,],
[	0.29510477	,	0.096311194	,	0.244025185	,	0.308171833	,],
[	0.395072957	,	0.351441463	,	0.082316606	,	0.300184703	,],
[	0.336893302	,	0.427038844	,	0.317197435	,	0.095461654	,]],
[[	0.09723117	,	0.341095009	,	0.335962292	,	0.25734375	,],
[	0.297432731	,	0.095559433	,	0.245803146	,	0.310356718	,],
[	0.394792824	,	0.351352026	,	0.083832975	,	0.299783142	,],
[	0.334869182	,	0.425064874	,	0.315009597	,	0.094442708	,]],
[[	0.09533402	,	0.338004433	,	0.332807458	,	0.254239152	,],
[	0.294334983	,	0.093989972	,	0.242960599	,	0.307326938	,],
[	0.398150694	,	0.354491649	,	0.085028806	,	0.303475751	,],
[	0.338573759	,	0.428652368	,	0.31859321	,	0.100643991	,]],
[[	0.10111482	,	0.341038535	,	0.335961704	,	0.257466627	,],
[	0.296817312	,	0.099450693	,	0.245872864	,	0.309729809	,],
[	0.394746103	,	0.351084739	,	0.082398902	,	0.299682146	,],
[	0.335065203	,	0.425258074	,	0.315229333	,	0.093915714	,]],
[[	0.095332895	,	0.338994152	,	0.333898571	,	0.255091408	,],
[	0.295453654	,	0.093851584	,	0.243728357	,	0.308218437	,],
[	0.396639941	,	0.353356542	,	0.086734676	,	0.30156933	,],
[	0.33571962	,	0.425882671	,	0.315721549	,	0.096238615	,]],
[[	0.096807978	,	0.338650667	,	0.333429947	,	0.254730984	,],
[	0.294500296	,	0.095232219	,	0.243226744	,	0.307397537	,],
[	0.396056843	,	0.352368665	,	0.08302509	,	0.301212163	,],
[	0.338084268	,	0.428121446	,	0.318320474	,	0.09644045	,]],
[[	0.098464224	,	0.34232922	,	0.337099175	,	0.258605803	,],
[	0.298675347	,	0.096661799	,	0.246927284	,	0.311409913	,],
[	0.394699191	,	0.351217372	,	0.08317905	,	0.29967819	,],
[	0.334711908	,	0.4250066	,	0.314975412	,	0.093987179	,]],
[[	0.095010158	,	0.338049358	,	0.332920328	,	0.254123795	,],
[	0.294397284	,	0.093600563	,	0.242913581	,	0.307341299	,],
[	0.398717315	,	0.355658772	,	0.090262116	,	0.303647099	,],
[	0.337667698	,	0.427649078	,	0.317556416	,	0.099111554	,]],
[[	0.099450113	,	0.339993075	,	0.334914083	,	0.256358252	,],
[	0.295860743	,	0.097984332	,	0.244832005	,	0.308821598	,],
[	0.394799432	,	0.351075842	,	0.082059755	,	0.299773145	,],
[	0.336154991	,	0.42622948	,	0.316185652	,	0.094559617	,]],
[[	0.096298516	,	0.339949021	,	0.334854329	,	0.256163316	,],
[	0.296300916	,	0.09465094	,	0.244610584	,	0.309058466	,],
[	0.395690426	,	0.352333784	,	0.085321941	,	0.300563708	,],
[	0.335404275	,	0.425593965	,	0.315386023	,	0.095348866	,]],
[[	0.097401335	,	0.352841546	,	0.345347945	,	0.256922118	,],
[	0.283948183	,	0.096023406	,	0.244275593	,	0.299070393	,],
[	0.390511821	,	0.360069795	,	0.084416001	,	0.296532958	,],
[	0.335423813	,	0.439328783	,	0.326904513	,	0.095720149	,]],
[[	0.097465997	,	0.355198573	,	0.347520612	,	0.259292782	,],
[	0.28781656	,	0.103036191	,	0.248615698	,	0.302942181	,],
[	0.388670981	,	0.358519901	,	0.084123952	,	0.294580546	,],
[	0.332613188	,	0.436844547	,	0.324104991	,	0.094037172	,]],
[[	0.095402637	,	0.352040275	,	0.344423311	,	0.255999293	,],
[	0.284345958	,	0.095044304	,	0.244413717	,	0.299501618	,],
[	0.392376815	,	0.362634091	,	0.090827507	,	0.298356478	,],
[	0.336021366	,	0.440025056	,	0.327363444	,	0.099722923	,]],
[[	0.100680219	,	0.354665679	,	0.347090339	,	0.258809037	,],
[	0.284988512	,	0.09833421	,	0.245544553	,	0.300192887	,],
[	0.389096593	,	0.358822207	,	0.083393726	,	0.295003381	,],
[	0.333571705	,	0.437582806	,	0.324940113	,	0.094088628	,]],
[[	0.095583684	,	0.353152956	,	0.345598289	,	0.257066851	,],
[	0.286889185	,	0.09660783	,	0.246586702	,	0.301763959	,],
[	0.390069394	,	0.360074954	,	0.086488843	,	0.295849634	,],
[	0.333674041	,	0.437680464	,	0.324796893	,	0.095686157	,]],
[[	0.096662884	,	0.352512773	,	0.344969712	,	0.256510989	,],
[	0.283998301	,	0.095558211	,	0.244096995	,	0.299079317	,],
[	0.391732812	,	0.361222699	,	0.085402671	,	0.297709786	,],
[	0.336513034	,	0.440484478	,	0.328070851	,	0.096794698	,]],
[[	0.098867225	,	0.356520474	,	0.348822286	,	0.260557514	,],
[	0.287071975	,	0.101723005	,	0.247678427	,	0.302119317	,],
[	0.3888057	,	0.358518559	,	0.083693219	,	0.294458917	,],
[	0.332879882	,	0.437020399	,	0.324304549	,	0.093921556	,]],
[[	0.095115432	,	0.352167502	,	0.344480506	,	0.256056875	,],
[	0.284690462	,	0.095119453	,	0.244729266	,	0.299790562	,],
[	0.391576887	,	0.36183055	,	0.089385309	,	0.297442843	,],
[	0.335334391	,	0.439382577	,	0.326724352	,	0.098452406	,]],
[[	0.099368494	,	0.353914069	,	0.346352664	,	0.257934859	,],
[	0.284619831	,	0.097346864	,	0.24497349	,	0.299762901	,],
[	0.389715686	,	0.359283511	,	0.083796644	,	0.295596081	,],
[	0.334287055	,	0.438250569	,	0.325695937	,	0.094606334	,]],
[[	0.096555917	,	0.354169775	,	0.346502347	,	0.258038903	,],
[	0.288022547	,	0.097733076	,	0.247737367	,	0.302917936	,],
[	0.389522744	,	0.359395328	,	0.085445695	,	0.295262512	,],
[	0.333196146	,	0.437336173	,	0.324468069	,	0.094934143	,]],
[[	0.097031876	,	0.363836216	,	0.35259702	,	0.257461727	,],
[	0.271528716	,	0.095765651	,	0.242501957	,	0.289520827	,],
[	0.383247	,	0.366107535	,	0.08592766	,	0.291393135	,],
[	0.332042316	,	0.447954891	,	0.332991039	,	0.095682258	,]],
[[	0.096744558	,	0.365836302	,	0.354526083	,	0.259482293	,],
[	0.27345665	,	0.100518644	,	0.244966962	,	0.291489441	,],
[	0.379823775	,	0.362875199	,	0.083504689	,	0.287798501	,],
[	0.328822036	,	0.445153209	,	0.32983578	,	0.093548851	,]],
[[	0.094734362	,	0.362741469	,	0.351399047	,	0.256320676	,],
[	0.27305688	,	0.096146213	,	0.243820299	,	0.290983801	,],
[	0.381873133	,	0.365508701	,	0.088290334	,	0.289832378	,],
[	0.332070741	,	0.448262193	,	0.333031351	,	0.099059293	,]],
[[	0.100397294	,	0.365608709	,	0.354397262	,	0.259390011	,],
[	0.271629652	,	0.096823739	,	0.242722612	,	0.289612419	,],
[	0.381083402	,	0.364138084	,	0.084004433	,	0.289252893	,],
[	0.329906279	,	0.445976775	,	0.33075566	,	0.093735078	,]],
[[	0.09467011	,	0.363619755	,	0.352411192	,	0.257194964	,],
[	0.275374613	,	0.104007995	,	0.247142111	,	0.29336001	,],
[	0.380049905	,	0.363335953	,	0.084591438	,	0.287966823	,],
[	0.329615774	,	0.445806582	,	0.330463906	,	0.095051487	,]],
[[	0.09615588	,	0.363338812	,	0.352176335	,	0.256837257	,],
[	0.271864265	,	0.095534784	,	0.242687649	,	0.289749834	,],
[	0.384127965	,	0.36801314	,	0.092040185	,	0.292225524	,],
[	0.332975888	,	0.44881682	,	0.333882591	,	0.096502119	,]],
[[	0.097736809	,	0.366837906	,	0.355480808	,	0.260545225	,],
[	0.272930298	,	0.099445364	,	0.244224172	,	0.2907921	,],
[	0.379883594	,	0.362903201	,	0.083267631	,	0.287818553	,],
[	0.329272518	,	0.445459322	,	0.330175743	,	0.093473838	,]],
[[	0.094509369	,	0.362901997	,	0.35152751	,	0.256371549	,],
[	0.2740619	,	0.096871872	,	0.2446691	,	0.29181854	,],
[	0.381185175	,	0.364703963	,	0.086887599	,	0.289098896	,],
[	0.331103554	,	0.447167374	,	0.332002213	,	0.097457055	,]],
[[	0.098935037	,	0.364809556	,	0.353520511	,	0.258488306	,],
[	0.271500053	,	0.096189506	,	0.242525639	,	0.289420323	,],
[	0.381906242	,	0.364808465	,	0.084597222	,	0.289951774	,],
[	0.330512686	,	0.446535155	,	0.331436157	,	0.094282366	,]],
[[	0.095560491	,	0.364603006	,	0.353294256	,	0.258066132	,],
[	0.27485725	,	0.102752232	,	0.24644738	,	0.292706354	,],
[	0.379644474	,	0.362856556	,	0.083886765	,	0.287583381	,],
[	0.329359961	,	0.445509918	,	0.330142195	,	0.094287795	,]],
[[	0.095774048	,	0.372862905	,	0.358909054	,	0.25712177	,],
[	0.260510145	,	0.095561688	,	0.24117486	,	0.280462998	,],
[	0.374286554	,	0.370037098	,	0.090409567	,	0.28409723	,],
[	0.329186034	,	0.456013392	,	0.339741907	,	0.095483016	,]],
[[	0.09595873	,	0.375177453	,	0.3611717	,	0.259523462	,],
[	0.260294464	,	0.097742224	,	0.241456492	,	0.28034079	,],
[	0.371530861	,	0.366504178	,	0.083417873	,	0.281220894	,],
[	0.325314009	,	0.452715812	,	0.336008165	,	0.092506179	,]],
[[	0.093617272	,	0.371818655	,	0.357772522	,	0.256017753	,],
[	0.263894123	,	0.098065777	,	0.244355643	,	0.283741612	,],
[	0.37163503	,	0.367076603	,	0.085779956	,	0.281304458	,],
[	0.327732526	,	0.454976002	,	0.338399045	,	0.097104102	,]],
[[	0.099125634	,	0.374767298	,	0.360708422	,	0.25909152	,],
[	0.25980141	,	0.095614234	,	0.240634516	,	0.279796157	,],
[	0.374145734	,	0.368835656	,	0.085433507	,	0.283913086	,],
[	0.327101313	,	0.454051212	,	0.337524808	,	0.093544108	,]],
[[	0.093852105	,	0.372962909	,	0.358919438	,	0.257162188	,],
[	0.261516142	,	0.100475606	,	0.242987669	,	0.281608652	,],
[	0.370981365	,	0.366025439	,	0.083635948	,	0.280585924	,],
[	0.32599522	,	0.453237687	,	0.33649699	,	0.093840812	,]],
[[	0.094965268	,	0.372444919	,	0.358437578	,	0.25658565	,],
[	0.261331111	,	0.096094625	,	0.241855137	,	0.281249232	,],
[	0.37340497	,	0.368974166	,	0.088787679	,	0.283082626	,],
[	0.33035081	,	0.457146859	,	0.340923415	,	0.096568547	,]],
[[	0.097335553	,	0.376669508	,	0.362543224	,	0.26097822	,],
[	0.259965087	,	0.096851683	,	0.240953227	,	0.279899578	,],
[	0.3718686	,	0.366705004	,	0.083481677	,	0.28151332	,],
[	0.325610346	,	0.452867286	,	0.336308179	,	0.092483782	,]],
[[	0.093510642	,	0.372188622	,	0.358017713	,	0.256310673	,],
[	0.26423119	,	0.104709503	,	0.245921599	,	0.284205762	,],
[	0.37119526	,	0.366587082	,	0.08479679	,	0.280831088	,],
[	0.327069939	,	0.454339116	,	0.337645287	,	0.095842653	,]],
[[	0.097717062	,	0.373888941	,	0.359846502	,	0.258115874	,],
[	0.260050398	,	0.095534977	,	0.24079522	,	0.280031753	,],
[	0.37504152	,	0.369783117	,	0.08631459	,	0.284770126	,],
[	0.327768421	,	0.454667266	,	0.338197094	,	0.094066082	,]],
[[	0.094993279	,	0.374197288	,	0.360183434	,	0.258419402	,],
[	0.260919215	,	0.099299233	,	0.242263352	,	0.280993349	,],
[	0.371126293	,	0.366091315	,	0.083247502	,	0.280621232	,],
[	0.325875965	,	0.453014709	,	0.336385748	,	0.093222482	,]],
[[	0.102823043	,	0.229602523	,	0.2600888	,	0.274534096	,],
[	0.39982535	,	0.084132425	,	0.255146469	,	0.407798349	,],
[	0.476777728	,	0.311265166	,	0.081086128	,	0.379818867	,],
[	0.366906156	,	0.332870974	,	0.247158329	,	0.097940746	,]],
[[	0.098378036	,	0.223886193	,	0.25454362	,	0.269188738	,],
[	0.403520029	,	0.085861215	,	0.259168857	,	0.411598887	,],
[	0.473730256	,	0.307741464	,	0.074013598	,	0.376609891	,],
[	0.364198858	,	0.329766782	,	0.243889356	,	0.096244107	,]],
[[	0.101865433	,	0.224516356	,	0.25510109	,	0.27024782	,],
[	0.399640545	,	0.082724619	,	0.255051966	,	0.407752085	,],
[	0.47421181	,	0.308330991	,	0.07648719	,	0.37697304	,],
[	0.36742207	,	0.332518879	,	0.246599718	,	0.101850278	,]],
[[	0.099980512	,	0.226828094	,	0.257284047	,	0.271685146	,],
[	0.401228902	,	0.086798843	,	0.256645825	,	0.409283177	,],
[	0.47594609	,	0.310294233	,	0.075936197	,	0.379007215	,],
[	0.364986176	,	0.330747555	,	0.244971113	,	0.09629766	,]],
[[	0.098681814	,	0.223339024	,	0.254081122	,	0.268777264	,],
[	0.401178697	,	0.083646105	,	0.256550951	,	0.409136081	,],
[	0.473361591	,	0.307347398	,	0.074183104	,	0.376113392	,],
[	0.364964308	,	0.330454646	,	0.244484351	,	0.097904629	,]],
[[	0.104407061	,	0.231370555	,	0.261763968	,	0.276140149	,],
[	0.399691366	,	0.083554033	,	0.254876702	,	0.407632479	,],
[	0.476204268	,	0.310418546	,	0.079813401	,	0.379115633	,],
[	0.367902066	,	0.333787055	,	0.248235914	,	0.098844308	,]],
[[	0.098856273	,	0.22458602	,	0.255139432	,	0.269855088	,],
[	0.403354393	,	0.090405884	,	0.258965591	,	0.411263637	,],
[	0.473893567	,	0.307942075	,	0.07411311	,	0.37675178	,],
[	0.364303326	,	0.329882399	,	0.243966768	,	0.09598729	,]],
[[	0.101037743	,	0.224233677	,	0.254777907	,	0.269848885	,],
[	0.399676151	,	0.082454965	,	0.254894456	,	0.407718423	,],
[	0.473921899	,	0.307783907	,	0.075534775	,	0.376431708	,],
[	0.366451479	,	0.331608408	,	0.245637833	,	0.100405285	,]],
[[	0.101113179	,	0.227842879	,	0.25836638	,	0.272731892	,],
[	0.400604018	,	0.085719585	,	0.256096521	,	0.408694216	,],
[	0.476725718	,	0.311033798	,	0.076690407	,	0.379812213	,],
[	0.365769911	,	0.331640048	,	0.245922345	,	0.096818913	,]],
[[	0.098687928	,	0.223711795	,	0.254301146	,	0.269010597	,],
[	0.402332045	,	0.084549096	,	0.257639038	,	0.41010537	,],
[	0.473521167	,	0.307483665	,	0.074021944	,	0.376275624	,],
[	0.364698346	,	0.330159699	,	0.244238485	,	0.097171505	,]],
[[	0.100426109	,	0.231849	,	0.25998181	,	0.265054064	,],
[	0.380889042	,	0.081933252	,	0.250639559	,	0.390696273	,],
[	0.451618499	,	0.301327559	,	0.072237635	,	0.359247932	,],
[	0.34974329	,	0.329429749	,	0.24603653	,	0.094642671	,]],
[[	0.094647948	,	0.230353004	,	0.258374113	,	0.262970829	,],
[	0.37865475	,	0.081075998	,	0.248226779	,	0.388512613	,],
[	0.453598177	,	0.303498316	,	0.07645993	,	0.361252306	,],
[	0.351944466	,	0.331977566	,	0.248854282	,	0.094585301	,]],
[[	0.096232696	,	0.229428801	,	0.257500089	,	0.262398083	,],
[	0.381941087	,	0.087330153	,	0.251800063	,	0.391684912	,],
[	0.453345648	,	0.30312317	,	0.073482368	,	0.361104249	,],
[	0.348930703	,	0.328551907	,	0.245192121	,	0.092454966	,]],
[[	0.097320352	,	0.234005356	,	0.261888551	,	0.266221504	,],
[	0.379532902	,	0.08092921	,	0.24917568	,	0.389291404	,],
[	0.451746079	,	0.301393451	,	0.072952442	,	0.359268686	,],
[	0.351633583	,	0.330975441	,	0.247558637	,	0.097623298	,]],
[[	0.094218704	,	0.229172428	,	0.257259124	,	0.261826066	,],
[	0.379818779	,	0.08331883	,	0.249467921	,	0.389560427	,],
[	0.455495184	,	0.3054712	,	0.07973731	,	0.363016552	,],
[	0.349820267	,	0.329968804	,	0.246608622	,	0.092710447	,]],
[[	0.098533586	,	0.230725186	,	0.258806739	,	0.263775925	,],
[	0.382019456	,	0.08286912	,	0.251683343	,	0.391616073	,],
[	0.452194404	,	0.301877291	,	0.072415373	,	0.359767938	,],
[	0.349247789	,	0.328940719	,	0.245548542	,	0.093712328	,]],
[[	0.09565332	,	0.231438925	,	0.259359693	,	0.263870502	,],
[	0.378861182	,	0.080950686	,	0.248479769	,	0.388632747	,],
[	0.452796724	,	0.302616524	,	0.075159235	,	0.36026959	,],
[	0.353041572	,	0.333229154	,	0.249964224	,	0.095522463	,]],
[[	0.0952263	,	0.229032566	,	0.256996805	,	0.261797134	,],
[	0.380893263	,	0.085878932	,	0.250677129	,	0.390581034	,],
[	0.454297711	,	0.304094188	,	0.074406611	,	0.362025461	,],
[	0.348661359	,	0.328512634	,	0.245114577	,	0.092053281	,]],
[[	0.098564558	,	0.235088896	,	0.263022593	,	0.267210249	,],
[	0.379975273	,	0.081316519	,	0.249671823	,	0.389768916	,],
[	0.451814464	,	0.301291198	,	0.072686006	,	0.3591846	,],
[	0.350718963	,	0.330032893	,	0.246594572	,	0.096019014	,]],
[[	0.094236744	,	0.229591868	,	0.25766352	,	0.262066158	,],
[	0.378791328	,	0.081811278	,	0.248327299	,	0.388526609	,],
[	0.454815084	,	0.304685142	,	0.078384674	,	0.362259257	,],
[	0.350764371	,	0.330704489	,	0.247480304	,	0.093385937	,]],
[[	0.096326638	,	0.238316434	,	0.263122062	,	0.258475327	,],
[	0.364542163	,	0.084052067	,	0.245825975	,	0.375553596	,],
[	0.441353312	,	0.304483773	,	0.075312847	,	0.35125511	,],
[	0.340426471	,	0.332923607	,	0.249213923	,	0.090865569	,]],
[[	0.092872217	,	0.238281467	,	0.263166923	,	0.258067055	,],
[	0.36478202	,	0.081055824	,	0.245917692	,	0.37584154	,],
[	0.43767009	,	0.300554342	,	0.071919946	,	0.347193188	,],
[	0.342887854	,	0.33504824	,	0.251243839	,	0.095619699	,]],
[[	0.092816987	,	0.236327247	,	0.26131672	,	0.256498367	,],
[	0.363012789	,	0.080736472	,	0.244130982	,	0.373889812	,],
[	0.439288347	,	0.302363096	,	0.075776937	,	0.348684126	,],
[	0.34145348	,	0.334036352	,	0.250473174	,	0.09108325	,]],
[[	0.096159249	,	0.242408853	,	0.2671779	,	0.261882907	,],
[	0.366562351	,	0.087487229	,	0.247971015	,	0.377521421	,],
[	0.439147542	,	0.302141859	,	0.073049138	,	0.348807197	,],
[	0.340876753	,	0.333234612	,	0.249348223	,	0.092021972	,]],
[[	0.091558186	,	0.236775371	,	0.261673651	,	0.256524174	,],
[	0.363485754	,	0.080125325	,	0.24454001	,	0.374444156	,],
[	0.437399917	,	0.300254862	,	0.072367721	,	0.346767098	,],
[	0.345027375	,	0.337790001	,	0.254309387	,	0.094269822	,]],
[[	0.095012279	,	0.237462184	,	0.262369712	,	0.257652992	,],
[	0.363956482	,	0.082787271	,	0.245185417	,	0.374959119	,],
[	0.441049918	,	0.304203069	,	0.079030494	,	0.350489717	,],
[	0.340521324	,	0.33308341	,	0.249341275	,	0.090604756	,]],
[[	0.093688981	,	0.239481903	,	0.264297738	,	0.259098247	,],
[	0.365727507	,	0.08173906	,	0.246849687	,	0.376622215	,],
[	0.43775299	,	0.300530511	,	0.071744058	,	0.347146846	,],
[	0.342123786	,	0.334367272	,	0.250537859	,	0.094279577	,]],
[[	0.092376092	,	0.236406441	,	0.261185186	,	0.256390295	,],
[	0.362773512	,	0.080165308	,	0.243749931	,	0.37369887	,],
[	0.438486799	,	0.301381378	,	0.07436669	,	0.347757805	,],
[	0.342591141	,	0.335218361	,	0.251537873	,	0.09195809	,]],
[[	0.097330635	,	0.243658002	,	0.268514467	,	0.263143672	,],
[	0.365491678	,	0.085887326	,	0.246844666	,	0.376444013	,],
[	0.440056093	,	0.303008433	,	0.073705859	,	0.349604393	,],
[	0.340669953	,	0.33299956	,	0.249171763	,	0.0913621	,]],
[[	0.092035366	,	0.237387736	,	0.262212546	,	0.257085812	,],
[	0.363838591	,	0.080126953	,	0.244762043	,	0.374629277	,],
[	0.437646254	,	0.300451976	,	0.072279134	,	0.346895444	,],
[	0.344573481	,	0.336641038	,	0.252690888	,	0.098032738	,]],
[[	0.094403875	,	0.245617205	,	0.269271791	,	0.255415599	,],
[	0.349138382	,	0.080732805	,	0.241803936	,	0.362237367	,],
[	0.426386918	,	0.301345972	,	0.076268029	,	0.338211477	,],
[	0.331917497	,	0.336517458	,	0.253263235	,	0.088562577	,]],
[[	0.091105729	,	0.245975299	,	0.269575675	,	0.2552606	,],
[	0.352929243	,	0.081919392	,	0.245486795	,	0.365921581	,],
[	0.425093359	,	0.299751997	,	0.071458787	,	0.337066996	,],
[	0.332947599	,	0.337221259	,	0.253905925	,	0.091594931	,]],
[[	0.090897869	,	0.243611433	,	0.267181773	,	0.253232531	,],
[	0.348665256	,	0.078637149	,	0.241083052	,	0.361687157	,],
[	0.424337687	,	0.299128869	,	0.072408821	,	0.336186151	,],
[	0.3342332	,	0.338866736	,	0.255662768	,	0.090255118	,]],
[[	0.094517678	,	0.250022835	,	0.273570999	,	0.259097059	,],
[	0.350298936	,	0.08307452	,	0.242944657	,	0.363306596	,],
[	0.428152918	,	0.30284177	,	0.074342918	,	0.340135765	,],
[	0.331670768	,	0.33600314	,	0.252663007	,	0.088951784	,]],
[[	0.089829999	,	0.244183379	,	0.267937646	,	0.25355673	,],
[	0.350520416	,	0.079764482	,	0.242915591	,	0.363453743	,],
[	0.423888903	,	0.29842041	,	0.070789507	,	0.335665601	,],
[	0.335110702	,	0.339174541	,	0.255719143	,	0.094850622	,]],
[[	0.09312472	,	0.245008784	,	0.268684059	,	0.254648757	,],
[	0.348851464	,	0.079833854	,	0.241307262	,	0.361785986	,],
[	0.426035103	,	0.300750914	,	0.075159112	,	0.337691601	,],
[	0.332240713	,	0.336896764	,	0.253556775	,	0.088629134	,]],
[[	0.092104705	,	0.247092676	,	0.27063008	,	0.256295959	,],
[	0.352384182	,	0.086657465	,	0.245332771	,	0.365303931	,],
[	0.425853005	,	0.300554385	,	0.072080134	,	0.337861878	,],
[	0.332532174	,	0.336863386	,	0.25343486	,	0.090610968	,]],
[[	0.090451265	,	0.243731873	,	0.267243741	,	0.253203811	,],
[	0.349253965	,	0.079015626	,	0.24159815	,	0.362152687	,],
[	0.424414871	,	0.299017256	,	0.072012809	,	0.336148998	,],
[	0.335054235	,	0.339680947	,	0.256572034	,	0.091041653	,]],
[[	0.096077102	,	0.251557142	,	0.27498146	,	0.260519163	,],
[	0.349780935	,	0.082043742	,	0.242338241	,	0.362686439	,],
[	0.42785467	,	0.302942454	,	0.078470608	,	0.339733501	,],
[	0.331884587	,	0.336290689	,	0.252906859	,	0.088687194	,]],
[[	0.090405859	,	0.244892794	,	0.268476137	,	0.254269328	,],
[	0.351433827	,	0.080534842	,	0.243771181	,	0.364260067	,],
[	0.424578297	,	0.299071326	,	0.071143884	,	0.336281963	,],
[	0.334471838	,	0.338506562	,	0.255037504	,	0.093640092	,]],
[[	0.092049501	,	0.254974748	,	0.277327673	,	0.253521332	,],
[	0.337425448	,	0.079182131	,	0.240768444	,	0.352645056	,],
[	0.415320866	,	0.301807878	,	0.073735291	,	0.328393753	,],
[	0.32812744	,	0.344580321	,	0.260718428	,	0.08808093	,]],
[[	0.090117474	,	0.256264803	,	0.278658422	,	0.254465541	,],
[	0.340046403	,	0.084674079	,	0.24373515	,	0.355226385	,],
[	0.41749844	,	0.303935966	,	0.073658917	,	0.330903276	,],
[	0.32785601	,	0.344194892	,	0.260250556	,	0.089566677	,]],
[[	0.089149173	,	0.253487002	,	0.276009023	,	0.252007641	,],
[	0.338936516	,	0.079684725	,	0.242133487	,	0.353938576	,],
[	0.414608461	,	0.300952258	,	0.071623399	,	0.327708702	,],
[	0.331158877	,	0.347521687	,	0.263889004	,	0.09068044	,]],
[[	0.09421933	,	0.260907051	,	0.283280197	,	0.258994598	,],
[	0.338106473	,	0.080795176	,	0.241404575	,	0.353253604	,],
[	0.417040008	,	0.303796198	,	0.076922861	,	0.330134249	,],
[	0.32750158	,	0.343668534	,	0.259797591	,	0.087816892	,]],
[[	0.088852682	,	0.254784719	,	0.277150559	,	0.252860688	,],
[	0.342229804	,	0.082421838	,	0.245552902	,	0.357258655	,],
[	0.415367808	,	0.301743979	,	0.071853487	,	0.328513779	,],
[	0.329687237	,	0.345672003	,	0.261666636	,	0.092258079	,]],
[[	0.091302366	,	0.254708562	,	0.277130208	,	0.253116835	,],
[	0.337603455	,	0.078966602	,	0.240827551	,	0.352708459	,],
[	0.414969394	,	0.301189842	,	0.072796967	,	0.327811158	,],
[	0.328994297	,	0.345439409	,	0.261685315	,	0.088632505	,]],
[[	0.091331317	,	0.257602971	,	0.279902799	,	0.255723206	,],
[	0.339394299	,	0.083444941	,	0.242941097	,	0.35455905	,],
[	0.41883339	,	0.305355317	,	0.075139147	,	0.332244426	,],
[	0.327629441	,	0.343747999	,	0.259857242	,	0.088647603	,]],
[[	0.088782914	,	0.25363494	,	0.275998186	,	0.251940975	,],
[	0.339450291	,	0.080035123	,	0.242678744	,	0.354545642	,],
[	0.414840963	,	0.30113405	,	0.071522168	,	0.327882865	,],
[	0.33202607	,	0.347897952	,	0.26402725	,	0.0957333	,]],
[[	0.094381449	,	0.256249195	,	0.278543283	,	0.254827448	,],
[	0.337921292	,	0.080282101	,	0.241111782	,	0.352984261	,],
[	0.416500447	,	0.303072309	,	0.075549854	,	0.329432613	,],
[	0.327576995	,	0.343900686	,	0.259968471	,	0.087606754	,]],
[[	0.089179012	,	0.255095195	,	0.277428831	,	0.253173116	,],
[	0.341668961	,	0.087277868	,	0.245311671	,	0.356688354	,],
[	0.416593881	,	0.302903329	,	0.072778967	,	0.329754682	,],
[	0.3291988	,	0.345193346	,	0.261201595	,	0.091211886	,]],
[[	0.091395354	,	0.266107221	,	0.286823106	,	0.25445321	,],
[	0.330207699	,	0.080765665	,	0.242614162	,	0.347115685	,],
[	0.406648789	,	0.303528629	,	0.071103465	,	0.321486949	,],
[	0.327467866	,	0.356273092	,	0.271026171	,	0.090043522	,]],
[[	0.090853196	,	0.268413992	,	0.288992197	,	0.256459766	,],
[	0.329005682	,	0.081533845	,	0.24151961	,	0.345919922	,],
[	0.408961872	,	0.306250271	,	0.075985088	,	0.323810446	,],
[	0.324438346	,	0.353190043	,	0.267585179	,	0.087919458	,]],
[[	0.088903799	,	0.264871533	,	0.285531092	,	0.253129594	,],
[	0.333018809	,	0.08868993	,	0.245866701	,	0.349739981	,],
[	0.407923459	,	0.304898344	,	0.071652979	,	0.322915274	,],
[	0.327051076	,	0.355583294	,	0.269879084	,	0.093031946	,]],
[[	0.094650969	,	0.267711174	,	0.288278154	,	0.256222947	,],
[	0.329233794	,	0.080380169	,	0.241537449	,	0.34614251	,],
[	0.407006536	,	0.304011987	,	0.072336169	,	0.321742576	,],
[	0.325516803	,	0.354223361	,	0.268609746	,	0.08813589	,]],
[[	0.088922012	,	0.266240739	,	0.286969307	,	0.254311875	,],
[	0.33024073	,	0.083853975	,	0.242716867	,	0.347033901	,],
[	0.411144671	,	0.308762056	,	0.079700224	,	0.325966312	,],
[	0.324874303	,	0.353401966	,	0.267624787	,	0.08910651	,]],
[[	0.090356736	,	0.265402555	,	0.286205698	,	0.253629153	,],
[	0.331395437	,	0.081665365	,	0.243714063	,	0.348185377	,],
[	0.407052083	,	0.304002311	,	0.071264277	,	0.321831924	,],
[	0.328363928	,	0.357263441	,	0.271919304	,	0.090894795	,]],
[[	0.091935433	,	0.269581733	,	0.290153075	,	0.257692794	,],
[	0.329051628	,	0.080942545	,	0.241421042	,	0.345909548	,],
[	0.408327423	,	0.305497316	,	0.074789588	,	0.322991784	,],
[	0.324270677	,	0.353092926	,	0.267463658	,	0.087398652	,]],
[[	0.0888	,	0.265241189	,	0.285768904	,	0.253375221	,],
[	0.332234405	,	0.087370737	,	0.244962747	,	0.348880737	,],
[	0.408910161	,	0.305818491	,	0.072576144	,	0.323867101	,],
[	0.32628602	,	0.354827665	,	0.26918669	,	0.091579424	,]],
[[	0.093270142	,	0.266945933	,	0.287390284	,	0.255319253	,],
[	0.329698913	,	0.08046019	,	0.241868259	,	0.346393545	,],
[	0.40687249	,	0.303763299	,	0.071667498	,	0.321518031	,],
[	0.326453927	,	0.355128208	,	0.269657732	,	0.088960133	,]],
[[	0.08976669	,	0.267209482	,	0.287875004	,	0.255215963	,],
[	0.329874853	,	0.082969645	,	0.242338318	,	0.346548124	,],
[	0.4103533	,	0.307768755	,	0.078331432	,	0.325086409	,],
[	0.324685872	,	0.35327842	,	0.267550663	,	0.088507658	,]],
[[	0.089900496	,	0.275515904	,	0.294196382	,	0.253375516	,],
[	0.321570042	,	0.082943719	,	0.244884487	,	0.340678867	,],
[	0.397860774	,	0.306246071	,	0.071627459	,	0.31444736	,],
[	0.324846002	,	0.364842212	,	0.278690013	,	0.091372533	,]],
[[	0.089700461	,	0.278068789	,	0.296637214	,	0.255620377	,],
[	0.317316615	,	0.079933176	,	0.240758096	,	0.336580041	,],
[	0.397648689	,	0.306145004	,	0.073158828	,	0.313968618	,],
[	0.320291762	,	0.360220816	,	0.273809591	,	0.087526853	,]],
[[	0.087510037	,	0.274406539	,	0.293006528	,	0.252166741	,],
[	0.319258395	,	0.084738677	,	0.243026376	,	0.338338321	,],
[	0.400753011	,	0.309263142	,	0.074275912	,	0.317530415	,],
[	0.32175196	,	0.361510892	,	0.275003361	,	0.090963122	,]],
[[	0.093163916	,	0.277126801	,	0.295646104	,	0.25517783	,],
[	0.319262827	,	0.080782659	,	0.242516221	,	0.338191512	,],
[	0.397046386	,	0.305435908	,	0.071248272	,	0.313391301	,],
[	0.322150242	,	0.361994854	,	0.27567401	,	0.088663227	,]],
[[	0.087622493	,	0.275783862	,	0.294438967	,	0.253321766	,],
[	0.317726501	,	0.081195648	,	0.241122733	,	0.336876153	,],
[	0.399346719	,	0.308039403	,	0.076063822	,	0.315578231	,],
[	0.320161797	,	0.359969151	,	0.273374426	,	0.088004674	,]],
[[	0.089038278	,	0.275050885	,	0.293689729	,	0.252760225	,],
[	0.321451744	,	0.088348348	,	0.245274257	,	0.340374773	,],
[	0.398596311	,	0.306966559	,	0.072087397	,	0.315103309	,],
[	0.324303526	,	0.363770145	,	0.277215531	,	0.094540224	,]],
[[	0.090685284	,	0.279157091	,	0.297716308	,	0.256718627	,],
[	0.317742045	,	0.079992961	,	0.241049514	,	0.336869878	,],
[	0.397485027	,	0.305971311	,	0.072466012	,	0.313728104	,],
[	0.320612439	,	0.360511547	,	0.274137483	,	0.087502735	,]],
[[	0.087220268	,	0.27455981	,	0.293040622	,	0.252188165	,],
[	0.318651694	,	0.083536206	,	0.242261272	,	0.337746093	,],
[	0.401385742	,	0.310389312	,	0.079518417	,	0.317678854	,],
[	0.321159288	,	0.360918148	,	0.274469303	,	0.089865561	,]],
[[	0.091909779	,	0.276512981	,	0.29496715	,	0.254334083	,],
[	0.32019227	,	0.081609029	,	0.243325383	,	0.339092422	,],
[	0.397244357	,	0.305592509	,	0.071186973	,	0.313542127	,],
[	0.323204535	,	0.363003633	,	0.276741731	,	0.089638907	,]],
[[	0.088521606	,	0.276740243	,	0.295326272	,	0.254234246	,],
[	0.317608218	,	0.080669194	,	0.24088566	,	0.336601217	,],
[	0.398588823	,	0.307249496	,	0.07475097	,	0.314765937	,],
[	0.320297555	,	0.360092326	,	0.273618908	,	0.087901242	,]],
[[	0.08852267	,	0.285629347	,	0.302333252	,	0.252215001	,],
[	0.310399234	,	0.087702487	,	0.244701802	,	0.331041228	,],
[	0.391408436	,	0.311040364	,	0.073888137	,	0.308928611	,],
[	0.320326157	,	0.371981451	,	0.284440031	,	0.09376288	,]],
[[	0.089112896	,	0.288891135	,	0.305519468	,	0.255304787	,],
[	0.308229487	,	0.08118832	,	0.242022297	,	0.328968142	,],
[	0.388975459	,	0.30872164	,	0.072541323	,	0.306237201	,],
[	0.316469967	,	0.36846774	,	0.281023448	,	0.08622075	,]],
[[	0.086804589	,	0.285227825	,	0.301948905	,	0.251736192	,],
[	0.307831198	,	0.083280238	,	0.242054745	,	0.328692158	,],
[	0.392346416	,	0.312592133	,	0.078929035	,	0.309650117	,],
[	0.317233298	,	0.36913963	,	0.28148672	,	0.088903902	,]],
[[	0.091954441	,	0.287508313	,	0.304081739	,	0.25418541	,],
[	0.311036199	,	0.083380382	,	0.24466862	,	0.331518038	,],
[	0.3897869	,	0.309452242	,	0.072468665	,	0.307179285	,],
[	0.319021337	,	0.370757824	,	0.28351058	,	0.088346106	,]],
[[	0.087014703	,	0.286546384	,	0.303302472	,	0.25296219	,],
[	0.307514751	,	0.081216143	,	0.24131373	,	0.32810931	,],
[	0.389893082	,	0.309873069	,	0.074555368	,	0.307158177	,],
[	0.316196062	,	0.368041357	,	0.280411057	,	0.086513705	,]],
[[	0.087974484	,	0.28546868	,	0.302107303	,	0.251989722	,],
[	0.309644775	,	0.086244342	,	0.243763121	,	0.330234142	,],
[	0.392593608	,	0.312342849	,	0.074886077	,	0.310066776	,],
[	0.319459334	,	0.371065557	,	0.283484471	,	0.092198174	,]],
[[	0.090180362	,	0.289929472	,	0.30659055	,	0.256402802	,],
[	0.308869248	,	0.081429238	,	0.242595604	,	0.32949984	,],
[	0.389195792	,	0.308905002	,	0.07227768	,	0.306417441	,],
[	0.316767908	,	0.36872564	,	0.281369474	,	0.086400917	,]],
[[	0.086572617	,	0.285157855	,	0.301821531	,	0.251653536	,],
[	0.307928077	,	0.082577966	,	0.241916005	,	0.328495572	,],
[	0.391464646	,	0.311617458	,	0.077434869	,	0.308621695	,],
[	0.317003323	,	0.368801715	,	0.281226513	,	0.088224479	,]],
[[	0.09038821	,	0.286608005	,	0.303193481	,	0.253168211	,],
[	0.31188463	,	0.09026063	,	0.246490206	,	0.332464568	,],
[	0.39058439	,	0.310275723	,	0.073086159	,	0.308050226	,],
[	0.320269678	,	0.372035169	,	0.284746821	,	0.089448906	,]],
[[	0.087904989	,	0.287515147	,	0.304207032	,	0.253870522	,],
[	0.307923166	,	0.081201592	,	0.241540029	,	0.328417611	,],
[	0.389533185	,	0.309358422	,	0.073732231	,	0.306682573	,],
[	0.316421135	,	0.368248555	,	0.280586135	,	0.086513527	,]],
[[	0.087111377	,	0.295253653	,	0.310483681	,	0.251355798	,],
[	0.297375928	,	0.084053252	,	0.242132592	,	0.319997869	,],
[	0.383587785	,	0.316005074	,	0.07983719	,	0.302510364	,],
[	0.314829667	,	0.378412405	,	0.289922793	,	0.090833623	,]],
[[	0.089196746	,	0.299485759	,	0.314772376	,	0.255620298	,],
[	0.300220279	,	0.083851965	,	0.244692548	,	0.32285775	,],
[	0.379909333	,	0.311442875	,	0.071840781	,	0.298765549	,],
[	0.313987076	,	0.377850528	,	0.289647281	,	0.08715944	,]],
[[	0.085724196	,	0.294843601	,	0.310120846	,	0.250965094	,],
[	0.296654824	,	0.081830278	,	0.241373956	,	0.319370943	,],
[	0.380602226	,	0.312376291	,	0.07478812	,	0.299363029	,],
[	0.312703702	,	0.376610407	,	0.288088945	,	0.087479217	,]],
[[	0.090400625	,	0.296971769	,	0.312136971	,	0.253211	,],
[	0.298981283	,	0.086930685	,	0.243950401	,	0.321591744	,],
[	0.382444792	,	0.313873448	,	0.073734604	,	0.301393596	,],
[	0.317184423	,	0.380810427	,	0.292344859	,	0.094586391	,]],
[[	0.086571763	,	0.296817606	,	0.312175452	,	0.252799743	,],
[	0.298350746	,	0.082238416	,	0.242709864	,	0.320856388	,],
[	0.379720245	,	0.311261664	,	0.072284023	,	0.298418706	,],
[	0.313040037	,	0.376638521	,	0.288262666	,	0.086492111	,]],
[[	0.086718066	,	0.295307964	,	0.310542858	,	0.251374054	,],
[	0.297074982	,	0.083235609	,	0.241727265	,	0.319670023	,],
[	0.38249756	,	0.314638663	,	0.078105496	,	0.301355298	,],
[	0.31436457	,	0.377999746	,	0.289374561	,	0.08996037	,]],
[[	0.090465833	,	0.300732268	,	0.315906045	,	0.256783694	,],
[	0.301312876	,	0.090886101	,	0.246567914	,	0.323862531	,],
[	0.380559382	,	0.312062521	,	0.072368613	,	0.299415789	,],
[	0.314936602	,	0.378776147	,	0.290539288	,	0.087950536	,]],
[[	0.085921408	,	0.295573361	,	0.310796526	,	0.251537004	,],
[	0.297131101	,	0.081829368	,	0.241640281	,	0.319812742	,],
[	0.380742639	,	0.312413959	,	0.074167484	,	0.299407563	,],
[	0.312732644	,	0.376537221	,	0.288095259	,	0.086891965	,]],
[[	0.08915032	,	0.296314578	,	0.311446993	,	0.252491369	,],
[	0.298405588	,	0.085813328	,	0.243208609	,	0.320981996	,],
[	0.383396255	,	0.314957877	,	0.07485638	,	0.30252948	,],
[	0.316382879	,	0.380023089	,	0.291592746	,	0.093117719	,]],
[[	0.087663177	,	0.298012838	,	0.313354716	,	0.253970545	,],
[	0.298991235	,	0.082745393	,	0.243349278	,	0.321513027	,],
[	0.37954876	,	0.311024145	,	0.071819468	,	0.298238634	,],
[	0.313461435	,	0.377224166	,	0.28870567	,	0.086754431	,]],
[[	0.087559427	,	0.306162411	,	0.319761912	,	0.25192512	,],
[	0.285688334	,	0.083656467	,	0.241161484	,	0.310643393	,],
[	0.37256312	,	0.316920678	,	0.0766761	,	0.292692122	,],
[	0.310417967	,	0.385999391	,	0.295996952	,	0.088871547	,]],
[[	0.088590163	,	0.309351738	,	0.322871028	,	0.255136294	,],
[	0.289745533	,	0.091106996	,	0.245717317	,	0.314488097	,],
[	0.372155076	,	0.315990727	,	0.072904552	,	0.292362174	,],
[	0.310667246	,	0.386317404	,	0.296690198	,	0.086538433	,]],
[[	0.085777998	,	0.305499954	,	0.319065623	,	0.25121782	,],
[	0.285957699	,	0.082440842	,	0.241250945	,	0.310740689	,],
[	0.370818132	,	0.31487787	,	0.07319756	,	0.290877589	,],
[	0.308751581	,	0.384476387	,	0.294562577	,	0.085827157	,]],
[[	0.090499308	,	0.30742053	,	0.320901025	,	0.253279106	,],
[	0.286819283	,	0.086094022	,	0.242455448	,	0.311657234	,],
[	0.37459054	,	0.319373807	,	0.08020562	,	0.294629734	,],
[	0.312365882	,	0.387853244	,	0.297956311	,	0.091848214	,]],
[[	0.085926159	,	0.306930622	,	0.320557097	,	0.252512716	,],
[	0.288361639	,	0.083932354	,	0.243445474	,	0.313012717	,],
[	0.371170368	,	0.315117608	,	0.072380584	,	0.291264987	,],
[	0.309323375	,	0.384885896	,	0.295127362	,	0.085389642	,]],
[[	0.086760881	,	0.305668562	,	0.319288972	,	0.251349206	,],
[	0.285430508	,	0.082893658	,	0.24082711	,	0.310257905	,],
[	0.372068576	,	0.31642775	,	0.075510258	,	0.292064609	,],
[	0.309955225	,	0.385420628	,	0.295486453	,	0.087899671	,]],
[[	0.089803964	,	0.310685697	,	0.324168501	,	0.256430218	,],
[	0.289066904	,	0.089708636	,	0.244816255	,	0.313684979	,],
[	0.373340197	,	0.317168898	,	0.073936205	,	0.293392737	,],
[	0.311610728	,	0.387242878	,	0.297684249	,	0.087269587	,]],
[[	0.085727923	,	0.305787566	,	0.319289116	,	0.251479486	,],
[	0.286514163	,	0.082465562	,	0.241619567	,	0.311134019	,],
[	0.37087935	,	0.314955874	,	0.072828913	,	0.290870045	,],
[	0.308882257	,	0.384583473	,	0.294621924	,	0.08539491	,]],
[[	0.089415275	,	0.307007958	,	0.320450856	,	0.25273595	,],
[	0.286360723	,	0.085011169	,	0.241919379	,	0.311102163	,],
[	0.373766853	,	0.318370335	,	0.078779405	,	0.293854263	,],
[	0.311351134	,	0.386874906	,	0.296883652	,	0.090309104	,]],
[[	0.086914856	,	0.307846433	,	0.321381015	,	0.253442968	,],
[	0.2891786	,	0.084526949	,	0.244203975	,	0.313770004	,],
[	0.371392514	,	0.315306922	,	0.072375294	,	0.291498911	,],
[	0.309804566	,	0.385410436	,	0.295640187	,	0.085691546	,]],
[[	0.085935442	,	0.314980426	,	0.327241161	,	0.251545602	,],
[	0.276163727	,	0.083380579	,	0.242343802	,	0.303049183	,],
[	0.360498764	,	0.316039148	,	0.072683827	,	0.282759163	,],
[	0.304342793	,	0.390489866	,	0.300219372	,	0.085284506	,]],
[[	0.088739156	,	0.319407911	,	0.331655241	,	0.256097819	,],
[	0.274876467	,	0.084386848	,	0.241400623	,	0.301826169	,],
[	0.363094775	,	0.319068305	,	0.077901792	,	0.28529775	,],
[	0.308066823	,	0.394158352	,	0.303707362	,	0.09147522	,]],
[[	0.085033333	,	0.314879726	,	0.327190871	,	0.251565353	,],
[	0.279267213	,	0.092191701	,	0.246223649	,	0.305974222	,],
[	0.361495886	,	0.316747482	,	0.072700411	,	0.28381496	,],
[	0.304834917	,	0.391259457	,	0.301031733	,	0.085069985	,]],
[[	0.088788031	,	0.31630156	,	0.328582965	,	0.25301128	,],
[	0.274888522	,	0.082914452	,	0.241183328	,	0.301806499	,],
[	0.361107576	,	0.316606792	,	0.073893901	,	0.283172879	,],
[	0.305083271	,	0.391447415	,	0.30114826	,	0.087007619	,]],
[[	0.086128401	,	0.316883615	,	0.329232639	,	0.253388891	,],
[	0.275858584	,	0.08666898	,	0.242557563	,	0.302807656	,],
[	0.364638688	,	0.319876718	,	0.075463874	,	0.28697835	,],
[	0.307740252	,	0.393689938	,	0.303587725	,	0.087414192	,]],
[[	0.085296404	,	0.314578427	,	0.326932934	,	0.251072823	,],
[	0.277091309	,	0.084095405	,	0.243164769	,	0.303842484	,],
[	0.360710238	,	0.31607203	,	0.072456071	,	0.282874212	,],
[	0.304532253	,	0.390734925	,	0.300301355	,	0.085139267	,]],
[[	0.090084669	,	0.320725848	,	0.332898037	,	0.257453495	,],
[	0.274671521	,	0.083538279	,	0.240879356	,	0.301489386	,],
[	0.362511625	,	0.318339156	,	0.076614047	,	0.284461504	,],
[	0.307109448	,	0.393114706	,	0.302757184	,	0.089961716	,]],
[[	0.085071355	,	0.315239361	,	0.327471692	,	0.251761338	,],
[	0.278406	,	0.090624516	,	0.245170879	,	0.305082359	,],
[	0.362284293	,	0.317538461	,	0.073315617	,	0.284558569	,],
[	0.305352236	,	0.391692654	,	0.301528496	,	0.085364786	,]],
[[	0.087782786	,	0.315890026	,	0.328027863	,	0.252464951	,],
[	0.275302629	,	0.082841184	,	0.241376317	,	0.302088239	,],
[	0.360796555	,	0.31635758	,	0.0733191	,	0.282846158	,],
[	0.304719827	,	0.391067347	,	0.300677592	,	0.086139917	,]],
[[	0.087048415	,	0.317774517	,	0.329991493	,	0.254245117	,],
[	0.275327659	,	0.085497822	,	0.241924061	,	0.302136312	,],
[	0.364667057	,	0.32088062	,	0.080387629	,	0.286748698	,],
[	0.308808528	,	0.394810772	,	0.304837045	,	0.088421517	,]],
[[	0.085771041	,	0.327879062	,	0.337950663	,	0.252745674	,],
[	0.265150587	,	0.083984897	,	0.242500243	,	0.294391925	,],
[	0.354796559	,	0.322729955	,	0.074958481	,	0.277519136	,],
[	0.304330228	,	0.402785377	,	0.310504808	,	0.087140455	,]],
[[	0.087945771	,	0.332079011	,	0.342063512	,	0.256972129	,],
[	0.265424953	,	0.086985173	,	0.243094936	,	0.294513712	,],
[	0.358667082	,	0.327298351	,	0.082400089	,	0.281532771	,],
[	0.306974564	,	0.405330587	,	0.313481541	,	0.087605043	,]],
[[	0.084377797	,	0.327594319	,	0.337613171	,	0.252344456	,],
[	0.268059018	,	0.085882864	,	0.245207153	,	0.297056037	,],
[	0.354593117	,	0.322302226	,	0.073574278	,	0.277440506	,],
[	0.303541047	,	0.402095196	,	0.309879919	,	0.084982862	,]],
[[	0.088797695	,	0.329348366	,	0.339355485	,	0.254237166	,],
[	0.26453383	,	0.084371003	,	0.242082029	,	0.293699318	,],
[	0.35584287	,	0.324008979	,	0.077385348	,	0.278508004	,],
[	0.306018054	,	0.40449843	,	0.31230971	,	0.089929879	,]],
[[	0.085368266	,	0.329362516	,	0.339513339	,	0.254222394	,],
[	0.267028673	,	0.090017544	,	0.24491401	,	0.295974733	,],
[	0.356702956	,	0.324195317	,	0.074774422	,	0.279520501	,],
[	0.30501142	,	0.403336872	,	0.311330912	,	0.085767928	,]],
[[	0.085269735	,	0.327763221	,	0.337831923	,	0.252515001	,],
[	0.265831934	,	0.08426284	,	0.243054632	,	0.294851747	,],
[	0.354460725	,	0.322278271	,	0.074176776	,	0.277221116	,],
[	0.30397029	,	0.402436541	,	0.310150399	,	0.086441737	,]],
[[	0.08901369	,	0.333180525	,	0.34311365	,	0.25813847	,],
[	0.264908477	,	0.085882567	,	0.242484832	,	0.293979688	,],
[	0.357546741	,	0.326095213	,	0.080675751	,	0.280405442	,],
[	0.308422595	,	0.406578808	,	0.314413468	,	0.093659261	,]],
[[	0.084569452	,	0.328101945	,	0.338132014	,	0.252906918	,],
[	0.269075959	,	0.086944212	,	0.246236261	,	0.298048871	,],
[	0.355084847	,	0.322636568	,	0.073530867	,	0.277835249	,],
[	0.303573185	,	0.402096512	,	0.309935119	,	0.084821235	,]],
[[	0.087585315	,	0.328738915	,	0.338696491	,	0.253432573	,],
[	0.264973346	,	0.084272644	,	0.242291328	,	0.29400675	,],
[	0.355167378	,	0.323283154	,	0.076108581	,	0.277849509	,],
[	0.305436395	,	0.403871682	,	0.311687269	,	0.08890906	,]],
[[	0.086212821	,	0.330418777	,	0.340447217	,	0.255156892	,],
[	0.266431309	,	0.088819882	,	0.244234645	,	0.295342328	,],
[	0.357538349	,	0.324958721	,	0.075458735	,	0.280369949	,],
[	0.305953512	,	0.404221407	,	0.312152943	,	0.086493798	,]],
[[	0.084982247	,	0.336424592	,	0.345288176	,	0.252080395	,],
[	0.257317177	,	0.085109111	,	0.244206825	,	0.287817104	,],
[	0.346463904	,	0.325052802	,	0.074805377	,	0.270419922	,],
[	0.301333904	,	0.409909556	,	0.31689628	,	0.086579156	,]],
[[	0.087061568	,	0.340428908	,	0.349282642	,	0.25635528	,],
[	0.255203627	,	0.085110495	,	0.242495226	,	0.285691207	,],
[	0.348855046	,	0.328250221	,	0.08041396	,	0.272889569	,],
[	0.304912438	,	0.41323966	,	0.320734718	,	0.088212345	,]],
[[	0.083384821	,	0.335715746	,	0.344655251	,	0.251517061	,],
[	0.25820472	,	0.091654078	,	0.246108471	,	0.28865807	,],
[	0.347632049	,	0.3258907	,	0.074892656	,	0.271541588	,],
[	0.300686926	,	0.409417596	,	0.316421316	,	0.084959462	,]],
[[	0.088078487	,	0.338111491	,	0.346838691	,	0.253722121	,],
[	0.255517914	,	0.083992045	,	0.242605383	,	0.286090618	,],
[	0.34703667	,	0.325854335	,	0.076467688	,	0.270838135	,],
[	0.302536889	,	0.411275325	,	0.318295774	,	0.088870487	,]],
[[	0.084422417	,	0.337923869	,	0.346839652	,	0.253631055	,],
[	0.255758005	,	0.087014228	,	0.243370317	,	0.286235577	,],
[	0.351000477	,	0.32915756	,	0.077804905	,	0.274918902	,],
[	0.302625099	,	0.411006536	,	0.318202585	,	0.086009208	,]],
[[	0.084400136	,	0.336253509	,	0.345230591	,	0.251787082	,],
[	0.258406027	,	0.085914652	,	0.245191813	,	0.288772904	,],
[	0.346402776	,	0.324924767	,	0.074347981	,	0.270378803	,],
[	0.301040576	,	0.409602727	,	0.316487318	,	0.085809543	,]],
[[	0.088349181	,	0.341731336	,	0.350525001	,	0.257546436	,],
[	0.255226553	,	0.084575754	,	0.242290291	,	0.285671902	,],
[	0.348375537	,	0.327456929	,	0.079132736	,	0.272183356	,],
[	0.305188121	,	0.413645489	,	0.320505609	,	0.092552282	,]],
[[	0.083541895	,	0.336416266	,	0.345251157	,	0.252052958	,],
[	0.257687778	,	0.090354736	,	0.245403834	,	0.287999112	,],
[	0.348338549	,	0.326536806	,	0.075423688	,	0.272261304	,],
[	0.301393403	,	0.409957074	,	0.317097774	,	0.085185711	,]],
[[	0.086711127	,	0.337255296	,	0.346054608	,	0.252840263	,],
[	0.256214337	,	0.084291442	,	0.243117351	,	0.286624316	,],
[	0.346637714	,	0.32533723	,	0.075465467	,	0.270384276	,],
[	0.302221509	,	0.410943966	,	0.317870208	,	0.087984506	,]],
[[	0.08551186	,	0.338957308	,	0.347939481	,	0.254656162	,],
[	0.255463642	,	0.085966847	,	0.242809386	,	0.285909534	,],
[	0.350400621	,	0.329817811	,	0.082672162	,	0.274176862	,],
[	0.303732171	,	0.412049467	,	0.319400466	,	0.086981928	,]],
[[	0.084609405	,	0.347958512	,	0.355173009	,	0.252893153	,],
[	0.247475961	,	0.087362832	,	0.246132019	,	0.280084556	,],
[	0.33823859	,	0.329446781	,	0.075057127	,	0.262908413	,],
[	0.298284647	,	0.418698381	,	0.324457774	,	0.085064454	,]],
[[	0.085884545	,	0.351260025	,	0.358497729	,	0.256472832	,],
[	0.243990717	,	0.086054771	,	0.243198382	,	0.276727318	,],
[	0.340391957	,	0.332367666	,	0.080478775	,	0.265098275	,],
[	0.302081822	,	0.422090239	,	0.328236712	,	0.08680461	,]],
[[	0.082629895	,	0.347158949	,	0.35436821	,	0.252141132	,],
[	0.246651831	,	0.092025448	,	0.246317799	,	0.279192557	,],
[	0.339187473	,	0.3301649	,	0.075048637	,	0.263980984	,],
[	0.297789197	,	0.418158941	,	0.324040016	,	0.083359864	,]],
[[	0.08754899	,	0.349333812	,	0.356415863	,	0.254324877	,],
[	0.245199863	,	0.085767864	,	0.24405165	,	0.277829038	,],
[	0.338277483	,	0.329724377	,	0.076305516	,	0.262897283	,],
[	0.299920363	,	0.420361919	,	0.326095268	,	0.087740891	,]],
[[	0.083305599	,	0.348702872	,	0.35597344	,	0.253683807	,],
[	0.244268144	,	0.087633332	,	0.243759631	,	0.277000433	,],
[	0.34267615	,	0.333482828	,	0.077978471	,	0.267530914	,],
[	0.299618656	,	0.419764678	,	0.325677645	,	0.08436326	,]],
[[	0.083843572	,	0.347433369	,	0.354745551	,	0.252410603	,],
[	0.248438341	,	0.088179151	,	0.247024091	,	0.280975563	,],
[	0.338204187	,	0.32940295	,	0.07459682	,	0.26293423	,],
[	0.29816876	,	0.418477566	,	0.324187674	,	0.084373355	,]],
[[	0.087034132	,	0.352362345	,	0.359435306	,	0.257478672	,],
[	0.244313392	,	0.085792042	,	0.243239437	,	0.276863852	,],
[	0.33960437	,	0.331552575	,	0.078981399	,	0.264444633	,],
[	0.302005148	,	0.422095275	,	0.327825048	,	0.09095527	,]],
[[	0.08281276	,	0.347519553	,	0.354693288	,	0.252460488	,],
[	0.246008998	,	0.090824612	,	0.245612358	,	0.278605417	,],
[	0.340165877	,	0.331110829	,	0.075784821	,	0.264904961	,],
[	0.298132817	,	0.418377147	,	0.324212182	,	0.083347549	,]],
[[	0.086427196	,	0.348755058	,	0.355893836	,	0.253621198	,],
[	0.246003527	,	0.086318967	,	0.244834682	,	0.278654566	,],
[	0.33822907	,	0.329580631	,	0.075612059	,	0.262820492	,],
[	0.298849144	,	0.419295823	,	0.324929107	,	0.086125125	,]],
[[	0.084419132	,	0.349730336	,	0.35696833	,	0.254774604	,],
[	0.244268882	,	0.086957874	,	0.24345282	,	0.276885732	,],
[	0.341679465	,	0.333842969	,	0.082529213	,	0.266321383	,],
[	0.300831162	,	0.420793673	,	0.326898471	,	0.08549106	,]],
[[	0.09886159	,	0.227510733	,	0.266085308	,	0.285656471	,],
[	0.383728818	,	0.07911572	,	0.255362964	,	0.409650513	,],
[	0.452494731	,	0.29353073	,	0.074043897	,	0.374608261	,],
[	0.340815961	,	0.311911294	,	0.236466561	,	0.097609052	,]],
[[	0.094019304	,	0.22151324	,	0.260216186	,	0.280133366	,],
[	0.385591289	,	0.083521427	,	0.257507535	,	0.411417711	,],
[	0.455452701	,	0.296754593	,	0.075021381	,	0.377840537	,],
[	0.34083302	,	0.312221331	,	0.236898807	,	0.094534764	,]],
[[	0.096873396	,	0.221663887	,	0.260585272	,	0.280745824	,],
[	0.385415825	,	0.080037074	,	0.256929487	,	0.411204218	,],
[	0.451960644	,	0.292929025	,	0.072141202	,	0.374110719	,],
[	0.339244745	,	0.310457649	,	0.23489006	,	0.094377753	,]],
[[	0.09594526	,	0.224510697	,	0.2631753	,	0.282769914	,],
[	0.383906983	,	0.080159459	,	0.255604187	,	0.409818905	,],
[	0.45396154	,	0.295143343	,	0.076510782	,	0.376128613	,],
[	0.342971499	,	0.313857569	,	0.238429123	,	0.100886052	,]],
[[	0.09423534	,	0.220927373	,	0.259832117	,	0.279687078	,],
[	0.387856572	,	0.08727935	,	0.260109416	,	0.413708891	,],
[	0.453221571	,	0.29449867	,	0.072985071	,	0.375592188	,],
[	0.339323354	,	0.310755246	,	0.235240652	,	0.093457086	,]],
[[	0.100025314	,	0.223698981	,	0.262602593	,	0.282844517	,],
[	0.384431034	,	0.079453562	,	0.255899644	,	0.410194328	,],
[	0.452126911	,	0.293160197	,	0.073056029	,	0.374268641	,],
[	0.340162354	,	0.311319164	,	0.235849218	,	0.096317851	,]],
[[	0.094513513	,	0.222284127	,	0.260917657	,	0.280724177	,],
[	0.385048807	,	0.082490228	,	0.256875748	,	0.410845329	,],
[	0.456303816	,	0.297855857	,	0.08044996	,	0.378563679	,],
[	0.341884104	,	0.313345525	,	0.23791884	,	0.095323523	,]],
[[	0.095961483	,	0.221391798	,	0.260135847	,	0.28029483	,],
[	0.386335251	,	0.080738439	,	0.257931876	,	0.412128744	,],
[	0.45199018	,	0.29298688	,	0.071859305	,	0.374066336	,],
[	0.338973441	,	0.31019204	,	0.234645248	,	0.093645263	,]],
[[	0.096985145	,	0.225645117	,	0.26420263	,	0.283801608	,],
[	0.383904497	,	0.079845027	,	0.255563392	,	0.409690591	,],
[	0.453525061	,	0.294654307	,	0.075711735	,	0.375591985	,],
[	0.341960165	,	0.312846838	,	0.237360493	,	0.099228788	,]],
[[	0.093804619	,	0.220863267	,	0.259686331	,	0.279477305	,],
[	0.387045181	,	0.085870158	,	0.259139746	,	0.412869298	,],
[	0.453950635	,	0.295166845	,	0.073365079	,	0.37615115	,],
[	0.339880679	,	0.311397484	,	0.235877579	,	0.093784379	,]],
[[	0.095776069	,	0.228291622	,	0.266974605	,	0.277505441	,],
[	0.369038449	,	0.084439277	,	0.256304884	,	0.398360031	,],
[	0.437413591	,	0.292153081	,	0.074424832	,	0.3640937	,],
[	0.32716005	,	0.311207771	,	0.239049043	,	0.091265826	,]],
[[	0.091549804	,	0.227819615	,	0.266278842	,	0.276428858	,],
[	0.366400719	,	0.077905908	,	0.252969085	,	0.395514101	,],
[	0.433680915	,	0.288162323	,	0.071228534	,	0.360138312	,],
[	0.328083522	,	0.311921029	,	0.239591372	,	0.093928221	,]],
[[	0.092017269	,	0.226179007	,	0.264728915	,	0.275134811	,],
[	0.366309984	,	0.079872299	,	0.253111471	,	0.395516807	,],
[	0.435921324	,	0.290618305	,	0.075643816	,	0.362350428	,],
[	0.329509527	,	0.313620672	,	0.241348076	,	0.092888526	,]],
[[	0.094629288	,	0.2315556	,	0.270006672	,	0.279800358	,],
[	0.369084357	,	0.080235348	,	0.255851077	,	0.398073728	,],
[	0.435044627	,	0.289658783	,	0.071863475	,	0.361615664	,],
[	0.326655831	,	0.310551628	,	0.23812243	,	0.091245709	,]],
[[	0.090663466	,	0.226408572	,	0.264968827	,	0.275072803	,],
[	0.365567463	,	0.077844696	,	0.252289508	,	0.394733033	,],
[	0.433788972	,	0.288329175	,	0.072054293	,	0.360127732	,],
[	0.330154425	,	0.31357697	,	0.241259896	,	0.09689594	,]],
[[	0.094375338	,	0.227442918	,	0.266155848	,	0.276499853	,],
[	0.367787729	,	0.082743735	,	0.254860253	,	0.397018165	,],
[	0.438191961	,	0.293219133	,	0.079581014	,	0.364735822	,],
[	0.327523162	,	0.31160725	,	0.239386609	,	0.091090158	,]],
[[	0.092031272	,	0.228506719	,	0.266892123	,	0.276950888	,],
[	0.367112206	,	0.07843424	,	0.253720251	,	0.39606689	,],
[	0.434124808	,	0.288543145	,	0.071304661	,	0.360447971	,],
[	0.32739773	,	0.31123725	,	0.238813263	,	0.092794318	,]],
[[	0.091146268	,	0.225770708	,	0.264341289	,	0.274594672	,],
[	0.365872736	,	0.078937416	,	0.252564441	,	0.394969436	,],
[	0.435336088	,	0.290002572	,	0.074670639	,	0.361704954	,],
[	0.330905564	,	0.315057613	,	0.242774711	,	0.094128279	,]],
[[	0.096010002	,	0.233061295	,	0.271282686	,	0.281066323	,],
[	0.370116616	,	0.086447676	,	0.257340867	,	0.399269379	,],
[	0.436073024	,	0.290589778	,	0.072816675	,	0.362463681	,],
[	0.326943682	,	0.310844072	,	0.238456741	,	0.091067498	,]],
[[	0.090547025	,	0.226477601	,	0.265027762	,	0.275014579	,],
[	0.365764058	,	0.077674186	,	0.252420458	,	0.394893944	,],
[	0.43402605	,	0.288560737	,	0.071948631	,	0.360373676	,],
[	0.329433385	,	0.313013352	,	0.240652363	,	0.095685658	,]],
[[	0.094531058	,	0.238487171	,	0.276273421	,	0.276412395	,],
[	0.351322758	,	0.079029958	,	0.249698502	,	0.381710463	,],
[	0.420446584	,	0.288195789	,	0.073907922	,	0.349881068	,],
[	0.320666924	,	0.31750007	,	0.245390123	,	0.091204532	,]],
[[	0.090427489	,	0.238176346	,	0.275815567	,	0.275554394	,],
[	0.35591389	,	0.086662958	,	0.254821754	,	0.386198411	,],
[	0.421181199	,	0.288883293	,	0.071947422	,	0.350744013	,],
[	0.31805547	,	0.314709268	,	0.242488011	,	0.089735477	,]],
[[	0.091024671	,	0.236677048	,	0.274400575	,	0.274431261	,],
[	0.351390253	,	0.077784371	,	0.249557105	,	0.381614587	,],
[	0.419174626	,	0.286886873	,	0.071072101	,	0.348584492	,],
[	0.321675227	,	0.317990698	,	0.245824057	,	0.095670372	,]],
[[	0.093578672	,	0.242078801	,	0.279604063	,	0.279089025	,],
[	0.35271201	,	0.08152788	,	0.251164013	,	0.383039565	,],
[	0.422490942	,	0.290519995	,	0.077201976	,	0.352016147	,],
[	0.31896056	,	0.315663894	,	0.243375026	,	0.089700034	,]],
[[	0.089522985	,	0.236835656	,	0.27449227	,	0.274202352	,],
[	0.353447836	,	0.079061546	,	0.251681741	,	0.383452086	,],
[	0.419722457	,	0.287354378	,	0.070760373	,	0.349161032	,],
[	0.31891951	,	0.315494446	,	0.243102408	,	0.091391436	,]],
[[	0.093197037	,	0.237701057	,	0.275497292	,	0.275455098	,],
[	0.351405519	,	0.078622762	,	0.249715587	,	0.381661027	,],
[	0.420031751	,	0.287670231	,	0.072778068	,	0.349276243	,],
[	0.321912298	,	0.318742129	,	0.246667247	,	0.09211047	,]],
[[	0.09126957	,	0.23927934	,	0.276697286	,	0.276426338	,],
[	0.354872705	,	0.084843582	,	0.253492758	,	0.385000148	,],
[	0.422589528	,	0.290359168	,	0.073288016	,	0.352146505	,],
[	0.318091146	,	0.314799126	,	0.242479964	,	0.089293206	,]],
[[	0.090522029	,	0.236793299	,	0.274385281	,	0.274356198	,],
[	0.351919462	,	0.077960781	,	0.250012989	,	0.382010915	,],
[	0.419019783	,	0.286591026	,	0.070629593	,	0.348390693	,],
[	0.320531418	,	0.316812651	,	0.244638717	,	0.093875327	,]],
[[	0.09498827	,	0.243359151	,	0.280871515	,	0.280300834	,],
[	0.352446531	,	0.080679955	,	0.250843121	,	0.382635345	,],
[	0.421566147	,	0.28946851	,	0.075721649	,	0.350996776	,],
[	0.319510634	,	0.316252991	,	0.244000606	,	0.089921947	,]],
[[	0.089751428	,	0.237274335	,	0.274916599	,	0.274541339	,],
[	0.354540016	,	0.079960285	,	0.252587113	,	0.384531001	,],
[	0.420438675	,	0.288146015	,	0.07124897	,	0.349824228	,],
[	0.318351538	,	0.314843485	,	0.242502305	,	0.090268926	,]],
[[	0.090376528	,	0.241447398	,	0.277691033	,	0.267940394	,],
[	0.338380934	,	0.076539669	,	0.247915472	,	0.37013807	,],
[	0.40741992	,	0.285337224	,	0.071698877	,	0.337632358	,],
[	0.315075947	,	0.322959204	,	0.252275026	,	0.090685858	,]],
[[	0.088181352	,	0.242401879	,	0.27837784	,	0.268422742	,],
[	0.340502612	,	0.081387971	,	0.250390964	,	0.37219832	,],
[	0.411038704	,	0.288982981	,	0.073712976	,	0.341585668	,],
[	0.311392979	,	0.319301714	,	0.248387698	,	0.088045181	,]],
[[	0.087143587	,	0.239633915	,	0.275782094	,	0.266026576	,],
[	0.339531927	,	0.076732814	,	0.248839077	,	0.37105326	,],
[	0.407066277	,	0.284847212	,	0.070228426	,	0.337309928	,],
[	0.313844764	,	0.321425545	,	0.25060284	,	0.092844317	,]],
[[	0.092400274	,	0.247175734	,	0.283162863	,	0.273018594	,],
[	0.338534262	,	0.07778439	,	0.248163552	,	0.370249304	,],
[	0.408653021	,	0.286693278	,	0.074117975	,	0.338868206	,],
[	0.312845049	,	0.320707429	,	0.249791537	,	0.088752108	,]],
[[	0.08673739	,	0.240514207	,	0.27667148	,	0.266573041	,],
[	0.343005644	,	0.079783538	,	0.252367977	,	0.374464364	,],
[	0.40897559	,	0.286718202	,	0.071439395	,	0.33915836	,],
[	0.312165016	,	0.319811652	,	0.248854852	,	0.08954555	,]],
[[	0.08896919	,	0.240383828	,	0.276663009	,	0.266912763	,],
[	0.338532737	,	0.076421882	,	0.248033571	,	0.370167591	,],
[	0.407055323	,	0.28486994	,	0.070896777	,	0.337188822	,],
[	0.316275091	,	0.324343021	,	0.253606512	,	0.091801955	,]],
[[	0.089147537	,	0.243556471	,	0.27938002	,	0.269465762	,],
[	0.339919973	,	0.080245964	,	0.249598599	,	0.371576159	,],
[	0.410573971	,	0.289000084	,	0.077768696	,	0.341055885	,],
[	0.311480314	,	0.319393676	,	0.248476944	,	0.08789815	,]],
[[	0.086893663	,	0.239820474	,	0.275789888	,	0.265971424	,],
[	0.340520069	,	0.077653874	,	0.249832912	,	0.371984568	,],
[	0.407562213	,	0.28539698	,	0.070501816	,	0.337779517	,],
[	0.31322883	,	0.32078955	,	0.249909757	,	0.091606237	,]],
[[	0.092379889	,	0.24246847	,	0.278478464	,	0.268995761	,],
[	0.33835772	,	0.077007751	,	0.247735247	,	0.369917629	,],
[	0.408141009	,	0.286184809	,	0.073216883	,	0.338433453	,],
[	0.313488084	,	0.321404426	,	0.25047621	,	0.089280945	,]],
[[	0.087268677	,	0.241364058	,	0.27737947	,	0.267313573	,],
[	0.341914175	,	0.083624222	,	0.251873292	,	0.373480825	,],
[	0.410074862	,	0.287918484	,	0.07254057	,	0.340360391	,],
[	0.311565611	,	0.319230034	,	0.248269501	,	0.088529482	,]],
[[	0.088983103	,	0.250945074	,	0.28526576	,	0.266115519	,],
[	0.329434193	,	0.076715885	,	0.247612088	,	0.361659134	,],
[	0.398183564	,	0.286657325	,	0.069293888	,	0.329309842	,],
[	0.313557242	,	0.332392106	,	0.260500171	,	0.092125863	,]],
[[	0.087593431	,	0.252556056	,	0.286574266	,	0.267335521	,],
[	0.328994007	,	0.078491248	,	0.24749656	,	0.361363228	,],
[	0.399705551	,	0.288420653	,	0.073147076	,	0.330822785	,],
[	0.308671124	,	0.327454699	,	0.255378909	,	0.08775037	,]],
[[	0.086322209	,	0.249514713	,	0.283699139	,	0.264540595	,],
[	0.332603002	,	0.079322862	,	0.250667377	,	0.36453903	,],
[	0.400244966	,	0.288668447	,	0.070650144	,	0.331362615	,],
[	0.310325427	,	0.328897607	,	0.256826255	,	0.09134844	,]],
[[	0.0915795	,	0.256939077	,	0.291026681	,	0.271577282	,],
[	0.328831906	,	0.076797576	,	0.247005337	,	0.361068081	,],
[	0.398275316	,	0.286779745	,	0.070111839	,	0.329428185	,],
[	0.311106958	,	0.329796396	,	0.257679712	,	0.089454284	,]],
[[	0.085577466	,	0.250274912	,	0.284471659	,	0.265087959	,],
[	0.330716341	,	0.081420934	,	0.249328959	,	0.362951568	,],
[	0.401974019	,	0.291092906	,	0.07690836	,	0.333356538	,],
[	0.309018118	,	0.32753802	,	0.255419537	,	0.088530429	,]],
[[	0.087802403	,	0.250120651	,	0.284466284	,	0.26528682	,],
[	0.330160815	,	0.07712405	,	0.248278765	,	0.362218753	,],
[	0.398607748	,	0.286968488	,	0.069621261	,	0.329659413	,],
[	0.312408236	,	0.330719277	,	0.258657157	,	0.094465427	,]],
[[	0.088348021	,	0.253444789	,	0.287353568	,	0.268158642	,],
[	0.328694083	,	0.077562461	,	0.247090421	,	0.360937703	,],
[	0.39907313	,	0.287713752	,	0.071912866	,	0.330067983	,],
[	0.309150166	,	0.327932482	,	0.255926086	,	0.087929207	,]],
[[	0.085869985	,	0.249604266	,	0.283737916	,	0.264540001	,],
[	0.33299993	,	0.085147493	,	0.25182903	,	0.364986151	,],
[	0.401178874	,	0.289674147	,	0.07170743	,	0.332352537	,],
[	0.309799036	,	0.328356158	,	0.25620771	,	0.090198561	,]],
[[	0.091212523	,	0.252180507	,	0.286305454	,	0.267376221	,],
[	0.328948267	,	0.07658413	,	0.247136031	,	0.361023527	,],
[	0.398483749	,	0.286875775	,	0.070034289	,	0.329492902	,],
[	0.311939714	,	0.330709977	,	0.25864249	,	0.090209529	,]],
[[	0.086242425	,	0.251136682	,	0.285277629	,	0.265818169	,],
[	0.330283253	,	0.080254206	,	0.248652835	,	0.362268537	,],
[	0.400823386	,	0.289680325	,	0.075152028	,	0.331846918	,],
[	0.308755125	,	0.327315265	,	0.25520179	,	0.088084784	,]],
[[	0.086494737	,	0.2578017	,	0.290980185	,	0.262559626	,],
[	0.320965264	,	0.080027777	,	0.250451199	,	0.354879505	,],
[	0.39005673	,	0.289517254	,	0.07178549	,	0.322317824	,],
[	0.305839783	,	0.335296311	,	0.263705135	,	0.091082503	,]],
[[	0.086603783	,	0.260549676	,	0.293536926	,	0.265028022	,],
[	0.316652108	,	0.076920333	,	0.246309238	,	0.350795214	,],
[	0.387437759	,	0.287165503	,	0.07048281	,	0.319773554	,],
[	0.305130943	,	0.334963807	,	0.263361311	,	0.087697572	,]],
[[	0.084220987	,	0.25692142	,	0.289916435	,	0.261501393	,],
[	0.318294198	,	0.081290969	,	0.248353739	,	0.352388759	,],
[	0.390614496	,	0.290821072	,	0.076646836	,	0.322939491	,],
[	0.303697688	,	0.333480895	,	0.261789881	,	0.087671246	,]],
[[	0.089942483	,	0.25977611	,	0.292877414	,	0.264644073	,],
[	0.318462953	,	0.07781907	,	0.247889007	,	0.352428108	,],
[	0.388106695	,	0.287573273	,	0.070176817	,	0.320400751	,],
[	0.308054441	,	0.337564816	,	0.265937021	,	0.094394468	,]],
[[	0.084710238	,	0.258508768	,	0.291509741	,	0.262914924	,],
[	0.316763736	,	0.077911615	,	0.246389936	,	0.350837422	,],
[	0.388334359	,	0.288215065	,	0.072397721	,	0.32068893	,],
[	0.304150434	,	0.333930891	,	0.262133219	,	0.086861864	,]],
[[	0.085476	,	0.25713615	,	0.290334111	,	0.261892983	,],
[	0.320821937	,	0.085044371	,	0.250943865	,	0.354746669	,],
[	0.390890421	,	0.290496439	,	0.072759864	,	0.323282333	,],
[	0.305083581	,	0.334583838	,	0.262898274	,	0.089879523	,]],
[[	0.087654301	,	0.261667751	,	0.294517776	,	0.266052318	,],
[	0.317019153	,	0.076933805	,	0.246564878	,	0.351082556	,],
[	0.387225685	,	0.28686241	,	0.06999778	,	0.319532808	,],
[	0.306165741	,	0.335997278	,	0.264383525	,	0.088557292	,]],
[[	0.083783013	,	0.256787817	,	0.289867543	,	0.261325418	,],
[	0.317923761	,	0.080232102	,	0.247772301	,	0.351989487	,],
[	0.389643306	,	0.289743951	,	0.075126292	,	0.322038748	,],
[	0.303641893	,	0.333297119	,	0.261600366	,	0.086903642	,]],
[[	0.088639081	,	0.258964009	,	0.29203308	,	0.263718577	,],
[	0.319407124	,	0.078464225	,	0.248668828	,	0.353073477	,],
[	0.389059348	,	0.288553291	,	0.070902076	,	0.321283174	,],
[	0.307121961	,	0.336508519	,	0.264888873	,	0.092830352	,]],
[[	0.08519397	,	0.259025402	,	0.29201287	,	0.263368517	,],
[	0.316714452	,	0.077415972	,	0.246232945	,	0.350654573	,],
[	0.387724107	,	0.28751961	,	0.071408487	,	0.31992577	,],
[	0.304422281	,	0.33408886	,	0.262435886	,	0.086910759	,]],
[[	0.084871072	,	0.26486851	,	0.296611984	,	0.259445846	,],
[	0.306084316	,	0.080530106	,	0.246639704	,	0.341080961	,],
[	0.377626057	,	0.289544995	,	0.075066942	,	0.311551328	,],
[	0.297358901	,	0.338183708	,	0.266145889	,	0.085833307	,]],
[[	0.086534707	,	0.268889378	,	0.300336095	,	0.263118852	,],
[	0.306649696	,	0.077622829	,	0.246622069	,	0.341399848	,],
[	0.377094537	,	0.288423413	,	0.070967136	,	0.310940498	,],
[	0.301351487	,	0.342402071	,	0.270547888	,	0.088140649	,]],
[[	0.083158222	,	0.264446417	,	0.296063007	,	0.258831564	,],
[	0.304813805	,	0.077489011	,	0.24500449	,	0.339727086	,],
[	0.375773606	,	0.287331481	,	0.071521407	,	0.309689261	,],
[	0.297074245	,	0.338072882	,	0.266115415	,	0.084561542	,]],
[[	0.088151604	,	0.266755706	,	0.298344486	,	0.261385583	,],
[	0.308477416	,	0.084042964	,	0.249054049	,	0.343124221	,],
[	0.379566662	,	0.291640396	,	0.078379746	,	0.313540541	,],
[	0.298928969	,	0.339782568	,	0.267703618	,	0.088432589	,]],
[[	0.084039207	,	0.266238028	,	0.297898489	,	0.260508274	,],
[	0.305300973	,	0.076721777	,	0.245163395	,	0.339947456	,],
[	0.375597624	,	0.287003403	,	0.07013126	,	0.309473871	,],
[	0.298885735	,	0.339725475	,	0.26780877	,	0.085720012	,]],
[[	0.084001789	,	0.264457332	,	0.296220981	,	0.258912332	,],
[	0.305511589	,	0.079327106	,	0.245844801	,	0.340210124	,],
[	0.377106137	,	0.288762508	,	0.073824889	,	0.310876305	,],
[	0.297141522	,	0.337971695	,	0.265987865	,	0.085329994	,]],
[[	0.087524892	,	0.269758709	,	0.301175951	,	0.263923515	,],
[	0.307795179	,	0.078572516	,	0.247587788	,	0.342441309	,],
[	0.37769767	,	0.289029696	,	0.071512615	,	0.311540326	,],
[	0.301782476	,	0.342298341	,	0.270323814	,	0.092255791	,]],
[[	0.083235227	,	0.264824699	,	0.296305447	,	0.259084031	,],
[	0.304655022	,	0.076818587	,	0.244724043	,	0.339391163	,],
[	0.375468779	,	0.286839724	,	0.070721767	,	0.309308657	,],
[	0.297270624	,	0.33822889	,	0.26625652	,	0.084318527	,]],
[[	0.086816733	,	0.266074139	,	0.297622691	,	0.260519877	,],
[	0.307488379	,	0.082541206	,	0.247970419	,	0.342123808	,],
[	0.378882526	,	0.290923951	,	0.077120481	,	0.312754862	,],
[	0.298484334	,	0.339380777	,	0.267207026	,	0.087511214	,]],
[[	0.084799565	,	0.267064154	,	0.298533912	,	0.261191055	,],
[	0.305813962	,	0.076932	,	0.245640064	,	0.340440453	,],
[	0.376262291	,	0.287537184	,	0.070421568	,	0.310072832	,],
[	0.299709677	,	0.340748967	,	0.268775483	,	0.086445729	,]],
[[	0.084343227	,	0.274773368	,	0.304730071	,	0.257473709	,],
[	0.295371077	,	0.079709856	,	0.245705666	,	0.330618562	,],
[	0.368337751	,	0.291250531	,	0.074882925	,	0.302379041	,],
[	0.295210464	,	0.346405506	,	0.274486064	,	0.086025486	,]],
[[	0.084656907	,	0.277645725	,	0.307285545	,	0.260075454	,],
[	0.297706051	,	0.078823954	,	0.247443093	,	0.33276565	,],
[	0.368450312	,	0.290584714	,	0.071316896	,	0.302319748	,],
[	0.297748255	,	0.349198105	,	0.277464367	,	0.086460021	,]],
[[	0.082080484	,	0.273880753	,	0.303641163	,	0.25644413	,],
[	0.294551266	,	0.077256126	,	0.244582372	,	0.329746334	,],
[	0.366732756	,	0.289230504	,	0.071455846	,	0.300595787	,],
[	0.29419256	,	0.345591008	,	0.273650488	,	0.083812086	,]],
[[	0.087505742	,	0.276600528	,	0.306268218	,	0.259246788	,],
[	0.297163455	,	0.082463742	,	0.247568668	,	0.332188616	,],
[	0.370627461	,	0.293707243	,	0.078564605	,	0.304620257	,],
[	0.296827632	,	0.348001427	,	0.276209911	,	0.088799858	,]],
[[	0.082406907	,	0.275265503	,	0.305053889	,	0.257725996	,],
[	0.295862429	,	0.077294667	,	0.245517198	,	0.330848487	,],
[	0.367145817	,	0.289426948	,	0.070562104	,	0.301058498	,],
[	0.295735	,	0.346951329	,	0.275050479	,	0.084438721	,]],
[[	0.083467281	,	0.274274715	,	0.304191825	,	0.256868226	,],
[	0.294647783	,	0.078272717	,	0.24471102	,	0.329767885	,],
[	0.367871087	,	0.290607925	,	0.073796623	,	0.301766854	,],
[	0.294876718	,	0.346107627	,	0.27405148	,	0.085185697	,]],
[[	0.086092595	,	0.279086174	,	0.308653369	,	0.261497005	,],
[	0.299100778	,	0.080090974	,	0.248732226	,	0.334026503	,],
[	0.369048473	,	0.291272953	,	0.071903033	,	0.302998945	,],
[	0.298823259	,	0.350072876	,	0.278340148	,	0.087309194	,]],
[[	0.082113746	,	0.274178213	,	0.303885361	,	0.256717565	,],
[	0.294541135	,	0.076801412	,	0.244375506	,	0.32964464	,],
[	0.366700797	,	0.289111189	,	0.070839662	,	0.300460631	,],
[	0.294260899	,	0.345602143	,	0.273666742	,	0.083570785	,]],
[[	0.085914494	,	0.27555326	,	0.30527707	,	0.258130939	,],
[	0.296294406	,	0.081169518	,	0.246545868	,	0.331385531	,],
[	0.36966467	,	0.29265022	,	0.076894423	,	0.303598779	,],
[	0.296539044	,	0.347792463	,	0.275778702	,	0.087834237	,]],
[[	0.083309455	,	0.276101122	,	0.305835237	,	0.258487783	,],
[	0.29632644	,	0.077653267	,	0.24597656	,	0.331338345	,],
[	0.367398472	,	0.289701024	,	0.070681881	,	0.301366745	,],
[	0.296374717	,	0.347614944	,	0.27569378	,	0.084930605	,]],
[[	0.083680261	,	0.285994844	,	0.314848334	,	0.257755775	,],
[	0.285320919	,	0.078956965	,	0.246339607	,	0.322487708	,],
[	0.359033418	,	0.293132106	,	0.071789472	,	0.29367114	,],
[	0.292425466	,	0.355683144	,	0.281969402	,	0.084642914	,]],
[[	0.085821939	,	0.29018165	,	0.318752409	,	0.261827263	,],
[	0.28889414	,	0.0856246	,	0.250382481	,	0.325861723	,],
[	0.362836983	,	0.296400458	,	0.073111096	,	0.297403162	,],
[	0.297162682	,	0.360067124	,	0.286487136	,	0.091676092	,]],
[[	0.082085594	,	0.285510914	,	0.314176459	,	0.257195391	,],
[	0.286016307	,	0.078423006	,	0.246722474	,	0.323124602	,],
[	0.358843529	,	0.29265103	,	0.070113402	,	0.29337359	,],
[	0.293017647	,	0.356353274	,	0.282706903	,	0.08418887	,]],
[[	0.086505428	,	0.287431189	,	0.316107881	,	0.259158534	,],
[	0.286415756	,	0.081035276	,	0.247514938	,	0.323466951	,],
[	0.360310772	,	0.294750209	,	0.074371953	,	0.29499604	,],
[	0.293903457	,	0.357017729	,	0.283432476	,	0.086889853	,]],
[[	0.083252248	,	0.287523591	,	0.316298382	,	0.259144238	,],
[	0.288932711	,	0.080545521	,	0.249513323	,	0.325811186	,],
[	0.360282081	,	0.294001847	,	0.070798849	,	0.29496192	,],
[	0.295695303	,	0.358718643	,	0.285164512	,	0.086155026	,]],
[[	0.082867791	,	0.285570399	,	0.3143951	,	0.257229819	,],
[	0.285501753	,	0.078621325	,	0.246272046	,	0.322481613	,],
[	0.358659503	,	0.292733577	,	0.071012395	,	0.293292958	,],
[	0.292727606	,	0.355844731	,	0.282147779	,	0.084315982	,]],
[[	0.087181551	,	0.291524649	,	0.320103202	,	0.263020368	,],
[	0.288053662	,	0.084165139	,	0.249397724	,	0.324975692	,],
[	0.362263072	,	0.296978248	,	0.077805374	,	0.297009072	,],
[	0.296084874	,	0.359007122	,	0.285383479	,	0.090050954	,]],
[[	0.08229798	,	0.285868249	,	0.314394625	,	0.257444273	,],
[	0.286535661	,	0.078508372	,	0.247212705	,	0.323495538	,],
[	0.358881188	,	0.292753515	,	0.069998957	,	0.293404111	,],
[	0.293696964	,	0.35682731	,	0.283263421	,	0.084418091	,]],
[[	0.08537226	,	0.286826226	,	0.315516063	,	0.258476941	,],
[	0.285640758	,	0.079872055	,	0.246657365	,	0.322574278	,],
[	0.359853008	,	0.293981091	,	0.073164662	,	0.294324621	,],
[	0.293547711	,	0.356611412	,	0.282911615	,	0.085801988	,]],
[[	0.084117408	,	0.288476431	,	0.317049435	,	0.259935649	,],
[	0.28997627	,	0.08148305	,	0.250491499	,	0.326813885	,],
[	0.361214583	,	0.294836698	,	0.071708767	,	0.29582549	,],
[	0.296515411	,	0.35961564	,	0.286051062	,	0.086991731	,]],
[[	0.083287943	,	0.296314081	,	0.325231296	,	0.258423335	,],
[	0.277394556	,	0.078863773	,	0.249237503	,	0.31611952	,],
[	0.352396635	,	0.296759565	,	0.072236242	,	0.287221401	,],
[	0.291862716	,	0.366113768	,	0.29233277	,	0.08438126	,]],
[[	0.084967048	,	0.300122774	,	0.328860254	,	0.262257394	,],
[	0.279856971	,	0.084286843	,	0.252169193	,	0.318424817	,],
[	0.355892227	,	0.300960137	,	0.079113113	,	0.290730461	,],
[	0.296000048	,	0.370110188	,	0.296510515	,	0.091173472	,]],
[[	0.08153162	,	0.295623109	,	0.324624124	,	0.257764278	,],
[	0.278885745	,	0.079132844	,	0.250388471	,	0.317559122	,],
[	0.35259835	,	0.296673556	,	0.071156373	,	0.287410574	,],
[	0.292167183	,	0.366478026	,	0.292758634	,	0.083793061	,]],
[[	0.086124678	,	0.297555777	,	0.326409107	,	0.25966292	,],
[	0.277719703	,	0.08012082	,	0.249584423	,	0.316150376	,],
[	0.353139356	,	0.297774834	,	0.074274726	,	0.287930551	,],
[	0.293019855	,	0.36738762	,	0.293668643	,	0.086554783	,]],
[[	0.082650491	,	0.297690849	,	0.326540056	,	0.259672118	,],
[	0.282318397	,	0.088331674	,	0.254874829	,	0.320703594	,],
[	0.355171368	,	0.299057175	,	0.073010231	,	0.28996617	,],
[	0.294810107	,	0.368862788	,	0.295269888	,	0.08591749	,]],
[[	0.082297959	,	0.295716774	,	0.324711355	,	0.257685828	,],
[	0.277703496	,	0.078565003	,	0.249275643	,	0.316189069	,],
[	0.35199705	,	0.296291868	,	0.071450325	,	0.286771244	,],
[	0.291854457	,	0.366094934	,	0.292258332	,	0.083972359	,]],
[[	0.086273414	,	0.301471713	,	0.330091916	,	0.263501201	,],
[	0.279190916	,	0.083084199	,	0.251289445	,	0.317573928	,],
[	0.354993848	,	0.299868679	,	0.077400211	,	0.289692692	,],
[	0.295054702	,	0.369158769	,	0.295413944	,	0.089606393	,]],
[[	0.081471944	,	0.295837415	,	0.324610321	,	0.257774571	,],
[	0.279520885	,	0.07959894	,	0.250996009	,	0.318003301	,],
[	0.353296641	,	0.297291744	,	0.071564497	,	0.288034574	,],
[	0.292578819	,	0.366894976	,	0.293186418	,	0.084037589	,]],
[[	0.084937937	,	0.297045442	,	0.325984576	,	0.259091082	,],
[	0.277649492	,	0.079450015	,	0.249462322	,	0.316187783	,],
[	0.35272389	,	0.297187701	,	0.073187077	,	0.287371302	,],
[	0.29248426	,	0.36677001	,	0.293029146	,	0.085485964	,]],
[[	0.08298977	,	0.298076839	,	0.326937295	,	0.259990559	,],
[	0.281534165	,	0.08671548	,	0.253922839	,	0.319834535	,],
[	0.355967089	,	0.29986884	,	0.073901006	,	0.290781406	,],
[	0.295901221	,	0.36994819	,	0.296412389	,	0.086765679	,]],
[[	0.082189304	,	0.307434123	,	0.335303864	,	0.259159642	,],
[	0.270875644	,	0.08099523	,	0.253447573	,	0.311651147	,],
[	0.346371282	,	0.301572433	,	0.073010386	,	0.282474597	,],
[	0.290150704	,	0.375895827	,	0.301469563	,	0.083553219	,]],
[[	0.0862285	,	0.31298791	,	0.340560644	,	0.264789016	,],
[	0.269425798	,	0.081506811	,	0.25224822	,	0.310147872	,],
[	0.346027284	,	0.301802335	,	0.075027534	,	0.282140518	,],
[	0.292278432	,	0.377764879	,	0.303291815	,	0.087418967	,]],
[[	0.081248333	,	0.307542507	,	0.335294454	,	0.259316921	,],
[	0.273889767	,	0.089437787	,	0.257353814	,	0.314546082	,],
[	0.349050764	,	0.30415157	,	0.075187057	,	0.285121569	,],
[	0.292171637	,	0.377712294	,	0.303417815	,	0.084835461	,]],
[[	0.084464226	,	0.308488428	,	0.336257992	,	0.260232845	,],
[	0.269667905	,	0.080345306	,	0.252279906	,	0.31038996	,],
[	0.345688596	,	0.301006052	,	0.073004655	,	0.281649882	,],
[	0.290211395	,	0.376038454	,	0.301457531	,	0.084186058	,]],
[[	0.083407715	,	0.310420106	,	0.338076351	,	0.262053671	,],
[	0.270640685	,	0.084093702	,	0.253754986	,	0.311345276	,],
[	0.347934964	,	0.303748397	,	0.078140981	,	0.283910262	,],
[	0.294451459	,	0.379990788	,	0.305592638	,	0.090773567	,]],
[[	0.081397814	,	0.306996754	,	0.334941312	,	0.258640516	,],
[	0.271733096	,	0.081399343	,	0.253883106	,	0.31233849	,],
[	0.347083863	,	0.302220041	,	0.073534118	,	0.283077114	,],
[	0.290615733	,	0.376277342	,	0.301686241	,	0.083584282	,]],
[[	0.087602357	,	0.314364963	,	0.341912645	,	0.266128057	,],
[	0.26924983	,	0.080888709	,	0.251855393	,	0.309913026	,],
[	0.345533862	,	0.301158896	,	0.074065084	,	0.281592729	,],
[	0.291501116	,	0.377133963	,	0.302541748	,	0.086244635	,]],
[[	0.081680524	,	0.308168144	,	0.335829265	,	0.259765959	,],
[	0.273052124	,	0.088006903	,	0.256360872	,	0.313580133	,],
[	0.350026694	,	0.305007572	,	0.076137221	,	0.286145229	,],
[	0.293302095	,	0.378861857	,	0.304587692	,	0.085761519	,]],
[[	0.083787402	,	0.308127349	,	0.335909484	,	0.259809141	,],
[	0.269997828	,	0.080332433	,	0.252543883	,	0.310675774	,],
[	0.345975246	,	0.301127053	,	0.072816552	,	0.281796511	,],
[	0.290243725	,	0.375909481	,	0.301319544	,	0.083708368	,]],
[[	0.084165923	,	0.310963897	,	0.338826103	,	0.262691971	,],
[	0.270269787	,	0.083097951	,	0.253303251	,	0.310898809	,],
[	0.346911611	,	0.302663756	,	0.076522802	,	0.282800605	,],
[	0.293684581	,	0.379258961	,	0.30474419	,	0.089447566	,]],
[[	0.082227827	,	0.317783503	,	0.344743016	,	0.25952421	,],
[	0.262805036	,	0.082117052	,	0.255965732	,	0.304833204	,],
[	0.339407582	,	0.30518618	,	0.073552079	,	0.275850257	,],
[	0.28915496	,	0.386006685	,	0.310571415	,	0.083896759	,]],
[[	0.084658231	,	0.322023359	,	0.348778408	,	0.263812836	,],
[	0.260844031	,	0.082252383	,	0.25455488	,	0.30309913	,],
[	0.339192248	,	0.305548353	,	0.075693721	,	0.275569218	,],
[	0.291864994	,	0.388576041	,	0.313319296	,	0.088808411	,]],
[[	0.08097093	,	0.31768643	,	0.344534192	,	0.259313766	,],
[	0.264766194	,	0.089827931	,	0.259200051	,	0.306899043	,],
[	0.342280836	,	0.308017839	,	0.075888031	,	0.278723059	,],
[	0.290677748	,	0.387447508	,	0.312174243	,	0.084702583	,]],
[[	0.085148387	,	0.319308463	,	0.346153678	,	0.261050089	,],
[	0.261052272	,	0.081142639	,	0.254516079	,	0.303256854	,],
[	0.338459194	,	0.304568481	,	0.073382765	,	0.274881056	,],
[	0.289717329	,	0.386506511	,	0.31119879	,	0.085116103	,]],
[[	0.081802454	,	0.319342017	,	0.346265888	,	0.26108997	,],
[	0.261668966	,	0.08459994	,	0.255802472	,	0.303977143	,],
[	0.340838698	,	0.307435363	,	0.07851887	,	0.27707583	,],
[	0.294254889	,	0.390658369	,	0.315556919	,	0.087930744	,]],
[[	0.081638678	,	0.31762772	,	0.344553415	,	0.259236089	,],
[	0.263819159	,	0.082881534	,	0.256894638	,	0.305758039	,],
[	0.340004459	,	0.305863986	,	0.074042771	,	0.276436205	,],
[	0.289557185	,	0.386251019	,	0.310852078	,	0.083825588	,]],
[[	0.086010067	,	0.323398637	,	0.349987386	,	0.265204536	,],
[	0.260917773	,	0.081826141	,	0.254334525	,	0.302860422	,],
[	0.338966082	,	0.305176068	,	0.074736333	,	0.275163365	,],
[	0.291469977	,	0.388216751	,	0.312842407	,	0.087842439	,]],
[[	0.080698468	,	0.317521073	,	0.344267485	,	0.259173261	,],
[	0.26417915	,	0.088417062	,	0.258282827	,	0.306179637	,],
[	0.342680895	,	0.309663964	,	0.082040193	,	0.278983425	,],
[	0.291678241	,	0.388244054	,	0.313085557	,	0.085305845	,]],
[[	0.084068017	,	0.318784263	,	0.345652353	,	0.260408212	,],
[	0.261900524	,	0.081524696	,	0.255201262	,	0.30387065	,],
[	0.338756979	,	0.304706765	,	0.073214349	,	0.275024836	,],
[	0.289419563	,	0.386173192	,	0.310847212	,	0.084361331	,]],
[[	0.083075759	,	0.320461496	,	0.347347982	,	0.262184002	,],
[	0.261651703	,	0.083838662	,	0.255494738	,	0.303732577	,],
[	0.339987447	,	0.306421773	,	0.077082479	,	0.276162873	,],
[	0.293277561	,	0.390046154	,	0.314686832	,	0.090889168	,]],
[[	0.082071891	,	0.327818832	,	0.354491729	,	0.259276125	,],
[	0.255412204	,	0.084142803	,	0.259808739	,	0.298423755	,],
[	0.332795923	,	0.309412791	,	0.074590948	,	0.268826196	,],
[	0.288414244	,	0.395679686	,	0.319585835	,	0.083338166	,]],
[[	0.085317635	,	0.332750424	,	0.359061189	,	0.264222372	,],
[	0.252342325	,	0.08294325	,	0.257252803	,	0.29550106	,],
[	0.331647536	,	0.308752299	,	0.075612268	,	0.267677016	,],
[	0.291490096	,	0.398632034	,	0.322525614	,	0.088564752	,]],
[[	0.081095298	,	0.327789928	,	0.354415145	,	0.259280892	,],
[	0.254897677	,	0.088864209	,	0.260434696	,	0.297947066	,],
[	0.335897566	,	0.312300052	,	0.077212675	,	0.272074951	,],
[	0.289732109	,	0.396859766	,	0.321016787	,	0.083822946	,]],
[[	0.085042363	,	0.329356549	,	0.355945413	,	0.260780811	,],
[	0.253412637	,	0.082710494	,	0.257996161	,	0.296519503	,],
[	0.33165939	,	0.308335769	,	0.073999591	,	0.267531512	,],
[	0.288834895	,	0.39627532	,	0.32014847	,	0.084466049	,]],
[[	0.082325121	,	0.329865749	,	0.356477465	,	0.261278145	,],
[	0.252524229	,	0.084306059	,	0.257750166	,	0.29579232	,],
[	0.333204828	,	0.310572927	,	0.078521285	,	0.269143095	,],
[	0.293367816	,	0.400073937	,	0.324246434	,	0.087080168	,]],
[[	0.081792829	,	0.327747951	,	0.354464552	,	0.25913768	,],
[	0.256990024	,	0.085639053	,	0.261260614	,	0.299894838	,],
[	0.333510379	,	0.310004037	,	0.074939815	,	0.269430563	,],
[	0.288858043	,	0.395969893	,	0.319928918	,	0.083222578	,]],
[[	0.086539522	,	0.333826717	,	0.36009384	,	0.265390283	,],
[	0.252521556	,	0.08256524	,	0.257179142	,	0.295558093	,],
[	0.331652604	,	0.308580222	,	0.074946209	,	0.267406686	,],
[	0.29034101	,	0.397542866	,	0.321439886	,	0.086979079	,]],
[[	0.081218084	,	0.328258872	,	0.35472406	,	0.259617812	,],
[	0.254258801	,	0.087452341	,	0.259624877	,	0.297250023	,],
[	0.335286256	,	0.312890923	,	0.082001374	,	0.27114215	,],
[	0.290861989	,	0.397823928	,	0.321993728	,	0.084675674	,]],
[[	0.083851921	,	0.328776107	,	0.355288397	,	0.2600577	,],
[	0.254343193	,	0.083416185	,	0.25878911	,	0.29738088	,],
[	0.33197867	,	0.308698211	,	0.074069513	,	0.267870017	,],
[	0.288490795	,	0.395737503	,	0.319605104	,	0.083714536	,]],
[[	0.083404999	,	0.330842139	,	0.357360832	,	0.262257305	,],
[	0.252304562	,	0.083617755	,	0.257305326	,	0.295471021	,],
[	0.332810214	,	0.309952013	,	0.077310137	,	0.268580447	,],
[	0.292557265	,	0.399718048	,	0.323778728	,	0.090366742	,]],
[[	0.082749803	,	0.338052987	,	0.363608106	,	0.259485343	,],
[	0.247776702	,	0.092688092	,	0.262945896	,	0.291389685	,],
[	0.325966426	,	0.312998031	,	0.075205076	,	0.261990092	,],
[	0.287156819	,	0.403560432	,	0.327270535	,	0.083215928	,]],
[[	0.085129416	,	0.342150798	,	0.367363372	,	0.263785365	,],
[	0.243891661	,	0.083324845	,	0.257894408	,	0.287546482	,],
[	0.323975688	,	0.311456776	,	0.07508081	,	0.259824669	,],
[	0.289514867	,	0.405967696	,	0.329685666	,	0.088042355	,]],
[[	0.081470738	,	0.337759092	,	0.363229814	,	0.259301321	,],
[	0.244901529	,	0.087562988	,	0.259653189	,	0.288594614	,],
[	0.327142454	,	0.315553167	,	0.081880749	,	0.263200899	,],
[	0.288468639	,	0.404819451	,	0.328600384	,	0.083694293	,]],
[[	0.085591313	,	0.339516498	,	0.36494996	,	0.260890954	,],
[	0.245844475	,	0.084203982	,	0.259584405	,	0.289487083	,],
[	0.324416737	,	0.311652762	,	0.074200177	,	0.260375303	,],
[	0.287586352	,	0.404205209	,	0.327774954	,	0.084480033	,]],
[[	0.082506249	,	0.33951319	,	0.365057008	,	0.261120661	,],
[	0.243237201	,	0.083945157	,	0.257604965	,	0.287038055	,],
[	0.325020201	,	0.312818592	,	0.077384719	,	0.260804211	,],
[	0.291968589	,	0.407818802	,	0.331760192	,	0.086729063	,]],
[[	0.081932659	,	0.3377048	,	0.363219762	,	0.259119835	,],
[	0.247117832	,	0.091249758	,	0.262107212	,	0.290716227	,],
[	0.326740395	,	0.313681421	,	0.075853577	,	0.26273253	,],
[	0.287184943	,	0.403484914	,	0.32707435	,	0.082865856	,]],
[[	0.086528973	,	0.343578903	,	0.368785592	,	0.265253992	,],
[	0.244336782	,	0.083381082	,	0.258102244	,	0.287920912	,],
[	0.323860523	,	0.311240548	,	0.074530503	,	0.259687623	,],
[	0.288778419	,	0.405203238	,	0.328896582	,	0.086643872	,]],
[[	0.081306859	,	0.338063819	,	0.363312081	,	0.259428133	,],
[	0.244637266	,	0.086657178	,	0.259154653	,	0.28831396	,],
[	0.326764258	,	0.314842635	,	0.080718264	,	0.262620057	,],
[	0.289359372	,	0.405444914	,	0.329295703	,	0.084293016	,]],
[[	0.084278457	,	0.338693795	,	0.364129865	,	0.260185974	,],
[	0.246970869	,	0.08511865	,	0.260514386	,	0.290487046	,],
[	0.324884766	,	0.311995738	,	0.074313862	,	0.260789048	,],
[	0.287143781	,	0.403654412	,	0.327248292	,	0.083670369	,]],
[[	0.08340197	,	0.340482683	,	0.365814659	,	0.261932385	,],
[	0.243551162	,	0.083636645	,	0.257586784	,	0.287215402	,],
[	0.324557193	,	0.312137758	,	0.07631655	,	0.260306294	,],
[	0.291237065	,	0.407601035	,	0.331408903	,	0.090239316	,]],
[[	0.095834508	,	0.221255552	,	0.269800164	,	0.297651289	,],
[	0.37180933	,	0.078306746	,	0.260822361	,	0.418743691	,],
[	0.4331674	,	0.281038865	,	0.07062585	,	0.377511818	,],
[	0.314559892	,	0.292038825	,	0.226635756	,	0.093151863	,]],
[[	0.090650495	,	0.21992916	,	0.268236923	,	0.295667368	,],
[	0.370662738	,	0.078814774	,	0.259770344	,	0.417648136	,],
[	0.434621582	,	0.282726212	,	0.074252968	,	0.379033688	,],
[	0.316815487	,	0.293939397	,	0.228585827	,	0.097591443	,]],
[[	0.09217571	,	0.219200399	,	0.267632904	,	0.295456586	,],
[	0.374270885	,	0.085579629	,	0.263901585	,	0.421386194	,],
[	0.43505242	,	0.28304572	,	0.072036323	,	0.379392587	,],
[	0.316422622	,	0.293847234	,	0.228514944	,	0.094108785	,]],
[[	0.093606842	,	0.223811094	,	0.271890614	,	0.299206665	,],
[	0.370382833	,	0.077382585	,	0.259445335	,	0.417384739	,],
[	0.433200065	,	0.28105081	,	0.071267351	,	0.377396328	,],
[	0.314819055	,	0.29202175	,	0.22656853	,	0.094118196	,]],
[[	0.090221989	,	0.218963602	,	0.267374286	,	0.294761965	,],
[	0.371556401	,	0.08101554	,	0.260843033	,	0.418578323	,],
[	0.436229847	,	0.284470601	,	0.077144279	,	0.380509472	,],
[	0.319591936	,	0.296259976	,	0.231026216	,	0.101310316	,]],
[[	0.094500566	,	0.220477997	,	0.268971676	,	0.296714155	,],
[	0.372445715	,	0.078894306	,	0.26142873	,	0.419273343	,],
[	0.433583943	,	0.2815013	,	0.070682667	,	0.377914545	,],
[	0.314782766	,	0.292271459	,	0.22678776	,	0.092979339	,]],
[[	0.091262344	,	0.220819613	,	0.268982703	,	0.296387825	,],
[	0.37012605	,	0.078015439	,	0.259197676	,	0.417136299	,],
[	0.434028196	,	0.282089803	,	0.073189423	,	0.378260904	,],
[	0.315833169	,	0.29300494	,	0.227565778	,	0.096104567	,]],
[[	0.09133218	,	0.218846014	,	0.267195058	,	0.294960732	,],
[	0.373817415	,	0.084482705	,	0.26328067	,	0.420794439	,],
[	0.436105	,	0.284071908	,	0.073010091	,	0.38032431	,],
[	0.317444166	,	0.294966019	,	0.229540898	,	0.094967695	,]],
[[	0.094912526	,	0.225101592	,	0.273162272	,	0.300285916	,],
[	0.370878759	,	0.077676471	,	0.259882506	,	0.417754906	,],
[	0.433063545	,	0.280843784	,	0.070773686	,	0.377150169	,],
[	0.314613303	,	0.291852219	,	0.226305742	,	0.093549391	,]],
[[	0.09038324	,	0.219372021	,	0.267653139	,	0.295035802	,],
[	0.371090759	,	0.079965544	,	0.260340536	,	0.4181476	,],
[	0.435530085	,	0.283645015	,	0.076016576	,	0.379741432	,],
[	0.318242952	,	0.295270566	,	0.229918468	,	0.099620932	,]],
[[	0.091537526	,	0.226477883	,	0.274696985	,	0.290084313	,],
[	0.353320895	,	0.077213585	,	0.256089963	,	0.400907712	,],
[	0.414708649	,	0.276032698	,	0.071800147	,	0.360951034	,],
[	0.306563833	,	0.296554781	,	0.234514552	,	0.092637792	,]],
[[	0.089157853	,	0.227484072	,	0.275390812	,	0.290256133	,],
[	0.356591306	,	0.078363306	,	0.259267623	,	0.403823355	,],
[	0.415702412	,	0.277204304	,	0.070553724	,	0.362113545	,],
[	0.302809212	,	0.292498671	,	0.230329763	,	0.089678785	,]],
[[	0.088455697	,	0.224909559	,	0.273027723	,	0.288241468	,],
[	0.352942149	,	0.075532838	,	0.255472717	,	0.400358799	,],
[	0.413599465	,	0.274713919	,	0.069085563	,	0.359737503	,],
[	0.304925518	,	0.294433596	,	0.232247944	,	0.093889489	,]],
[[	0.092718719	,	0.23166802	,	0.279422155	,	0.294058507	,],
[	0.354863742	,	0.079974707	,	0.257804558	,	0.402409906	,],
[	0.416858435	,	0.278430934	,	0.07514834	,	0.363142116	,],
[	0.304408798	,	0.294201076	,	0.232008063	,	0.090573332	,]],
[[	0.087736511	,	0.225515569	,	0.273574929	,	0.288453527	,],
[	0.354570717	,	0.076415277	,	0.257162547	,	0.401909314	,],
[	0.414112685	,	0.275350011	,	0.068927082	,	0.360344704	,],
[	0.302821264	,	0.29236175	,	0.230057099	,	0.090429785	,]],
[[	0.090730616	,	0.226170524	,	0.274394795	,	0.28947378	,],
[	0.35321691	,	0.076565644	,	0.255911098	,	0.400718609	,],
[	0.414310722	,	0.275592455	,	0.070842833	,	0.36039752	,],
[	0.307143226	,	0.296201949	,	0.234241591	,	0.097223866	,]],
[[	0.09018681	,	0.228612644	,	0.276401346	,	0.291242941	,],
[	0.35706013	,	0.083554559	,	0.260238568	,	0.404515636	,],
[	0.417005822	,	0.278453586	,	0.071561802	,	0.363290932	,],
[	0.303040461	,	0.292844862	,	0.230541499	,	0.089433687	,]],
[[	0.08820872	,	0.225037347	,	0.273093787	,	0.288212526	,],
[	0.353104793	,	0.075383222	,	0.255562632	,	0.400355681	,],
[	0.413579523	,	0.274757267	,	0.068797269	,	0.359666205	,],
[	0.304234111	,	0.293748176	,	0.231511683	,	0.092707822	,]],
[[	0.09375179	,	0.227683031	,	0.275777395	,	0.291207608	,],
[	0.354346141	,	0.079027121	,	0.257267923	,	0.401880987	,],
[	0.416039544	,	0.277546793	,	0.073755592	,	0.362218264	,],
[	0.305386933	,	0.295256301	,	0.233031909	,	0.091274132	,]],
[[	0.088295014	,	0.226317962	,	0.274365394	,	0.289101875	,],
[	0.355214286	,	0.076994665	,	0.257838347	,	0.402499773	,],
[	0.414984893	,	0.276292176	,	0.069670849	,	0.361207726	,],
[	0.302763158	,	0.292327368	,	0.229987018	,	0.089829843	,]],
[[	0.088914102	,	0.232301462	,	0.279661029	,	0.284039761	,],
[	0.340644835	,	0.075354385	,	0.254593896	,	0.388694568	,],
[	0.401537392	,	0.273489047	,	0.069871852	,	0.348539343	,],
[	0.300627661	,	0.300919227	,	0.240204659	,	0.094740328	,]],
[[	0.08710433	,	0.233590119	,	0.280660558	,	0.284767625	,],
[	0.34301995	,	0.080537411	,	0.257404606	,	0.391156534	,],
[	0.405319736	,	0.277692479	,	0.076501162	,	0.352358549	,],
[	0.297404133	,	0.298192882	,	0.237383137	,	0.088185518	,]],
[[	0.086084605	,	0.230921387	,	0.278059233	,	0.282511672	,],
[	0.341411112	,	0.075240507	,	0.255243508	,	0.389459911	,],
[	0.402216354	,	0.274137087	,	0.069547059	,	0.349149143	,],
[	0.297800659	,	0.298436566	,	0.237654703	,	0.090401329	,]],
[[	0.09126979	,	0.238332827	,	0.285228236	,	0.289092047	,],
[	0.341057942	,	0.076805722	,	0.255024748	,	0.389108693	,],
[	0.402723479	,	0.274797966	,	0.072062653	,	0.349785982	,],
[	0.300652197	,	0.301417176	,	0.240717668	,	0.090880048	,]],
[[	0.085732322	,	0.231958857	,	0.27909151	,	0.283165164	,],
[	0.344618231	,	0.078084106	,	0.258443397	,	0.39233707	,],
[	0.404404085	,	0.276481714	,	0.071439546	,	0.351413977	,],
[	0.297065557	,	0.297691823	,	0.236844505	,	0.088293962	,]],
[[	0.087994216	,	0.231853034	,	0.279162159	,	0.28352252	,],
[	0.340778612	,	0.07513987	,	0.254604646	,	0.388760919	,],
[	0.40185623	,	0.2736943	,	0.069720433	,	0.34874791	,],
[	0.299412258	,	0.299779244	,	0.239042716	,	0.093070336	,]],
[[	0.088438403	,	0.235171106	,	0.282001484	,	0.286164655	,],
[	0.342513562	,	0.079502533	,	0.256750331	,	0.390693504	,],
[	0.404527954	,	0.276797063	,	0.075135977	,	0.351556411	,],
[	0.298129555	,	0.298977071	,	0.238177324	,	0.088669425	,]],
[[	0.085717626	,	0.230973306	,	0.278148793	,	0.282445401	,],
[	0.342463525	,	0.07621711	,	0.256257431	,	0.390229532	,],
[	0.402508076	,	0.274429435	,	0.069704748	,	0.349548677	,],
[	0.297436688	,	0.298092279	,	0.237266304	,	0.089482194	,]],
[[	0.091270116	,	0.233679987	,	0.280909271	,	0.285539973	,],
[	0.340723094	,	0.075953176	,	0.254659855	,	0.388730846	,],
[	0.402459367	,	0.274400086	,	0.071290009	,	0.349299553	,],
[	0.301681889	,	0.302538328	,	0.241731042	,	0.091871731	,]],
[[	0.086363348	,	0.232657554	,	0.279741897	,	0.283743183	,],
[	0.344805506	,	0.083111565	,	0.259271589	,	0.392884279	,],
[	0.405820487	,	0.277894984	,	0.072782656	,	0.352714658	,],
[	0.296784481	,	0.29749342	,	0.236642246	,	0.087771011	,]],
[[	0.086056231	,	0.237610575	,	0.283950292	,	0.278177731	,],
[	0.32881276	,	0.074934334	,	0.253587923	,	0.377261146	,],
[	0.390684402	,	0.272814837	,	0.069549584	,	0.338312355	,],
[	0.293460854	,	0.304280173	,	0.244888602	,	0.090044174	,]],
[[	0.084858401	,	0.239466244	,	0.285575717	,	0.279509953	,],
[	0.328178159	,	0.076367002	,	0.253317754	,	0.376866319	,],
[	0.39051385	,	0.272795184	,	0.071214875	,	0.338123752	,],
[	0.293488675	,	0.30473248	,	0.245333413	,	0.08724672	,]],
[[	0.083227463	,	0.23614028	,	0.282452741	,	0.276560946	,],
[	0.331822898	,	0.077484619	,	0.256582166	,	0.38022521	,],
[	0.393316935	,	0.275507424	,	0.071862093	,	0.340934595	,],
[	0.291620201	,	0.302725337	,	0.243255859	,	0.086841478	,]],
[[	0.088868374	,	0.243882918	,	0.289834667	,	0.283595908	,],
[	0.328215845	,	0.074956905	,	0.253045623	,	0.376730654	,],
[	0.390079861	,	0.27209082	,	0.069329498	,	0.337566395	,],
[	0.295637683	,	0.306314793	,	0.247047387	,	0.093340395	,]],
[[	0.083294953	,	0.237531053	,	0.283726254	,	0.277582316	,],
[	0.329953047	,	0.079293427	,	0.255158459	,	0.378572983	,],
[	0.392101016	,	0.274437455	,	0.073835543	,	0.339624046	,],
[	0.292355076	,	0.303543037	,	0.243980418	,	0.086433948	,]],
[[	0.085193981	,	0.237163385	,	0.283493345	,	0.277561221	,],
[	0.329706106	,	0.075537893	,	0.254367786	,	0.378067043	,],
[	0.391313904	,	0.273345215	,	0.069973675	,	0.338873022	,],
[	0.292590542	,	0.303380052	,	0.244025986	,	0.088627745	,]],
[[	0.085703636	,	0.24037821	,	0.286396466	,	0.280418999	,],
[	0.328028223	,	0.075647338	,	0.25293831	,	0.376575347	,],
[	0.390129747	,	0.272279319	,	0.070286788	,	0.337605208	,],
[	0.294674492	,	0.30589709	,	0.24655484	,	0.088302976	,]],
[[	0.083013099	,	0.236321199	,	0.282424635	,	0.276533048	,],
[	0.332292463	,	0.082908984	,	0.257660975	,	0.380780465	,],
[	0.394209544	,	0.276870035	,	0.077387083	,	0.341886114	,],
[	0.291439742	,	0.302371228	,	0.243037551	,	0.086154176	,]],
[[	0.08830313	,	0.238829414	,	0.285191021	,	0.279505106	,],
[	0.328140514	,	0.074657741	,	0.252921585	,	0.376583466	,],
[	0.389911558	,	0.271989272	,	0.068986221	,	0.337407326	,],
[	0.29487826	,	0.305520568	,	0.246179976	,	0.091937715	,]],
[[	0.083626479	,	0.237948037	,	0.284081056	,	0.277988845	,],
[	0.329349448	,	0.077970827	,	0.254343803	,	0.377800913	,],
[	0.391366444	,	0.273584826	,	0.07262592	,	0.338852059	,],
[	0.292737919	,	0.303874003	,	0.244320738	,	0.086623032	,]],
[[	0.084820538	,	0.245461863	,	0.29120702	,	0.27447757	,],
[	0.320256155	,	0.076299522	,	0.255181247	,	0.368663772	,],
[	0.382342151	,	0.274932721	,	0.070572515	,	0.330342817	,],
[	0.288776735	,	0.310208722	,	0.250346683	,	0.087819703	,]],
[[	0.084324268	,	0.247748672	,	0.293135263	,	0.276246485	,],
[	0.317333415	,	0.074616913	,	0.252288345	,	0.365905025	,],
[	0.379719652	,	0.272203373	,	0.069016501	,	0.327569163	,],
[	0.291097483	,	0.312912837	,	0.253157468	,	0.0880237	,]],
[[	0.082453198	,	0.244518402	,	0.290115068	,	0.273409616	,],
[	0.320024124	,	0.079865224	,	0.255459594	,	0.368558076	,],
[	0.382849107	,	0.275778252	,	0.074908053	,	0.330785329	,],
[	0.287530909	,	0.309444421	,	0.249591877	,	0.085627513	,]],
[[	0.088043578	,	0.247235944	,	0.292862754	,	0.276378166	,],
[	0.318603	,	0.074815517	,	0.253431284	,	0.366964961	,],
[	0.380614652	,	0.273121084	,	0.06904822	,	0.328503003	,],
[	0.290586593	,	0.31203311	,	0.25235304	,	0.09081252	,]],
[[	0.082494583	,	0.245748624	,	0.2912633	,	0.274229368	,],
[	0.318007619	,	0.076173643	,	0.253058454	,	0.36644664	,],
[	0.380126495	,	0.272860121	,	0.070520309	,	0.328034176	,],
[	0.28936582	,	0.311021172	,	0.251062799	,	0.086427255	,]],
[[	0.083921715	,	0.24507759	,	0.290742966	,	0.273958157	,],
[	0.321577973	,	0.077310055	,	0.256277126	,	0.369788319	,],
[	0.383620794	,	0.276070864	,	0.071708855	,	0.33137576	,],
[	0.288336077	,	0.309779177	,	0.249842787	,	0.086943069	,]],
[[	0.085478771	,	0.248986886	,	0.294303986	,	0.277410334	,],
[	0.317293885	,	0.074112238	,	0.252136081	,	0.365697936	,],
[	0.379754406	,	0.272207446	,	0.068645889	,	0.32741614	,],
[	0.292469634	,	0.314273713	,	0.254452076	,	0.089172555	,]],
[[	0.082086971	,	0.244460256	,	0.289998445	,	0.273184992	,],
[	0.319106144	,	0.078651973	,	0.254444347	,	0.367550199	,],
[	0.382129834	,	0.274912727	,	0.0735787	,	0.329967224	,],
[	0.288003678	,	0.309686491	,	0.249827263	,	0.085465281	,]],
[[	0.086785913	,	0.246603995	,	0.292212748	,	0.275584133	,],
[	0.319089187	,	0.075166322	,	0.253847782	,	0.367266873	,],
[	0.381407536	,	0.273971898	,	0.069704323	,	0.329301563	,],
[	0.289888179	,	0.3113596	,	0.251578909	,	0.089517376	,]],
[[	0.08343996	,	0.246835929	,	0.292262355	,	0.275197532	,],
[	0.317350713	,	0.075264224	,	0.252439322	,	0.365893649	,],
[	0.380266454	,	0.272826584	,	0.069954966	,	0.328026441	,],
[	0.289842515	,	0.311640527	,	0.251737168	,	0.086799166	,]],
[[	0.083372795	,	0.253595947	,	0.298721124	,	0.272724856	,],
[	0.310361835	,	0.079856255	,	0.255821674	,	0.359767001	,],
[	0.372723157	,	0.275510706	,	0.073933628	,	0.321188617	,],
[	0.285771551	,	0.317354416	,	0.258417759	,	0.085530119	,]],
[[	0.08544613	,	0.257875316	,	0.302757401	,	0.276689009	,],
[	0.309347633	,	0.075098926	,	0.254137071	,	0.35869139	,],
[	0.371535151	,	0.273872793	,	0.06916658	,	0.320002519	,],
[	0.289444511	,	0.320685564	,	0.261928026	,	0.091794563	,]],
[[	0.081844797	,	0.253196958	,	0.298325891	,	0.2722875	,],
[	0.308300514	,	0.075985721	,	0.253326726	,	0.357647637	,],
[	0.370662991	,	0.273178	,	0.0699367	,	0.319112891	,],
[	0.285941164	,	0.317796931	,	0.258923238	,	0.084917005	,]],
[[	0.086180455	,	0.25485428	,	0.300150368	,	0.27422983	,],
[	0.312734003	,	0.07799324	,	0.257316191	,	0.361843969	,],
[	0.374430947	,	0.276913737	,	0.071940012	,	0.322994445	,],
[	0.28653489	,	0.318151839	,	0.259184072	,	0.087300138	,]],
[[	0.083157192	,	0.255456642	,	0.300546644	,	0.274263053	,],
[	0.308506153	,	0.074693209	,	0.253155658	,	0.357747325	,],
[	0.370546018	,	0.272891058	,	0.068608508	,	0.319013963	,],
[	0.288880243	,	0.320642619	,	0.261687792	,	0.087305146	,]],
[[	0.082567038	,	0.25321267	,	0.2984457	,	0.272270774	,],
[	0.31009788	,	0.078897029	,	0.255305294	,	0.359378917	,],
[	0.372066951	,	0.274789798	,	0.072665859	,	0.320415319	,],
[	0.285660832	,	0.317230812	,	0.2582054	,	0.085085431	,]],
[[	0.087018235	,	0.259561581	,	0.304431198	,	0.278200758	,],
[	0.310255792	,	0.075783788	,	0.254916415	,	0.359446614	,],
[	0.372180691	,	0.274533875	,	0.069591953	,	0.320582636	,],
[	0.288676392	,	0.320014083	,	0.261150393	,	0.090426613	,]],
[[	0.082135787	,	0.253722189	,	0.298756766	,	0.272729195	,],
[	0.308598692	,	0.075820323	,	0.253470437	,	0.357865109	,],
[	0.370585332	,	0.272898966	,	0.069423634	,	0.318915282	,],
[	0.286692444	,	0.318499673	,	0.259514216	,	0.085345595	,]],
[[	0.085178824	,	0.254509997	,	0.299588092	,	0.273620602	,],
[	0.312097976	,	0.082327566	,	0.257499368	,	0.361416788	,],
[	0.374491361	,	0.277359433	,	0.076370742	,	0.322870289	,],
[	0.285959169	,	0.317593219	,	0.258636906	,	0.086294977	,]],
[[	0.084054571	,	0.256407715	,	0.301372521	,	0.27512111	,],
[	0.308843259	,	0.074803581	,	0.253444491	,	0.358015878	,],
[	0.370855202	,	0.273159612	,	0.068678707	,	0.319161951	,],
[	0.289936771	,	0.321683885	,	0.262799379	,	0.088310684	,]],
[[	0.081036689	,	0.259344384	,	0.303602137	,	0.267597296	,],
[	0.295850689	,	0.075593941	,	0.251747766	,	0.345302488	,],
[	0.357635042	,	0.271344779	,	0.069170006	,	0.306641996	,],
[	0.279293761	,	0.321573045	,	0.262972549	,	0.083463505	,]],
[[	0.083944697	,	0.26438391	,	0.308354459	,	0.272216731	,],
[	0.299960465	,	0.082848689	,	0.256301647	,	0.349319845	,],
[	0.361767627	,	0.275970659	,	0.07614604	,	0.310779068	,],
[	0.28109889	,	0.323136273	,	0.264663679	,	0.087187365	,]],
[[	0.079979917	,	0.259307571	,	0.303463052	,	0.267510799	,],
[	0.295759348	,	0.074316225	,	0.251449843	,	0.34512839	,],
[	0.358056663	,	0.271593798	,	0.068388052	,	0.307011158	,],
[	0.281346901	,	0.323791031	,	0.265247359	,	0.084815371	,]],
[[	0.083894884	,	0.260718792	,	0.30494945	,	0.269161306	,],
[	0.296947033	,	0.077978481	,	0.253073439	,	0.34636596	,],
[	0.358808079	,	0.272654105	,	0.071301104	,	0.307767362	,],
[	0.279060659	,	0.32139054	,	0.26281968	,	0.083851145	,]],
[[	0.081246479	,	0.261478303	,	0.305571953	,	0.269376853	,],
[	0.298538883	,	0.076082155	,	0.253919513	,	0.347626233	,],
[	0.360409514	,	0.274019647	,	0.070374249	,	0.309576025	,],
[	0.28281431	,	0.32485211	,	0.266414904	,	0.090173893	,]],
[[	0.080521217	,	0.259213913	,	0.303371768	,	0.267345012	,],
[	0.295607292	,	0.07480439	,	0.251319039	,	0.344922885	,],
[	0.357694001	,	0.271303484	,	0.068786204	,	0.306603785	,],
[	0.279648466	,	0.321931007	,	0.263300784	,	0.083454789	,]],
[[	0.084968947	,	0.265268759	,	0.30915114	,	0.273195079	,],
[	0.298994775	,	0.081385489	,	0.255244507	,	0.348295196	,],
[	0.360811731	,	0.274905463	,	0.074623037	,	0.309780762	,],
[	0.280541189	,	0.322478544	,	0.263952095	,	0.086071976	,]],
[[	0.080133517	,	0.259757266	,	0.303803316	,	0.267782191	,],
[	0.296417585	,	0.074465166	,	0.251976333	,	0.345569009	,],
[	0.358639747	,	0.272021863	,	0.068673879	,	0.307608353	,],
[	0.282217686	,	0.324639651	,	0.266067501	,	0.085587248	,]],
[[	0.082768877	,	0.260139985	,	0.304385708	,	0.268445571	,],
[	0.296411371	,	0.076990371	,	0.252365141	,	0.345807329	,],
[	0.358355795	,	0.272081163	,	0.070419521	,	0.307271433	,],
[	0.279030163	,	0.321317437	,	0.262756228	,	0.083473207	,]],
[[	0.082113386	,	0.262408498	,	0.306509387	,	0.270271904	,],
[	0.299366004	,	0.076850364	,	0.254687853	,	0.348280989	,],
[	0.361611298	,	0.275116617	,	0.0714483	,	0.310577482	,],
[	0.282047508	,	0.32408957	,	0.26559756	,	0.088761631	,]],
[[	0.078098336	,	0.261688104	,	0.304101699	,	0.260446197	,],
[	0.280225781	,	0.073053741	,	0.246099062	,	0.329053272	,],
[	0.34048312	,	0.265573008	,	0.066374911	,	0.291139319	,],
[	0.269712337	,	0.320191917	,	0.263392468	,	0.080441523	,]],
[[	0.081976112	,	0.267188652	,	0.309322183	,	0.265860099	,],
[	0.283412789	,	0.07924281	,	0.249908241	,	0.332214593	,],
[	0.3428569	,	0.26840557	,	0.071530443	,	0.293415755	,],
[	0.270541659	,	0.320739951	,	0.264071292	,	0.083092733	,]],
[[	0.077425766	,	0.261819718	,	0.304082854	,	0.260644306	,],
[	0.281439049	,	0.073006636	,	0.247114255	,	0.33011126	,],
[	0.341770399	,	0.266713148	,	0.066793535	,	0.292427645	,],
[	0.272425353	,	0.322773786	,	0.266089594	,	0.082618185	,]],
[[	0.080802125	,	0.262915389	,	0.305301142	,	0.261878638	,],
[	0.28087399	,	0.074812648	,	0.246983234	,	0.32964091	,],
[	0.340984863	,	0.266242302	,	0.06775365	,	0.291559389	,],
[	0.269465026	,	0.319878671	,	0.263081239	,	0.080805429	,]],
[[	0.078939559	,	0.264292805	,	0.306544084	,	0.262885017	,],
[	0.284731016	,	0.075840978	,	0.250204103	,	0.333240675	,],
[	0.345089224	,	0.270176361	,	0.069752931	,	0.29579267	,],
[	0.272346703	,	0.322519711	,	0.265872105	,	0.086035366	,]],
[[	0.077684053	,	0.261455771	,	0.303897986	,	0.260256113	,],
[	0.280426889	,	0.072783474	,	0.246224572	,	0.329125505	,],
[	0.340729967	,	0.265700688	,	0.06621927	,	0.291260052	,],
[	0.270408179	,	0.320787123	,	0.263936406	,	0.080932579	,]],
[[	0.083139037	,	0.268473149	,	0.310520681	,	0.2671001	,],
[	0.282380851	,	0.077564189	,	0.248611349	,	0.331167371	,],
[	0.342223787	,	0.267645303	,	0.070247463	,	0.292775897	,],
[	0.270363105	,	0.320486418	,	0.263801774	,	0.082454836	,]],
[[	0.077721309	,	0.262431962	,	0.304633497	,	0.261101847	,],
[	0.282089863	,	0.073552484	,	0.247684681	,	0.330700545	,],
[	0.342655007	,	0.26772368	,	0.067523229	,	0.293197225	,],
[	0.273555527	,	0.323963241	,	0.267198051	,	0.083595896	,]],
[[	0.079756946	,	0.262399767	,	0.304658743	,	0.261273129	,],
[	0.280478744	,	0.073968775	,	0.246411913	,	0.329148068	,],
[	0.340999171	,	0.266046991	,	0.067198142	,	0.291423781	,],
[	0.269545486	,	0.319934642	,	0.263107902	,	0.080518954	,]],
[[	0.07993643	,	0.265215675	,	0.307381611	,	0.26377258	,],
[	0.284441616	,	0.08115992	,	0.251048669	,	0.333148567	,],
[	0.344257406	,	0.269968499	,	0.073771651	,	0.294906522	,],
[	0.27159059	,	0.321772369	,	0.265085063	,	0.084765075	,]],
[[	0.074898425	,	0.261906342	,	0.303076276	,	0.25009039	,],
[	0.262822512	,	0.070892294	,	0.240827973	,	0.311241965	,],
[	0.321986033	,	0.259555229	,	0.064244981	,	0.274387472	,],
[	0.259969552	,	0.319262588	,	0.264952836	,	0.077401949	,]],
[[	0.07864199	,	0.267364282	,	0.30820372	,	0.255309198	,],
[	0.263476668	,	0.07402034	,	0.241996383	,	0.311909087	,],
[	0.322151348	,	0.260082745	,	0.06670646	,	0.274509751	,],
[	0.260024091	,	0.319322708	,	0.265082136	,	0.079505141	,]],
[[	0.073941444	,	0.261840678	,	0.302854741	,	0.24993988	,],
[	0.265673816	,	0.072979708	,	0.243504964	,	0.313868249	,],
[	0.324628474	,	0.262096857	,	0.066214381	,	0.276908902	,],
[	0.262725823	,	0.322122952	,	0.267949464	,	0.079740824	,]],
[[	0.077385079	,	0.262979313	,	0.304108234	,	0.251304229	,],
[	0.262418862	,	0.071335556	,	0.240593751	,	0.310885028	,],
[	0.321381956	,	0.258987768	,	0.064476441	,	0.273627511	,],
[	0.258922277	,	0.318408883	,	0.264179175	,	0.077217237	,]],
[[	0.075736028	,	0.264360943	,	0.30535232	,	0.252371786	,],
[	0.265160243	,	0.077191048	,	0.243956366	,	0.313456928	,],
[	0.324153048	,	0.262441705	,	0.070342077	,	0.276621618	,],
[	0.261578422	,	0.32083371	,	0.266636031	,	0.082250379	,]],
[[	0.074189608	,	0.261476796	,	0.30263953	,	0.249661334	,],
[	0.263679119	,	0.071319447	,	0.241424377	,	0.311795988	,],
[	0.322624586	,	0.260127035	,	0.064665078	,	0.274994712	,],
[	0.260384524	,	0.319734171	,	0.265412837	,	0.07779222	,]],
[[	0.079958841	,	0.268603773	,	0.309418416	,	0.256571006	,],
[	0.263104173	,	0.073318539	,	0.241544242	,	0.31138514	,],
[	0.322025036	,	0.259935589	,	0.066113917	,	0.27432231	,],
[	0.259689917	,	0.318924456	,	0.264754827	,	0.078651708	,]],
[[	0.074398254	,	0.262544979	,	0.30349331	,	0.25062629	,],
[	0.266703732	,	0.073859478	,	0.244467953	,	0.314805978	,],
[	0.325601838	,	0.263081539	,	0.06729559	,	0.278021655	,],
[	0.263977908	,	0.323397977	,	0.269198903	,	0.080958694	,]],
[[	0.076338383	,	0.262440807	,	0.30335855	,	0.25061926	,],
[	0.262502215	,	0.071053501	,	0.240542001	,	0.310713076	,],
[	0.321303973	,	0.258966898	,	0.063954995	,	0.27358669	,],
[	0.25925906	,	0.318623041	,	0.264316548	,	0.077095425	,]],
[[	0.07665923	,	0.265459375	,	0.306312909	,	0.253336804	,],
[	0.264523548	,	0.075918435	,	0.243185571	,	0.312715979	,],
[	0.323119364	,	0.261342771	,	0.068761928	,	0.27559825	,],
[	0.260978434	,	0.32019402	,	0.266000747	,	0.081040458	,]],
[[	0.07982986	,	0.289459382	,	0.333161953	,	0.266362773	,],
[	0.275252102	,	0.077449481	,	0.261203211	,	0.326925772	,],
[	0.340812738	,	0.28330276	,	0.072738993	,	0.288913092	,],
[	0.279881474	,	0.353065278	,	0.294920913	,	0.084505912	,]],
[[	0.08499191	,	0.295836336	,	0.339284403	,	0.272657124	,],
[	0.273650684	,	0.078021487	,	0.259982619	,	0.32568279	,],
[	0.338949758	,	0.281690497	,	0.072246757	,	0.286955937	,],
[	0.278611429	,	0.351918312	,	0.293689824	,	0.084662746	,]],
[[	0.079446334	,	0.289758034	,	0.333346435	,	0.266681837	,],
[	0.278176604	,	0.085520227	,	0.265049923	,	0.329928049	,],
[	0.342347028	,	0.285678974	,	0.078812968	,	0.2904593	,],
[	0.282979217	,	0.355943659	,	0.297879433	,	0.091403043	,]],
[[	0.082418217	,	0.290554475	,	0.334292804	,	0.267595016	,],
[	0.274122789	,	0.07706368	,	0.260077505	,	0.325805945	,],
[	0.339302718	,	0.281835599	,	0.071590686	,	0.287310005	,],
[	0.278638339	,	0.35205105	,	0.293812711	,	0.08365735	,]],
[[	0.081747431	,	0.292692812	,	0.336240383	,	0.26942854	,],
[	0.275042888	,	0.08050646	,	0.261617599	,	0.32689996	,],
[	0.339890147	,	0.282791391	,	0.074425794	,	0.287859654	,],
[	0.279805352	,	0.35310855	,	0.29486231	,	0.086764539	,]],
[[	0.079681853	,	0.289488014	,	0.333190824	,	0.266406676	,],
[	0.276181587	,	0.078220082	,	0.261868077	,	0.327707436	,],
[	0.341749643	,	0.284161573	,	0.073413997	,	0.289701202	,],
[	0.280827716	,	0.353973772	,	0.29577363	,	0.085278104	,]],
[[	0.085312114	,	0.292397246	,	0.336261258	,	0.269437721	,],
[	0.273826928	,	0.077575486	,	0.260030799	,	0.325587648	,],
[	0.338816128	,	0.281490499	,	0.071886472	,	0.286789083	,],
[	0.278286507	,	0.351564253	,	0.293182571	,	0.083961918	,]],
[[	0.080092751	,	0.290659813	,	0.33412039	,	0.267534788	,],
[	0.277411173	,	0.084153144	,	0.264117848	,	0.329064101	,],
[	0.34178478	,	0.285000342	,	0.077648377	,	0.289756824	,],
[	0.282100746	,	0.355163112	,	0.297028378	,	0.0900787	,]],
[[	0.081289283	,	0.289980654	,	0.333665984	,	0.266992433	,],
[	0.274571702	,	0.077157993	,	0.260484047	,	0.326302795	,],
[	0.339839607	,	0.28233397	,	0.071883255	,	0.287785061	,],
[	0.278946926	,	0.35226237	,	0.294099605	,	0.083784107	,]],
[[	0.082892484	,	0.293935013	,	0.337375024	,	0.270661619	,],
[	0.274611725	,	0.079502749	,	0.261070268	,	0.326448381	,],
[	0.33958948	,	0.282506845	,	0.073576948	,	0.287570625	,],
[	0.279079557	,	0.35237292	,	0.294163901	,	0.085519661	,]],
[[	0.0797154	,	0.299698557	,	0.342267176	,	0.265275951	,],
[	0.26796056	,	0.080356178	,	0.265070447	,	0.31984248	,],
[	0.333838516	,	0.287736907	,	0.07423204	,	0.281313847	,],
[	0.279679305	,	0.363921324	,	0.304843642	,	0.084626592	,]],
[[	0.083724864	,	0.305139383	,	0.347305445	,	0.27071758	,],
[	0.264874366	,	0.078797427	,	0.262405298	,	0.317006107	,],
[	0.330227416	,	0.284371355	,	0.071835003	,	0.277680153	,],
[	0.277561626	,	0.361882693	,	0.302754865	,	0.083776077	,]],
[[	0.078852284	,	0.299861505	,	0.342244555	,	0.26535818	,],
[	0.267568462	,	0.084418459	,	0.265720202	,	0.319692797	,],
[	0.33251389	,	0.287298153	,	0.076962197	,	0.280098784	,],
[	0.281820344	,	0.365857071	,	0.306914297	,	0.090099784	,]],
[[	0.082026556	,	0.30074398	,	0.343207745	,	0.266439648	,],
[	0.266000977	,	0.078931079	,	0.263259709	,	0.318003259	,],
[	0.331557477	,	0.285517882	,	0.072256425	,	0.278947757	,],
[	0.278028612	,	0.362409224	,	0.303334743	,	0.0832861	,]],
[[	0.08065833	,	0.302232253	,	0.344527235	,	0.267663821	,],
[	0.265144828	,	0.080174868	,	0.262964337	,	0.317235282	,],
[	0.330758027	,	0.285177366	,	0.073349171	,	0.278242711	,],
[	0.278497099	,	0.362914175	,	0.303692967	,	0.085388654	,]],
[[	0.079031096	,	0.299294034	,	0.341863699	,	0.264915141	,],
[	0.269346546	,	0.081581019	,	0.266396224	,	0.321075947	,],
[	0.334876314	,	0.288731928	,	0.075190805	,	0.282305705	,],
[	0.28056344	,	0.364668942	,	0.305677316	,	0.085290111	,]],
[[	0.084971064	,	0.306450439	,	0.348603885	,	0.272006339	,],
[	0.265066039	,	0.078678336	,	0.262463786	,	0.316958623	,],
[	0.33044453	,	0.284551723	,	0.07176295	,	0.277773308	,],
[	0.277414725	,	0.361732327	,	0.302510771	,	0.083175936	,]],
[[	0.079015933	,	0.300054942	,	0.34241355	,	0.26567245	,],
[	0.266840543	,	0.083143846	,	0.264869186	,	0.318869639	,],
[	0.332025233	,	0.286623486	,	0.075827186	,	0.279499424	,],
[	0.280949179	,	0.365124161	,	0.306124441	,	0.088819786	,]],
[[	0.080993225	,	0.300265111	,	0.34277249	,	0.265870493	,],
[	0.266953911	,	0.079691814	,	0.264237268	,	0.318873503	,],
[	0.332118583	,	0.286117611	,	0.072636486	,	0.279502569	,],
[	0.27877616	,	0.362986915	,	0.303937494	,	0.083614446	,]],
[[	0.08162601	,	0.303151517	,	0.345507573	,	0.268748536	,],
[	0.265091783	,	0.079545662	,	0.262739165	,	0.31708272	,],
[	0.330302545	,	0.28462354	,	0.072520694	,	0.277711746	,],
[	0.278186207	,	0.362485339	,	0.30337848	,	0.084592645	,]],
[[	0.079948788	,	0.31043973	,	0.353990124	,	0.266324715	,],
[	0.263323465	,	0.082543713	,	0.270736658	,	0.315993334	,],
[	0.329745757	,	0.29314812	,	0.076711415	,	0.277011265	,],
[	0.279655036	,	0.374123795	,	0.314722891	,	0.083955092	,]],
[[	0.084328574	,	0.316139741	,	0.359260942	,	0.271978159	,],
[	0.258911004	,	0.079609222	,	0.266940674	,	0.31197297	,],
[	0.325142897	,	0.288947022	,	0.073324626	,	0.272406467	,],
[	0.27830429	,	0.372913128	,	0.313558095	,	0.08407915	,]],
[[	0.0791936	,	0.310540057	,	0.353883322	,	0.266410288	,],
[	0.260680895	,	0.084241049	,	0.269222455	,	0.313800523	,],
[	0.32692342	,	0.291245035	,	0.077602168	,	0.274266419	,],
[	0.282983727	,	0.377317547	,	0.318161247	,	0.087104971	,]],
[[	0.082147679	,	0.311504997	,	0.354975421	,	0.267501938	,],
[	0.260714679	,	0.08032545	,	0.268351925	,	0.313618205	,],
[	0.327215839	,	0.290766668	,	0.074496515	,	0.274466174	,],
[	0.278208364	,	0.372790661	,	0.313443804	,	0.083006048	,]],
[[	0.081253512	,	0.313256688	,	0.356541348	,	0.269000504	,],
[	0.25885374	,	0.080596317	,	0.267074	,	0.312002718	,],
[	0.325783232	,	0.289704524	,	0.074705041	,	0.272995523	,],
[	0.279540636	,	0.374213912	,	0.314830503	,	0.086314001	,]],
[[	0.079475162	,	0.310350423	,	0.353856795	,	0.266174936	,],
[	0.263007005	,	0.087804965	,	0.271761504	,	0.315955314	,],
[	0.329124884	,	0.29372796	,	0.081411994	,	0.276494449	,],
[	0.28061296	,	0.374938665	,	0.315582588	,	0.08479126	,]],
[[	0.085699577	,	0.317474397	,	0.360471863	,	0.273251605	,],
[	0.259137602	,	0.079309177	,	0.266812744	,	0.312013438	,],
[	0.32550327	,	0.289245788	,	0.073290611	,	0.272797648	,],
[	0.278354143	,	0.37297806	,	0.313518901	,	0.08381432	,]],
[[	0.079507185	,	0.310968511	,	0.354194691	,	0.266819424	,],
[	0.260397578	,	0.083140201	,	0.268661139	,	0.313337486	,],
[	0.32630422	,	0.290534017	,	0.076445956	,	0.273571273	,],
[	0.2825209	,	0.376840936	,	0.317621226	,	0.090139005	,]],
[[	0.081486384	,	0.311141612	,	0.354485447	,	0.266972442	,],
[	0.261756678	,	0.081113301	,	0.269234851	,	0.314586182	,],
[	0.328129489	,	0.291710637	,	0.075268248	,	0.27536173	,],
[	0.278688767	,	0.373181324	,	0.31383829	,	0.083160289	,]],
[[	0.082392329	,	0.314437369	,	0.357584582	,	0.270128821	,],
[	0.258835483	,	0.079979183	,	0.266887689	,	0.311779009	,],
[	0.325427205	,	0.289269388	,	0.073932054	,	0.272596304	,],
[	0.27939804	,	0.37397468	,	0.314564021	,	0.08550095	,]],
[[	0.079744799	,	0.317733693	,	0.36097116	,	0.26435689	,],
[	0.252672995	,	0.087309748	,	0.272150941	,	0.305766468	,],
[	0.31985672	,	0.294147988	,	0.081106768	,	0.266814695	,],
[	0.276716972	,	0.380019081	,	0.321602106	,	0.082974118	,]],
[[	0.083871716	,	0.323150991	,	0.366124378	,	0.269927648	,],
[	0.249431569	,	0.079734458	,	0.267825374	,	0.302461246	,],
[	0.316982725	,	0.290283597	,	0.073918786	,	0.26370645	,],
[	0.276030558	,	0.379479142	,	0.320999743	,	0.083783422	,]],
[[	0.078835961	,	0.317635485	,	0.360828382	,	0.264381619	,],
[	0.25007332	,	0.082796351	,	0.268967129	,	0.303127273	,],
[	0.316922271	,	0.290862632	,	0.076231791	,	0.263767185	,],
[	0.280295882	,	0.38341938	,	0.325148809	,	0.086175616	,]],
[[	0.081980387	,	0.318683907	,	0.361929057	,	0.265352911	,],
[	0.251915597	,	0.081334432	,	0.270108608	,	0.304843843	,],
[	0.319414169	,	0.29250593	,	0.075783518	,	0.266179029	,],
[	0.275828409	,	0.379327673	,	0.320695882	,	0.082381011	,]],
[[	0.080900318	,	0.320365033	,	0.363499503	,	0.266963887	,],
[	0.248621116	,	0.079666312	,	0.267192587	,	0.301556297	,],
[	0.316550462	,	0.29007215	,	0.074210654	,	0.263293737	,],
[	0.277102718	,	0.380671238	,	0.322191405	,	0.085834976	,]],
[[	0.079370071	,	0.317648078	,	0.360897584	,	0.264158945	,],
[	0.251861182	,	0.085978944	,	0.271193407	,	0.304871016	,],
[	0.319033985	,	0.293135786	,	0.079610442	,	0.265815493	,],
[	0.277661638	,	0.380846772	,	0.322379963	,	0.083723914	,]],
[[	0.08517079	,	0.320819179	,	0.364193383	,	0.267518519	,],
[	0.249802063	,	0.079787337	,	0.268079912	,	0.302661444	,],
[	0.317295703	,	0.290623853	,	0.073932776	,	0.264016673	,],
[	0.275919663	,	0.379251519	,	0.320767951	,	0.083144419	,]],
[[	0.07921573	,	0.31808155	,	0.361200499	,	0.264720998	,],
[	0.249734669	,	0.081882714	,	0.268493919	,	0.302625319	,],
[	0.317018483	,	0.290680557	,	0.075696925	,	0.263603197	,],
[	0.280042488	,	0.38333961	,	0.324827559	,	0.089750465	,]],
[[	0.080954931	,	0.318203778	,	0.361477742	,	0.264814006	,],
[	0.253300776	,	0.082535413	,	0.271327942	,	0.306025656	,],
[	0.320662477	,	0.29373807	,	0.07685039	,	0.267334323	,],
[	0.276114795	,	0.3796062	,	0.321031817	,	0.082603762	,]],
[[	0.081893324	,	0.321199908	,	0.364236849	,	0.267842733	,],
[	0.249195262	,	0.079725068	,	0.267625809	,	0.302030181	,],
[	0.316488293	,	0.289821407	,	0.073707819	,	0.263049847	,],
[	0.276753224	,	0.380186279	,	0.321724928	,	0.084930709	,]],
[[	0.079794884	,	0.330305221	,	0.373899663	,	0.266266658	,],
[	0.244522921	,	0.086589455	,	0.27669333	,	0.300146022	,],
[	0.311707564	,	0.297278124	,	0.080207173	,	0.25912082	,],
[	0.278218021	,	0.392928513	,	0.334766646	,	0.083663825	,]],
[[	0.084473745	,	0.336029791	,	0.379207587	,	0.272132114	,],
[	0.243903403	,	0.08195341	,	0.274977331	,	0.299318362	,],
[	0.310789172	,	0.295466358	,	0.075537137	,	0.25822619	,],
[	0.276723279	,	0.391745431	,	0.333376988	,	0.083617285	,]],
[[	0.079284803	,	0.330560394	,	0.373942358	,	0.266629511	,],
[	0.242600887	,	0.082866508	,	0.274303733	,	0.298046366	,],
[	0.309721143	,	0.295015376	,	0.076545501	,	0.257216668	,],
[	0.280964906	,	0.395898135	,	0.337690403	,	0.090480005	,]],
[[	0.082316832	,	0.3315636	,	0.375110804	,	0.267561731	,],
[	0.24691719	,	0.090406817	,	0.279258175	,	0.302250602	,],
[	0.313919089	,	0.299974645	,	0.084136887	,	0.261529413	,],
[	0.276635183	,	0.391707859	,	0.333425347	,	0.082573009	,]],
[[	0.081285367	,	0.333133337	,	0.376482588	,	0.26918968	,],
[	0.242447999	,	0.081288622	,	0.273800949	,	0.297877972	,],
[	0.309825398	,	0.294734183	,	0.075215411	,	0.257392955	,],
[	0.27750549	,	0.392713016	,	0.334375866	,	0.085454166	,]],
[[	0.079333487	,	0.330082145	,	0.373680161	,	0.266057699	,],
[	0.243966214	,	0.085409625	,	0.275833212	,	0.299362977	,],
[	0.311140548	,	0.296667367	,	0.079068209	,	0.2585357	,],
[	0.279192396	,	0.393857046	,	0.335667178	,	0.084585899	,]],
[[	0.08506129	,	0.33321776	,	0.376929312	,	0.269238035	,],
[	0.24466065	,	0.082504669	,	0.275639292	,	0.299885388	,],
[	0.311797473	,	0.296488718	,	0.076333546	,	0.25924863	,],
[	0.276507203	,	0.391463147	,	0.333146427	,	0.08303509	,]],
[[	0.079854359	,	0.331253736	,	0.374657514	,	0.267363307	,],
[	0.242838359	,	0.082422399	,	0.274285261	,	0.298169156	,],
[	0.309602052	,	0.294705088	,	0.075800816	,	0.256924919	,],
[	0.280235957	,	0.395083614	,	0.336836239	,	0.089109065	,]],
[[	0.081183845	,	0.330865171	,	0.37441683	,	0.26692116	,],
[	0.245973143	,	0.08890827	,	0.278205316	,	0.30125129	,],
[	0.313336774	,	0.299202185	,	0.082835472	,	0.260851847	,],
[	0.277255767	,	0.392109427	,	0.333888152	,	0.082793547	,]],
[[	0.082437118	,	0.334071687	,	0.377360183	,	0.270074115	,],
[	0.243056984	,	0.08137674	,	0.274264418	,	0.29836039	,],
[	0.310344953	,	0.295118659	,	0.075264822	,	0.257643915	,],
[	0.277224433	,	0.392266932	,	0.334007577	,	0.084604586	]]],dtype=np.float32)
								
y_train_x=np.array([[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	]],dtype=np.float32)						
								
y_train_y=np.array([[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	]],dtype=np.float32)						
								
y_test_x=np.array([[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.060606061	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.121212121	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.181818182	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.242424242	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.303030303	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.363636364	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.424242424	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.484848485	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.545454545	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.606060606	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.666666667	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.727272727	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.787878788	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.03030303	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.090909091	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.151515152	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.212121212	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.272727273	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.393939394	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.454545455	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.515151515	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.575757576	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.636363636	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.696969697	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	],						
[	0.757575758	]],dtype=np.float32)						
								
y_test_y=np.array([[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.055555556	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.111111111	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.166666667	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.222222222	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.277777778	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.333333333	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	],						
[	0.388888889	]],dtype=np.float32)						


In [3]:
x_test=np.expand_dims(x_test,axis=-1)
x_train=np.expand_dims(x_train,axis=-1)
x_train[:,:,0,0]+=-0.00238
x_train[:,:,1,0]+=0.001496742
x_train[:,:,2,0]+=0.009334
x_train[:,:,3,0]+=-0.00588
x_test[:,:,0,0]+=-0.00238
x_test[:,:,1,0]+=0.001496742
x_test[:,:,2,0]+=0.009334
x_test[:,:,3,0]+=-0.00588
x_train[:,0,0,0]=0
x_train[:,1,1,0]=0
x_train[:,2,2,0]=0
x_train[:,3,3,0]=0
x_test[:,0,0,0]=0
x_test[:,1,1,0]=0
x_test[:,2,2,0]=0
x_test[:,3,3,0]=0
for i in range(len(x_train)):
    for j in range(4):
        x_train[i,j,0:4,:]=x_train[i,j,0:4,:]/max(x_train[i,j,0:4,:])
for i in range(len(x_test)):
    for j in range(4):
        x_test[i,j,0:4,:]=x_test[i,j,0:4,:]/max(x_test[i,j,0:4,:])

In [4]:
len(x_test)

1330

In [5]:
x_test.shape

(1330, 4, 4, 1)

In [6]:
x_train.shape

(560, 4, 4, 1)

In [7]:
y_train_x.shape

(560, 1)

In [8]:
y_train_y.shape

(560, 1)

In [9]:
y_test_x.shape

(1330, 1)

In [10]:
y_test_y.shape

(1330, 1)